In [1]:
import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error
pd.options.display.precision = 15

import lightgbm as lgb
import xgboost as xgb
import time
import datetime
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
import gc
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from scipy.signal import hilbert
from scipy.signal import hann
from scipy.signal import convolve
from scipy import stats
from sklearn.kernel_ridge import KernelRidge

/Users/shuozhang/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
%%time
train = pd.read_csv('train.csv', dtype={'acoustic_data': np.int16, 'time_to_failure': np.float32})

CPU times: user 3min 42s, sys: 33.9 s, total: 4min 16s
Wall time: 4min 59s


In [8]:
rows = 150000
segments = int(np.floor(train.shape[0] / rows))

def add_trend_feature(arr, abs_values=False):
    idx = np.array(range(len(arr)))
    if abs_values:
        arr = np.abs(arr)
    lr = LinearRegression()
    lr.fit(idx.reshape(-1, 1), arr)
    return lr.coef_[0]

def classic_sta_lta(x, length_sta, length_lta):
    
    sta = np.cumsum(x ** 2)

    # Convert to float
    sta = np.require(sta, dtype=np.float)

    # Copy for LTA
    lta = sta.copy()

    # Compute the STA and the LTA
    sta[length_sta:] = sta[length_sta:] - sta[:-length_sta]
    sta /= length_sta
    lta[length_lta:] = lta[length_lta:] - lta[:-length_lta]
    lta /= length_lta

    # Pad zeros
    sta[:length_lta - 1] = 0

    # Avoid division by zero by setting zero values to tiny float
    dtiny = np.finfo(0.0).tiny
    idx = lta < dtiny
    lta[idx] = dtiny

    return sta / lta

X_tr = pd.DataFrame(index=range(segments), dtype=np.float64)

y_tr = pd.DataFrame(index=range(segments), dtype=np.float64, columns=['time_to_failure'])

total_mean = train['acoustic_data'].mean()
total_std = train['acoustic_data'].std()
total_max = train['acoustic_data'].max()
total_min = train['acoustic_data'].min()
total_sum = train['acoustic_data'].sum()
total_abs_sum = np.abs(train['acoustic_data']).sum()

for segment in tqdm_notebook(range(segments)):
    seg = train.iloc[segment*rows:segment*rows+rows]
    x = pd.Series(seg['acoustic_data'].values)
    y = seg['time_to_failure'].values[-1]
    
    y_tr.loc[segment, 'time_to_failure'] = y
    X_tr.loc[segment, 'mean'] = x.mean()
    X_tr.loc[segment, 'std'] = x.std()
    X_tr.loc[segment, 'max'] = x.max()
    X_tr.loc[segment, 'min'] = x.min()
    
    
    X_tr.loc[segment, 'mean_change_abs'] = np.mean(np.diff(x))
    X_tr.loc[segment, 'mean_change_rate'] = np.mean(np.nonzero((np.diff(x) / x[:-1]))[0])
    X_tr.loc[segment, 'abs_max'] = np.abs(x).max()
    X_tr.loc[segment, 'abs_min'] = np.abs(x).min()
    
    X_tr.loc[segment, 'std_first_50000'] = x[:50000].std()
    X_tr.loc[segment, 'std_last_50000'] = x[-50000:].std()
    X_tr.loc[segment, 'std_first_10000'] = x[:10000].std()
    X_tr.loc[segment, 'std_last_10000'] = x[-10000:].std()
    
    X_tr.loc[segment, 'avg_first_50000'] = x[:50000].mean()
    X_tr.loc[segment, 'avg_last_50000'] = x[-50000:].mean()
    X_tr.loc[segment, 'avg_first_10000'] = x[:10000].mean()
    X_tr.loc[segment, 'avg_last_10000'] = x[-10000:].mean()
    
    X_tr.loc[segment, 'min_first_50000'] = x[:50000].min()
    X_tr.loc[segment, 'min_last_50000'] = x[-50000:].min()
    X_tr.loc[segment, 'min_first_10000'] = x[:10000].min()
    X_tr.loc[segment, 'min_last_10000'] = x[-10000:].min()
    
    X_tr.loc[segment, 'max_first_50000'] = x[:50000].max()
    X_tr.loc[segment, 'max_last_50000'] = x[-50000:].max()
    X_tr.loc[segment, 'max_first_10000'] = x[:10000].max()
    X_tr.loc[segment, 'max_last_10000'] = x[-10000:].max()
    
    X_tr.loc[segment, 'max_to_min'] = x.max() / np.abs(x.min())
    X_tr.loc[segment, 'max_to_min_diff'] = x.max() - np.abs(x.min())
    X_tr.loc[segment, 'count_big'] = len(x[np.abs(x) > 500])
    X_tr.loc[segment, 'sum'] = x.sum()
    
    X_tr.loc[segment, 'mean_change_rate_first_50000'] = np.mean(np.nonzero((np.diff(x[:50000]) / x[:50000][:-1]))[0])
    X_tr.loc[segment, 'mean_change_rate_last_50000'] = np.mean(np.nonzero((np.diff(x[-50000:]) / x[-50000:][:-1]))[0])
    X_tr.loc[segment, 'mean_change_rate_first_10000'] = np.mean(np.nonzero((np.diff(x[:10000]) / x[:10000][:-1]))[0])
    X_tr.loc[segment, 'mean_change_rate_last_10000'] = np.mean(np.nonzero((np.diff(x[-10000:]) / x[-10000:][:-1]))[0])
    
    X_tr.loc[segment, 'q95'] = np.quantile(x, 0.95)
    X_tr.loc[segment, 'q99'] = np.quantile(x, 0.99)
    X_tr.loc[segment, 'q05'] = np.quantile(x, 0.05)
    X_tr.loc[segment, 'q01'] = np.quantile(x, 0.01)
    
    X_tr.loc[segment, 'abs_q95'] = np.quantile(np.abs(x), 0.95)
    X_tr.loc[segment, 'abs_q99'] = np.quantile(np.abs(x), 0.99)
    X_tr.loc[segment, 'abs_q05'] = np.quantile(np.abs(x), 0.05)
    X_tr.loc[segment, 'abs_q01'] = np.quantile(np.abs(x), 0.01)
    
    X_tr.loc[segment, 'trend'] = add_trend_feature(x)
    X_tr.loc[segment, 'abs_trend'] = add_trend_feature(x, abs_values=True)
    X_tr.loc[segment, 'abs_mean'] = np.abs(x).mean()
    X_tr.loc[segment, 'abs_std'] = np.abs(x).std()
    
    X_tr.loc[segment, 'mad'] = x.mad()
    X_tr.loc[segment, 'kurt'] = x.kurtosis()
    X_tr.loc[segment, 'skew'] = x.skew()
    X_tr.loc[segment, 'med'] = x.median()
    
    X_tr.loc[segment, 'Hilbert_mean'] = np.abs(hilbert(x)).mean()
    X_tr.loc[segment, 'Hann_window_mean'] = (convolve(x, hann(150), mode='same') / sum(hann(150))).mean()
    X_tr.loc[segment, 'classic_sta_lta1_mean'] = classic_sta_lta(x, 500, 10000).mean()
    X_tr.loc[segment, 'classic_sta_lta2_mean'] = classic_sta_lta(x, 5000, 100000).mean()
    X_tr.loc[segment, 'classic_sta_lta3_mean'] = classic_sta_lta(x, 3333, 6666).mean()
    X_tr.loc[segment, 'classic_sta_lta4_mean'] = classic_sta_lta(x, 10000, 25000).mean()
    X_tr.loc[segment, 'Moving_average_700_mean'] = x.rolling(window=700).mean().mean(skipna=True)
    X_tr.loc[segment, 'Moving_average_1500_mean'] = x.rolling(window=1500).mean().mean(skipna=True)
    X_tr.loc[segment, 'Moving_average_3000_mean'] = x.rolling(window=3000).mean().mean(skipna=True)
    X_tr.loc[segment, 'Moving_average_6000_mean'] = x.rolling(window=6000).mean().mean(skipna=True)
    ewma = pd.Series.ewm
    X_tr.loc[segment, 'exp_Moving_average_300_mean'] = (ewma(x, span=300).mean()).mean(skipna=True)
    X_tr.loc[segment, 'exp_Moving_average_3000_mean'] = ewma(x, span=3000).mean().mean(skipna=True)
    X_tr.loc[segment, 'exp_Moving_average_30000_mean'] = ewma(x, span=6000).mean().mean(skipna=True)
    no_of_std = 2
    X_tr.loc[segment, 'MA_700MA_std_mean'] = x.rolling(window=700).std().mean()
    X_tr.loc[segment,'MA_700MA_BB_high_mean'] = (X_tr.loc[segment, 'Moving_average_700_mean'] + no_of_std * X_tr.loc[segment, 'MA_700MA_std_mean']).mean()
    X_tr.loc[segment,'MA_700MA_BB_low_mean'] = (X_tr.loc[segment, 'Moving_average_700_mean'] - no_of_std * X_tr.loc[segment, 'MA_700MA_std_mean']).mean()
    X_tr.loc[segment, 'MA_400MA_std_mean'] = x.rolling(window=400).std().mean()
    X_tr.loc[segment,'MA_400MA_BB_high_mean'] = (X_tr.loc[segment, 'Moving_average_700_mean'] + no_of_std * X_tr.loc[segment, 'MA_400MA_std_mean']).mean()
    X_tr.loc[segment,'MA_400MA_BB_low_mean'] = (X_tr.loc[segment, 'Moving_average_700_mean'] - no_of_std * X_tr.loc[segment, 'MA_400MA_std_mean']).mean()
    X_tr.loc[segment, 'MA_1000MA_std_mean'] = x.rolling(window=1000).std().mean()
    
    X_tr.loc[segment, 'iqr'] = np.subtract(*np.percentile(x, [75, 25]))
    X_tr.loc[segment, 'q999'] = np.quantile(x,0.999)
    X_tr.loc[segment, 'q001'] = np.quantile(x,0.001)
    X_tr.loc[segment, 'ave10'] = stats.trim_mean(x, 0.1)
    
    for windows in [10, 100, 1000]:
        x_roll_std = x.rolling(windows).std().dropna().values
        x_roll_mean = x.rolling(windows).mean().dropna().values
        
        X_tr.loc[segment, 'ave_roll_std_' + str(windows)] = x_roll_std.mean()
        X_tr.loc[segment, 'std_roll_std_' + str(windows)] = x_roll_std.std()
        X_tr.loc[segment, 'max_roll_std_' + str(windows)] = x_roll_std.max()
        X_tr.loc[segment, 'min_roll_std_' + str(windows)] = x_roll_std.min()
        X_tr.loc[segment, 'q01_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.01)
        X_tr.loc[segment, 'q05_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.05)
        X_tr.loc[segment, 'q95_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.95)
        X_tr.loc[segment, 'q99_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.99)
        X_tr.loc[segment, 'av_change_abs_roll_std_' + str(windows)] = np.mean(np.diff(x_roll_std))
        X_tr.loc[segment, 'av_change_rate_roll_std_' + str(windows)] = np.mean(np.nonzero((np.diff(x_roll_std) / x_roll_std[:-1]))[0])
        X_tr.loc[segment, 'abs_max_roll_std_' + str(windows)] = np.abs(x_roll_std).max()
        
        X_tr.loc[segment, 'ave_roll_mean_' + str(windows)] = x_roll_mean.mean()
        X_tr.loc[segment, 'std_roll_mean_' + str(windows)] = x_roll_mean.std()
        X_tr.loc[segment, 'max_roll_mean_' + str(windows)] = x_roll_mean.max()
        X_tr.loc[segment, 'min_roll_mean_' + str(windows)] = x_roll_mean.min()
        X_tr.loc[segment, 'q01_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.01)
        X_tr.loc[segment, 'q05_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.05)
        X_tr.loc[segment, 'q95_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.95)
        X_tr.loc[segment, 'q99_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.99)
        X_tr.loc[segment, 'av_change_abs_roll_mean_' + str(windows)] = np.mean(np.diff(x_roll_mean))
        X_tr.loc[segment, 'av_change_rate_roll_mean_' + str(windows)] = np.mean(np.nonzero((np.diff(x_roll_mean) / x_roll_mean[:-1]))[0])
        X_tr.loc[segment, 'abs_max_roll_mean_' + str(windows)] = np.abs(x_roll_mean).max()

In [9]:
submission = pd.read_csv('sample_submission.csv', index_col='seg_id')
X_test = pd.DataFrame(columns=X_tr.columns, dtype=np.float64, index=submission.index)
plt.figure(figsize=(22, 16))

for i, seg_id in enumerate(tqdm_notebook(X_test.index)):
    seg = pd.read_csv('test/' + seg_id + '.csv')
    
    x = pd.Series(seg['acoustic_data'].values)
    X_test.loc[seg_id, 'mean'] = x.mean()
    X_test.loc[seg_id, 'std'] = x.std()
    X_test.loc[seg_id, 'max'] = x.max()
    X_test.loc[seg_id, 'min'] = x.min()
        
    X_test.loc[seg_id, 'mean_change_abs'] = np.mean(np.diff(x))
    X_test.loc[seg_id, 'mean_change_rate'] = np.mean(np.nonzero((np.diff(x) / x[:-1]))[0])
    X_test.loc[seg_id, 'abs_max'] = np.abs(x).max()
    X_test.loc[seg_id, 'abs_min'] = np.abs(x).min()
    
    X_test.loc[seg_id, 'std_first_50000'] = x[:50000].std()
    X_test.loc[seg_id, 'std_last_50000'] = x[-50000:].std()
    X_test.loc[seg_id, 'std_first_10000'] = x[:10000].std()
    X_test.loc[seg_id, 'std_last_10000'] = x[-10000:].std()
    
    X_test.loc[seg_id, 'avg_first_50000'] = x[:50000].mean()
    X_test.loc[seg_id, 'avg_last_50000'] = x[-50000:].mean()
    X_test.loc[seg_id, 'avg_first_10000'] = x[:10000].mean()
    X_test.loc[seg_id, 'avg_last_10000'] = x[-10000:].mean()
    
    X_test.loc[seg_id, 'min_first_50000'] = x[:50000].min()
    X_test.loc[seg_id, 'min_last_50000'] = x[-50000:].min()
    X_test.loc[seg_id, 'min_first_10000'] = x[:10000].min()
    X_test.loc[seg_id, 'min_last_10000'] = x[-10000:].min()
    
    X_test.loc[seg_id, 'max_first_50000'] = x[:50000].max()
    X_test.loc[seg_id, 'max_last_50000'] = x[-50000:].max()
    X_test.loc[seg_id, 'max_first_10000'] = x[:10000].max()
    X_test.loc[seg_id, 'max_last_10000'] = x[-10000:].max()
    
    X_test.loc[seg_id, 'max_to_min'] = x.max() / np.abs(x.min())
    X_test.loc[seg_id, 'max_to_min_diff'] = x.max() - np.abs(x.min())
    X_test.loc[seg_id, 'count_big'] = len(x[np.abs(x) > 500])
    X_test.loc[seg_id, 'sum'] = x.sum()
    
    X_test.loc[seg_id, 'mean_change_rate_first_50000'] = np.mean(np.nonzero((np.diff(x[:50000]) / x[:50000][:-1]))[0])
    X_test.loc[seg_id, 'mean_change_rate_last_50000'] = np.mean(np.nonzero((np.diff(x[-50000:]) / x[-50000:][:-1]))[0])
    X_test.loc[seg_id, 'mean_change_rate_first_10000'] = np.mean(np.nonzero((np.diff(x[:10000]) / x[:10000][:-1]))[0])
    X_test.loc[seg_id, 'mean_change_rate_last_10000'] = np.mean(np.nonzero((np.diff(x[-10000:]) / x[-10000:][:-1]))[0])
    
    X_test.loc[seg_id, 'q95'] = np.quantile(x,0.95)
    X_test.loc[seg_id, 'q99'] = np.quantile(x,0.99)
    X_test.loc[seg_id, 'q05'] = np.quantile(x,0.05)
    X_test.loc[seg_id, 'q01'] = np.quantile(x,0.01)
    
    X_test.loc[seg_id, 'abs_q95'] = np.quantile(np.abs(x), 0.95)
    X_test.loc[seg_id, 'abs_q99'] = np.quantile(np.abs(x), 0.99)
    X_test.loc[seg_id, 'abs_q05'] = np.quantile(np.abs(x), 0.05)
    X_test.loc[seg_id, 'abs_q01'] = np.quantile(np.abs(x), 0.01)
    
    X_test.loc[seg_id, 'trend'] = add_trend_feature(x)
    X_test.loc[seg_id, 'abs_trend'] = add_trend_feature(x, abs_values=True)
    X_test.loc[seg_id, 'abs_mean'] = np.abs(x).mean()
    X_test.loc[seg_id, 'abs_std'] = np.abs(x).std()
    
    X_test.loc[seg_id, 'mad'] = x.mad()
    X_test.loc[seg_id, 'kurt'] = x.kurtosis()
    X_test.loc[seg_id, 'skew'] = x.skew()
    X_test.loc[seg_id, 'med'] = x.median()
    
    X_test.loc[seg_id, 'Hilbert_mean'] = np.abs(hilbert(x)).mean()
    X_test.loc[seg_id, 'Hann_window_mean'] = (convolve(x, hann(150), mode='same') / sum(hann(150))).mean()
    X_test.loc[seg_id, 'classic_sta_lta1_mean'] = classic_sta_lta(x, 500, 10000).mean()
    X_test.loc[seg_id, 'classic_sta_lta2_mean'] = classic_sta_lta(x, 5000, 100000).mean()
    X_test.loc[seg_id, 'classic_sta_lta3_mean'] = classic_sta_lta(x, 3333, 6666).mean()
    X_test.loc[seg_id, 'classic_sta_lta4_mean'] = classic_sta_lta(x, 10000, 25000).mean()
    X_test.loc[seg_id, 'Moving_average_700_mean'] = x.rolling(window=700).mean().mean(skipna=True)
    X_test.loc[seg_id, 'Moving_average_1500_mean'] = x.rolling(window=1500).mean().mean(skipna=True)
    X_test.loc[seg_id, 'Moving_average_3000_mean'] = x.rolling(window=3000).mean().mean(skipna=True)
    X_test.loc[seg_id, 'Moving_average_6000_mean'] = x.rolling(window=6000).mean().mean(skipna=True)
    ewma = pd.Series.ewm
    X_test.loc[seg_id, 'exp_Moving_average_300_mean'] = (ewma(x, span=300).mean()).mean(skipna=True)
    X_test.loc[seg_id, 'exp_Moving_average_3000_mean'] = ewma(x, span=3000).mean().mean(skipna=True)
    X_test.loc[seg_id, 'exp_Moving_average_30000_mean'] = ewma(x, span=6000).mean().mean(skipna=True)
    no_of_std = 2
    X_test.loc[seg_id, 'MA_700MA_std_mean'] = x.rolling(window=700).std().mean()
    X_test.loc[seg_id,'MA_700MA_BB_high_mean'] = (X_test.loc[seg_id, 'Moving_average_700_mean'] + no_of_std * X_test.loc[seg_id, 'MA_700MA_std_mean']).mean()
    X_test.loc[seg_id,'MA_700MA_BB_low_mean'] = (X_test.loc[seg_id, 'Moving_average_700_mean'] - no_of_std * X_test.loc[seg_id, 'MA_700MA_std_mean']).mean()
    X_test.loc[seg_id, 'MA_400MA_std_mean'] = x.rolling(window=400).std().mean()
    X_test.loc[seg_id,'MA_400MA_BB_high_mean'] = (X_test.loc[seg_id, 'Moving_average_700_mean'] + no_of_std * X_test.loc[seg_id, 'MA_400MA_std_mean']).mean()
    X_test.loc[seg_id,'MA_400MA_BB_low_mean'] = (X_test.loc[seg_id, 'Moving_average_700_mean'] - no_of_std * X_test.loc[seg_id, 'MA_400MA_std_mean']).mean()
    X_test.loc[seg_id, 'MA_1000MA_std_mean'] = x.rolling(window=1000).std().mean()
    
    X_test.loc[seg_id, 'iqr'] = np.subtract(*np.percentile(x, [75, 25]))
    X_test.loc[seg_id, 'q999'] = np.quantile(x,0.999)
    X_test.loc[seg_id, 'q001'] = np.quantile(x,0.001)
    X_test.loc[seg_id, 'ave10'] = stats.trim_mean(x, 0.1)
    
    for windows in [10, 100, 1000]:
        x_roll_std = x.rolling(windows).std().dropna().values
        x_roll_mean = x.rolling(windows).mean().dropna().values
        
        X_test.loc[seg_id, 'ave_roll_std_' + str(windows)] = x_roll_std.mean()
        X_test.loc[seg_id, 'std_roll_std_' + str(windows)] = x_roll_std.std()
        X_test.loc[seg_id, 'max_roll_std_' + str(windows)] = x_roll_std.max()
        X_test.loc[seg_id, 'min_roll_std_' + str(windows)] = x_roll_std.min()
        X_test.loc[seg_id, 'q01_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.01)
        X_test.loc[seg_id, 'q05_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.05)
        X_test.loc[seg_id, 'q95_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.95)
        X_test.loc[seg_id, 'q99_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.99)
        X_test.loc[seg_id, 'av_change_abs_roll_std_' + str(windows)] = np.mean(np.diff(x_roll_std))
        X_test.loc[seg_id, 'av_change_rate_roll_std_' + str(windows)] = np.mean(np.nonzero((np.diff(x_roll_std) / x_roll_std[:-1]))[0])
        X_test.loc[seg_id, 'abs_max_roll_std_' + str(windows)] = np.abs(x_roll_std).max()
        
        X_test.loc[seg_id, 'ave_roll_mean_' + str(windows)] = x_roll_mean.mean()
        X_test.loc[seg_id, 'std_roll_mean_' + str(windows)] = x_roll_mean.std()
        X_test.loc[seg_id, 'max_roll_mean_' + str(windows)] = x_roll_mean.max()
        X_test.loc[seg_id, 'min_roll_mean_' + str(windows)] = x_roll_mean.min()
        X_test.loc[seg_id, 'q01_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.01)
        X_test.loc[seg_id, 'q05_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.05)
        X_test.loc[seg_id, 'q95_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.95)
        X_test.loc[seg_id, 'q99_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.99)
        X_test.loc[seg_id, 'av_change_abs_roll_mean_' + str(windows)] = np.mean(np.diff(x_roll_mean))
        X_test.loc[seg_id, 'av_change_rate_roll_mean_' + str(windows)] = np.mean(np.nonzero((np.diff(x_roll_mean) / x_roll_mean[:-1]))[0])
        X_test.loc[seg_id, 'abs_max_roll_mean_' + str(windows)] = np.abs(x_roll_mean).max()

In [10]:
def activation(x) :
    return x/(1+np.abs(x))

In [11]:
def activation1(x) :
    return x/np.sqrt(1+(x)**2)

In [12]:
def activation2(x) :
    return (2/np.pi)*np.arctan((np.pi/2)*x)

In [13]:
from scipy.special import erf

In [14]:
def GPI_activation(data):
    return (5.591662 +
            0.0399999991*activation(((data["mean_change_rate_first_10000"]) - (((73.0) * ((((((7.0) * (((((((((data["q95"]) + (((0.3183098733) + (data["q05_roll_std_100"]))))) * 2.0)) * 2.0)) + (((data["q95_roll_std_10"]) + (data["iqr"]))))))) + (data["iqr"]))/2.0)))))) +
            0.0399999991*activation(((((3.6923100948) + (data["Moving_average_6000_mean"]))) * (((73.0) * (((data["MA_400MA_BB_low_mean"]) - (((((data["Moving_average_6000_mean"]) * (data["q05_roll_std_10"]))) + (((data["iqr"]) + (((data["q05_roll_std_10"]) * (((3.6923100948) * 2.0)))))))))))))) +
            0.0399999991*activation(((((((((((((((((((((((data["q05"]) * 2.0)) * 2.0)) - (data["iqr"]))) - (data["max_roll_std_1000"]))) * 2.0)) * 2.0)) * 2.0)) - (data["iqr"]))) - (3.6923100948))) - (((data["q05_roll_std_10"]) * (73.0))))) * 2.0)) +
            0.0399999991*activation(((((((((((((((((((data["q05_roll_std_10"]) * (((((-3.0) * 2.0)) * 2.0)))) - (((data["iqr"]) + (data["max_roll_std_1000"]))))) * 2.0)) * 2.0)) * 2.0)) - (data["max_roll_mean_100"]))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*activation(((73.0) * ((((data["max_first_50000"]) + ((((data["max_first_50000"]) + (((73.0) * (((data["q05"]) - (((data["q05_roll_std_100"]) + (((0.5555559993) + (((data["mad"]) + (data["q95"]))))))))))))/2.0)))/2.0)))) +
            0.0399999991*activation(((((((data["q05_roll_std_10"]) * (73.0))) * 2.0)) - ((((((((data["q05_roll_std_10"]) * (73.0))) + (((((((((data["iqr"]) + (((data["MA_1000MA_std_mean"]) + (data["abs_max_roll_mean_10"]))))) * 2.0)) * 2.0)) * 2.0)))/2.0)) * (73.0))))) +
            0.0399999991*activation(((((data["mean_change_abs"]) - (((((data["q95"]) + (((data["q95"]) + (((((data["q05_roll_std_100"]) / 2.0)) + (((((((0.3183098733) * 2.0)) - (data["q05"]))) - (data["q05"]))))))))) * (73.0))))) * 2.0)) +
            0.0399999991*activation((-1.0*(((((((7.48210620880126953)) * 2.0)) + (((((data["iqr"]) + ((((-1.0*((data["avg_first_10000"])))) * 2.0)))) + (((73.0) * (((data["iqr"]) + (((((data["q05_roll_std_10"]) * 2.0)) * 2.0))))))))))))) +
            0.0399999991*activation((((((data["q01_roll_std_10"]) + (((((data["q05_roll_mean_10"]) - (((((data["q05_roll_std_100"]) - (((data["q05"]) - (0.3026320040))))) + (((((((0.5434780121) + (0.3026320040))/2.0)) + (data["iqr"]))/2.0)))))) * (73.0))))/2.0)) * (73.0))) +
            0.0399999991*activation(((((data["trend"]) + (((data["q95_roll_std_10"]) + (data["trend"]))))) - (((73.0) * (((((((data["ave_roll_std_10"]) + (((((((data["q95_roll_mean_10"]) + (data["q05_roll_std_1000"]))/2.0)) + (data["q05_roll_std_1000"]))/2.0)))/2.0)) + ((((0.5434780121) + (data["q95_roll_mean_10"]))/2.0)))/2.0)))))) +
            0.0399999991*activation(((73.0) - (((((73.0) * 2.0)) * (((73.0) * ((((((((((data["q05_roll_std_1000"]) / 2.0)) + (data["iqr"]))/2.0)) + (data["q05_roll_std_10"]))) + (((0.2183910012) + (data["q95"]))))))))))) +
            0.0399999991*activation(((73.0) * (((73.0) * (((((73.0) * (((data["q05_roll_mean_10"]) - (((((data["q05_roll_std_100"]) + (data["q95"]))) + (0.5434780121))))))) + (data["q05_roll_std_100"]))))))) +
            0.0399999991*activation((-1.0*((((73.0) * (((73.0) * (((73.0) * (((data["q05_roll_std_100"]) + ((((data["q95_roll_mean_10"]) + ((((data["Moving_average_3000_mean"]) + (((((data["iqr"]) + (0.9756100178))) + (data["abs_mean"]))))/2.0)))/2.0))))))))))))) +
            0.0399999991*activation(((73.0) * (((((data["classic_sta_lta3_mean"]) + (data["mad"]))) + (((((((73.0) * (((data["q05_roll_mean_10"]) - (((data["q05_roll_std_100"]) + (((data["mad"]) + (0.5555559993))))))))) - (data["iqr"]))) - (data["iqr"]))))))) +
            0.0399999991*activation(((((activation((data["q95_roll_std_10"]))) + (((((((data["q05_roll_std_10"]) + ((((((0.2439019978) + (data["ave10"]))/2.0)) / 2.0)))) * 2.0)) * 2.0)))) * (((data["q05_roll_std_10"]) - (((73.0) * (((73.0) * (73.0))))))))) +
            0.0399999991*activation(((3.6923100948) * (((73.0) * (((3.6923100948) * ((((((-1.0*((data["q99_roll_mean_1000"])))) * (data["avg_last_50000"]))) - (((data["q05_roll_std_10"]) * (((3.6923100948) + (((3.6923100948) + (data["abs_max_roll_mean_100"]))))))))))))))) +
            0.0399999991*activation(((((((data["Moving_average_1500_mean"]) + (((((((((data["MA_400MA_BB_low_mean"]) + ((((-1.0*((data["q05_roll_std_1000"])))) - (data["iqr"]))))) * (73.0))) - (data["min_roll_std_1000"]))) - (data["q05_roll_std_1000"]))))) * (73.0))) * (73.0))) +
            0.0399999991*activation(((73.0) - (((73.0) * (((73.0) * (((data["q05_roll_std_10"]) + (activation(((((((data["exp_Moving_average_3000_mean"]) + (((data["iqr"]) + (activation((((data["q05_roll_std_100"]) + (data["exp_Moving_average_30000_mean"]))))))))/2.0)) / 2.0)))))))))))) +
            0.0399999991*activation(((73.0) * (((73.0) * (((data["q05_roll_mean_10"]) - (((activation((activation((activation((activation((activation((data["q05_roll_std_100"]))))))))))) + ((((data["q95_roll_std_10"]) + (activation((2.8437500000))))/2.0)))))))))) +
            0.0399999991*activation(((((((data["iqr"]) / 2.0)) + (((data["mean"]) + (((data["q05_roll_std_100"]) + (((data["q05_roll_std_10"]) * 2.0)))))))) * ((-1.0*((((73.0) - (((((((((data["mean"]) * 2.0)) * 2.0)) * 2.0)) * 2.0))))))))) +
            0.0399999991*activation(((((((((0.5217390060) - (((((data["q05_roll_std_10"]) * ((-1.0*((7.0)))))) + (data["exp_Moving_average_3000_mean"]))))) * ((-1.0*((73.0)))))) - (((data["classic_sta_lta4_mean"]) * (7.0))))) - ((((7.0) + (data["classic_sta_lta4_mean"]))/2.0)))) +
            0.0399999991*activation(((((((((((((((((((activation(((-1.0*((((data["q05_roll_mean_100"]) + (0.2183910012)))))))) - ((((data["q05_roll_std_100"]) + (((data["q05_roll_std_10"]) * 2.0)))/2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) - (data["exp_Moving_average_30000_mean"]))) * 2.0)) +
            0.0399999991*activation(((data["trend"]) - (((73.0) * ((((3.6923100948) + ((((9.0)) * (((activation((activation((activation((activation((activation((activation((activation((data["q05_roll_std_100"]))))))))))))))) - (data["q05_roll_mean_10"]))))))/2.0)))))) +
            0.0399999991*activation(((data["q05_roll_std_1000"]) - (((73.0) * (((data["avg_last_50000"]) + (((((73.0) * 2.0)) * (((data["ave_roll_std_100"]) + (((data["Moving_average_6000_mean"]) + (((data["iqr"]) + (((data["q05_roll_std_1000"]) + (((data["q95"]) * 2.0)))))))))))))))))) +
            0.0399999991*activation(((((data["q05_roll_mean_10"]) - (2.0769200325))) - ((((((6.08570814132690430)) * (((2.0769200325) + (((((((((data["q05_roll_std_10"]) - (data["q05_roll_mean_10"]))) * 2.0)) * 2.0)) * 2.0)))))) + (((((data["av_change_rate_roll_std_100"]) - (data["avg_last_10000"]))) * 2.0)))))) +
            0.0399999991*activation((-1.0*((((((((((((((((((((((((((((((((data["q05_roll_std_1000"]) + (data["q05_roll_mean_100"]))/2.0)) * 2.0)) + (data["iqr"]))/2.0)) + (data["q95_roll_mean_10"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0))))) +
            0.0399999991*activation((((-1.0*((((((((((data["q05_roll_std_100"]) - (activation(((-1.0*((((data["MA_400MA_std_mean"]) + (((data["exp_Moving_average_300_mean"]) * (((data["q05_roll_std_100"]) * 2.0))))))))))))) * 2.0)) * 2.0)) + (activation((data["q05_roll_std_100"])))))))) * (73.0))) +
            0.0399999991*activation(((((data["abs_q05"]) - ((((((2.0769200325) * ((((((73.0) * ((((((0.5555559993) + (((data["q05_roll_std_100"]) + (0.7446810007))))/2.0)) + (data["q05_roll_std_100"]))))) + (0.5555559993))/2.0)))) + (0.5555559993))/2.0)))) * 2.0)) +
            0.0399999991*activation((((10.75933361053466797)) - (((7.0) * (((7.0) * (((7.0) * ((((((data["q95_roll_mean_10"]) + (data["q05_roll_std_100"]))) + ((((data["q95_roll_mean_10"]) + (((data["iqr"]) + (((data["Moving_average_700_mean"]) * 2.0)))))/2.0)))/2.0)))))))))) +
            0.0399999991*activation(((73.0) * (((((data["q05_roll_std_1000"]) + ((((data["q05_roll_std_10"]) + ((((((data["q05_roll_std_1000"]) + (activation((data["q05_roll_std_1000"]))))/2.0)) - (((data["q95_roll_mean_1000"]) - (activation((73.0))))))))/2.0)))) * (((2.8437500000) - (73.0))))))) +
            0.0399999991*activation(((((((((-3.0) + (data["Moving_average_3000_mean"]))) - (((73.0) * (((data["ave_roll_mean_10"]) - ((((((data["q05_roll_std_100"]) + (data["q95_roll_mean_10"]))/2.0)) * (((-3.0) - (((data["Moving_average_700_mean"]) + (data["q95_roll_mean_10"]))))))))))))) * 2.0)) * 2.0)) +
            0.0399999991*activation(((((((data["classic_sta_lta3_mean"]) * 2.0)) + (data["q05_roll_mean_100"]))) - (((73.0) * (((((((0.6582279801) + (data["q05_roll_std_10"]))) - (((data["q05_roll_mean_10"]) * 2.0)))) + ((((-1.0*((data["abs_q05"])))) + (((data["q05_roll_std_100"]) * 2.0)))))))))) +
            0.0399999991*activation(((((data["skew"]) + (((data["kurt"]) + (data["med"]))))) - ((((7.02540111541748047)) * (((((data["iqr"]) + (((data["q95_roll_std_10"]) + (((((data["q95_roll_mean_10"]) + (((data["iqr"]) * (data["Moving_average_1500_mean"]))))) * 2.0)))))) * 2.0)))))) +
            0.0399999991*activation(((((data["q05_roll_std_100"]) - (((-2.0) + (((((data["iqr"]) * (-2.0))) - ((-1.0*((data["av_change_rate_roll_std_10"])))))))))) - ((((((data["q05_roll_std_100"]) + (activation((((data["q05_roll_std_100"]) * (data["ave_roll_mean_10"]))))))/2.0)) * (73.0))))) +
            0.0399999991*activation(((((((((((data["q05_roll_std_10"]) + (((data["av_change_abs_roll_std_1000"]) - (((data["av_change_abs_roll_mean_10"]) * 2.0)))))) * 2.0)) + (data["av_change_abs_roll_std_1000"]))) * 2.0)) - (((73.0) * (((((data["abs_max_roll_mean_1000"]) + (((0.5555559993) + (data["q05_roll_std_10"]))))) * 2.0)))))) +
            0.0399999991*activation((((((((((((((((((activation((((((data["q95_roll_mean_10"]) + (data["q95_roll_std_100"]))) - (data["q05_roll_std_1000"]))))) + (data["q95_roll_std_100"]))/2.0)) + (data["q95_roll_std_100"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) - (((73.0) * (data["q95_roll_mean_10"]))))) * 2.0)) +
            0.0399999991*activation(((73.0) - (((((((((((data["q05_roll_std_10"]) * 2.0)) - (data["ave10"]))) + (((data["q05_roll_std_100"]) + (0.9756100178))))) * (73.0))) * (((73.0) * 2.0)))))) +
            0.0399999991*activation(((((data["Moving_average_6000_mean"]) + (((7.0) + (((data["ave10"]) * 2.0)))))) * (((((((((data["MA_700MA_BB_low_mean"]) * (data["avg_last_50000"]))) + ((-1.0*((((data["q95_roll_mean_10"]) * 2.0))))))) - (((data["ave_roll_mean_10"]) + (data["q05_roll_std_100"]))))) * 2.0)))) +
            0.0399999991*activation(((73.0) * (((data["q05_roll_std_100"]) - (((((((data["iqr"]) - (activation((((73.0) * (((data["ave10"]) - (((data["iqr"]) + (((((data["q05_roll_std_1000"]) + (0.5434780121))) * 2.0)))))))))))) * 2.0)) * 2.0)))))) +
            0.0399999991*activation(((((((((((((((data["exp_Moving_average_30000_mean"]) * (((data["exp_Moving_average_30000_mean"]) + ((-1.0*((data["q01_roll_std_10"])))))))) - (((data["q05_roll_std_10"]) + (((data["q05_roll_std_10"]) + (((data["q95_roll_mean_100"]) + (data["q05_roll_std_10"]))))))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*activation(((73.0) - (((((data["q05_roll_std_100"]) * (73.0))) * (((73.0) * (((data["q05_roll_std_100"]) * (((data["q05_roll_std_100"]) - (((data["q05"]) - ((((0.2525250018) + (2.0))/2.0)))))))))))))) +
            0.0399999991*activation(((((((((7.0) - ((((data["sum"]) + (((73.0) * (((((data["sum"]) + (data["q05_roll_std_10"]))) + (((((((activation((data["abs_q05"]))) + (data["q05_roll_std_10"]))) * 2.0)) * 2.0)))))))/2.0)))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*activation((((((1.0)) - (((((((((data["q05_roll_std_100"]) + (activation((((data["abs_q05"]) * (((((data["exp_Moving_average_30000_mean"]) - (data["abs_q05"]))) + (((data["q05_roll_std_100"]) + (((data["q95_roll_mean_10"]) * 2.0)))))))))))) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) +
            0.0399999991*activation(((((activation((((activation((data["Moving_average_1500_mean"]))) * 2.0)))) - (((data["Moving_average_1500_mean"]) + ((-1.0*((((((data["q05_roll_std_100"]) + (activation((0.6582279801))))) * 2.0))))))))) * ((((-1.0*((((data["Moving_average_1500_mean"]) * 2.0))))) - (73.0))))) +
            0.0399999991*activation(((73.0) - ((((((((((((data["q05_roll_std_100"]) + (((data["q05_roll_std_100"]) * (data["Moving_average_6000_mean"]))))/2.0)) + (((data["Moving_average_6000_mean"]) + (((data["q05_roll_std_100"]) + (data["q95"]))))))) * 2.0)) * 2.0)) * (((73.0) + (data["Moving_average_6000_mean"]))))))) +
            0.0399999991*activation(((((((((((((data["mean_change_rate_last_10000"]) - (data["mean_change_abs"]))) - (((((((((((((data["q05_roll_std_10"]) + (((0.5434780121) - (data["q05"]))))) * 2.0)) - (data["Hann_window_mean"]))) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*activation(((((((-1.0*((((data["avg_last_50000"]) * (3.6923100948)))))) + (((73.0) / 2.0)))/2.0)) - (((73.0) * ((((((data["Moving_average_6000_mean"]) + (((data["q05_roll_std_10"]) - (((data["exp_Moving_average_30000_mean"]) * (data["exp_Moving_average_30000_mean"]))))))/2.0)) + (data["q05_roll_std_10"]))))))) +
            0.0399999991*activation(((73.0) * ((-1.0*((((data["q01_roll_std_1000"]) + (((73.0) * (((data["q05_roll_std_100"]) + ((((1.4411799908) + ((((((data["q01_roll_std_1000"]) - (data["q99_roll_mean_1000"]))) + (((data["q99_roll_mean_1000"]) * (data["q99_roll_mean_1000"]))))/2.0)))/2.0))))))))))))) +
            0.0399999991*activation((((12.50784111022949219)) + ((((12.50784111022949219)) - ((((((activation((((data["q05_roll_std_10"]) * (data["q95_roll_mean_1000"]))))) + (((((((data["q05_roll_std_10"]) + (activation((((data["q05_roll_std_10"]) * (data["exp_Moving_average_30000_mean"]))))))) * 2.0)) * 2.0)))/2.0)) * (73.0))))))) +
            0.0399999991*activation(((((((data["skew"]) - (((data["ave_roll_mean_100"]) + (((((((data["ave_roll_mean_100"]) * (data["abs_q05"]))) + (((((2.0) + (((((data["q05_roll_std_100"]) - (data["q05_roll_mean_10"]))) * 2.0)))) * 2.0)))) * 2.0)))))) * 2.0)) * 2.0)) +
            0.0399999991*activation(((((((((data["q95_roll_mean_1000"]) * (data["ave_roll_mean_100"]))) * 2.0)) - (((((data["exp_Moving_average_30000_mean"]) + (((((((((data["exp_Moving_average_30000_mean"]) * (data["q05_roll_std_10"]))) + (((((data["q05_roll_std_10"]) - (0.2439019978))) * 2.0)))) * 2.0)) * 2.0)))) * 2.0)))) * 2.0)) +
            0.0399999991*activation(((((((((data["q05_roll_std_10"]) - (((((data["q95"]) - (((((data["iqr"]) - (((data["exp_Moving_average_3000_mean"]) * (activation((((data["q05_roll_std_10"]) * 2.0)))))))) - (((data["q05_roll_std_10"]) * 2.0)))))) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*activation(((((((data["q95_roll_mean_100"]) + (((data["mean_change_rate_first_10000"]) + (((data["abs_q05"]) + (data["abs_q05"]))))))) * (3.6923100948))) + (((((((((data["q05_roll_std_1000"]) + (0.7446810007))) * (((3.6923100948) - (73.0))))) * 2.0)) * 2.0)))) +
            0.0399999991*activation((-1.0*((((((73.0) * ((((((data["q01_roll_mean_1000"]) - (((data["q05"]) * 2.0)))) + (((data["q05_roll_std_1000"]) + (((((data["q05_roll_std_100"]) - (data["q95_roll_mean_1000"]))) * (data["Moving_average_700_mean"]))))))/2.0)))) + (((((data["Moving_average_1500_mean"]) * 2.0)) * 2.0))))))) +
            0.0399999991*activation(((((((((data["q05"]) - (((73.0) * ((((-1.0*((((data["q05"]) * 2.0))))) + (((data["q05_roll_std_1000"]) + (1.9019600153))))))))) + (((((((data["q05"]) + (data["max_to_min"]))) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) +
            0.0399999991*activation(((((((data["av_change_abs_roll_mean_100"]) - (((data["q01_roll_mean_1000"]) * 2.0)))) - (((73.0) * (((((((data["Hann_window_mean"]) * (((data["q95"]) - (data["q95_roll_mean_1000"]))))) / 2.0)) + (((((data["q95"]) * 2.0)) + (data["q01_roll_mean_1000"]))))))))) * 2.0)) +
            0.0399999991*activation(((((((((((data["skew"]) - (((((data["q05_roll_mean_10"]) + (((((data["iqr"]) + (((3.1415927410) * (((1.0) - (((data["q05_roll_mean_10"]) * 2.0)))))))) * 2.0)))) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*activation((((((((6.0)) - (((((((((data["q05_roll_std_100"]) + (((data["q05_roll_std_100"]) * 2.0)))) * 2.0)) + (((((data["ave10"]) * (((((data["q05_roll_std_100"]) * 2.0)) * 2.0)))) + (activation((data["q05_roll_std_100"]))))))) * 2.0)))) * 2.0)) * 2.0)) +
            0.0399999991*activation(((((((((data["q05_roll_std_10"]) + (((((((data["q95_roll_mean_1000"]) + (((((((data["skew"]) + (((((((data["abs_q05"]) - (((data["q05_roll_std_10"]) * 2.0)))) * 2.0)) * 2.0)))) * 2.0)) * 2.0)))) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*activation(((((((((data["max_to_min"]) + (data["q95_roll_std_1000"]))) - (((((73.0) * ((((data["q05_roll_std_1000"]) + (((0.2183910012) + (data["abs_q05"]))))/2.0)))) * (((data["q05_roll_std_1000"]) * (((data["q05_roll_std_10"]) + (data["q01_roll_std_10"]))))))))) * 2.0)) * 2.0)) +
            0.0399999991*activation(((((((((((data["abs_q05"]) + (data["q05_roll_std_100"]))) - (((7.0) * (((data["iqr"]) - (((data["q99_roll_mean_1000"]) + ((-1.0*((((((((data["q05_roll_std_100"]) + (1.0))) * 2.0)) * 2.0))))))))))))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*activation((-1.0*((((7.0) * (((73.0) * (((((7.0) * (((((data["Moving_average_6000_mean"]) * (((((-1.0*((data["Moving_average_6000_mean"])))) + (activation((activation((data["q01_roll_std_10"]))))))/2.0)))) + (data["q95_roll_mean_10"]))))) + (data["q95_roll_mean_10"])))))))))) +
            0.0399999991*activation(((((((((((data["skew"]) + (data["ave10"]))) + (data["q95_roll_std_100"]))) - (((7.0) * (((((data["q05_roll_std_100"]) * (data["q95"]))) * ((((((data["q05_roll_std_100"]) * (7.0))) + (7.0))/2.0)))))))) * 2.0)) * 2.0)) +
            0.0399999991*activation((((((9.64285850524902344)) * (((((data["Moving_average_700_mean"]) - ((((((9.64285850524902344)) * ((((data["q05_roll_std_100"]) + (data["exp_Moving_average_30000_mean"]))/2.0)))) - ((((data["skew"]) + ((3.98402547836303711)))/2.0)))))) * 2.0)))) * (((data["iqr"]) * (data["q05_roll_std_100"]))))) +
            0.0399999991*activation(((((((((((((((((data["Moving_average_1500_mean"]) - (((activation((data["exp_Moving_average_30000_mean"]))) + (activation((((data["abs_q05"]) + ((((data["q95_roll_std_10"]) + (data["q01_roll_std_10"]))/2.0)))))))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) - (data["iqr"]))) * 2.0)) +
            0.0399999991*activation((((-1.0*((data["q95"])))) * (((73.0) + ((-1.0*((((data["abs_q05"]) * ((((((((-1.0*((data["abs_q05"])))) - (data["min_roll_std_10"]))) - (73.0))) - ((((-1.0*((data["q05_roll_mean_10"])))) * (73.0)))))))))))))) +
            0.0399999991*activation(((data["q05_roll_std_10"]) * (((((((((data["q05_roll_std_100"]) * ((11.90774154663085938)))) * (((data["ave10"]) - (-2.0))))) - ((11.90774154663085938)))) * ((-1.0*((((data["q05_roll_std_1000"]) + (((data["q05_roll_std_100"]) - (-2.0)))))))))))) +
            0.0399999991*activation(((((((data["iqr"]) * ((9.63457870483398438)))) * (((data["q95_roll_mean_1000"]) - ((((9.63457870483398438)) * (((data["q05_roll_std_10"]) * ((((data["q01_roll_std_10"]) + (((data["iqr"]) * (data["q95_roll_mean_1000"]))))/2.0)))))))))) - (((-3.0) * (data["q01_roll_std_10"]))))) +
            0.0399999991*activation(((((((((-1.0) - (((activation((data["q05_roll_std_10"]))) + (((((((data["q05_roll_std_100"]) - (activation((((data["classic_sta_lta1_mean"]) + (((((((data["q95_roll_std_100"]) * 2.0)) * 2.0)) * 2.0)))))))) * 2.0)) * 2.0)))))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*activation(((((((((data["q95_roll_mean_10"]) - (((((((data["q95_roll_mean_10"]) + (((((data["q05_roll_std_10"]) + (((3.6923100948) * ((((data["ave10"]) + (((data["q05_roll_std_10"]) + (-1.0))))/2.0)))))) * 2.0)))) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*activation(((((data["q05_roll_std_100"]) + (((((data["q95_roll_mean_1000"]) + (((((data["q05_roll_std_100"]) + ((((3.1415927410) + (data["q05_roll_std_1000"]))/2.0)))) * (((((((data["q05_roll_mean_10"]) * 2.0)) * 2.0)) * (((3.1415927410) * (data["q05_roll_std_100"]))))))))) * 2.0)))) * 2.0)) +
            0.0399999991*activation(((((((((((((data["mean_change_rate_first_10000"]) - (data["Moving_average_6000_mean"]))) * 2.0)) - (data["q01_roll_mean_1000"]))) - (((data["av_change_abs_roll_mean_10"]) + (((data["mean_change_rate_last_50000"]) + (((data["av_change_abs_roll_std_100"]) - ((((((13.93686485290527344)) / 2.0)) * (data["q05"]))))))))))) * 2.0)) * 2.0)) +
            0.0399999991*activation(((((((((data["q05_roll_std_10"]) * (data["q05_roll_std_10"]))) * ((((-1.0*((((2.0) - ((-1.0*((((data["q05_roll_std_100"]) + (((((data["q05_roll_std_100"]) * (data["abs_q05"]))) * 2.0)))))))))))) * 2.0)))) - ((-1.0*((data["kurt"])))))) * 2.0)) +
            0.0399999991*activation(((((((((((((((data["q05"]) - (activation((((data["iqr"]) + (((((data["Moving_average_1500_mean"]) + (((data["q05"]) + (data["Moving_average_3000_mean"]))))) * 2.0)))))))) * 2.0)) + (data["q05"]))) * 2.0)) * 2.0)) - (data["av_change_abs_roll_mean_10"]))) * 2.0)) +
            0.0399999991*activation(((((((((activation((((((((((((((data["Moving_average_6000_mean"]) - (data["q05_roll_std_100"]))) * 2.0)) * 2.0)) - (((1.6428600550) + (data["q01_roll_std_10"]))))) * 2.0)) - (data["av_change_abs_roll_std_100"]))))) * 2.0)) - (data["abs_q05"]))) * 2.0)) * 2.0)) +
            0.0399999991*activation((((((((((((((data["skew"]) + (data["q95_roll_std_10"]))/2.0)) - (((((data["ave_roll_std_100"]) + (((data["max_roll_mean_10"]) * 2.0)))) * (data["abs_q05"]))))) + (((((activation((data["q95_roll_std_100"]))) - (data["q01_roll_std_10"]))) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*activation(((data["skew"]) + (((((data["q01_roll_std_100"]) * 2.0)) + (((data["Moving_average_700_mean"]) + (((data["q05_roll_std_10"]) * (((data["q05"]) * ((((((73.0) * (((data["Moving_average_700_mean"]) + (((data["q05_roll_std_100"]) / 2.0)))))) + (data["q05_roll_std_10"]))/2.0)))))))))))) +
            0.0399999991*activation(((((((data["q05_roll_mean_10"]) * (((data["q05_roll_std_100"]) * (data["q05_roll_std_100"]))))) * (((((data["q05_roll_std_100"]) * 2.0)) * (((data["q05_roll_std_1000"]) * 2.0)))))) - (((((((data["q05_roll_std_1000"]) * 2.0)) * (data["q05_roll_std_100"]))) - (((data["q01_roll_std_1000"]) * 2.0)))))) +
            0.0399999991*activation(((((((-1.0) + (((((((activation((data["q01_roll_std_10"]))) * 2.0)) * 2.0)) + (((data["max_to_min_diff"]) + (((data["q05"]) * ((((14.98138904571533203)) + (((data["mean_change_rate_last_10000"]) * 2.0)))))))))))) * (73.0))) * (73.0))) +
            0.0399999991*activation(((((((data["q05_roll_std_10"]) - (((((data["q05_roll_std_1000"]) + (0.7446810007))) * (((data["q05_roll_std_10"]) * (((((((data["q01_roll_std_10"]) * (((data["ave10"]) + (((data["q95_roll_mean_10"]) * (data["q05_roll_std_10"]))))))) * 2.0)) * 2.0)))))))) * 2.0)) * 2.0)) +
            0.0399999991*activation(((((((activation((data["q01_roll_std_100"]))) - (((((((((data["q05_roll_std_1000"]) + (((((data["q95"]) - (activation((data["q95_roll_std_100"]))))) * 2.0)))) - (activation((activation((data["q95"]))))))) - (data["std_roll_mean_1000"]))) * 2.0)))) * 2.0)) * 2.0)) +
            0.0399999991*activation(((((data["q05_roll_std_10"]) + (((data["ave_roll_mean_1000"]) * 2.0)))) * (((((((data["ave_roll_mean_1000"]) * (data["ave_roll_mean_1000"]))) - (data["q05_roll_std_1000"]))) + (((data["q05_roll_mean_10"]) * (((data["Moving_average_3000_mean"]) + (((((data["q05_roll_std_10"]) * 2.0)) * 2.0)))))))))) +
            0.0399999991*activation(((((((((((data["q05_roll_std_1000"]) + (((data["q05_roll_std_100"]) - (((data["Moving_average_6000_mean"]) * (data["Moving_average_6000_mean"]))))))) * (((data["q95_roll_std_100"]) * ((((-1.0*((data["iqr"])))) - (((data["Moving_average_6000_mean"]) + (data["Moving_average_1500_mean"]))))))))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*activation(((((((((data["sum"]) + (0.9756100178))) * 2.0)) * 2.0)) * (((data["q01_roll_std_100"]) - (((activation((((data["q05_roll_std_10"]) * (data["q05_roll_std_10"]))))) + (((data["q05_roll_std_1000"]) * (((((data["q01_roll_std_100"]) * (data["q01_roll_std_10"]))) * 2.0)))))))))) +
            0.0399999991*activation((((((((((-1.0*((data["q05_roll_std_100"])))) - (((data["q01_roll_std_100"]) * (data["mean"]))))) * (((data["q05_roll_std_100"]) * (data["q05_roll_std_100"]))))) * (((data["q05_roll_std_100"]) * (data["q05_roll_std_100"]))))) + (((((data["av_change_rate_roll_std_10"]) * (data["max_to_min"]))) * 2.0)))) +
            0.0399999991*activation(((((((data["q05_roll_std_10"]) * 2.0)) * 2.0)) * (((((0.5434780121) * 2.0)) - (((((data["q05_roll_std_10"]) * (((((((((data["q95_roll_std_10"]) * 2.0)) * (data["q05_roll_std_10"]))) + (data["q05_roll_std_10"]))) * (data["q05_roll_std_10"]))))) + (data["skew"]))))))) +
            0.0399999991*activation(((((data["abs_q05"]) * (data["q95_roll_std_100"]))) - (((((data["q05_roll_std_100"]) * (((data["q05_roll_std_100"]) * (((data["q05_roll_std_100"]) * (((data["q05_roll_std_100"]) * (((data["abs_q05"]) * (data["q05_roll_std_100"]))))))))))) + ((((data["min_roll_std_100"]) + (data["av_change_abs_roll_mean_100"]))/2.0)))))) +
            0.0399999991*activation(((((data["Moving_average_6000_mean"]) + (0.8873239756))) * (((((data["q05_roll_std_100"]) + (data["q95_roll_std_1000"]))) * (((1.6428600550) - (((data["q05_roll_std_1000"]) * (((((data["q05_roll_std_100"]) + (((data["exp_Moving_average_3000_mean"]) + (0.9756100178))))) + (data["q05_roll_std_100"]))))))))))) +
            0.0399999991*activation(((((data["q95_roll_mean_10"]) + (((data["q95_roll_std_1000"]) * (((data["med"]) + (((((data["q05_roll_mean_100"]) + (((data["q95_roll_mean_10"]) * (data["q95_roll_mean_10"]))))) * (data["q05_roll_std_100"]))))))))) * (((((data["exp_Moving_average_30000_mean"]) - (data["q05_roll_std_100"]))) - (data["q05_roll_std_1000"]))))) +
            0.0399999991*activation(((((((data["min"]) + ((((-1.0*((data["MA_400MA_BB_high_mean"])))) + (activation((((((0.3183098733) + (((((data["kurt"]) * 2.0)) * 2.0)))) * ((((3.0)) + ((-1.0*((data["kurt"])))))))))))))) * 2.0)) * 2.0)) +
            0.0399999991*activation(((((((data["min_roll_mean_1000"]) * 2.0)) * 2.0)) + ((((-1.0*((((((data["q05_roll_std_1000"]) + (((data["q01_roll_std_10"]) * (data["q95"]))))) + (((data["avg_last_50000"]) * (((data["q01_roll_std_10"]) - (data["avg_last_50000"])))))))))) * (data["q01_roll_std_100"]))))) +
            0.0399921872*activation(((((data["q95_roll_std_1000"]) - ((((((((data["av_change_abs_roll_mean_10"]) + (data["std_first_10000"]))/2.0)) + (((data["q95"]) - (data["min_last_10000"]))))) - (((data["min"]) + (((data["q95_roll_std_1000"]) + (data["mean_change_rate_last_10000"]))))))))) - (((data["q95"]) + (data["q95"]))))) +
            0.0399999991*activation(((((((data["min_roll_mean_1000"]) - (((data["q05_roll_std_1000"]) * (((activation((((data["med"]) * 2.0)))) + ((((((data["skew"]) * (data["kurt"]))) + (data["q05_roll_std_1000"]))/2.0)))))))) + (((data["kurt"]) * (data["skew"]))))) + (data["min_roll_mean_1000"]))) +
            0.0399921872*activation(((data["exp_Moving_average_30000_mean"]) * (((((data["max_roll_std_100"]) - (((((data["exp_Moving_average_30000_mean"]) * (data["MA_400MA_std_mean"]))) + (data["q05_roll_mean_10"]))))) * (((((data["mean_change_rate_first_10000"]) * (73.0))) - (((((data["abs_mean"]) * (73.0))) * (73.0))))))))) +
            0.0399999991*activation(((data["q01_roll_std_10"]) + ((-1.0*((((data["q05_roll_std_1000"]) * (((data["av_change_rate_roll_mean_100"]) + ((((-1.0*((data["classic_sta_lta4_mean"])))) + (((data["med"]) + (((data["q05_roll_std_1000"]) + (((data["iqr"]) * (((data["iqr"]) + (data["max_last_10000"])))))))))))))))))))) +
            0.0399921872*activation((((((((data["q05"]) + (data["mean_change_rate_first_10000"]))/2.0)) + (((data["kurt"]) - (((((data["kurt"]) * (data["abs_max_roll_std_1000"]))) * (data["kurt"]))))))) + (((((data["av_change_abs_roll_std_100"]) * ((-1.0*((data["classic_sta_lta3_mean"])))))) * 2.0)))) +
            0.0399999991*activation(((((((((((((((data["skew"]) - ((((data["min_roll_std_1000"]) + (data["q95_roll_mean_100"]))/2.0)))) + (((data["q95_roll_mean_100"]) * (data["q05"]))))) * (data["MA_400MA_std_mean"]))) * (data["MA_400MA_BB_high_mean"]))) * 2.0)) * 2.0)) + ((((data["mean_change_abs"]) + (data["min_roll_std_1000"]))/2.0)))) +
            0.0399921872*activation((((((4.12892913818359375)) * ((((((data["exp_Moving_average_300_mean"]) + (data["med"]))/2.0)) - (data["q01_roll_std_100"]))))) * (((data["skew"]) + (((0.2439019978) + ((((data["MA_1000MA_std_mean"]) + ((((((data["av_change_abs_roll_std_1000"]) + (data["med"]))) + (data["iqr"]))/2.0)))/2.0)))))))) +
            0.0399999991*activation(((((((((data["av_change_abs_roll_std_10"]) * (((((data["mean_change_rate_last_50000"]) + (data["av_change_abs_roll_mean_10"]))) + (data["min_last_50000"]))))) * 2.0)) * 2.0)) + ((((((((1.0)) + (((data["av_change_rate_roll_std_10"]) + (data["skew"]))))/2.0)) + (data["q01_roll_mean_1000"]))/2.0)))) +
            0.0399999991*activation(((((((((data["q05_roll_std_10"]) * (((((((data["med"]) + (data["q05_roll_std_10"]))) * (data["q05_roll_mean_10"]))) * (data["q95_roll_std_1000"]))))) * (data["q05_roll_std_10"]))) + (((((data["q05_roll_mean_10"]) + (data["q95_roll_std_100"]))) + (data["q95_roll_std_100"]))))) * 2.0)) +
            0.0399921872*activation((((data["skew"]) + (((((((((data["min_roll_std_10"]) * ((-1.0*((data["min_roll_std_10"])))))) + (data["q05_roll_mean_1000"]))) * (data["Hilbert_mean"]))) - ((((data["av_change_rate_roll_std_100"]) + (((data["max_roll_std_100"]) * (((data["skew"]) * (data["skew"]))))))/2.0)))))/2.0)) +
            0.0399999991*activation(((data["q05_roll_std_10"]) * (((((((((data["q01_roll_mean_1000"]) - (activation((((data["Moving_average_6000_mean"]) - (((data["ave_roll_std_10"]) * (((data["kurt"]) - ((((data["av_change_abs_roll_mean_10"]) + ((((data["Moving_average_6000_mean"]) + (data["ave_roll_std_10"]))/2.0)))/2.0)))))))))))) * 2.0)) * 2.0)) * 2.0)))) +
            0.0399843715*activation(((data["min"]) + ((-1.0*((((data["av_change_rate_roll_std_100"]) * ((((((((data["mean_change_rate"]) + (data["min"]))/2.0)) + (data["min_roll_std_1000"]))) + (((activation((data["min_last_10000"]))) + ((((((data["min_roll_std_100"]) * (data["av_change_rate_roll_mean_10"]))) + (data["av_change_abs_roll_std_10"]))/2.0))))))))))))) +
            0.0399999991*activation(((data["min_roll_mean_100"]) * (((((data["min_roll_std_1000"]) * (((data["q01_roll_std_1000"]) * (((((((data["q01_roll_std_1000"]) * (data["q99_roll_std_100"]))) + (data["exp_Moving_average_300_mean"]))) + (data["avg_first_10000"]))))))) + (((((data["mean_change_rate"]) * (3.1415927410))) - (data["avg_first_10000"]))))))) +
            0.0399921872*activation(((((data["q95_roll_mean_1000"]) * (((((((((data["q95_roll_mean_1000"]) + (((data["avg_last_10000"]) + (((3.0) / 2.0)))))/2.0)) + (data["q05_roll_mean_100"]))/2.0)) - (data["q95_roll_std_1000"]))))) * (((((((data["q95_roll_std_1000"]) * 2.0)) + (data["q99_roll_mean_10"]))) + (data["classic_sta_lta4_mean"]))))) +
            0.0399296731*activation((((((((((((data["q01_roll_std_100"]) + (activation((((data["q01"]) + ((-1.0*((((data["q01_roll_std_100"]) * 2.0))))))))))/2.0)) * ((-1.0*(((((data["q95"]) + (((data["exp_Moving_average_30000_mean"]) * 2.0)))/2.0))))))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399921872*activation(((((activation((data["q99"]))) + ((-1.0*(((((data["max_first_10000"]) + (((data["av_change_rate_roll_mean_1000"]) - (data["q99"]))))/2.0))))))) - ((((((((data["av_change_rate_roll_mean_1000"]) * (data["av_change_rate_roll_mean_1000"]))) + (data["q05_roll_std_10"]))/2.0)) * (((data["q05_roll_std_10"]) * (data["iqr"]))))))) +
            0.0399999991*activation(((data["min_roll_mean_1000"]) * (((((((((((data["std_roll_std_100"]) + (((data["q05"]) + (((((data["q95_roll_mean_1000"]) * (data["q95_roll_std_1000"]))) + (((data["q95_roll_mean_1000"]) * (((data["q95_roll_mean_1000"]) * (data["q95_roll_std_1000"]))))))))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) +
            0.0399921872*activation(((((data["mean_change_abs"]) * (data["min_roll_std_10"]))) + (((((((data["std_last_50000"]) * (3.6923100948))) * (((data["q95_roll_mean_1000"]) + (((data["classic_sta_lta4_mean"]) - (data["min_roll_std_10"]))))))) * ((((((data["min_roll_std_10"]) + (data["q05_roll_mean_100"]))/2.0)) - (data["max_roll_std_10"]))))))) +
            0.0399999991*activation(((((data["q01_roll_std_10"]) + (data["min_roll_std_100"]))) * (((data["abs_q05"]) - (((data["avg_last_50000"]) - (((((data["classic_sta_lta2_mean"]) / 2.0)) + (((data["max_roll_mean_1000"]) * (((((data["avg_last_50000"]) - (activation((data["min_roll_std_100"]))))) - (data["min_roll_std_100"]))))))))))))) +
            0.0399999991*activation(((((data["q95_roll_mean_100"]) * (data["min_roll_mean_1000"]))) - (((((data["av_change_abs_roll_std_100"]) * (((((data["classic_sta_lta3_mean"]) + (data["mean_change_rate_first_50000"]))) + (((data["q95_roll_mean_100"]) * (data["classic_sta_lta3_mean"]))))))) + (((data["mean_change_abs"]) * (((data["classic_sta_lta3_mean"]) - (data["mean_change_rate_first_50000"]))))))))) +
            0.0398359038*activation(((data["q95_roll_mean_10"]) * (((((data["q95_roll_mean_10"]) * (((data["min_roll_mean_1000"]) + (data["av_change_rate_roll_std_10"]))))) - (((data["max_to_min"]) + (((((((data["max_to_min"]) * 2.0)) * 2.0)) * (((data["av_change_rate_roll_std_10"]) + (data["q01_roll_mean_1000"]))))))))))) +
            0.0399999991*activation(((data["q01_roll_std_10"]) * (((((((data["q01_roll_std_10"]) * (data["skew"]))) + (((data["classic_sta_lta4_mean"]) - (data["classic_sta_lta2_mean"]))))) + ((((data["av_change_abs_roll_mean_1000"]) + (((((-1.0*((data["q01_roll_std_100"])))) + (((data["q01_roll_std_10"]) * ((-1.0*((data["q01_roll_std_10"])))))))/2.0)))/2.0)))))) +
            0.0385153331*activation((((((data["std_roll_mean_1000"]) + (data["av_change_abs_roll_mean_100"]))) + (((data["max_first_10000"]) * (((data["MA_700MA_std_mean"]) * (((73.0) * (((activation((data["max_to_min"]))) - (((data["av_change_rate_roll_mean_1000"]) + (((((data["std_roll_mean_1000"]) * 2.0)) * 2.0)))))))))))))/2.0)) +
            0.0399999991*activation(((((data["std_roll_mean_1000"]) * (((data["av_change_rate_roll_std_1000"]) - (((((((data["skew"]) - (data["min_roll_std_10"]))) - (data["classic_sta_lta4_mean"]))) + (((data["std_roll_mean_1000"]) - (2.0))))))))) + (((data["classic_sta_lta4_mean"]) * (((data["av_change_abs_roll_mean_1000"]) - (data["q99"]))))))) +
            0.0399843715*activation((-1.0*((((data["kurt"]) * (((data["kurt"]) * ((((((data["av_change_abs_roll_mean_10"]) + (((((data["med"]) - (((data["med"]) * (data["av_change_abs_roll_mean_10"]))))) - (data["min_roll_std_10"]))))/2.0)) + (((data["q99_roll_std_1000"]) * (data["kurt"])))))))))))) +
            0.0399921872*activation(((data["mean_change_rate_first_10000"]) * (((data["av_change_abs_roll_mean_100"]) * ((-1.0*((((((activation((((((data["classic_sta_lta3_mean"]) + (data["mean_change_rate_first_10000"]))) * 2.0)))) + (((1.9019600153) - (data["av_change_abs_roll_mean_100"]))))) - (((((data["classic_sta_lta3_mean"]) * 2.0)) * (data["mean_change_rate_first_10000"])))))))))))) +
            0.0399921872*activation(((((data["av_change_rate_roll_std_100"]) * (((data["classic_sta_lta3_mean"]) * (((((data["min_roll_std_1000"]) - (data["min"]))) * (data["q05"]))))))) + (((data["mean_change_rate_last_10000"]) * (((((data["av_change_abs_roll_std_1000"]) - (data["min"]))) - (((data["classic_sta_lta3_mean"]) * (data["q01_roll_std_1000"]))))))))) +
            0.0399999991*activation(((data["q95_roll_std_1000"]) + (((data["q05_roll_mean_10"]) - ((-1.0*((((data["q95_roll_std_1000"]) * (((data["q95_roll_std_1000"]) * (((((data["q01_roll_std_10"]) * ((((((((data["q95_roll_std_1000"]) / 2.0)) + (data["q01_roll_mean_1000"]))/2.0)) - (data["med"]))))) * (data["q95_roll_std_1000"])))))))))))))) +
            0.0399843715*activation(((data["av_change_abs_roll_mean_100"]) * (((data["classic_sta_lta4_mean"]) * (((((((((3.65123486518859863)) + ((((3.65123486518859863)) * (data["abs_max_roll_std_10"]))))/2.0)) + (((((data["min_last_50000"]) * (((data["abs_max_roll_std_1000"]) - (data["av_change_abs_roll_mean_10"]))))) - (data["classic_sta_lta4_mean"]))))) * (data["ave_roll_mean_10"]))))))) +
            0.0399999991*activation(((((data["max_to_min_diff"]) * (((data["min_roll_std_100"]) * (((data["av_change_rate_roll_std_100"]) + (((data["q01_roll_std_100"]) + (((((((data["av_change_rate_roll_mean_100"]) + (data["Hilbert_mean"]))) * (data["q95_roll_mean_1000"]))) + (((data["q01"]) * (data["min_roll_std_100"]))))))))))))) * 2.0)) +
            0.0399999991*activation(((((activation((((((data["q05"]) + (((activation((data["avg_first_10000"]))) + (data["mean_change_rate_last_10000"]))))) + (data["q05"]))))) + (((((data["kurt"]) * (data["mean_change_rate_last_10000"]))) + (data["q95_roll_std_1000"]))))) * (((data["q01_roll_mean_1000"]) + (data["av_change_rate_roll_mean_100"]))))) +
            0.0399921872*activation(((data["min_roll_mean_1000"]) + ((((((((((((data["av_change_rate_roll_std_10"]) + (((data["av_change_abs_roll_std_1000"]) * (data["min_roll_mean_1000"]))))/2.0)) * (data["classic_sta_lta3_mean"]))) + (data["av_change_rate_roll_std_10"]))/2.0)) + (((data["abs_max_roll_mean_100"]) - (((data["av_change_abs_roll_std_1000"]) * (data["classic_sta_lta1_mean"]))))))/2.0)))) +
            0.0399999991*activation(((data["max_to_min"]) * (((((data["MA_400MA_std_mean"]) - (data["q95_roll_std_100"]))) + ((((((data["min_roll_std_10"]) * (((((((data["min_roll_std_10"]) * (data["classic_sta_lta1_mean"]))) - (data["min_roll_std_10"]))) - (data["kurt"]))))) + ((((data["kurt"]) + (data["min_roll_std_10"]))/2.0)))/2.0)))))) +
            0.0399921872*activation(((data["min_roll_std_10"]) * ((((((((((((activation((data["min_roll_std_10"]))) + (data["avg_last_10000"]))/2.0)) - (data["q95_roll_std_100"]))) - (data["MA_700MA_BB_high_mean"]))) + (((data["abs_max_roll_mean_100"]) - (data["classic_sta_lta4_mean"]))))) * (((data["min_roll_std_10"]) - (((3.0) / 2.0)))))))) +
            0.0399765596*activation(((((data["avg_last_10000"]) * (((data["min_roll_mean_1000"]) + (((((data["avg_last_10000"]) + (activation((data["mean_change_rate_last_50000"]))))) * (((data["min_roll_std_10"]) * (data["min_last_10000"]))))))))) + (activation((activation((((((data["mean_change_rate_last_50000"]) * (data["min_roll_std_1000"]))) * 2.0)))))))) +
            0.0399921872*activation(activation((activation((activation((((((((data["q95_roll_std_10"]) - (data["q05"]))) * (((data["abs_q05"]) * (((data["mean_change_rate_first_10000"]) + (((((data["mean_change_rate_first_10000"]) + (data["q05"]))) * (((data["abs_q05"]) * (data["q95_roll_mean_1000"]))))))))))) * 2.0)))))))) +
            0.0399843715*activation(((data["q95_roll_std_1000"]) * (((data["max_roll_mean_100"]) + (((((((((data["q01_roll_mean_1000"]) * 2.0)) * 2.0)) * (data["min_roll_mean_1000"]))) * (((((((data["q01_roll_mean_1000"]) + ((-1.0*((data["max_to_min"])))))) * 2.0)) * 2.0)))))))) +
            0.0399765596*activation(((data["q05_roll_std_100"]) * ((-1.0*((activation((((((data["med"]) + (((((data["med"]) + (((data["q01_roll_std_10"]) * (((data["avg_first_50000"]) + (data["q01_roll_std_10"]))))))) * ((((1.16074943542480469)) + (data["avg_first_50000"]))))))) * 2.0))))))))) +
            0.0399062298*activation((-1.0*((((data["classic_sta_lta2_mean"]) * ((((((activation((data["kurt"]))) + (data["avg_last_50000"]))/2.0)) + (((data["q05_roll_mean_100"]) * ((-1.0*(((((((((data["kurt"]) + (data["q05_roll_mean_100"]))/2.0)) - (activation((data["kurt"]))))) + (data["std_last_10000"]))))))))))))))) +
            0.0399453007*activation(((data["std_last_50000"]) * (((((activation((((((((((((data["av_change_abs_roll_mean_100"]) + (activation((activation((((((((((data["skew"]) + (data["Moving_average_6000_mean"]))) * 2.0)) * 2.0)) * 2.0)))))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) * 2.0)))) +
            0.0390935726*activation((-1.0*((((data["max_roll_mean_100"]) * (((((((activation((((((data["classic_sta_lta3_mean"]) + ((((data["q99_roll_mean_1000"]) + (data["mean_change_rate_first_10000"]))/2.0)))) + (((data["av_change_abs_roll_mean_10"]) + (((data["classic_sta_lta3_mean"]) * (data["q95_roll_mean_1000"]))))))))) * 2.0)) * 2.0)) * 2.0))))))) +
            0.0399296731*activation(((data["q95_roll_std_100"]) + ((-1.0*((((data["q95_roll_std_10"]) + ((((((((data["kurt"]) * ((((data["mean_change_rate_last_10000"]) + (data["trend"]))/2.0)))) * (data["kurt"]))) + ((((((data["med"]) + (data["max_first_50000"]))/2.0)) * (data["min_roll_std_100"]))))/2.0))))))))) +
            0.0398984179*activation(((((activation((((data["q01"]) - (data["q05"]))))) + (((((((((activation((data["q05_roll_std_100"]))) + (data["q01"]))) * 2.0)) + (data["q95_roll_mean_1000"]))) * 2.0)))) * ((-1.0*((((data["q01"]) - (data["q05"])))))))) +
            0.0399999991*activation(activation((activation(((-1.0*((((data["mean_change_rate_first_10000"]) * (((data["min_last_10000"]) + (((data["min_last_10000"]) + (((((data["abs_q95"]) + (((data["avg_last_50000"]) + ((((data["min_last_10000"]) + (0.8873239756))/2.0)))))) * ((-1.0*((data["q05_roll_std_100"]))))))))))))))))))) +
            0.0399999991*activation(((data["min_first_50000"]) * ((((-1.0*((data["av_change_abs_roll_std_100"])))) - (((data["ave_roll_mean_10"]) * ((-1.0*((((data["max_to_min"]) - (((((((data["MA_400MA_BB_low_mean"]) - (((data["av_change_rate_roll_mean_100"]) + (((data["q01_roll_mean_1000"]) * (data["av_change_rate_roll_mean_100"]))))))) * 2.0)) * 2.0))))))))))))) +
            0.0399999991*activation(((((((((data["max_to_min_diff"]) * (((data["av_change_rate_roll_std_100"]) + (((((data["q95_roll_std_10"]) + (data["max_first_50000"]))) + (((data["mean_change_abs"]) + (((((data["MA_400MA_BB_low_mean"]) * (data["max_first_50000"]))) * (data["max_first_50000"]))))))))))) * 2.0)) * 2.0)) * (data["min_roll_std_100"]))) +
            0.0399999991*activation(((((data["max"]) * (((data["std_first_50000"]) * ((((((-1.0*((data["max_roll_std_100"])))) + (((((((((data["q001"]) + (((data["q001"]) * (data["classic_sta_lta4_mean"]))))) + (data["classic_sta_lta4_mean"]))) * 2.0)) * 2.0)))) * 2.0)))))) * (data["q05_roll_std_1000"]))) +
            0.0399765596*activation(((((data["abs_max_roll_std_1000"]) * 2.0)) * (((data["q95_roll_std_10"]) - (((data["q999"]) - (activation((((data["max_last_10000"]) * (((((((((((((data["abs_max_roll_std_1000"]) + (data["Moving_average_6000_mean"]))) * 2.0)) + (data["mean_change_abs"]))) * 2.0)) * 2.0)) * 2.0)))))))))))) +
            0.0399687439*activation(((data["min_roll_std_10"]) * ((((((data["avg_last_50000"]) + (((((((data["q01_roll_std_1000"]) * (data["classic_sta_lta4_mean"]))) * (data["mean_change_rate_last_50000"]))) - (((data["min_roll_std_10"]) * (data["av_change_abs_roll_mean_10"]))))))/2.0)) + ((((((data["classic_sta_lta4_mean"]) * (data["Moving_average_6000_mean"]))) + (data["mean_change_rate_last_50000"]))/2.0)))))) +
                   
            0.0398906022*activation((((activation((((data["av_change_abs_roll_mean_1000"]) - (((data["Hann_window_mean"]) - (data["skew"]))))))) + (((((((data["min_roll_mean_1000"]) * 2.0)) * 2.0)) * (((data["skew"]) + (((((data["std_roll_mean_1000"]) - (data["q99_roll_mean_10"]))) - (data["med"]))))))))/2.0)) +
                   
            0.0399999991*activation(((data["std_first_10000"]) * ((-1.0*((((data["min_roll_std_10"]) * (((((((((data["kurt"]) + (((((data["av_change_rate_roll_std_1000"]) - (data["trend"]))) + (data["mean_change_rate_first_10000"]))))) - (data["max_last_10000"]))) + (((data["trend"]) * (data["trend"]))))) * 2.0))))))))) +
                   
            0.0399765596*activation(((data["q99"]) * ((-1.0*((((data["iqr"]) - (((((((1.9019600153) + (((data["abs_max_roll_mean_100"]) + (data["q99"]))))) * (((((data["std_roll_mean_1000"]) * 2.0)) * (((data["max_roll_mean_1000"]) - (data["std_roll_mean_1000"]))))))) + (data["min_roll_std_1000"])))))))))) +
                   
            0.0399765596*activation(((((data["mean_change_rate_last_10000"]) + (activation((((((activation((data["skew"]))) + (((data["max_roll_std_1000"]) * 2.0)))) * 2.0)))))) * (((data["abs_max_roll_std_10"]) + ((-1.0*((((data["skew"]) * (((((data["abs_max"]) / 2.0)) - (data["ave_roll_std_1000"])))))))))))) +
                   
            0.0355928876*activation((((((((data["min_roll_std_100"]) * (data["q05_roll_mean_1000"]))) * (data["min_roll_std_1000"]))) + (((((data["min_roll_std_100"]) * (data["min_roll_std_100"]))) - (((((data["max_to_min"]) + ((((((data["q999"]) + (data["min_roll_std_100"]))/2.0)) + (data["q999"]))))) * (data["classic_sta_lta3_mean"]))))))/2.0)) +
                   
            0.0399843715*activation((((((-1.0*((((((((data["q01_roll_std_10"]) * (data["q05_roll_std_1000"]))) * (data["max_first_50000"]))) * (data["q01_roll_std_10"])))))) * (((data["max_first_50000"]) + (data["max_to_min_diff"]))))) * (((((data["min"]) - (data["q05"]))) - (data["q05"]))))) +
                   
            0.0399765596*activation(((data["med"]) * (((activation((((((((((data["av_change_abs_roll_std_100"]) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) - (((((data["min_roll_std_10"]) + ((((data["q01_roll_std_1000"]) + ((((data["max_to_min_diff"]) + (data["min_roll_std_1000"]))/2.0)))/2.0)))) * (((data["av_change_abs_roll_std_100"]) * 2.0)))))))) +
                   
            0.0387106836*activation((-1.0*((((((data["av_change_rate_roll_mean_10"]) * (((data["q95_roll_std_1000"]) + (((((((((((data["min_roll_mean_1000"]) * 2.0)) * 2.0)) * (((data["q95_roll_std_1000"]) + (data["min_roll_mean_1000"]))))) * 2.0)) * (((data["q95_roll_std_100"]) - (((data["min_roll_mean_1000"]) * 2.0)))))))))) * 2.0))))) +
                   
            0.0399999991*activation(((data["max_last_10000"]) * ((((((((data["MA_700MA_BB_low_mean"]) * (((((data["avg_first_10000"]) - (data["av_change_abs_roll_mean_1000"]))) - (((data["avg_first_10000"]) * (((data["avg_first_10000"]) - (((data["min_roll_std_1000"]) * 2.0)))))))))) * 2.0)) + (((data["avg_first_10000"]) - (data["mean_change_rate_last_10000"]))))/2.0)))) +
                   
            0.0399999991*activation(activation((((((((((data["av_change_rate_roll_std_100"]) * (((data["min"]) * (((data["q05_roll_std_100"]) + (((((data["av_change_abs_roll_std_100"]) + (((data["mean_change_rate_first_10000"]) * (data["av_change_rate_roll_std_100"]))))) + (((data["med"]) - (data["mean_change_rate_first_10000"]))))))))))) * 2.0)) * 2.0)) * 2.0)))) +
                   
            0.0359679610*activation((-1.0*((((activation((((((((((((((((((data["min_roll_std_1000"]) - (data["av_change_rate_roll_std_1000"]))) - (((-2.0) + ((-1.0*((data["min_roll_std_1000"])))))))) - (activation((data["av_change_abs_roll_std_100"]))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) / 2.0))))) +
                   
            0.0373354182*activation((-1.0*((((data["max_to_min"]) * (((data["avg_first_50000"]) * (((((data["ave_roll_std_100"]) - (((data["classic_sta_lta1_mean"]) + (data["mean_change_rate_first_10000"]))))) * (((data["classic_sta_lta2_mean"]) - ((-1.0*((((data["ave_roll_std_100"]) / 2.0)))))))))))))))) +
                   
            0.0398671627*activation(((((0.8873239756) + (data["avg_last_50000"]))) * (((data["iqr"]) * (((((0.8873239756) + (data["Moving_average_6000_mean"]))) + (((data["q05_roll_mean_10"]) * (((((((data["ave10"]) + (data["q01_roll_std_10"]))) * (data["q01_roll_std_10"]))) * (data["iqr"]))))))))))) +
                   
            0.0399453007*activation((((activation((((((((((((data["mean_change_rate_last_10000"]) + (activation(((((((-1.0*((data["mean_change_rate_last_50000"])))) * 2.0)) * (data["q95_roll_mean_1000"]))))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) + (activation((activation(((((-1.0*((data["mean_change_rate_last_50000"])))) * 2.0)))))))/2.0)) +
                   
            0.0399687439*activation(((((((data["max_roll_std_1000"]) + (((data["max_roll_std_1000"]) + (data["trend"]))))) + (data["count_big"]))) * (((data["max_to_min"]) * (((((((-1.0*((data["max_to_min"])))) + (data["max_roll_mean_1000"]))/2.0)) - (data["q99_roll_std_10"]))))))) +
                   
            0.0399843715*activation(((data["classic_sta_lta2_mean"]) * (((data["abs_mean"]) * (((data["classic_sta_lta1_mean"]) * (((((((data["mean_change_rate_first_50000"]) + ((-1.0*((data["mean_change_abs"])))))) + (data["classic_sta_lta4_mean"]))) + (((((((-2.0) + (data["av_change_rate_roll_std_10"]))/2.0)) + (data["av_change_abs_roll_std_1000"]))/2.0)))))))))) +
                   
            0.0399687439*activation(((data["min_roll_std_100"]) * (((((((data["mean_change_abs"]) + (((((((data["av_change_abs_roll_std_100"]) * (data["min_roll_std_100"]))) + (data["min_roll_std_1000"]))) * (((data["std_roll_mean_1000"]) * (((data["min_last_10000"]) + (data["av_change_abs_roll_std_100"]))))))))) * 2.0)) * (data["av_change_abs_roll_std_100"]))))) +
                   
            0.0399374850*activation(((((data["min_first_10000"]) - (((data["mean_change_rate_first_10000"]) * (data["kurt"]))))) * (((((((data["abs_trend"]) * (data["kurt"]))) - (data["mean_change_abs"]))) + (((data["std_roll_mean_1000"]) + (((data["std_roll_mean_1000"]) + (((data["min_first_10000"]) + (data["abs_std"]))))))))))) +
                   
            0.0399921872*activation(((data["q95_roll_std_10"]) * ((-1.0*((((((-1.0*((data["abs_max_roll_mean_100"])))) + ((((-1.0*((((data["classic_sta_lta4_mean"]) / 2.0))))) - (((data["q001"]) * (((data["q95_roll_mean_1000"]) - ((-1.0*((((((((data["classic_sta_lta4_mean"]) / 2.0)) / 2.0)) / 2.0))))))))))))/2.0))))))) +
                   
            0.0399843715*activation(((((data["mean_change_rate_last_10000"]) * (((data["abs_max"]) * (((((((((((data["min_roll_std_1000"]) + (data["max_to_min"]))) - (data["count_big"]))) * 2.0)) - (data["max_to_min"]))) + (((data["av_change_abs_roll_std_1000"]) + (data["mean_change_abs"]))))))))) + (data["count_big"]))) +
                   
            0.0383434258*activation(((((data["med"]) * ((-1.0*((((((((data["trend"]) * (((activation((data["av_change_abs_roll_std_100"]))) - (data["mean_change_abs"]))))) + (data["mean_change_abs"]))) * 2.0))))))) * (((((data["av_change_abs_roll_std_100"]) + (activation((data["min_last_10000"]))))) + (data["av_change_abs_roll_std_100"]))))) +
                   
            0.0399999991*activation(activation((((((((data["min_roll_mean_100"]) * 2.0)) * 2.0)) * (((((((data["max_last_10000"]) - (0.5434780121))) * (data["av_change_rate_roll_mean_100"]))) + (((data["av_change_rate_roll_std_10"]) - (((data["min_roll_std_10"]) * (data["av_change_rate_roll_std_10"]))))))))))) +
                   
            0.0399921872*activation((-1.0*((((((((((data["min_roll_std_100"]) + (data["av_change_abs_roll_std_10"]))/2.0)) * (((data["av_change_abs_roll_mean_1000"]) + (data["av_change_abs_roll_std_10"]))))) + (((data["av_change_abs_roll_std_100"]) * (((data["classic_sta_lta4_mean"]) * (((data["av_change_abs_roll_std_10"]) + (((data["min_roll_std_1000"]) * (data["av_change_abs_roll_std_100"]))))))))))/2.0))))) +
                   
            0.0293416679*activation(activation((((data["abs_q95"]) * (((data["q01_roll_std_10"]) * (((data["q01_roll_std_10"]) * (((((data["max_last_50000"]) - ((((((data["av_change_abs_roll_mean_10"]) - (data["classic_sta_lta4_mean"]))) + (data["av_change_abs_roll_mean_100"]))/2.0)))) - ((-1.0*((((data["av_change_abs_roll_mean_10"]) * (data["classic_sta_lta4_mean"])))))))))))))))) +
                   
            0.0399765596*activation(((((data["trend"]) * (data["max_first_10000"]))) * (((data["med"]) + (((data["av_change_rate_roll_mean_1000"]) + (((((2.0769200325) - (((activation((((((((((data["classic_sta_lta1_mean"]) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) * 2.0)))) / 2.0)))))))) +
                   
            0.0399921872*activation(((((data["mean_change_rate_last_50000"]) * ((-1.0*((((data["min_first_10000"]) - (data["std_first_10000"])))))))) * ((((((((((-1.0*((data["std_first_10000"])))) - (data["min_last_50000"]))) - (data["abs_max_roll_std_10"]))) - (((data["mean_change_rate_last_50000"]) * (data["std_first_10000"]))))) - (data["classic_sta_lta2_mean"]))))) +
                   
            0.0399921872*activation(((data["av_change_abs_roll_mean_10"]) * (((data["abs_trend"]) + ((((((((data["mean_change_abs"]) * (data["min_last_10000"]))) + (((data["mean_change_rate_first_10000"]) * (data["max_first_10000"]))))) + ((((data["av_change_abs_roll_mean_10"]) + (((((data["med"]) - (data["q95_roll_mean_1000"]))) - (data["mean_change_rate_first_10000"]))))/2.0)))/2.0)))))) +
            0.0399531126*activation(((data["ave_roll_std_100"]) * (((data["min_roll_std_100"]) * (((((data["max_roll_mean_100"]) + ((((data["max_to_min_diff"]) + (data["q99"]))/2.0)))) * (((((((data["max_to_min_diff"]) * ((((-1.0*((data["std_first_50000"])))) * (data["q95_roll_std_100"]))))) - (data["skew"]))) * 2.0)))))))) +
            0.0399843715*activation(((data["std_roll_std_100"]) * (((data["min_roll_std_1000"]) * (((data["max_roll_mean_10"]) - (((data["kurt"]) * ((((((data["avg_last_10000"]) * (data["avg_last_10000"]))) + ((((data["q99_roll_mean_100"]) + ((((((data["avg_last_10000"]) * (data["avg_last_10000"]))) + (data["avg_first_10000"]))/2.0)))/2.0)))/2.0)))))))))) +
            0.0353584699*activation(((data["med"]) * (((activation((((activation(((((((-1.0*((data["max_to_min"])))) - (data["trend"]))) * 2.0)))) - (((data["min_roll_std_100"]) * (data["med"]))))))) - (activation((activation((data["min_roll_std_100"]))))))))) +
            0.0399765596*activation((((((((data["ave_roll_mean_10"]) + (data["min_roll_std_100"]))/2.0)) * (((((data["classic_sta_lta4_mean"]) + (data["kurt"]))) + (((((data["q95_roll_mean_10"]) + (((data["q95"]) * (data["q95"]))))) * (data["sum"]))))))) * ((((data["q05_roll_std_100"]) + (data["ave_roll_mean_10"]))/2.0)))) +
            0.0399843715*activation(((data["min_roll_std_10"]) * ((((((-1.0*((((data["av_change_abs_roll_std_10"]) * (data["av_change_abs_roll_mean_100"])))))) * (data["av_change_abs_roll_std_10"]))) + (activation(((((-1.0*((data["q05"])))) + (((data["min_roll_std_1000"]) * ((-1.0*((((data["av_change_abs_roll_mean_100"]) * (data["av_change_abs_roll_mean_100"])))))))))))))))) +
            0.0399999991*activation((-1.0*((((data["classic_sta_lta1_mean"]) * (((data["max_last_50000"]) + (((((data["abs_q95"]) + (((data["av_change_abs_roll_std_10"]) * 2.0)))) * (((((data["abs_std"]) + (data["max_last_50000"]))) * (((((data["iqr"]) * 2.0)) * (data["avg_last_10000"])))))))))))))) +
            0.0388200805*activation(((((data["min_first_10000"]) * (((data["mean_change_rate"]) * (((((data["min_roll_std_100"]) + (data["max_last_50000"]))) + (((((data["mean_change_rate"]) * (((data["mean_change_abs"]) * (((data["min_roll_std_100"]) + (data["min_first_10000"]))))))) - (data["mean_change_abs"]))))))))) * 2.0)) +
            0.0389529206*activation(((activation((activation((((data["max_to_min"]) + (data["max_to_min"]))))))) - (activation((((((((((((((((((((data["min_first_10000"]) * 2.0)) + (data["max_to_min"]))) * 2.0)) * 2.0)) + (data["max_last_50000"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)))))) +
            0.0399843715*activation((-1.0*((((((data["max_first_50000"]) * 2.0)) * (((activation(((((((-1.0*((((data["q95_roll_std_1000"]) + (((data["q01_roll_std_10"]) + (data["av_change_abs_roll_std_1000"])))))))) * 2.0)) * 2.0)))) + (((data["q01_roll_std_10"]) + (((data["max_first_50000"]) * (data["mean_change_abs"])))))))))))) +
            0.0399453007*activation((((((data["av_change_abs_roll_std_1000"]) + (data["mean_change_abs"]))/2.0)) * (((data["av_change_abs_roll_std_100"]) + (((data["trend"]) + (((((data["av_change_abs_roll_mean_1000"]) + ((((data["av_change_abs_roll_std_100"]) + (((data["av_change_rate_roll_std_100"]) + (((data["av_change_abs_roll_std_1000"]) + (data["min_roll_std_10"]))))))/2.0)))) * (data["av_change_abs_roll_mean_1000"]))))))))) +
            0.0399140455*activation(((data["std_first_50000"]) * (((data["mean_change_rate_last_50000"]) * ((-1.0*((((data["avg_first_50000"]) * ((((data["max_first_10000"]) + (((data["min_roll_mean_10"]) - (((data["avg_first_50000"]) - (((((data["mean_change_rate_last_50000"]) - (data["min_last_10000"]))) * (data["min_last_10000"]))))))))/2.0))))))))))) +
            0.0399999991*activation(((((data["min_last_10000"]) + (activation((((data["ave_roll_mean_1000"]) + (((((((data["av_change_abs_roll_mean_100"]) + (data["av_change_abs_roll_std_10"]))) * (data["ave_roll_mean_1000"]))) * (((data["min_roll_std_1000"]) * 2.0)))))))))) * (((data["min_roll_std_1000"]) * (((((data["std_roll_mean_1000"]) * 2.0)) * 2.0)))))) +
            0.0399765596*activation((((((data["av_change_abs_roll_mean_100"]) + (data["q99_roll_std_10"]))/2.0)) * (activation((((activation((((((((((((((data["avg_first_10000"]) + (data["max_roll_std_1000"]))) + (data["count_big"]))) * (73.0))) - (data["av_change_rate_roll_mean_100"]))) * 2.0)) + (data["avg_first_10000"]))))) * 2.0)))))) +
            0.0371556953*activation(((((data["max_last_10000"]) * (((((data["std_last_50000"]) * (data["avg_last_50000"]))) + (((((((data["min_roll_std_10"]) * 2.0)) + (((data["min_last_10000"]) * (((data["max_first_50000"]) * (((((data["min_roll_std_10"]) * 2.0)) * 2.0)))))))) * (data["max_first_10000"]))))))) * 2.0)) +
            0.0399609283*activation(((data["av_change_abs_roll_std_10"]) * ((((((((((data["min_last_50000"]) + (((data["trend"]) * (data["av_change_abs_roll_mean_10"]))))/2.0)) + (data["max_to_min"]))/2.0)) + (((data["min_last_50000"]) - (((data["min_last_50000"]) * (((data["min_last_50000"]) * (data["q05_roll_mean_1000"]))))))))/2.0)))) +
            0.0391638987*activation(((((data["trend"]) - ((((data["min_roll_std_100"]) + (data["q001"]))/2.0)))) * (((activation((activation((((((data["classic_sta_lta3_mean"]) + ((((data["classic_sta_lta3_mean"]) + ((((data["kurt"]) + (data["q95_roll_mean_100"]))/2.0)))/2.0)))) * 2.0)))))) * ((-1.0*((data["min_roll_std_100"])))))))) +
            0.0398749746*activation(((data["ave10"]) * ((-1.0*((((data["av_change_abs_roll_mean_10"]) * (((data["av_change_abs_roll_mean_100"]) + ((-1.0*(((((data["av_change_abs_roll_mean_10"]) + (((((data["classic_sta_lta2_mean"]) * 2.0)) + (((data["mean_change_rate_last_10000"]) * (data["av_change_rate_roll_mean_100"]))))))/2.0)))))))))))))) +
            0.0385778472*activation((-1.0*(((((activation((((((((data["trend"]) + (((data["min_roll_std_10"]) + (data["Moving_average_6000_mean"]))))) * 2.0)) * 2.0)))) + (((((-1.0*((((data["q95_roll_std_100"]) * (data["ave10"])))))) + (((data["mean_change_rate_first_50000"]) * (data["av_change_abs_roll_mean_10"]))))/2.0)))/2.0))))) +
            0.0399921872*activation(((((((((data["min_roll_std_10"]) * ((((data["ave_roll_std_10"]) + (data["avg_last_50000"]))/2.0)))) + (((((((((data["avg_last_10000"]) + (data["min_roll_std_10"]))/2.0)) + (data["min_roll_std_10"]))/2.0)) - (data["ave_roll_std_10"]))))/2.0)) + (((((data["q95_roll_std_100"]) - (data["ave_roll_std_10"]))) * 2.0)))/2.0)) +
            0.0399921872*activation((((((-1.0*((((data["mean_change_rate_first_10000"]) * 2.0))))) * ((((((((((data["av_change_rate_roll_std_10"]) * (data["avg_last_10000"]))) + (((data["min_last_10000"]) * (data["av_change_rate_roll_std_10"]))))) - (((data["avg_first_10000"]) / 2.0)))) + (data["av_change_rate_roll_std_1000"]))/2.0)))) * (data["av_change_abs_roll_std_1000"]))) +
            0.0399453007*activation(((data["av_change_abs_roll_std_1000"]) * (((data["avg_last_10000"]) * (((((((data["avg_last_10000"]) * (((data["avg_last_10000"]) * (((((data["med"]) * (((data["q001"]) - (data["classic_sta_lta1_mean"]))))) - (data["min_roll_std_10"]))))))) - (data["max_roll_mean_10"]))) - (data["min_roll_std_10"]))))))) +
            0.0398749746*activation(activation((activation((((((((((((-1.0*((data["av_change_abs_roll_std_100"])))) + ((-1.0*((data["mean_change_rate_first_50000"])))))/2.0)) / 2.0)) - (((data["mean_change_abs"]) * (((((((data["avg_last_50000"]) * (data["q99_roll_mean_10"]))) - (data["mean_change_rate_first_50000"]))) - (data["std_last_10000"]))))))) * 2.0)))))) +
            0.0363899209*activation(((data["max_last_10000"]) * (((((activation((((((((data["av_change_rate_roll_mean_100"]) + (((data["min_roll_std_1000"]) * 2.0)))) * 2.0)) * 2.0)))) + (((data["av_change_abs_roll_std_100"]) + ((-1.0*((data["mean_change_abs"])))))))) + (((((data["av_change_rate_roll_mean_10"]) * (data["max_last_50000"]))) * 2.0)))))) +
            0.0399999991*activation(((data["min_roll_std_100"]) * (((activation((data["min_last_10000"]))) + (((((activation((data["std_first_10000"]))) - ((((activation((data["min_last_10000"]))) + (((data["exp_Moving_average_30000_mean"]) * (data["min_last_10000"]))))/2.0)))) * ((((((data["avg_last_50000"]) + (data["mean_change_abs"]))/2.0)) * 2.0)))))))) +
            0.0351396762*activation(activation((((((data["av_change_abs_roll_std_1000"]) * (data["mean_change_rate_last_10000"]))) + (((data["med"]) * (activation((((data["av_change_rate_roll_std_100"]) - (((((data["avg_last_10000"]) - (((data["mean_change_rate_first_10000"]) - (((data["avg_last_10000"]) * (data["mean_change_rate_last_10000"]))))))) - (data["av_change_abs_roll_std_1000"]))))))))))))) +
            0.0399921872*activation((((activation((((((((((data["min_roll_std_10"]) + (((data["mean_change_rate_first_50000"]) + (((((data["mean_change_rate_first_50000"]) + (data["mean_change_rate_first_50000"]))) * (data["classic_sta_lta4_mean"]))))))) * (data["av_change_rate_roll_mean_1000"]))) * 2.0)) * 2.0)))) + (((data["av_change_abs_roll_mean_10"]) * (activation((data["med"]))))))/2.0)) +
            0.0379214697*activation((-1.0*((((((data["max_to_min_diff"]) * (((((((((((((data["min_last_10000"]) - (data["med"]))) - (data["mean_change_rate_first_10000"]))) + (data["q05_roll_std_100"]))) + (data["q01_roll_std_1000"]))) * 2.0)) * 2.0)))) * ((-1.0*((((data["mean_change_rate_first_10000"]) * (data["min_last_10000"]))))))))))) +
            0.0399218611*activation(((data["min_first_10000"]) * (activation((((((((((activation((((data["max_to_min"]) * 2.0)))) + (((((((data["min_first_50000"]) * 2.0)) + (((activation((activation((data["mean_change_rate_last_10000"]))))) + (data["min_roll_mean_10"]))))) * 2.0)))) * 2.0)) * 2.0)) * 2.0)))))) +
            0.0328423530*activation((-1.0*((((activation((((data["kurt"]) * (((((((data["mean_change_rate_last_50000"]) * 2.0)) * 2.0)) * (data["skew"]))))))) + (activation((((data["skew"]) * (((((data["mean_change_rate_last_50000"]) * 2.0)) * (data["skew"])))))))))))) +
            0.0346083194*activation((((((data["av_change_abs_roll_std_10"]) / 2.0)) + ((-1.0*((((activation((data["q05_roll_mean_1000"]))) + (activation((((((data["min_roll_std_100"]) - (data["mean_change_rate_last_50000"]))) - (((((data["q95_roll_mean_1000"]) - (data["av_change_abs_roll_std_10"]))) - (((data["av_change_abs_roll_mean_1000"]) - (data["mean_change_rate_last_50000"])))))))))))))))/2.0)) +
            0.0395155288*activation(((data["avg_last_50000"]) * ((-1.0*(((((activation((activation((((((((((((data["av_change_rate_roll_mean_100"]) * ((-1.0*((data["max_to_min"])))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)))))) + ((((((data["av_change_rate_roll_std_10"]) * (data["avg_last_10000"]))) + (data["av_change_rate_roll_std_10"]))/2.0)))/2.0))))))) +
            0.0316311792*activation(((data["max_first_50000"]) - (((data["classic_sta_lta3_mean"]) * (((((data["max_to_min_diff"]) + (data["mean_change_abs"]))) - (((data["trend"]) * (((((data["mean_change_abs"]) - (((activation((data["max_to_min_diff"]))) * 2.0)))) - (data["av_change_abs_roll_mean_100"]))))))))))) +
            0.0399609283*activation(((data["trend"]) * (((((((data["min_roll_std_1000"]) * (data["std_last_10000"]))) * ((((data["av_change_abs_roll_mean_10"]) + (((data["std_last_50000"]) - ((((((data["av_change_rate_roll_mean_10"]) * (data["q05_roll_std_1000"]))) + (data["min_roll_std_1000"]))/2.0)))))/2.0)))) + (((data["min_roll_std_1000"]) - (data["q05_roll_std_1000"]))))))) +
            0.0362180099*activation((((activation((((((((((((data["q01_roll_std_100"]) - (data["mean_change_rate_last_10000"]))) - ((((data["av_change_abs_roll_mean_10"]) + (data["abs_max_roll_mean_100"]))/2.0)))) - (0.2525250018))) * (data["min_roll_std_10"]))) * (73.0))))) + (((data["min_roll_mean_1000"]) + (data["abs_max_roll_mean_100"]))))/2.0)) +
            0.0399999991*activation((((activation((((73.0) * (((((73.0) * (data["av_change_abs_roll_std_100"]))) - (data["av_change_abs_roll_mean_100"]))))))) + (activation((((((((((data["av_change_rate_roll_mean_100"]) - (data["av_change_abs_roll_std_100"]))) - (data["av_change_abs_roll_std_100"]))) - (data["std_first_50000"]))) - (data["av_change_abs_roll_std_100"]))))))/2.0)) +
            0.0374213718*activation((((-1.0*((((data["q05"]) * (((data["max_last_10000"]) * (((((data["min_roll_std_10"]) - (data["std_last_10000"]))) - (data["std_last_10000"])))))))))) + (((data["max_last_50000"]) * (((data["mean_change_rate_last_10000"]) - (activation((((((data["av_change_rate_roll_std_10"]) * 2.0)) * 2.0)))))))))) +
            0.0331470966*activation(((((((((data["abs_max_roll_std_1000"]) * 2.0)) * 2.0)) * 2.0)) * (((((data["mean_change_rate_last_50000"]) * ((((((data["abs_max_roll_std_1000"]) * (((data["mean_change_abs"]) * 2.0)))) + ((((data["std_first_50000"]) + (data["mean_change_rate_last_50000"]))/2.0)))/2.0)))) + (((data["mean_change_abs"]) * (data["av_change_rate_roll_std_1000"]))))))) +
            0.0396796241*activation(((activation((activation((((((data["avg_first_10000"]) * (((((data["Moving_average_3000_mean"]) + (data["skew"]))) * 2.0)))) + (activation(((-1.0*((((((data["ave_roll_mean_100"]) + (((data["av_change_abs_roll_std_1000"]) + (((data["avg_first_10000"]) * 2.0)))))) * 2.0))))))))))))) / 2.0)) +
            0.0399765596*activation(((activation((((((data["av_change_abs_roll_mean_10"]) + (data["exp_Moving_average_300_mean"]))) * (((((data["classic_sta_lta4_mean"]) + (data["av_change_abs_roll_mean_10"]))) + (((((data["av_change_abs_roll_mean_10"]) + (((activation((((activation((data["avg_last_50000"]))) * 2.0)))) * 2.0)))) * (data["av_change_abs_roll_mean_10"]))))))))) / 2.0)) +
            0.0396014862*activation((((((data["max_last_50000"]) + (((((activation((activation((((((activation((activation(((((data["q95"]) + (data["mean_change_abs"]))/2.0)))))) + (((data["av_change_abs_roll_std_100"]) * 2.0)))) * 2.0)))))) * 2.0)) - (data["av_change_abs_roll_std_100"]))))/2.0)) * (data["classic_sta_lta4_mean"]))) +
            0.0317015015*activation(((activation((((((data["avg_last_10000"]) - (((data["iqr"]) - (((((data["avg_last_10000"]) - (((data["iqr"]) * (data["av_change_rate_roll_std_100"]))))) - (((((((((data["min_roll_std_100"]) * (data["av_change_rate_roll_std_100"]))) * 2.0)) * 2.0)) * 2.0)))))))) * 2.0)))) / 2.0)) +
            0.0399296731*activation((-1.0*((((data["q05_roll_std_1000"]) * (((((((((data["avg_first_10000"]) + (data["mean_change_abs"]))/2.0)) * (data["std_last_50000"]))) + (((((((data["avg_last_10000"]) + (((data["mean_change_rate_first_10000"]) * (data["mean_change_abs"]))))/2.0)) + (data["max_to_min_diff"]))/2.0)))/2.0))))))) +
            0.0377104878*activation((((activation((activation((((((((data["count_big"]) + (((data["min_last_10000"]) * (data["min_first_50000"]))))) * 2.0)) * 2.0)))))) + ((((data["count_big"]) + ((-1.0*((((data["max_roll_mean_100"]) - (activation((((data["min_last_10000"]) * (data["avg_last_10000"])))))))))))/2.0)))/2.0)) +
            0.0309357308*activation(((activation((((((((data["max_to_min"]) * (data["classic_sta_lta2_mean"]))) + (((data["classic_sta_lta2_mean"]) * (((data["max_to_min"]) * (data["classic_sta_lta2_mean"]))))))) * 2.0)))) / 2.0)) +
            0.0355303772*activation(((((activation((activation((((((((((data["av_change_abs_roll_std_10"]) - (activation((((data["skew"]) * 2.0)))))) * (data["mean_change_rate_first_10000"]))) * (data["mean_change_rate_first_10000"]))) * (((data["mean_change_rate_first_10000"]) + (data["av_change_abs_roll_std_10"]))))))))) * (data["mean_change_rate_first_10000"]))) * (data["mean_change_rate_first_10000"]))) +
            0.0399843715*activation(((data["av_change_abs_roll_std_1000"]) * (((((((data["trend"]) + (activation((((data["min_roll_std_1000"]) * ((((8.0)) + (((data["min_roll_std_1000"]) * ((8.0)))))))))))/2.0)) + ((-1.0*((((data["q01_roll_std_1000"]) + (((data["q01_roll_std_1000"]) * (data["mean_change_rate_first_50000"])))))))))/2.0)))) +
            0.0398437195*activation((-1.0*((((data["std_first_10000"]) * ((((((data["q99"]) / 2.0)) + (((data["skew"]) + (((((data["std_first_10000"]) * 2.0)) * (((data["std_first_10000"]) * (((data["std_first_10000"]) * (((data["std_first_10000"]) - (((data["MA_700MA_std_mean"]) * 2.0)))))))))))))/2.0))))))) +
            0.0399843715*activation((-1.0*((((data["mean_change_abs"]) * ((((((data["av_change_abs_roll_mean_1000"]) + ((((((((data["abs_max_roll_mean_10"]) + (((((data["min_roll_mean_100"]) * (data["av_change_abs_roll_mean_1000"]))) * (data["min_roll_std_10"]))))) - ((((data["min_roll_std_10"]) + (data["av_change_abs_roll_std_100"]))/2.0)))) + (data["skew"]))/2.0)))/2.0)) / 2.0))))))) +
            0.0295057632*activation(((((((((((((((data["q99_roll_mean_100"]) + (((data["q99_roll_mean_100"]) + (data["q95_roll_mean_10"]))))) * (((data["q05_roll_std_100"]) + ((((((data["av_change_abs_roll_std_1000"]) - (data["min_roll_std_100"]))) + (data["av_change_abs_roll_mean_100"]))/2.0)))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * (data["av_change_abs_roll_std_1000"]))) +
            0.0346864611*activation(((activation((((((((data["mean_change_rate"]) * (((data["mean_change_rate_first_10000"]) + (((data["mean_change_rate_first_10000"]) + (data["mean_change_rate_last_50000"]))))))) + (((data["q01_roll_std_100"]) * (((((data["mean_change_rate_first_10000"]) * (((data["q99_roll_std_10"]) * 2.0)))) + (data["mean_change_rate_first_10000"]))))))) * 2.0)))) / 2.0)) +
            0.0397265106*activation(activation((((((data["min_roll_std_100"]) * ((-1.0*((((data["av_change_abs_roll_mean_100"]) * ((((data["q05_roll_mean_100"]) + (data["mean_change_rate_first_10000"]))/2.0))))))))) - (((((((-1.0*((data["classic_sta_lta4_mean"])))) / 2.0)) + ((((data["av_change_rate_roll_mean_100"]) + ((((data["Moving_average_6000_mean"]) + (data["mean_change_rate_first_10000"]))/2.0)))/2.0)))/2.0)))))) +
            0.0399687439*activation(((activation((((data["min_first_50000"]) * (((((((((activation((((((data["q01_roll_std_1000"]) + (activation(((-1.0*((((((data["skew"]) + (data["classic_sta_lta2_mean"]))) + (data["q95_roll_std_10"])))))))))) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)))))) / 2.0)) +
            0.0365696438*activation(((activation((((((((((((((((data["mean_change_rate_last_10000"]) * (((data["ave_roll_std_10"]) - (((data["q95"]) - ((((((data["abs_mean"]) + (((data["iqr"]) / 2.0)))/2.0)) / 2.0)))))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) / 2.0)) +
            0.0393358096*activation(((((data["max_roll_std_10"]) * (((((data["q95_roll_mean_100"]) + (((data["q95_roll_mean_100"]) + (((data["q05_roll_mean_1000"]) * (data["classic_sta_lta2_mean"]))))))) * (((data["max_roll_std_10"]) * (((((data["skew"]) * (((data["q05_roll_mean_100"]) + (data["classic_sta_lta2_mean"]))))) * 2.0)))))))) * 2.0)) +
            0.0381871462*activation((((activation((((((((data["q05_roll_mean_10"]) - (((data["q05"]) + (data["ave_roll_std_100"]))))) * (73.0))) * (73.0))))) + (((((((data["abs_q05"]) * (((data["std_roll_mean_1000"]) * (data["av_change_rate_roll_std_10"]))))) * 2.0)) - (data["q05"]))))/2.0)) +
            0.0383043550*activation(((data["max_last_50000"]) * (((((data["av_change_rate_roll_std_100"]) * 2.0)) * (activation((((((((((((((activation((((data["mean_change_abs"]) - (data["classic_sta_lta3_mean"]))))) - (data["classic_sta_lta3_mean"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))))))) +
            0.0397812054*activation(activation((activation((((data["min_roll_mean_100"]) * (((((data["av_change_rate_roll_mean_1000"]) - (data["min_roll_mean_100"]))) - (((((((data["min_last_50000"]) * (((data["av_change_rate_roll_mean_1000"]) - (activation((((data["min_roll_std_10"]) - (((data["min_roll_mean_100"]) * 2.0)))))))))) * 2.0)) * 2.0)))))))))) +
            0.0399921872*activation((-1.0*((((((activation((((((((((((((((((data["mean_change_abs"]) * 2.0)) - ((((((-1.0*((data["mean_change_rate_last_50000"])))) * 2.0)) - (data["trend"]))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) / 2.0)) / 2.0))))) +
            0.0329673775*activation(activation(((-1.0*((((data["std_first_10000"]) * (((((((((data["av_change_abs_roll_mean_10"]) + (data["max_last_10000"]))) * (((((data["max_last_10000"]) * (0.8873239756))) + (data["classic_sta_lta2_mean"]))))) * 2.0)) * (data["max_last_10000"])))))))))) +
            0.0367962494*activation(((data["avg_last_10000"]) * (((((data["std_roll_mean_1000"]) * ((-1.0*(((((((((data["mean_change_rate_first_50000"]) - (((data["mean_change_abs"]) * (data["max_last_10000"]))))) + (data["max_to_min_diff"]))/2.0)) - (((data["max_last_10000"]) * (((data["max_to_min"]) * (data["q05_roll_mean_100"])))))))))))) * 2.0)))) +
            0.0399921872*activation(((data["av_change_abs_roll_mean_10"]) * (((((data["av_change_abs_roll_std_1000"]) - ((((data["av_change_abs_roll_mean_10"]) + ((((data["min_roll_std_10"]) + (((((((data["av_change_rate_roll_mean_10"]) * (data["av_change_abs_roll_std_10"]))) + (data["av_change_abs_roll_mean_10"]))) * (data["av_change_abs_roll_mean_10"]))))/2.0)))/2.0)))) * (data["min_roll_std_10"]))))) +
            0.0395311601*activation(((activation(((-1.0*((((data["std_first_10000"]) + (data["min_first_10000"])))))))) + (activation((((data["max_roll_std_10"]) * (((data["q95_roll_mean_100"]) * ((((((((data["mean_change_abs"]) + (data["max_roll_std_10"]))/2.0)) - ((-1.0*((data["min_first_10000"])))))) + (data["mean_change_rate_last_50000"]))))))))))) +
            0.0399296731*activation(((((data["min_roll_std_10"]) + (((data["min_roll_std_10"]) * (data["med"]))))) * ((((((data["classic_sta_lta1_mean"]) + (data["min_roll_mean_1000"]))/2.0)) - (((data["q05_roll_std_10"]) * ((((((data["med"]) * ((((data["classic_sta_lta1_mean"]) + (data["min_roll_mean_1000"]))/2.0)))) + (data["med"]))/2.0)))))))) +
            0.0399453007*activation(activation((((((((activation((data["min_roll_std_10"]))) * (data["std_roll_mean_1000"]))) * 2.0)) + ((((((((((((data["max_to_min_diff"]) * 2.0)) * (data["std_last_50000"]))) + (data["min_roll_std_10"]))/2.0)) / 2.0)) + (((data["min_roll_std_10"]) * (activation((data["max_to_min_diff"]))))))))))) +
            0.0394999012*activation(((data["mean_change_rate_first_10000"]) * (((data["min_first_10000"]) * ((((((((data["min_roll_std_10"]) * (((data["av_change_abs_roll_std_100"]) + (data["av_change_abs_roll_std_100"]))))) + ((((data["mean_change_rate_first_10000"]) + (((data["min_roll_std_10"]) * (data["mean_change_rate_first_10000"]))))/2.0)))/2.0)) + (((data["min_roll_std_10"]) - (data["av_change_abs_roll_std_100"]))))))))) +
            0.0399687439*activation(((data["max_first_10000"]) * (((((data["std_first_10000"]) * (((data["max_first_10000"]) * (((data["std_first_10000"]) * (((activation((((data["std_first_10000"]) * (data["av_change_abs_roll_mean_10"]))))) + (data["std_roll_mean_100"]))))))))) - (((((data["av_change_abs_roll_mean_10"]) * (data["min_roll_std_100"]))) / 2.0)))))) +
            0.0397499502*activation(((data["trend"]) * ((((((data["std_first_50000"]) + (activation((activation((((data["av_change_rate_roll_mean_10"]) + (((((((data["av_change_rate_roll_mean_1000"]) + ((((data["min_roll_std_100"]) + (((data["Hilbert_mean"]) + (data["av_change_rate_roll_mean_10"]))))/2.0)))) * 2.0)) * 2.0)))))))))/2.0)) * (data["min_roll_std_100"]))))) +
            0.0274897423*activation((((activation((((((((((((((data["q95_roll_std_100"]) - (data["ave_roll_std_10"]))) - (data["abs_max_roll_mean_10"]))) - (data["max_last_50000"]))) - (((data["min_first_50000"]) + (data["ave_roll_std_10"]))))) * 2.0)) * 2.0)))) + (((((data["q95_roll_std_100"]) - (data["ave_roll_std_10"]))) * 2.0)))/2.0)) +
            0.0399531126*activation(((((data["mean_change_rate_first_10000"]) * (activation((((((((((data["av_change_abs_roll_mean_10"]) + (data["mean_change_rate_last_10000"]))/2.0)) * (((data["avg_first_10000"]) - (activation((data["av_change_abs_roll_mean_100"]))))))) + (activation((activation((data["avg_first_10000"]))))))/2.0)))))) * ((((-1.0*((data["mean_change_abs"])))) * 2.0)))) +
            0.0340613388*activation(((data["skew"]) * (((data["max_first_50000"]) * (((((((((activation((((((((data["iqr"]) * 2.0)) - (data["av_change_abs_roll_mean_10"]))) - (data["std_last_10000"]))))) + (data["av_change_abs_roll_mean_10"]))) - (((data["av_change_rate_roll_mean_10"]) * (data["avg_first_50000"]))))) * 2.0)) * 2.0)))))) +
            0.0399921872*activation((((((data["min_roll_std_10"]) + (((data["min_roll_std_10"]) * (data["avg_first_10000"]))))/2.0)) * (activation((((73.0) * (((data["avg_first_10000"]) * (((data["min_roll_std_10"]) + (((73.0) * (((73.0) * ((-1.0*((data["av_change_abs_roll_std_1000"])))))))))))))))))) +
            0.0356475860*activation(activation((((data["av_change_abs_roll_mean_1000"]) * ((((((((((((((data["avg_last_50000"]) + (data["kurt"]))) + (0.2783510089))/2.0)) + (data["av_change_rate_roll_std_1000"]))) + (0.2783510089))) + (data["kurt"]))) * ((((activation((data["avg_first_10000"]))) + (data["mean_change_rate_first_50000"]))/2.0)))))))) +
            0.0329986326*activation((((((((((data["kurt"]) + (data["q05_roll_mean_10"]))) * (data["min_roll_mean_10"]))) + (activation((((((data["q05_roll_mean_10"]) - (((((data["max_to_min"]) - (data["kurt"]))) + (((data["q05_roll_mean_10"]) * (data["max_to_min"]))))))) * 2.0)))))/2.0)) / 2.0)) +
            0.0399999991*activation(((data["std_roll_mean_1000"]) * (((data["med"]) * ((((-3.0) + (((((data["avg_first_10000"]) * 2.0)) * (((data["mean_change_rate_first_10000"]) - (((((((data["std_roll_mean_1000"]) / 2.0)) * (((data["std_roll_mean_1000"]) * (-3.0))))) * (data["abs_max_roll_mean_1000"]))))))))/2.0)))))) +
            0.0303496774*activation((-1.0*((((activation((activation((((((((data["classic_sta_lta2_mean"]) + (((data["av_change_abs_roll_std_10"]) + (((data["av_change_abs_roll_mean_10"]) + (data["classic_sta_lta1_mean"]))))))) * (data["abs_q05"]))) * (3.6923100948))))))) / 2.0))))) +
            0.0399687439*activation((((-1.0*((data["max_first_50000"])))) * (((((data["std_roll_mean_1000"]) + (((data["av_change_abs_roll_std_10"]) * (((data["ave_roll_mean_10"]) + (((data["med"]) + (1.0))))))))) * (((data["avg_last_50000"]) + (((data["med"]) + (2.0769200325))))))))) +
            0.0399843715*activation(((activation((((data["av_change_rate_roll_mean_100"]) * (((activation((((((((((data["av_change_abs_roll_mean_100"]) * 2.0)) + (data["mean_change_rate_first_50000"]))) * 2.0)) * 2.0)))) - (((data["av_change_abs_roll_mean_100"]) * (((((data["av_change_rate_roll_std_1000"]) - (data["classic_sta_lta2_mean"]))) - (data["classic_sta_lta2_mean"]))))))))))) / 2.0)) +
            0.0399999991*activation(((data["mean_change_abs"]) * ((((data["av_change_abs_roll_std_100"]) + (activation((((((((((((((((((data["min_first_10000"]) - (data["av_change_rate_roll_std_10"]))) - (((data["av_change_abs_roll_std_100"]) * (3.6923100948))))) * 2.0)) * 2.0)) * 2.0)) + (data["mean_change_abs"]))) * 2.0)) * 2.0)))))/2.0)))) +
            0.0285446383*activation(((activation((((((73.0) * (data["mean_change_rate_first_10000"]))) + (((((((((data["trend"]) + (((data["av_change_abs_roll_mean_10"]) + (data["min_last_50000"]))))) + (data["mean_change_rate_first_10000"]))) * 2.0)) * 2.0)))))) + ((-1.0*((activation((data["mean_change_rate_first_10000"])))))))) +
            0.0399687439*activation(((((activation((((((((((((((((((((activation(((((((data["kurt"]) + (data["classic_sta_lta4_mean"]))/2.0)) - (data["max_first_10000"]))))) + (data["skew"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) / 2.0)) / 2.0)) +
            0.0398593470*activation(((activation(((-1.0*((activation((((data["avg_first_10000"]) * (((data["min_roll_std_10"]) - (((((data["mean_change_rate_last_10000"]) * (data["min_roll_std_10"]))) + ((((data["mean_change_abs"]) + ((((((data["ave_roll_mean_100"]) + (data["avg_first_10000"]))/2.0)) + (data["skew"]))))/2.0))))))))))))))) / 2.0)) +
            0.0399296731*activation(((activation((activation((activation((activation((((((((data["mean_change_abs"]) * ((((data["av_change_abs_roll_mean_100"]) + (0.7446810007))/2.0)))) - (((activation((data["skew"]))) + ((((data["trend"]) + (0.7446810007))/2.0)))))) * 2.0)))))))))) / 2.0)) +
            0.0387341268*activation((((((((data["min_roll_std_100"]) * (activation((((((((data["av_change_rate_roll_std_100"]) * 2.0)) + (((data["abs_q05"]) + (activation((data["iqr"]))))))) * 2.0)))))) + (activation((((activation((((data["iqr"]) * 2.0)))) - (data["min_roll_std_10"]))))))/2.0)) / 2.0)) +
            0.0384997055*activation(((data["av_change_abs_roll_mean_1000"]) * (((activation((((((activation((data["std_first_50000"]))) + ((((-1.0*((data["max_to_min"])))) * (data["std_first_10000"]))))) + (((data["av_change_abs_roll_mean_1000"]) * (data["min_roll_mean_1000"]))))))) + ((((-1.0*((data["min_roll_std_10"])))) * (data["std_first_10000"]))))))) +
            0.0384918936*activation(((activation((activation((((((73.0) * (((73.0) * (((((data["q95_roll_std_100"]) + (data["std_roll_mean_1000"]))) * 2.0)))))) * (((data["std_first_50000"]) + (((activation((activation((data["max_roll_std_1000"]))))) * (data["kurt"]))))))))))) / 2.0)) +
            0.0343192033*activation(((((-1.0) - (activation((((((((0.8873239756) + (data["mean_change_abs"]))) - (data["Hann_window_mean"]))) * (((((((((data["Hann_window_mean"]) - (data["min_last_10000"]))) * 2.0)) * 2.0)) + (data["av_change_rate_roll_std_100"]))))))))) / 2.0)) +
            0.0372416489*activation(((activation((((((data["classic_sta_lta2_mean"]) + (((data["q95_roll_mean_1000"]) * (data["classic_sta_lta2_mean"]))))) + ((((-1.0*((((((data["std_last_10000"]) + (data["MA_400MA_BB_low_mean"]))) + (activation((((data["min_roll_std_100"]) + (data["classic_sta_lta2_mean"])))))))))) * (data["avg_last_10000"]))))))) / 2.0)) +
            0.0377808139*activation(activation((((((data["mean_change_rate_last_10000"]) * (((((((data["mean_change_rate_first_10000"]) + ((((data["min_roll_std_10"]) + ((-1.0*((-3.0)))))/2.0)))/2.0)) + (data["Moving_average_1500_mean"]))/2.0)))) * (activation((((((data["mean_change_rate_last_10000"]) + (((data["av_change_abs_roll_std_1000"]) + (1.9019600153))))) * 2.0)))))))) +
            0.0369056463*activation(activation((activation((((((data["max_first_50000"]) * (data["max_first_50000"]))) * ((((((((data["max_first_50000"]) + (data["kurt"]))/2.0)) - (data["avg_last_50000"]))) - (((activation((((data["mean_change_rate_first_10000"]) * 2.0)))) - (((data["kurt"]) - (0.5555559993))))))))))))))
def GPI_activation1(data):
    return (5.591662 +
            0.0399999991*activation1(((data["mean_change_rate_first_10000"]) - (((73.0) * ((((((7.0) * (((((((((data["q95"]) + (((0.3183098733) + (data["q05_roll_std_100"]))))) * 2.0)) * 2.0)) + (((data["q95_roll_std_10"]) + (data["iqr"]))))))) + (data["iqr"]))/2.0)))))) +
            0.0399999991*activation1(((((3.6923100948) + (data["Moving_average_6000_mean"]))) * (((73.0) * (((data["MA_400MA_BB_low_mean"]) - (((((data["Moving_average_6000_mean"]) * (data["q05_roll_std_10"]))) + (((data["iqr"]) + (((data["q05_roll_std_10"]) * (((3.6923100948) * 2.0)))))))))))))) +
            0.0399999991*activation1(((((((((((((((((((((((data["q05"]) * 2.0)) * 2.0)) - (data["iqr"]))) - (data["max_roll_std_1000"]))) * 2.0)) * 2.0)) * 2.0)) - (data["iqr"]))) - (3.6923100948))) - (((data["q05_roll_std_10"]) * (73.0))))) * 2.0)) +
            0.0399999991*activation1(((((((((((((((((((data["q05_roll_std_10"]) * (((((-3.0) * 2.0)) * 2.0)))) - (((data["iqr"]) + (data["max_roll_std_1000"]))))) * 2.0)) * 2.0)) * 2.0)) - (data["max_roll_mean_100"]))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*activation1(((73.0) * ((((data["max_first_50000"]) + ((((data["max_first_50000"]) + (((73.0) * (((data["q05"]) - (((data["q05_roll_std_100"]) + (((0.5555559993) + (((data["mad"]) + (data["q95"]))))))))))))/2.0)))/2.0)))) +
            0.0399999991*activation1(((((((data["q05_roll_std_10"]) * (73.0))) * 2.0)) - ((((((((data["q05_roll_std_10"]) * (73.0))) + (((((((((data["iqr"]) + (((data["MA_1000MA_std_mean"]) + (data["abs_max_roll_mean_10"]))))) * 2.0)) * 2.0)) * 2.0)))/2.0)) * (73.0))))) +
            0.0399999991*activation1(((((data["mean_change_abs"]) - (((((data["q95"]) + (((data["q95"]) + (((((data["q05_roll_std_100"]) / 2.0)) + (((((((0.3183098733) * 2.0)) - (data["q05"]))) - (data["q05"]))))))))) * (73.0))))) * 2.0)) +
            0.0399999991*activation1((-1.0*(((((((7.48210620880126953)) * 2.0)) + (((((data["iqr"]) + ((((-1.0*((data["avg_first_10000"])))) * 2.0)))) + (((73.0) * (((data["iqr"]) + (((((data["q05_roll_std_10"]) * 2.0)) * 2.0))))))))))))) +
            0.0399999991*activation1((((((data["q01_roll_std_10"]) + (((((data["q05_roll_mean_10"]) - (((((data["q05_roll_std_100"]) - (((data["q05"]) - (0.3026320040))))) + (((((((0.5434780121) + (0.3026320040))/2.0)) + (data["iqr"]))/2.0)))))) * (73.0))))/2.0)) * (73.0))) +
            0.0399999991*activation1(((((data["trend"]) + (((data["q95_roll_std_10"]) + (data["trend"]))))) - (((73.0) * (((((((data["ave_roll_std_10"]) + (((((((data["q95_roll_mean_10"]) + (data["q05_roll_std_1000"]))/2.0)) + (data["q05_roll_std_1000"]))/2.0)))/2.0)) + ((((0.5434780121) + (data["q95_roll_mean_10"]))/2.0)))/2.0)))))) +
            0.0399999991*activation1(((73.0) - (((((73.0) * 2.0)) * (((73.0) * ((((((((((data["q05_roll_std_1000"]) / 2.0)) + (data["iqr"]))/2.0)) + (data["q05_roll_std_10"]))) + (((0.2183910012) + (data["q95"]))))))))))) +
            0.0399999991*activation1(((73.0) * (((73.0) * (((((73.0) * (((data["q05_roll_mean_10"]) - (((((data["q05_roll_std_100"]) + (data["q95"]))) + (0.5434780121))))))) + (data["q05_roll_std_100"]))))))) +
            0.0399999991*activation1((-1.0*((((73.0) * (((73.0) * (((73.0) * (((data["q05_roll_std_100"]) + ((((data["q95_roll_mean_10"]) + ((((data["Moving_average_3000_mean"]) + (((((data["iqr"]) + (0.9756100178))) + (data["abs_mean"]))))/2.0)))/2.0))))))))))))) +
            0.0399999991*activation1(((73.0) * (((((data["classic_sta_lta3_mean"]) + (data["mad"]))) + (((((((73.0) * (((data["q05_roll_mean_10"]) - (((data["q05_roll_std_100"]) + (((data["mad"]) + (0.5555559993))))))))) - (data["iqr"]))) - (data["iqr"]))))))) +
            0.0399999991*activation1(((((activation1((data["q95_roll_std_10"]))) + (((((((data["q05_roll_std_10"]) + ((((((0.2439019978) + (data["ave10"]))/2.0)) / 2.0)))) * 2.0)) * 2.0)))) * (((data["q05_roll_std_10"]) - (((73.0) * (((73.0) * (73.0))))))))) +
            0.0399999991*activation1(((3.6923100948) * (((73.0) * (((3.6923100948) * ((((((-1.0*((data["q99_roll_mean_1000"])))) * (data["avg_last_50000"]))) - (((data["q05_roll_std_10"]) * (((3.6923100948) + (((3.6923100948) + (data["abs_max_roll_mean_100"]))))))))))))))) +
            0.0399999991*activation1(((((((data["Moving_average_1500_mean"]) + (((((((((data["MA_400MA_BB_low_mean"]) + ((((-1.0*((data["q05_roll_std_1000"])))) - (data["iqr"]))))) * (73.0))) - (data["min_roll_std_1000"]))) - (data["q05_roll_std_1000"]))))) * (73.0))) * (73.0))) +
            0.0399999991*activation1(((73.0) - (((73.0) * (((73.0) * (((data["q05_roll_std_10"]) + (activation1(((((((data["exp_Moving_average_3000_mean"]) + (((data["iqr"]) + (activation1((((data["q05_roll_std_100"]) + (data["exp_Moving_average_30000_mean"]))))))))/2.0)) / 2.0)))))))))))) +
            0.0399999991*activation1(((73.0) * (((73.0) * (((data["q05_roll_mean_10"]) - (((activation1((activation1((activation1((activation1((activation1((data["q05_roll_std_100"]))))))))))) + ((((data["q95_roll_std_10"]) + (activation1((2.8437500000))))/2.0)))))))))) +
            0.0399999991*activation1(((((((data["iqr"]) / 2.0)) + (((data["mean"]) + (((data["q05_roll_std_100"]) + (((data["q05_roll_std_10"]) * 2.0)))))))) * ((-1.0*((((73.0) - (((((((((data["mean"]) * 2.0)) * 2.0)) * 2.0)) * 2.0))))))))) +
            0.0399999991*activation1(((((((((0.5217390060) - (((((data["q05_roll_std_10"]) * ((-1.0*((7.0)))))) + (data["exp_Moving_average_3000_mean"]))))) * ((-1.0*((73.0)))))) - (((data["classic_sta_lta4_mean"]) * (7.0))))) - ((((7.0) + (data["classic_sta_lta4_mean"]))/2.0)))) +
            0.0399999991*activation1(((((((((((((((((((activation1(((-1.0*((((data["q05_roll_mean_100"]) + (0.2183910012)))))))) - ((((data["q05_roll_std_100"]) + (((data["q05_roll_std_10"]) * 2.0)))/2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) - (data["exp_Moving_average_30000_mean"]))) * 2.0)) +
            0.0399999991*activation1(((data["trend"]) - (((73.0) * ((((3.6923100948) + ((((9.0)) * (((activation1((activation1((activation1((activation1((activation1((activation1((activation1((data["q05_roll_std_100"]))))))))))))))) - (data["q05_roll_mean_10"]))))))/2.0)))))) +
            0.0399999991*activation1(((data["q05_roll_std_1000"]) - (((73.0) * (((data["avg_last_50000"]) + (((((73.0) * 2.0)) * (((data["ave_roll_std_100"]) + (((data["Moving_average_6000_mean"]) + (((data["iqr"]) + (((data["q05_roll_std_1000"]) + (((data["q95"]) * 2.0)))))))))))))))))) +
            0.0399999991*activation1(((((data["q05_roll_mean_10"]) - (2.0769200325))) - ((((((6.08570814132690430)) * (((2.0769200325) + (((((((((data["q05_roll_std_10"]) - (data["q05_roll_mean_10"]))) * 2.0)) * 2.0)) * 2.0)))))) + (((((data["av_change_rate_roll_std_100"]) - (data["avg_last_10000"]))) * 2.0)))))) +
            0.0399999991*activation1((-1.0*((((((((((((((((((((((((((((((((data["q05_roll_std_1000"]) + (data["q05_roll_mean_100"]))/2.0)) * 2.0)) + (data["iqr"]))/2.0)) + (data["q95_roll_mean_10"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0))))) +
            0.0399999991*activation1((((-1.0*((((((((((data["q05_roll_std_100"]) - (activation1(((-1.0*((((data["MA_400MA_std_mean"]) + (((data["exp_Moving_average_300_mean"]) * (((data["q05_roll_std_100"]) * 2.0))))))))))))) * 2.0)) * 2.0)) + (activation1((data["q05_roll_std_100"])))))))) * (73.0))) +
            0.0399999991*activation1(((((data["abs_q05"]) - ((((((2.0769200325) * ((((((73.0) * ((((((0.5555559993) + (((data["q05_roll_std_100"]) + (0.7446810007))))/2.0)) + (data["q05_roll_std_100"]))))) + (0.5555559993))/2.0)))) + (0.5555559993))/2.0)))) * 2.0)) +
            0.0399999991*activation1((((10.75933361053466797)) - (((7.0) * (((7.0) * (((7.0) * ((((((data["q95_roll_mean_10"]) + (data["q05_roll_std_100"]))) + ((((data["q95_roll_mean_10"]) + (((data["iqr"]) + (((data["Moving_average_700_mean"]) * 2.0)))))/2.0)))/2.0)))))))))) +
            0.0399999991*activation1(((73.0) * (((((data["q05_roll_std_1000"]) + ((((data["q05_roll_std_10"]) + ((((((data["q05_roll_std_1000"]) + (activation1((data["q05_roll_std_1000"]))))/2.0)) - (((data["q95_roll_mean_1000"]) - (activation1((73.0))))))))/2.0)))) * (((2.8437500000) - (73.0))))))) +
            0.0399999991*activation1(((((((((-3.0) + (data["Moving_average_3000_mean"]))) - (((73.0) * (((data["ave_roll_mean_10"]) - ((((((data["q05_roll_std_100"]) + (data["q95_roll_mean_10"]))/2.0)) * (((-3.0) - (((data["Moving_average_700_mean"]) + (data["q95_roll_mean_10"]))))))))))))) * 2.0)) * 2.0)) +
            0.0399999991*activation1(((((((data["classic_sta_lta3_mean"]) * 2.0)) + (data["q05_roll_mean_100"]))) - (((73.0) * (((((((0.6582279801) + (data["q05_roll_std_10"]))) - (((data["q05_roll_mean_10"]) * 2.0)))) + ((((-1.0*((data["abs_q05"])))) + (((data["q05_roll_std_100"]) * 2.0)))))))))) +
            0.0399999991*activation1(((((data["skew"]) + (((data["kurt"]) + (data["med"]))))) - ((((7.02540111541748047)) * (((((data["iqr"]) + (((data["q95_roll_std_10"]) + (((((data["q95_roll_mean_10"]) + (((data["iqr"]) * (data["Moving_average_1500_mean"]))))) * 2.0)))))) * 2.0)))))) +
            0.0399999991*activation1(((((data["q05_roll_std_100"]) - (((-2.0) + (((((data["iqr"]) * (-2.0))) - ((-1.0*((data["av_change_rate_roll_std_10"])))))))))) - ((((((data["q05_roll_std_100"]) + (activation1((((data["q05_roll_std_100"]) * (data["ave_roll_mean_10"]))))))/2.0)) * (73.0))))) +
            0.0399999991*activation1(((((((((((data["q05_roll_std_10"]) + (((data["av_change_abs_roll_std_1000"]) - (((data["av_change_abs_roll_mean_10"]) * 2.0)))))) * 2.0)) + (data["av_change_abs_roll_std_1000"]))) * 2.0)) - (((73.0) * (((((data["abs_max_roll_mean_1000"]) + (((0.5555559993) + (data["q05_roll_std_10"]))))) * 2.0)))))) +
            0.0399999991*activation1((((((((((((((((((activation1((((((data["q95_roll_mean_10"]) + (data["q95_roll_std_100"]))) - (data["q05_roll_std_1000"]))))) + (data["q95_roll_std_100"]))/2.0)) + (data["q95_roll_std_100"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) - (((73.0) * (data["q95_roll_mean_10"]))))) * 2.0)) +
            0.0399999991*activation1(((73.0) - (((((((((((data["q05_roll_std_10"]) * 2.0)) - (data["ave10"]))) + (((data["q05_roll_std_100"]) + (0.9756100178))))) * (73.0))) * (((73.0) * 2.0)))))) +
            0.0399999991*activation1(((((data["Moving_average_6000_mean"]) + (((7.0) + (((data["ave10"]) * 2.0)))))) * (((((((((data["MA_700MA_BB_low_mean"]) * (data["avg_last_50000"]))) + ((-1.0*((((data["q95_roll_mean_10"]) * 2.0))))))) - (((data["ave_roll_mean_10"]) + (data["q05_roll_std_100"]))))) * 2.0)))) +
            0.0399999991*activation1(((73.0) * (((data["q05_roll_std_100"]) - (((((((data["iqr"]) - (activation1((((73.0) * (((data["ave10"]) - (((data["iqr"]) + (((((data["q05_roll_std_1000"]) + (0.5434780121))) * 2.0)))))))))))) * 2.0)) * 2.0)))))) +
            0.0399999991*activation1(((((((((((((((data["exp_Moving_average_30000_mean"]) * (((data["exp_Moving_average_30000_mean"]) + ((-1.0*((data["q01_roll_std_10"])))))))) - (((data["q05_roll_std_10"]) + (((data["q05_roll_std_10"]) + (((data["q95_roll_mean_100"]) + (data["q05_roll_std_10"]))))))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*activation1(((73.0) - (((((data["q05_roll_std_100"]) * (73.0))) * (((73.0) * (((data["q05_roll_std_100"]) * (((data["q05_roll_std_100"]) - (((data["q05"]) - ((((0.2525250018) + (2.0))/2.0)))))))))))))) +
            0.0399999991*activation1(((((((((7.0) - ((((data["sum"]) + (((73.0) * (((((data["sum"]) + (data["q05_roll_std_10"]))) + (((((((activation1((data["abs_q05"]))) + (data["q05_roll_std_10"]))) * 2.0)) * 2.0)))))))/2.0)))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*activation1((((((1.0)) - (((((((((data["q05_roll_std_100"]) + (activation1((((data["abs_q05"]) * (((((data["exp_Moving_average_30000_mean"]) - (data["abs_q05"]))) + (((data["q05_roll_std_100"]) + (((data["q95_roll_mean_10"]) * 2.0)))))))))))) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) +
            0.0399999991*activation1(((((activation1((((activation1((data["Moving_average_1500_mean"]))) * 2.0)))) - (((data["Moving_average_1500_mean"]) + ((-1.0*((((((data["q05_roll_std_100"]) + (activation1((0.6582279801))))) * 2.0))))))))) * ((((-1.0*((((data["Moving_average_1500_mean"]) * 2.0))))) - (73.0))))) +
            0.0399999991*activation1(((73.0) - ((((((((((((data["q05_roll_std_100"]) + (((data["q05_roll_std_100"]) * (data["Moving_average_6000_mean"]))))/2.0)) + (((data["Moving_average_6000_mean"]) + (((data["q05_roll_std_100"]) + (data["q95"]))))))) * 2.0)) * 2.0)) * (((73.0) + (data["Moving_average_6000_mean"]))))))) +
            0.0399999991*activation1(((((((((((((data["mean_change_rate_last_10000"]) - (data["mean_change_abs"]))) - (((((((((((((data["q05_roll_std_10"]) + (((0.5434780121) - (data["q05"]))))) * 2.0)) - (data["Hann_window_mean"]))) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*activation1(((((((-1.0*((((data["avg_last_50000"]) * (3.6923100948)))))) + (((73.0) / 2.0)))/2.0)) - (((73.0) * ((((((data["Moving_average_6000_mean"]) + (((data["q05_roll_std_10"]) - (((data["exp_Moving_average_30000_mean"]) * (data["exp_Moving_average_30000_mean"]))))))/2.0)) + (data["q05_roll_std_10"]))))))) +
            0.0399999991*activation1(((73.0) * ((-1.0*((((data["q01_roll_std_1000"]) + (((73.0) * (((data["q05_roll_std_100"]) + ((((1.4411799908) + ((((((data["q01_roll_std_1000"]) - (data["q99_roll_mean_1000"]))) + (((data["q99_roll_mean_1000"]) * (data["q99_roll_mean_1000"]))))/2.0)))/2.0))))))))))))) +
            0.0399999991*activation1((((12.50784111022949219)) + ((((12.50784111022949219)) - ((((((activation1((((data["q05_roll_std_10"]) * (data["q95_roll_mean_1000"]))))) + (((((((data["q05_roll_std_10"]) + (activation1((((data["q05_roll_std_10"]) * (data["exp_Moving_average_30000_mean"]))))))) * 2.0)) * 2.0)))/2.0)) * (73.0))))))) +
            0.0399999991*activation1(((((((data["skew"]) - (((data["ave_roll_mean_100"]) + (((((((data["ave_roll_mean_100"]) * (data["abs_q05"]))) + (((((2.0) + (((((data["q05_roll_std_100"]) - (data["q05_roll_mean_10"]))) * 2.0)))) * 2.0)))) * 2.0)))))) * 2.0)) * 2.0)) +
            0.0399999991*activation1(((((((((data["q95_roll_mean_1000"]) * (data["ave_roll_mean_100"]))) * 2.0)) - (((((data["exp_Moving_average_30000_mean"]) + (((((((((data["exp_Moving_average_30000_mean"]) * (data["q05_roll_std_10"]))) + (((((data["q05_roll_std_10"]) - (0.2439019978))) * 2.0)))) * 2.0)) * 2.0)))) * 2.0)))) * 2.0)) +
            0.0399999991*activation1(((((((((data["q05_roll_std_10"]) - (((((data["q95"]) - (((((data["iqr"]) - (((data["exp_Moving_average_3000_mean"]) * (activation1((((data["q05_roll_std_10"]) * 2.0)))))))) - (((data["q05_roll_std_10"]) * 2.0)))))) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*activation1(((((((data["q95_roll_mean_100"]) + (((data["mean_change_rate_first_10000"]) + (((data["abs_q05"]) + (data["abs_q05"]))))))) * (3.6923100948))) + (((((((((data["q05_roll_std_1000"]) + (0.7446810007))) * (((3.6923100948) - (73.0))))) * 2.0)) * 2.0)))) +
            0.0399999991*activation1((-1.0*((((((73.0) * ((((((data["q01_roll_mean_1000"]) - (((data["q05"]) * 2.0)))) + (((data["q05_roll_std_1000"]) + (((((data["q05_roll_std_100"]) - (data["q95_roll_mean_1000"]))) * (data["Moving_average_700_mean"]))))))/2.0)))) + (((((data["Moving_average_1500_mean"]) * 2.0)) * 2.0))))))) +
            0.0399999991*activation1(((((((((data["q05"]) - (((73.0) * ((((-1.0*((((data["q05"]) * 2.0))))) + (((data["q05_roll_std_1000"]) + (1.9019600153))))))))) + (((((((data["q05"]) + (data["max_to_min"]))) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) +
            0.0399999991*activation1(((((((data["av_change_abs_roll_mean_100"]) - (((data["q01_roll_mean_1000"]) * 2.0)))) - (((73.0) * (((((((data["Hann_window_mean"]) * (((data["q95"]) - (data["q95_roll_mean_1000"]))))) / 2.0)) + (((((data["q95"]) * 2.0)) + (data["q01_roll_mean_1000"]))))))))) * 2.0)) +
            0.0399999991*activation1(((((((((((data["skew"]) - (((((data["q05_roll_mean_10"]) + (((((data["iqr"]) + (((3.1415927410) * (((1.0) - (((data["q05_roll_mean_10"]) * 2.0)))))))) * 2.0)))) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*activation1((((((((6.0)) - (((((((((data["q05_roll_std_100"]) + (((data["q05_roll_std_100"]) * 2.0)))) * 2.0)) + (((((data["ave10"]) * (((((data["q05_roll_std_100"]) * 2.0)) * 2.0)))) + (activation1((data["q05_roll_std_100"]))))))) * 2.0)))) * 2.0)) * 2.0)) +
            0.0399999991*activation1(((((((((data["q05_roll_std_10"]) + (((((((data["q95_roll_mean_1000"]) + (((((((data["skew"]) + (((((((data["abs_q05"]) - (((data["q05_roll_std_10"]) * 2.0)))) * 2.0)) * 2.0)))) * 2.0)) * 2.0)))) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*activation1(((((((((data["max_to_min"]) + (data["q95_roll_std_1000"]))) - (((((73.0) * ((((data["q05_roll_std_1000"]) + (((0.2183910012) + (data["abs_q05"]))))/2.0)))) * (((data["q05_roll_std_1000"]) * (((data["q05_roll_std_10"]) + (data["q01_roll_std_10"]))))))))) * 2.0)) * 2.0)) +
            0.0399999991*activation1(((((((((((data["abs_q05"]) + (data["q05_roll_std_100"]))) - (((7.0) * (((data["iqr"]) - (((data["q99_roll_mean_1000"]) + ((-1.0*((((((((data["q05_roll_std_100"]) + (1.0))) * 2.0)) * 2.0))))))))))))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*activation1((-1.0*((((7.0) * (((73.0) * (((((7.0) * (((((data["Moving_average_6000_mean"]) * (((((-1.0*((data["Moving_average_6000_mean"])))) + (activation1((activation1((data["q01_roll_std_10"]))))))/2.0)))) + (data["q95_roll_mean_10"]))))) + (data["q95_roll_mean_10"])))))))))) +
            0.0399999991*activation1(((((((((((data["skew"]) + (data["ave10"]))) + (data["q95_roll_std_100"]))) - (((7.0) * (((((data["q05_roll_std_100"]) * (data["q95"]))) * ((((((data["q05_roll_std_100"]) * (7.0))) + (7.0))/2.0)))))))) * 2.0)) * 2.0)) +
            0.0399999991*activation1((((((9.64285850524902344)) * (((((data["Moving_average_700_mean"]) - ((((((9.64285850524902344)) * ((((data["q05_roll_std_100"]) + (data["exp_Moving_average_30000_mean"]))/2.0)))) - ((((data["skew"]) + ((3.98402547836303711)))/2.0)))))) * 2.0)))) * (((data["iqr"]) * (data["q05_roll_std_100"]))))) +
            0.0399999991*activation1(((((((((((((((((data["Moving_average_1500_mean"]) - (((activation1((data["exp_Moving_average_30000_mean"]))) + (activation1((((data["abs_q05"]) + ((((data["q95_roll_std_10"]) + (data["q01_roll_std_10"]))/2.0)))))))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) - (data["iqr"]))) * 2.0)) +
            0.0399999991*activation1((((-1.0*((data["q95"])))) * (((73.0) + ((-1.0*((((data["abs_q05"]) * ((((((((-1.0*((data["abs_q05"])))) - (data["min_roll_std_10"]))) - (73.0))) - ((((-1.0*((data["q05_roll_mean_10"])))) * (73.0)))))))))))))) +
            0.0399999991*activation1(((data["q05_roll_std_10"]) * (((((((((data["q05_roll_std_100"]) * ((11.90774154663085938)))) * (((data["ave10"]) - (-2.0))))) - ((11.90774154663085938)))) * ((-1.0*((((data["q05_roll_std_1000"]) + (((data["q05_roll_std_100"]) - (-2.0)))))))))))) +
            0.0399999991*activation1(((((((data["iqr"]) * ((9.63457870483398438)))) * (((data["q95_roll_mean_1000"]) - ((((9.63457870483398438)) * (((data["q05_roll_std_10"]) * ((((data["q01_roll_std_10"]) + (((data["iqr"]) * (data["q95_roll_mean_1000"]))))/2.0)))))))))) - (((-3.0) * (data["q01_roll_std_10"]))))) +
            0.0399999991*activation1(((((((((-1.0) - (((activation1((data["q05_roll_std_10"]))) + (((((((data["q05_roll_std_100"]) - (activation1((((data["classic_sta_lta1_mean"]) + (((((((data["q95_roll_std_100"]) * 2.0)) * 2.0)) * 2.0)))))))) * 2.0)) * 2.0)))))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*activation1(((((((((data["q95_roll_mean_10"]) - (((((((data["q95_roll_mean_10"]) + (((((data["q05_roll_std_10"]) + (((3.6923100948) * ((((data["ave10"]) + (((data["q05_roll_std_10"]) + (-1.0))))/2.0)))))) * 2.0)))) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*activation1(((((data["q05_roll_std_100"]) + (((((data["q95_roll_mean_1000"]) + (((((data["q05_roll_std_100"]) + ((((3.1415927410) + (data["q05_roll_std_1000"]))/2.0)))) * (((((((data["q05_roll_mean_10"]) * 2.0)) * 2.0)) * (((3.1415927410) * (data["q05_roll_std_100"]))))))))) * 2.0)))) * 2.0)) +
            0.0399999991*activation1(((((((((((((data["mean_change_rate_first_10000"]) - (data["Moving_average_6000_mean"]))) * 2.0)) - (data["q01_roll_mean_1000"]))) - (((data["av_change_abs_roll_mean_10"]) + (((data["mean_change_rate_last_50000"]) + (((data["av_change_abs_roll_std_100"]) - ((((((13.93686485290527344)) / 2.0)) * (data["q05"]))))))))))) * 2.0)) * 2.0)) +
            0.0399999991*activation1(((((((((data["q05_roll_std_10"]) * (data["q05_roll_std_10"]))) * ((((-1.0*((((2.0) - ((-1.0*((((data["q05_roll_std_100"]) + (((((data["q05_roll_std_100"]) * (data["abs_q05"]))) * 2.0)))))))))))) * 2.0)))) - ((-1.0*((data["kurt"])))))) * 2.0)) +
            0.0399999991*activation1(((((((((((((((data["q05"]) - (activation1((((data["iqr"]) + (((((data["Moving_average_1500_mean"]) + (((data["q05"]) + (data["Moving_average_3000_mean"]))))) * 2.0)))))))) * 2.0)) + (data["q05"]))) * 2.0)) * 2.0)) - (data["av_change_abs_roll_mean_10"]))) * 2.0)) +
            0.0399999991*activation1(((((((((activation1((((((((((((((data["Moving_average_6000_mean"]) - (data["q05_roll_std_100"]))) * 2.0)) * 2.0)) - (((1.6428600550) + (data["q01_roll_std_10"]))))) * 2.0)) - (data["av_change_abs_roll_std_100"]))))) * 2.0)) - (data["abs_q05"]))) * 2.0)) * 2.0)) +
            0.0399999991*activation1((((((((((((((data["skew"]) + (data["q95_roll_std_10"]))/2.0)) - (((((data["ave_roll_std_100"]) + (((data["max_roll_mean_10"]) * 2.0)))) * (data["abs_q05"]))))) + (((((activation1((data["q95_roll_std_100"]))) - (data["q01_roll_std_10"]))) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*activation1(((data["skew"]) + (((((data["q01_roll_std_100"]) * 2.0)) + (((data["Moving_average_700_mean"]) + (((data["q05_roll_std_10"]) * (((data["q05"]) * ((((((73.0) * (((data["Moving_average_700_mean"]) + (((data["q05_roll_std_100"]) / 2.0)))))) + (data["q05_roll_std_10"]))/2.0)))))))))))) +
            0.0399999991*activation1(((((((data["q05_roll_mean_10"]) * (((data["q05_roll_std_100"]) * (data["q05_roll_std_100"]))))) * (((((data["q05_roll_std_100"]) * 2.0)) * (((data["q05_roll_std_1000"]) * 2.0)))))) - (((((((data["q05_roll_std_1000"]) * 2.0)) * (data["q05_roll_std_100"]))) - (((data["q01_roll_std_1000"]) * 2.0)))))) +
            0.0399999991*activation1(((((((-1.0) + (((((((activation1((data["q01_roll_std_10"]))) * 2.0)) * 2.0)) + (((data["max_to_min_diff"]) + (((data["q05"]) * ((((14.98138904571533203)) + (((data["mean_change_rate_last_10000"]) * 2.0)))))))))))) * (73.0))) * (73.0))) +
            0.0399999991*activation1(((((((data["q05_roll_std_10"]) - (((((data["q05_roll_std_1000"]) + (0.7446810007))) * (((data["q05_roll_std_10"]) * (((((((data["q01_roll_std_10"]) * (((data["ave10"]) + (((data["q95_roll_mean_10"]) * (data["q05_roll_std_10"]))))))) * 2.0)) * 2.0)))))))) * 2.0)) * 2.0)) +
            0.0399999991*activation1(((((((activation1((data["q01_roll_std_100"]))) - (((((((((data["q05_roll_std_1000"]) + (((((data["q95"]) - (activation1((data["q95_roll_std_100"]))))) * 2.0)))) - (activation1((activation1((data["q95"]))))))) - (data["std_roll_mean_1000"]))) * 2.0)))) * 2.0)) * 2.0)) +
            0.0399999991*activation1(((((data["q05_roll_std_10"]) + (((data["ave_roll_mean_1000"]) * 2.0)))) * (((((((data["ave_roll_mean_1000"]) * (data["ave_roll_mean_1000"]))) - (data["q05_roll_std_1000"]))) + (((data["q05_roll_mean_10"]) * (((data["Moving_average_3000_mean"]) + (((((data["q05_roll_std_10"]) * 2.0)) * 2.0)))))))))) +
            0.0399999991*activation1(((((((((((data["q05_roll_std_1000"]) + (((data["q05_roll_std_100"]) - (((data["Moving_average_6000_mean"]) * (data["Moving_average_6000_mean"]))))))) * (((data["q95_roll_std_100"]) * ((((-1.0*((data["iqr"])))) - (((data["Moving_average_6000_mean"]) + (data["Moving_average_1500_mean"]))))))))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*activation1(((((((((data["sum"]) + (0.9756100178))) * 2.0)) * 2.0)) * (((data["q01_roll_std_100"]) - (((activation1((((data["q05_roll_std_10"]) * (data["q05_roll_std_10"]))))) + (((data["q05_roll_std_1000"]) * (((((data["q01_roll_std_100"]) * (data["q01_roll_std_10"]))) * 2.0)))))))))) +
            0.0399999991*activation1((((((((((-1.0*((data["q05_roll_std_100"])))) - (((data["q01_roll_std_100"]) * (data["mean"]))))) * (((data["q05_roll_std_100"]) * (data["q05_roll_std_100"]))))) * (((data["q05_roll_std_100"]) * (data["q05_roll_std_100"]))))) + (((((data["av_change_rate_roll_std_10"]) * (data["max_to_min"]))) * 2.0)))) +
            0.0399999991*activation1(((((((data["q05_roll_std_10"]) * 2.0)) * 2.0)) * (((((0.5434780121) * 2.0)) - (((((data["q05_roll_std_10"]) * (((((((((data["q95_roll_std_10"]) * 2.0)) * (data["q05_roll_std_10"]))) + (data["q05_roll_std_10"]))) * (data["q05_roll_std_10"]))))) + (data["skew"]))))))) +
            0.0399999991*activation1(((((data["abs_q05"]) * (data["q95_roll_std_100"]))) - (((((data["q05_roll_std_100"]) * (((data["q05_roll_std_100"]) * (((data["q05_roll_std_100"]) * (((data["q05_roll_std_100"]) * (((data["abs_q05"]) * (data["q05_roll_std_100"]))))))))))) + ((((data["min_roll_std_100"]) + (data["av_change_abs_roll_mean_100"]))/2.0)))))) +
            0.0399999991*activation1(((((data["Moving_average_6000_mean"]) + (0.8873239756))) * (((((data["q05_roll_std_100"]) + (data["q95_roll_std_1000"]))) * (((1.6428600550) - (((data["q05_roll_std_1000"]) * (((((data["q05_roll_std_100"]) + (((data["exp_Moving_average_3000_mean"]) + (0.9756100178))))) + (data["q05_roll_std_100"]))))))))))) +
            0.0399999991*activation1(((((data["q95_roll_mean_10"]) + (((data["q95_roll_std_1000"]) * (((data["med"]) + (((((data["q05_roll_mean_100"]) + (((data["q95_roll_mean_10"]) * (data["q95_roll_mean_10"]))))) * (data["q05_roll_std_100"]))))))))) * (((((data["exp_Moving_average_30000_mean"]) - (data["q05_roll_std_100"]))) - (data["q05_roll_std_1000"]))))) +
            0.0399999991*activation1(((((((data["min"]) + ((((-1.0*((data["MA_400MA_BB_high_mean"])))) + (activation1((((((0.3183098733) + (((((data["kurt"]) * 2.0)) * 2.0)))) * ((((3.0)) + ((-1.0*((data["kurt"])))))))))))))) * 2.0)) * 2.0)) +
            0.0399999991*activation1(((((((data["min_roll_mean_1000"]) * 2.0)) * 2.0)) + ((((-1.0*((((((data["q05_roll_std_1000"]) + (((data["q01_roll_std_10"]) * (data["q95"]))))) + (((data["avg_last_50000"]) * (((data["q01_roll_std_10"]) - (data["avg_last_50000"])))))))))) * (data["q01_roll_std_100"]))))) +
            0.0399921872*activation1(((((data["q95_roll_std_1000"]) - ((((((((data["av_change_abs_roll_mean_10"]) + (data["std_first_10000"]))/2.0)) + (((data["q95"]) - (data["min_last_10000"]))))) - (((data["min"]) + (((data["q95_roll_std_1000"]) + (data["mean_change_rate_last_10000"]))))))))) - (((data["q95"]) + (data["q95"]))))) +
            0.0399999991*activation1(((((((data["min_roll_mean_1000"]) - (((data["q05_roll_std_1000"]) * (((activation1((((data["med"]) * 2.0)))) + ((((((data["skew"]) * (data["kurt"]))) + (data["q05_roll_std_1000"]))/2.0)))))))) + (((data["kurt"]) * (data["skew"]))))) + (data["min_roll_mean_1000"]))) +
            0.0399921872*activation1(((data["exp_Moving_average_30000_mean"]) * (((((data["max_roll_std_100"]) - (((((data["exp_Moving_average_30000_mean"]) * (data["MA_400MA_std_mean"]))) + (data["q05_roll_mean_10"]))))) * (((((data["mean_change_rate_first_10000"]) * (73.0))) - (((((data["abs_mean"]) * (73.0))) * (73.0))))))))) +
            0.0399999991*activation1(((data["q01_roll_std_10"]) + ((-1.0*((((data["q05_roll_std_1000"]) * (((data["av_change_rate_roll_mean_100"]) + ((((-1.0*((data["classic_sta_lta4_mean"])))) + (((data["med"]) + (((data["q05_roll_std_1000"]) + (((data["iqr"]) * (((data["iqr"]) + (data["max_last_10000"])))))))))))))))))))) +
            0.0399921872*activation1((((((((data["q05"]) + (data["mean_change_rate_first_10000"]))/2.0)) + (((data["kurt"]) - (((((data["kurt"]) * (data["abs_max_roll_std_1000"]))) * (data["kurt"]))))))) + (((((data["av_change_abs_roll_std_100"]) * ((-1.0*((data["classic_sta_lta3_mean"])))))) * 2.0)))) +
            0.0399999991*activation1(((((((((((((((data["skew"]) - ((((data["min_roll_std_1000"]) + (data["q95_roll_mean_100"]))/2.0)))) + (((data["q95_roll_mean_100"]) * (data["q05"]))))) * (data["MA_400MA_std_mean"]))) * (data["MA_400MA_BB_high_mean"]))) * 2.0)) * 2.0)) + ((((data["mean_change_abs"]) + (data["min_roll_std_1000"]))/2.0)))) +
            0.0399921872*activation1((((((4.12892913818359375)) * ((((((data["exp_Moving_average_300_mean"]) + (data["med"]))/2.0)) - (data["q01_roll_std_100"]))))) * (((data["skew"]) + (((0.2439019978) + ((((data["MA_1000MA_std_mean"]) + ((((((data["av_change_abs_roll_std_1000"]) + (data["med"]))) + (data["iqr"]))/2.0)))/2.0)))))))) +
            0.0399999991*activation1(((((((((data["av_change_abs_roll_std_10"]) * (((((data["mean_change_rate_last_50000"]) + (data["av_change_abs_roll_mean_10"]))) + (data["min_last_50000"]))))) * 2.0)) * 2.0)) + ((((((((1.0)) + (((data["av_change_rate_roll_std_10"]) + (data["skew"]))))/2.0)) + (data["q01_roll_mean_1000"]))/2.0)))) +
            0.0399999991*activation1(((((((((data["q05_roll_std_10"]) * (((((((data["med"]) + (data["q05_roll_std_10"]))) * (data["q05_roll_mean_10"]))) * (data["q95_roll_std_1000"]))))) * (data["q05_roll_std_10"]))) + (((((data["q05_roll_mean_10"]) + (data["q95_roll_std_100"]))) + (data["q95_roll_std_100"]))))) * 2.0)) +
            0.0399921872*activation1((((data["skew"]) + (((((((((data["min_roll_std_10"]) * ((-1.0*((data["min_roll_std_10"])))))) + (data["q05_roll_mean_1000"]))) * (data["Hilbert_mean"]))) - ((((data["av_change_rate_roll_std_100"]) + (((data["max_roll_std_100"]) * (((data["skew"]) * (data["skew"]))))))/2.0)))))/2.0)) +
            0.0399999991*activation1(((data["q05_roll_std_10"]) * (((((((((data["q01_roll_mean_1000"]) - (activation1((((data["Moving_average_6000_mean"]) - (((data["ave_roll_std_10"]) * (((data["kurt"]) - ((((data["av_change_abs_roll_mean_10"]) + ((((data["Moving_average_6000_mean"]) + (data["ave_roll_std_10"]))/2.0)))/2.0)))))))))))) * 2.0)) * 2.0)) * 2.0)))) +
            0.0399843715*activation1(((data["min"]) + ((-1.0*((((data["av_change_rate_roll_std_100"]) * ((((((((data["mean_change_rate"]) + (data["min"]))/2.0)) + (data["min_roll_std_1000"]))) + (((activation1((data["min_last_10000"]))) + ((((((data["min_roll_std_100"]) * (data["av_change_rate_roll_mean_10"]))) + (data["av_change_abs_roll_std_10"]))/2.0))))))))))))) +
            0.0399999991*activation1(((data["min_roll_mean_100"]) * (((((data["min_roll_std_1000"]) * (((data["q01_roll_std_1000"]) * (((((((data["q01_roll_std_1000"]) * (data["q99_roll_std_100"]))) + (data["exp_Moving_average_300_mean"]))) + (data["avg_first_10000"]))))))) + (((((data["mean_change_rate"]) * (3.1415927410))) - (data["avg_first_10000"]))))))) +
            0.0399921872*activation1(((((data["q95_roll_mean_1000"]) * (((((((((data["q95_roll_mean_1000"]) + (((data["avg_last_10000"]) + (((3.0) / 2.0)))))/2.0)) + (data["q05_roll_mean_100"]))/2.0)) - (data["q95_roll_std_1000"]))))) * (((((((data["q95_roll_std_1000"]) * 2.0)) + (data["q99_roll_mean_10"]))) + (data["classic_sta_lta4_mean"]))))) +
            0.0399296731*activation1((((((((((((data["q01_roll_std_100"]) + (activation1((((data["q01"]) + ((-1.0*((((data["q01_roll_std_100"]) * 2.0))))))))))/2.0)) * ((-1.0*(((((data["q95"]) + (((data["exp_Moving_average_30000_mean"]) * 2.0)))/2.0))))))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399921872*activation1(((((activation1((data["q99"]))) + ((-1.0*(((((data["max_first_10000"]) + (((data["av_change_rate_roll_mean_1000"]) - (data["q99"]))))/2.0))))))) - ((((((((data["av_change_rate_roll_mean_1000"]) * (data["av_change_rate_roll_mean_1000"]))) + (data["q05_roll_std_10"]))/2.0)) * (((data["q05_roll_std_10"]) * (data["iqr"]))))))) +
            0.0399999991*activation1(((data["min_roll_mean_1000"]) * (((((((((((data["std_roll_std_100"]) + (((data["q05"]) + (((((data["q95_roll_mean_1000"]) * (data["q95_roll_std_1000"]))) + (((data["q95_roll_mean_1000"]) * (((data["q95_roll_mean_1000"]) * (data["q95_roll_std_1000"]))))))))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) +
            0.0399921872*activation1(((((data["mean_change_abs"]) * (data["min_roll_std_10"]))) + (((((((data["std_last_50000"]) * (3.6923100948))) * (((data["q95_roll_mean_1000"]) + (((data["classic_sta_lta4_mean"]) - (data["min_roll_std_10"]))))))) * ((((((data["min_roll_std_10"]) + (data["q05_roll_mean_100"]))/2.0)) - (data["max_roll_std_10"]))))))) +
            0.0399999991*activation1(((((data["q01_roll_std_10"]) + (data["min_roll_std_100"]))) * (((data["abs_q05"]) - (((data["avg_last_50000"]) - (((((data["classic_sta_lta2_mean"]) / 2.0)) + (((data["max_roll_mean_1000"]) * (((((data["avg_last_50000"]) - (activation1((data["min_roll_std_100"]))))) - (data["min_roll_std_100"]))))))))))))) +
            0.0399999991*activation1(((((data["q95_roll_mean_100"]) * (data["min_roll_mean_1000"]))) - (((((data["av_change_abs_roll_std_100"]) * (((((data["classic_sta_lta3_mean"]) + (data["mean_change_rate_first_50000"]))) + (((data["q95_roll_mean_100"]) * (data["classic_sta_lta3_mean"]))))))) + (((data["mean_change_abs"]) * (((data["classic_sta_lta3_mean"]) - (data["mean_change_rate_first_50000"]))))))))) +
            0.0398359038*activation1(((data["q95_roll_mean_10"]) * (((((data["q95_roll_mean_10"]) * (((data["min_roll_mean_1000"]) + (data["av_change_rate_roll_std_10"]))))) - (((data["max_to_min"]) + (((((((data["max_to_min"]) * 2.0)) * 2.0)) * (((data["av_change_rate_roll_std_10"]) + (data["q01_roll_mean_1000"]))))))))))) +
            0.0399999991*activation1(((data["q01_roll_std_10"]) * (((((((data["q01_roll_std_10"]) * (data["skew"]))) + (((data["classic_sta_lta4_mean"]) - (data["classic_sta_lta2_mean"]))))) + ((((data["av_change_abs_roll_mean_1000"]) + (((((-1.0*((data["q01_roll_std_100"])))) + (((data["q01_roll_std_10"]) * ((-1.0*((data["q01_roll_std_10"])))))))/2.0)))/2.0)))))) +
            0.0385153331*activation1((((((data["std_roll_mean_1000"]) + (data["av_change_abs_roll_mean_100"]))) + (((data["max_first_10000"]) * (((data["MA_700MA_std_mean"]) * (((73.0) * (((activation1((data["max_to_min"]))) - (((data["av_change_rate_roll_mean_1000"]) + (((((data["std_roll_mean_1000"]) * 2.0)) * 2.0)))))))))))))/2.0)) +
            0.0399999991*activation1(((((data["std_roll_mean_1000"]) * (((data["av_change_rate_roll_std_1000"]) - (((((((data["skew"]) - (data["min_roll_std_10"]))) - (data["classic_sta_lta4_mean"]))) + (((data["std_roll_mean_1000"]) - (2.0))))))))) + (((data["classic_sta_lta4_mean"]) * (((data["av_change_abs_roll_mean_1000"]) - (data["q99"]))))))) +
            0.0399843715*activation1((-1.0*((((data["kurt"]) * (((data["kurt"]) * ((((((data["av_change_abs_roll_mean_10"]) + (((((data["med"]) - (((data["med"]) * (data["av_change_abs_roll_mean_10"]))))) - (data["min_roll_std_10"]))))/2.0)) + (((data["q99_roll_std_1000"]) * (data["kurt"])))))))))))) +
            0.0399921872*activation1(((data["mean_change_rate_first_10000"]) * (((data["av_change_abs_roll_mean_100"]) * ((-1.0*((((((activation1((((((data["classic_sta_lta3_mean"]) + (data["mean_change_rate_first_10000"]))) * 2.0)))) + (((1.9019600153) - (data["av_change_abs_roll_mean_100"]))))) - (((((data["classic_sta_lta3_mean"]) * 2.0)) * (data["mean_change_rate_first_10000"])))))))))))) +
            0.0399921872*activation1(((((data["av_change_rate_roll_std_100"]) * (((data["classic_sta_lta3_mean"]) * (((((data["min_roll_std_1000"]) - (data["min"]))) * (data["q05"]))))))) + (((data["mean_change_rate_last_10000"]) * (((((data["av_change_abs_roll_std_1000"]) - (data["min"]))) - (((data["classic_sta_lta3_mean"]) * (data["q01_roll_std_1000"]))))))))) +
            0.0399999991*activation1(((data["q95_roll_std_1000"]) + (((data["q05_roll_mean_10"]) - ((-1.0*((((data["q95_roll_std_1000"]) * (((data["q95_roll_std_1000"]) * (((((data["q01_roll_std_10"]) * ((((((((data["q95_roll_std_1000"]) / 2.0)) + (data["q01_roll_mean_1000"]))/2.0)) - (data["med"]))))) * (data["q95_roll_std_1000"])))))))))))))) +
            0.0399843715*activation1(((data["av_change_abs_roll_mean_100"]) * (((data["classic_sta_lta4_mean"]) * (((((((((3.65123486518859863)) + ((((3.65123486518859863)) * (data["abs_max_roll_std_10"]))))/2.0)) + (((((data["min_last_50000"]) * (((data["abs_max_roll_std_1000"]) - (data["av_change_abs_roll_mean_10"]))))) - (data["classic_sta_lta4_mean"]))))) * (data["ave_roll_mean_10"]))))))) +
            0.0399999991*activation1(((((data["max_to_min_diff"]) * (((data["min_roll_std_100"]) * (((data["av_change_rate_roll_std_100"]) + (((data["q01_roll_std_100"]) + (((((((data["av_change_rate_roll_mean_100"]) + (data["Hilbert_mean"]))) * (data["q95_roll_mean_1000"]))) + (((data["q01"]) * (data["min_roll_std_100"]))))))))))))) * 2.0)) +
            0.0399999991*activation1(((((activation1((((((data["q05"]) + (((activation1((data["avg_first_10000"]))) + (data["mean_change_rate_last_10000"]))))) + (data["q05"]))))) + (((((data["kurt"]) * (data["mean_change_rate_last_10000"]))) + (data["q95_roll_std_1000"]))))) * (((data["q01_roll_mean_1000"]) + (data["av_change_rate_roll_mean_100"]))))) +
            0.0399921872*activation1(((data["min_roll_mean_1000"]) + ((((((((((((data["av_change_rate_roll_std_10"]) + (((data["av_change_abs_roll_std_1000"]) * (data["min_roll_mean_1000"]))))/2.0)) * (data["classic_sta_lta3_mean"]))) + (data["av_change_rate_roll_std_10"]))/2.0)) + (((data["abs_max_roll_mean_100"]) - (((data["av_change_abs_roll_std_1000"]) * (data["classic_sta_lta1_mean"]))))))/2.0)))) +
            0.0399999991*activation1(((data["max_to_min"]) * (((((data["MA_400MA_std_mean"]) - (data["q95_roll_std_100"]))) + ((((((data["min_roll_std_10"]) * (((((((data["min_roll_std_10"]) * (data["classic_sta_lta1_mean"]))) - (data["min_roll_std_10"]))) - (data["kurt"]))))) + ((((data["kurt"]) + (data["min_roll_std_10"]))/2.0)))/2.0)))))) +
            0.0399921872*activation1(((data["min_roll_std_10"]) * ((((((((((((activation1((data["min_roll_std_10"]))) + (data["avg_last_10000"]))/2.0)) - (data["q95_roll_std_100"]))) - (data["MA_700MA_BB_high_mean"]))) + (((data["abs_max_roll_mean_100"]) - (data["classic_sta_lta4_mean"]))))) * (((data["min_roll_std_10"]) - (((3.0) / 2.0)))))))) +
            0.0399765596*activation1(((((data["avg_last_10000"]) * (((data["min_roll_mean_1000"]) + (((((data["avg_last_10000"]) + (activation1((data["mean_change_rate_last_50000"]))))) * (((data["min_roll_std_10"]) * (data["min_last_10000"]))))))))) + (activation1((activation1((((((data["mean_change_rate_last_50000"]) * (data["min_roll_std_1000"]))) * 2.0)))))))) +
            0.0399921872*activation1(activation1((activation1((activation1((((((((data["q95_roll_std_10"]) - (data["q05"]))) * (((data["abs_q05"]) * (((data["mean_change_rate_first_10000"]) + (((((data["mean_change_rate_first_10000"]) + (data["q05"]))) * (((data["abs_q05"]) * (data["q95_roll_mean_1000"]))))))))))) * 2.0)))))))) +
            0.0399843715*activation1(((data["q95_roll_std_1000"]) * (((data["max_roll_mean_100"]) + (((((((((data["q01_roll_mean_1000"]) * 2.0)) * 2.0)) * (data["min_roll_mean_1000"]))) * (((((((data["q01_roll_mean_1000"]) + ((-1.0*((data["max_to_min"])))))) * 2.0)) * 2.0)))))))) +
            0.0399765596*activation1(((data["q05_roll_std_100"]) * ((-1.0*((activation1((((((data["med"]) + (((((data["med"]) + (((data["q01_roll_std_10"]) * (((data["avg_first_50000"]) + (data["q01_roll_std_10"]))))))) * ((((1.16074943542480469)) + (data["avg_first_50000"]))))))) * 2.0))))))))) +
            0.0399062298*activation1((-1.0*((((data["classic_sta_lta2_mean"]) * ((((((activation1((data["kurt"]))) + (data["avg_last_50000"]))/2.0)) + (((data["q05_roll_mean_100"]) * ((-1.0*(((((((((data["kurt"]) + (data["q05_roll_mean_100"]))/2.0)) - (activation1((data["kurt"]))))) + (data["std_last_10000"]))))))))))))))) +
            0.0399453007*activation1(((data["std_last_50000"]) * (((((activation1((((((((((((data["av_change_abs_roll_mean_100"]) + (activation1((activation1((((((((((data["skew"]) + (data["Moving_average_6000_mean"]))) * 2.0)) * 2.0)) * 2.0)))))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) * 2.0)))) +
            0.0390935726*activation1((-1.0*((((data["max_roll_mean_100"]) * (((((((activation1((((((data["classic_sta_lta3_mean"]) + ((((data["q99_roll_mean_1000"]) + (data["mean_change_rate_first_10000"]))/2.0)))) + (((data["av_change_abs_roll_mean_10"]) + (((data["classic_sta_lta3_mean"]) * (data["q95_roll_mean_1000"]))))))))) * 2.0)) * 2.0)) * 2.0))))))) +
            0.0399296731*activation1(((data["q95_roll_std_100"]) + ((-1.0*((((data["q95_roll_std_10"]) + ((((((((data["kurt"]) * ((((data["mean_change_rate_last_10000"]) + (data["trend"]))/2.0)))) * (data["kurt"]))) + ((((((data["med"]) + (data["max_first_50000"]))/2.0)) * (data["min_roll_std_100"]))))/2.0))))))))) +
            0.0398984179*activation1(((((activation1((((data["q01"]) - (data["q05"]))))) + (((((((((activation1((data["q05_roll_std_100"]))) + (data["q01"]))) * 2.0)) + (data["q95_roll_mean_1000"]))) * 2.0)))) * ((-1.0*((((data["q01"]) - (data["q05"])))))))) +
            0.0399999991*activation1(activation1((activation1(((-1.0*((((data["mean_change_rate_first_10000"]) * (((data["min_last_10000"]) + (((data["min_last_10000"]) + (((((data["abs_q95"]) + (((data["avg_last_50000"]) + ((((data["min_last_10000"]) + (0.8873239756))/2.0)))))) * ((-1.0*((data["q05_roll_std_100"]))))))))))))))))))) +
            0.0399999991*activation1(((data["min_first_50000"]) * ((((-1.0*((data["av_change_abs_roll_std_100"])))) - (((data["ave_roll_mean_10"]) * ((-1.0*((((data["max_to_min"]) - (((((((data["MA_400MA_BB_low_mean"]) - (((data["av_change_rate_roll_mean_100"]) + (((data["q01_roll_mean_1000"]) * (data["av_change_rate_roll_mean_100"]))))))) * 2.0)) * 2.0))))))))))))) +
            0.0399999991*activation1(((((((((data["max_to_min_diff"]) * (((data["av_change_rate_roll_std_100"]) + (((((data["q95_roll_std_10"]) + (data["max_first_50000"]))) + (((data["mean_change_abs"]) + (((((data["MA_400MA_BB_low_mean"]) * (data["max_first_50000"]))) * (data["max_first_50000"]))))))))))) * 2.0)) * 2.0)) * (data["min_roll_std_100"]))) +
            0.0399999991*activation1(((((data["max"]) * (((data["std_first_50000"]) * ((((((-1.0*((data["max_roll_std_100"])))) + (((((((((data["q001"]) + (((data["q001"]) * (data["classic_sta_lta4_mean"]))))) + (data["classic_sta_lta4_mean"]))) * 2.0)) * 2.0)))) * 2.0)))))) * (data["q05_roll_std_1000"]))) +
            0.0399765596*activation1(((((data["abs_max_roll_std_1000"]) * 2.0)) * (((data["q95_roll_std_10"]) - (((data["q999"]) - (activation1((((data["max_last_10000"]) * (((((((((((((data["abs_max_roll_std_1000"]) + (data["Moving_average_6000_mean"]))) * 2.0)) + (data["mean_change_abs"]))) * 2.0)) * 2.0)) * 2.0)))))))))))) +
            0.0399687439*activation1(((data["min_roll_std_10"]) * ((((((data["avg_last_50000"]) + (((((((data["q01_roll_std_1000"]) * (data["classic_sta_lta4_mean"]))) * (data["mean_change_rate_last_50000"]))) - (((data["min_roll_std_10"]) * (data["av_change_abs_roll_mean_10"]))))))/2.0)) + ((((((data["classic_sta_lta4_mean"]) * (data["Moving_average_6000_mean"]))) + (data["mean_change_rate_last_50000"]))/2.0)))))) +
                   
            0.0398906022*activation1((((activation1((((data["av_change_abs_roll_mean_1000"]) - (((data["Hann_window_mean"]) - (data["skew"]))))))) + (((((((data["min_roll_mean_1000"]) * 2.0)) * 2.0)) * (((data["skew"]) + (((((data["std_roll_mean_1000"]) - (data["q99_roll_mean_10"]))) - (data["med"]))))))))/2.0)) +
                   
            0.0399999991*activation1(((data["std_first_10000"]) * ((-1.0*((((data["min_roll_std_10"]) * (((((((((data["kurt"]) + (((((data["av_change_rate_roll_std_1000"]) - (data["trend"]))) + (data["mean_change_rate_first_10000"]))))) - (data["max_last_10000"]))) + (((data["trend"]) * (data["trend"]))))) * 2.0))))))))) +
                   
            0.0399765596*activation1(((data["q99"]) * ((-1.0*((((data["iqr"]) - (((((((1.9019600153) + (((data["abs_max_roll_mean_100"]) + (data["q99"]))))) * (((((data["std_roll_mean_1000"]) * 2.0)) * (((data["max_roll_mean_1000"]) - (data["std_roll_mean_1000"]))))))) + (data["min_roll_std_1000"])))))))))) +
                   
            0.0399765596*activation1(((((data["mean_change_rate_last_10000"]) + (activation1((((((activation1((data["skew"]))) + (((data["max_roll_std_1000"]) * 2.0)))) * 2.0)))))) * (((data["abs_max_roll_std_10"]) + ((-1.0*((((data["skew"]) * (((((data["abs_max"]) / 2.0)) - (data["ave_roll_std_1000"])))))))))))) +
                   
            0.0355928876*activation1((((((((data["min_roll_std_100"]) * (data["q05_roll_mean_1000"]))) * (data["min_roll_std_1000"]))) + (((((data["min_roll_std_100"]) * (data["min_roll_std_100"]))) - (((((data["max_to_min"]) + ((((((data["q999"]) + (data["min_roll_std_100"]))/2.0)) + (data["q999"]))))) * (data["classic_sta_lta3_mean"]))))))/2.0)) +
                   
            0.0399843715*activation1((((((-1.0*((((((((data["q01_roll_std_10"]) * (data["q05_roll_std_1000"]))) * (data["max_first_50000"]))) * (data["q01_roll_std_10"])))))) * (((data["max_first_50000"]) + (data["max_to_min_diff"]))))) * (((((data["min"]) - (data["q05"]))) - (data["q05"]))))) +
                   
            0.0399765596*activation1(((data["med"]) * (((activation1((((((((((data["av_change_abs_roll_std_100"]) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) - (((((data["min_roll_std_10"]) + ((((data["q01_roll_std_1000"]) + ((((data["max_to_min_diff"]) + (data["min_roll_std_1000"]))/2.0)))/2.0)))) * (((data["av_change_abs_roll_std_100"]) * 2.0)))))))) +
                   
            0.0387106836*activation1((-1.0*((((((data["av_change_rate_roll_mean_10"]) * (((data["q95_roll_std_1000"]) + (((((((((((data["min_roll_mean_1000"]) * 2.0)) * 2.0)) * (((data["q95_roll_std_1000"]) + (data["min_roll_mean_1000"]))))) * 2.0)) * (((data["q95_roll_std_100"]) - (((data["min_roll_mean_1000"]) * 2.0)))))))))) * 2.0))))) +
                   
            0.0399999991*activation1(((data["max_last_10000"]) * ((((((((data["MA_700MA_BB_low_mean"]) * (((((data["avg_first_10000"]) - (data["av_change_abs_roll_mean_1000"]))) - (((data["avg_first_10000"]) * (((data["avg_first_10000"]) - (((data["min_roll_std_1000"]) * 2.0)))))))))) * 2.0)) + (((data["avg_first_10000"]) - (data["mean_change_rate_last_10000"]))))/2.0)))) +
                   
            0.0399999991*activation1(activation1((((((((((data["av_change_rate_roll_std_100"]) * (((data["min"]) * (((data["q05_roll_std_100"]) + (((((data["av_change_abs_roll_std_100"]) + (((data["mean_change_rate_first_10000"]) * (data["av_change_rate_roll_std_100"]))))) + (((data["med"]) - (data["mean_change_rate_first_10000"]))))))))))) * 2.0)) * 2.0)) * 2.0)))) +
                   
            0.0359679610*activation1((-1.0*((((activation1((((((((((((((((((data["min_roll_std_1000"]) - (data["av_change_rate_roll_std_1000"]))) - (((-2.0) + ((-1.0*((data["min_roll_std_1000"])))))))) - (activation1((data["av_change_abs_roll_std_100"]))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) / 2.0))))) +
                   
            0.0373354182*activation1((-1.0*((((data["max_to_min"]) * (((data["avg_first_50000"]) * (((((data["ave_roll_std_100"]) - (((data["classic_sta_lta1_mean"]) + (data["mean_change_rate_first_10000"]))))) * (((data["classic_sta_lta2_mean"]) - ((-1.0*((((data["ave_roll_std_100"]) / 2.0)))))))))))))))) +
                   
            0.0398671627*activation1(((((0.8873239756) + (data["avg_last_50000"]))) * (((data["iqr"]) * (((((0.8873239756) + (data["Moving_average_6000_mean"]))) + (((data["q05_roll_mean_10"]) * (((((((data["ave10"]) + (data["q01_roll_std_10"]))) * (data["q01_roll_std_10"]))) * (data["iqr"]))))))))))) +
                   
            0.0399453007*activation1((((activation1((((((((((((data["mean_change_rate_last_10000"]) + (activation1(((((((-1.0*((data["mean_change_rate_last_50000"])))) * 2.0)) * (data["q95_roll_mean_1000"]))))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) + (activation1((activation1(((((-1.0*((data["mean_change_rate_last_50000"])))) * 2.0)))))))/2.0)) +
                   
            0.0399687439*activation1(((((((data["max_roll_std_1000"]) + (((data["max_roll_std_1000"]) + (data["trend"]))))) + (data["count_big"]))) * (((data["max_to_min"]) * (((((((-1.0*((data["max_to_min"])))) + (data["max_roll_mean_1000"]))/2.0)) - (data["q99_roll_std_10"]))))))) +
                   
            0.0399843715*activation1(((data["classic_sta_lta2_mean"]) * (((data["abs_mean"]) * (((data["classic_sta_lta1_mean"]) * (((((((data["mean_change_rate_first_50000"]) + ((-1.0*((data["mean_change_abs"])))))) + (data["classic_sta_lta4_mean"]))) + (((((((-2.0) + (data["av_change_rate_roll_std_10"]))/2.0)) + (data["av_change_abs_roll_std_1000"]))/2.0)))))))))) +
                   
            0.0399687439*activation1(((data["min_roll_std_100"]) * (((((((data["mean_change_abs"]) + (((((((data["av_change_abs_roll_std_100"]) * (data["min_roll_std_100"]))) + (data["min_roll_std_1000"]))) * (((data["std_roll_mean_1000"]) * (((data["min_last_10000"]) + (data["av_change_abs_roll_std_100"]))))))))) * 2.0)) * (data["av_change_abs_roll_std_100"]))))) +
                   
            0.0399374850*activation1(((((data["min_first_10000"]) - (((data["mean_change_rate_first_10000"]) * (data["kurt"]))))) * (((((((data["abs_trend"]) * (data["kurt"]))) - (data["mean_change_abs"]))) + (((data["std_roll_mean_1000"]) + (((data["std_roll_mean_1000"]) + (((data["min_first_10000"]) + (data["abs_std"]))))))))))) +
                   
            0.0399921872*activation1(((data["q95_roll_std_10"]) * ((-1.0*((((((-1.0*((data["abs_max_roll_mean_100"])))) + ((((-1.0*((((data["classic_sta_lta4_mean"]) / 2.0))))) - (((data["q001"]) * (((data["q95_roll_mean_1000"]) - ((-1.0*((((((((data["classic_sta_lta4_mean"]) / 2.0)) / 2.0)) / 2.0))))))))))))/2.0))))))) +
                   
            0.0399843715*activation1(((((data["mean_change_rate_last_10000"]) * (((data["abs_max"]) * (((((((((((data["min_roll_std_1000"]) + (data["max_to_min"]))) - (data["count_big"]))) * 2.0)) - (data["max_to_min"]))) + (((data["av_change_abs_roll_std_1000"]) + (data["mean_change_abs"]))))))))) + (data["count_big"]))) +
                   
            0.0383434258*activation1(((((data["med"]) * ((-1.0*((((((((data["trend"]) * (((activation1((data["av_change_abs_roll_std_100"]))) - (data["mean_change_abs"]))))) + (data["mean_change_abs"]))) * 2.0))))))) * (((((data["av_change_abs_roll_std_100"]) + (activation1((data["min_last_10000"]))))) + (data["av_change_abs_roll_std_100"]))))) +
                   
            0.0399999991*activation1(activation1((((((((data["min_roll_mean_100"]) * 2.0)) * 2.0)) * (((((((data["max_last_10000"]) - (0.5434780121))) * (data["av_change_rate_roll_mean_100"]))) + (((data["av_change_rate_roll_std_10"]) - (((data["min_roll_std_10"]) * (data["av_change_rate_roll_std_10"]))))))))))) +
                   
            0.0399921872*activation1((-1.0*((((((((((data["min_roll_std_100"]) + (data["av_change_abs_roll_std_10"]))/2.0)) * (((data["av_change_abs_roll_mean_1000"]) + (data["av_change_abs_roll_std_10"]))))) + (((data["av_change_abs_roll_std_100"]) * (((data["classic_sta_lta4_mean"]) * (((data["av_change_abs_roll_std_10"]) + (((data["min_roll_std_1000"]) * (data["av_change_abs_roll_std_100"]))))))))))/2.0))))) +
                   
            0.0293416679*activation1(activation1((((data["abs_q95"]) * (((data["q01_roll_std_10"]) * (((data["q01_roll_std_10"]) * (((((data["max_last_50000"]) - ((((((data["av_change_abs_roll_mean_10"]) - (data["classic_sta_lta4_mean"]))) + (data["av_change_abs_roll_mean_100"]))/2.0)))) - ((-1.0*((((data["av_change_abs_roll_mean_10"]) * (data["classic_sta_lta4_mean"])))))))))))))))) +
                   
            0.0399765596*activation1(((((data["trend"]) * (data["max_first_10000"]))) * (((data["med"]) + (((data["av_change_rate_roll_mean_1000"]) + (((((2.0769200325) - (((activation1((((((((((data["classic_sta_lta1_mean"]) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) * 2.0)))) / 2.0)))))))) +
                   
            0.0399921872*activation1(((((data["mean_change_rate_last_50000"]) * ((-1.0*((((data["min_first_10000"]) - (data["std_first_10000"])))))))) * ((((((((((-1.0*((data["std_first_10000"])))) - (data["min_last_50000"]))) - (data["abs_max_roll_std_10"]))) - (((data["mean_change_rate_last_50000"]) * (data["std_first_10000"]))))) - (data["classic_sta_lta2_mean"]))))) +
                   
            0.0399921872*activation1(((data["av_change_abs_roll_mean_10"]) * (((data["abs_trend"]) + ((((((((data["mean_change_abs"]) * (data["min_last_10000"]))) + (((data["mean_change_rate_first_10000"]) * (data["max_first_10000"]))))) + ((((data["av_change_abs_roll_mean_10"]) + (((((data["med"]) - (data["q95_roll_mean_1000"]))) - (data["mean_change_rate_first_10000"]))))/2.0)))/2.0)))))) +
            0.0399531126*activation1(((data["ave_roll_std_100"]) * (((data["min_roll_std_100"]) * (((((data["max_roll_mean_100"]) + ((((data["max_to_min_diff"]) + (data["q99"]))/2.0)))) * (((((((data["max_to_min_diff"]) * ((((-1.0*((data["std_first_50000"])))) * (data["q95_roll_std_100"]))))) - (data["skew"]))) * 2.0)))))))) +
            0.0399843715*activation1(((data["std_roll_std_100"]) * (((data["min_roll_std_1000"]) * (((data["max_roll_mean_10"]) - (((data["kurt"]) * ((((((data["avg_last_10000"]) * (data["avg_last_10000"]))) + ((((data["q99_roll_mean_100"]) + ((((((data["avg_last_10000"]) * (data["avg_last_10000"]))) + (data["avg_first_10000"]))/2.0)))/2.0)))/2.0)))))))))) +
            0.0353584699*activation1(((data["med"]) * (((activation1((((activation1(((((((-1.0*((data["max_to_min"])))) - (data["trend"]))) * 2.0)))) - (((data["min_roll_std_100"]) * (data["med"]))))))) - (activation1((activation1((data["min_roll_std_100"]))))))))) +
            0.0399765596*activation1((((((((data["ave_roll_mean_10"]) + (data["min_roll_std_100"]))/2.0)) * (((((data["classic_sta_lta4_mean"]) + (data["kurt"]))) + (((((data["q95_roll_mean_10"]) + (((data["q95"]) * (data["q95"]))))) * (data["sum"]))))))) * ((((data["q05_roll_std_100"]) + (data["ave_roll_mean_10"]))/2.0)))) +
            0.0399843715*activation1(((data["min_roll_std_10"]) * ((((((-1.0*((((data["av_change_abs_roll_std_10"]) * (data["av_change_abs_roll_mean_100"])))))) * (data["av_change_abs_roll_std_10"]))) + (activation1(((((-1.0*((data["q05"])))) + (((data["min_roll_std_1000"]) * ((-1.0*((((data["av_change_abs_roll_mean_100"]) * (data["av_change_abs_roll_mean_100"])))))))))))))))) +
            0.0399999991*activation1((-1.0*((((data["classic_sta_lta1_mean"]) * (((data["max_last_50000"]) + (((((data["abs_q95"]) + (((data["av_change_abs_roll_std_10"]) * 2.0)))) * (((((data["abs_std"]) + (data["max_last_50000"]))) * (((((data["iqr"]) * 2.0)) * (data["avg_last_10000"])))))))))))))) +
            0.0388200805*activation1(((((data["min_first_10000"]) * (((data["mean_change_rate"]) * (((((data["min_roll_std_100"]) + (data["max_last_50000"]))) + (((((data["mean_change_rate"]) * (((data["mean_change_abs"]) * (((data["min_roll_std_100"]) + (data["min_first_10000"]))))))) - (data["mean_change_abs"]))))))))) * 2.0)) +
            0.0389529206*activation1(((activation1((activation1((((data["max_to_min"]) + (data["max_to_min"]))))))) - (activation1((((((((((((((((((((data["min_first_10000"]) * 2.0)) + (data["max_to_min"]))) * 2.0)) * 2.0)) + (data["max_last_50000"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)))))) +
            0.0399843715*activation1((-1.0*((((((data["max_first_50000"]) * 2.0)) * (((activation1(((((((-1.0*((((data["q95_roll_std_1000"]) + (((data["q01_roll_std_10"]) + (data["av_change_abs_roll_std_1000"])))))))) * 2.0)) * 2.0)))) + (((data["q01_roll_std_10"]) + (((data["max_first_50000"]) * (data["mean_change_abs"])))))))))))) +
            0.0399453007*activation1((((((data["av_change_abs_roll_std_1000"]) + (data["mean_change_abs"]))/2.0)) * (((data["av_change_abs_roll_std_100"]) + (((data["trend"]) + (((((data["av_change_abs_roll_mean_1000"]) + ((((data["av_change_abs_roll_std_100"]) + (((data["av_change_rate_roll_std_100"]) + (((data["av_change_abs_roll_std_1000"]) + (data["min_roll_std_10"]))))))/2.0)))) * (data["av_change_abs_roll_mean_1000"]))))))))) +
            0.0399140455*activation1(((data["std_first_50000"]) * (((data["mean_change_rate_last_50000"]) * ((-1.0*((((data["avg_first_50000"]) * ((((data["max_first_10000"]) + (((data["min_roll_mean_10"]) - (((data["avg_first_50000"]) - (((((data["mean_change_rate_last_50000"]) - (data["min_last_10000"]))) * (data["min_last_10000"]))))))))/2.0))))))))))) +
            0.0399999991*activation1(((((data["min_last_10000"]) + (activation1((((data["ave_roll_mean_1000"]) + (((((((data["av_change_abs_roll_mean_100"]) + (data["av_change_abs_roll_std_10"]))) * (data["ave_roll_mean_1000"]))) * (((data["min_roll_std_1000"]) * 2.0)))))))))) * (((data["min_roll_std_1000"]) * (((((data["std_roll_mean_1000"]) * 2.0)) * 2.0)))))) +
            0.0399765596*activation1((((((data["av_change_abs_roll_mean_100"]) + (data["q99_roll_std_10"]))/2.0)) * (activation1((((activation1((((((((((((((data["avg_first_10000"]) + (data["max_roll_std_1000"]))) + (data["count_big"]))) * (73.0))) - (data["av_change_rate_roll_mean_100"]))) * 2.0)) + (data["avg_first_10000"]))))) * 2.0)))))) +
            0.0371556953*activation1(((((data["max_last_10000"]) * (((((data["std_last_50000"]) * (data["avg_last_50000"]))) + (((((((data["min_roll_std_10"]) * 2.0)) + (((data["min_last_10000"]) * (((data["max_first_50000"]) * (((((data["min_roll_std_10"]) * 2.0)) * 2.0)))))))) * (data["max_first_10000"]))))))) * 2.0)) +
            0.0399609283*activation1(((data["av_change_abs_roll_std_10"]) * ((((((((((data["min_last_50000"]) + (((data["trend"]) * (data["av_change_abs_roll_mean_10"]))))/2.0)) + (data["max_to_min"]))/2.0)) + (((data["min_last_50000"]) - (((data["min_last_50000"]) * (((data["min_last_50000"]) * (data["q05_roll_mean_1000"]))))))))/2.0)))) +
            0.0391638987*activation1(((((data["trend"]) - ((((data["min_roll_std_100"]) + (data["q001"]))/2.0)))) * (((activation1((activation1((((((data["classic_sta_lta3_mean"]) + ((((data["classic_sta_lta3_mean"]) + ((((data["kurt"]) + (data["q95_roll_mean_100"]))/2.0)))/2.0)))) * 2.0)))))) * ((-1.0*((data["min_roll_std_100"])))))))) +
            0.0398749746*activation1(((data["ave10"]) * ((-1.0*((((data["av_change_abs_roll_mean_10"]) * (((data["av_change_abs_roll_mean_100"]) + ((-1.0*(((((data["av_change_abs_roll_mean_10"]) + (((((data["classic_sta_lta2_mean"]) * 2.0)) + (((data["mean_change_rate_last_10000"]) * (data["av_change_rate_roll_mean_100"]))))))/2.0)))))))))))))) +
            0.0385778472*activation1((-1.0*(((((activation1((((((((data["trend"]) + (((data["min_roll_std_10"]) + (data["Moving_average_6000_mean"]))))) * 2.0)) * 2.0)))) + (((((-1.0*((((data["q95_roll_std_100"]) * (data["ave10"])))))) + (((data["mean_change_rate_first_50000"]) * (data["av_change_abs_roll_mean_10"]))))/2.0)))/2.0))))) +
            0.0399921872*activation1(((((((((data["min_roll_std_10"]) * ((((data["ave_roll_std_10"]) + (data["avg_last_50000"]))/2.0)))) + (((((((((data["avg_last_10000"]) + (data["min_roll_std_10"]))/2.0)) + (data["min_roll_std_10"]))/2.0)) - (data["ave_roll_std_10"]))))/2.0)) + (((((data["q95_roll_std_100"]) - (data["ave_roll_std_10"]))) * 2.0)))/2.0)) +
            0.0399921872*activation1((((((-1.0*((((data["mean_change_rate_first_10000"]) * 2.0))))) * ((((((((((data["av_change_rate_roll_std_10"]) * (data["avg_last_10000"]))) + (((data["min_last_10000"]) * (data["av_change_rate_roll_std_10"]))))) - (((data["avg_first_10000"]) / 2.0)))) + (data["av_change_rate_roll_std_1000"]))/2.0)))) * (data["av_change_abs_roll_std_1000"]))) +
            0.0399453007*activation1(((data["av_change_abs_roll_std_1000"]) * (((data["avg_last_10000"]) * (((((((data["avg_last_10000"]) * (((data["avg_last_10000"]) * (((((data["med"]) * (((data["q001"]) - (data["classic_sta_lta1_mean"]))))) - (data["min_roll_std_10"]))))))) - (data["max_roll_mean_10"]))) - (data["min_roll_std_10"]))))))) +
            0.0398749746*activation1(activation1((activation1((((((((((((-1.0*((data["av_change_abs_roll_std_100"])))) + ((-1.0*((data["mean_change_rate_first_50000"])))))/2.0)) / 2.0)) - (((data["mean_change_abs"]) * (((((((data["avg_last_50000"]) * (data["q99_roll_mean_10"]))) - (data["mean_change_rate_first_50000"]))) - (data["std_last_10000"]))))))) * 2.0)))))) +
            0.0363899209*activation1(((data["max_last_10000"]) * (((((activation1((((((((data["av_change_rate_roll_mean_100"]) + (((data["min_roll_std_1000"]) * 2.0)))) * 2.0)) * 2.0)))) + (((data["av_change_abs_roll_std_100"]) + ((-1.0*((data["mean_change_abs"])))))))) + (((((data["av_change_rate_roll_mean_10"]) * (data["max_last_50000"]))) * 2.0)))))) +
            0.0399999991*activation1(((data["min_roll_std_100"]) * (((activation1((data["min_last_10000"]))) + (((((activation1((data["std_first_10000"]))) - ((((activation1((data["min_last_10000"]))) + (((data["exp_Moving_average_30000_mean"]) * (data["min_last_10000"]))))/2.0)))) * ((((((data["avg_last_50000"]) + (data["mean_change_abs"]))/2.0)) * 2.0)))))))) +
            0.0351396762*activation1(activation1((((((data["av_change_abs_roll_std_1000"]) * (data["mean_change_rate_last_10000"]))) + (((data["med"]) * (activation1((((data["av_change_rate_roll_std_100"]) - (((((data["avg_last_10000"]) - (((data["mean_change_rate_first_10000"]) - (((data["avg_last_10000"]) * (data["mean_change_rate_last_10000"]))))))) - (data["av_change_abs_roll_std_1000"]))))))))))))) +
            0.0399921872*activation1((((activation1((((((((((data["min_roll_std_10"]) + (((data["mean_change_rate_first_50000"]) + (((((data["mean_change_rate_first_50000"]) + (data["mean_change_rate_first_50000"]))) * (data["classic_sta_lta4_mean"]))))))) * (data["av_change_rate_roll_mean_1000"]))) * 2.0)) * 2.0)))) + (((data["av_change_abs_roll_mean_10"]) * (activation1((data["med"]))))))/2.0)) +
            0.0379214697*activation1((-1.0*((((((data["max_to_min_diff"]) * (((((((((((((data["min_last_10000"]) - (data["med"]))) - (data["mean_change_rate_first_10000"]))) + (data["q05_roll_std_100"]))) + (data["q01_roll_std_1000"]))) * 2.0)) * 2.0)))) * ((-1.0*((((data["mean_change_rate_first_10000"]) * (data["min_last_10000"]))))))))))) +
            0.0399218611*activation1(((data["min_first_10000"]) * (activation1((((((((((activation1((((data["max_to_min"]) * 2.0)))) + (((((((data["min_first_50000"]) * 2.0)) + (((activation1((activation1((data["mean_change_rate_last_10000"]))))) + (data["min_roll_mean_10"]))))) * 2.0)))) * 2.0)) * 2.0)) * 2.0)))))) +
            0.0328423530*activation1((-1.0*((((activation1((((data["kurt"]) * (((((((data["mean_change_rate_last_50000"]) * 2.0)) * 2.0)) * (data["skew"]))))))) + (activation1((((data["skew"]) * (((((data["mean_change_rate_last_50000"]) * 2.0)) * (data["skew"])))))))))))) +
            0.0346083194*activation1((((((data["av_change_abs_roll_std_10"]) / 2.0)) + ((-1.0*((((activation1((data["q05_roll_mean_1000"]))) + (activation1((((((data["min_roll_std_100"]) - (data["mean_change_rate_last_50000"]))) - (((((data["q95_roll_mean_1000"]) - (data["av_change_abs_roll_std_10"]))) - (((data["av_change_abs_roll_mean_1000"]) - (data["mean_change_rate_last_50000"])))))))))))))))/2.0)) +
            0.0395155288*activation1(((data["avg_last_50000"]) * ((-1.0*(((((activation1((activation1((((((((((((data["av_change_rate_roll_mean_100"]) * ((-1.0*((data["max_to_min"])))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)))))) + ((((((data["av_change_rate_roll_std_10"]) * (data["avg_last_10000"]))) + (data["av_change_rate_roll_std_10"]))/2.0)))/2.0))))))) +
            0.0316311792*activation1(((data["max_first_50000"]) - (((data["classic_sta_lta3_mean"]) * (((((data["max_to_min_diff"]) + (data["mean_change_abs"]))) - (((data["trend"]) * (((((data["mean_change_abs"]) - (((activation1((data["max_to_min_diff"]))) * 2.0)))) - (data["av_change_abs_roll_mean_100"]))))))))))) +
            0.0399609283*activation1(((data["trend"]) * (((((((data["min_roll_std_1000"]) * (data["std_last_10000"]))) * ((((data["av_change_abs_roll_mean_10"]) + (((data["std_last_50000"]) - ((((((data["av_change_rate_roll_mean_10"]) * (data["q05_roll_std_1000"]))) + (data["min_roll_std_1000"]))/2.0)))))/2.0)))) + (((data["min_roll_std_1000"]) - (data["q05_roll_std_1000"]))))))) +
            0.0362180099*activation1((((activation1((((((((((((data["q01_roll_std_100"]) - (data["mean_change_rate_last_10000"]))) - ((((data["av_change_abs_roll_mean_10"]) + (data["abs_max_roll_mean_100"]))/2.0)))) - (0.2525250018))) * (data["min_roll_std_10"]))) * (73.0))))) + (((data["min_roll_mean_1000"]) + (data["abs_max_roll_mean_100"]))))/2.0)) +
            0.0399999991*activation1((((activation1((((73.0) * (((((73.0) * (data["av_change_abs_roll_std_100"]))) - (data["av_change_abs_roll_mean_100"]))))))) + (activation1((((((((((data["av_change_rate_roll_mean_100"]) - (data["av_change_abs_roll_std_100"]))) - (data["av_change_abs_roll_std_100"]))) - (data["std_first_50000"]))) - (data["av_change_abs_roll_std_100"]))))))/2.0)) +
            0.0374213718*activation1((((-1.0*((((data["q05"]) * (((data["max_last_10000"]) * (((((data["min_roll_std_10"]) - (data["std_last_10000"]))) - (data["std_last_10000"])))))))))) + (((data["max_last_50000"]) * (((data["mean_change_rate_last_10000"]) - (activation1((((((data["av_change_rate_roll_std_10"]) * 2.0)) * 2.0)))))))))) +
            0.0331470966*activation1(((((((((data["abs_max_roll_std_1000"]) * 2.0)) * 2.0)) * 2.0)) * (((((data["mean_change_rate_last_50000"]) * ((((((data["abs_max_roll_std_1000"]) * (((data["mean_change_abs"]) * 2.0)))) + ((((data["std_first_50000"]) + (data["mean_change_rate_last_50000"]))/2.0)))/2.0)))) + (((data["mean_change_abs"]) * (data["av_change_rate_roll_std_1000"]))))))) +
            0.0396796241*activation1(((activation1((activation1((((((data["avg_first_10000"]) * (((((data["Moving_average_3000_mean"]) + (data["skew"]))) * 2.0)))) + (activation1(((-1.0*((((((data["ave_roll_mean_100"]) + (((data["av_change_abs_roll_std_1000"]) + (((data["avg_first_10000"]) * 2.0)))))) * 2.0))))))))))))) / 2.0)) +
            0.0399765596*activation1(((activation1((((((data["av_change_abs_roll_mean_10"]) + (data["exp_Moving_average_300_mean"]))) * (((((data["classic_sta_lta4_mean"]) + (data["av_change_abs_roll_mean_10"]))) + (((((data["av_change_abs_roll_mean_10"]) + (((activation1((((activation1((data["avg_last_50000"]))) * 2.0)))) * 2.0)))) * (data["av_change_abs_roll_mean_10"]))))))))) / 2.0)) +
            0.0396014862*activation1((((((data["max_last_50000"]) + (((((activation1((activation1((((((activation1((activation1(((((data["q95"]) + (data["mean_change_abs"]))/2.0)))))) + (((data["av_change_abs_roll_std_100"]) * 2.0)))) * 2.0)))))) * 2.0)) - (data["av_change_abs_roll_std_100"]))))/2.0)) * (data["classic_sta_lta4_mean"]))) +
            0.0317015015*activation1(((activation1((((((data["avg_last_10000"]) - (((data["iqr"]) - (((((data["avg_last_10000"]) - (((data["iqr"]) * (data["av_change_rate_roll_std_100"]))))) - (((((((((data["min_roll_std_100"]) * (data["av_change_rate_roll_std_100"]))) * 2.0)) * 2.0)) * 2.0)))))))) * 2.0)))) / 2.0)) +
            0.0399296731*activation1((-1.0*((((data["q05_roll_std_1000"]) * (((((((((data["avg_first_10000"]) + (data["mean_change_abs"]))/2.0)) * (data["std_last_50000"]))) + (((((((data["avg_last_10000"]) + (((data["mean_change_rate_first_10000"]) * (data["mean_change_abs"]))))/2.0)) + (data["max_to_min_diff"]))/2.0)))/2.0))))))) +
            0.0377104878*activation1((((activation1((activation1((((((((data["count_big"]) + (((data["min_last_10000"]) * (data["min_first_50000"]))))) * 2.0)) * 2.0)))))) + ((((data["count_big"]) + ((-1.0*((((data["max_roll_mean_100"]) - (activation1((((data["min_last_10000"]) * (data["avg_last_10000"])))))))))))/2.0)))/2.0)) +
            0.0309357308*activation1(((activation1((((((((data["max_to_min"]) * (data["classic_sta_lta2_mean"]))) + (((data["classic_sta_lta2_mean"]) * (((data["max_to_min"]) * (data["classic_sta_lta2_mean"]))))))) * 2.0)))) / 2.0)) +
            0.0355303772*activation1(((((activation1((activation1((((((((((data["av_change_abs_roll_std_10"]) - (activation1((((data["skew"]) * 2.0)))))) * (data["mean_change_rate_first_10000"]))) * (data["mean_change_rate_first_10000"]))) * (((data["mean_change_rate_first_10000"]) + (data["av_change_abs_roll_std_10"]))))))))) * (data["mean_change_rate_first_10000"]))) * (data["mean_change_rate_first_10000"]))) +
            0.0399843715*activation1(((data["av_change_abs_roll_std_1000"]) * (((((((data["trend"]) + (activation1((((data["min_roll_std_1000"]) * ((((8.0)) + (((data["min_roll_std_1000"]) * ((8.0)))))))))))/2.0)) + ((-1.0*((((data["q01_roll_std_1000"]) + (((data["q01_roll_std_1000"]) * (data["mean_change_rate_first_50000"])))))))))/2.0)))) +
            0.0398437195*activation1((-1.0*((((data["std_first_10000"]) * ((((((data["q99"]) / 2.0)) + (((data["skew"]) + (((((data["std_first_10000"]) * 2.0)) * (((data["std_first_10000"]) * (((data["std_first_10000"]) * (((data["std_first_10000"]) - (((data["MA_700MA_std_mean"]) * 2.0)))))))))))))/2.0))))))) +
            0.0399843715*activation1((-1.0*((((data["mean_change_abs"]) * ((((((data["av_change_abs_roll_mean_1000"]) + ((((((((data["abs_max_roll_mean_10"]) + (((((data["min_roll_mean_100"]) * (data["av_change_abs_roll_mean_1000"]))) * (data["min_roll_std_10"]))))) - ((((data["min_roll_std_10"]) + (data["av_change_abs_roll_std_100"]))/2.0)))) + (data["skew"]))/2.0)))/2.0)) / 2.0))))))) +
            0.0295057632*activation1(((((((((((((((data["q99_roll_mean_100"]) + (((data["q99_roll_mean_100"]) + (data["q95_roll_mean_10"]))))) * (((data["q05_roll_std_100"]) + ((((((data["av_change_abs_roll_std_1000"]) - (data["min_roll_std_100"]))) + (data["av_change_abs_roll_mean_100"]))/2.0)))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * (data["av_change_abs_roll_std_1000"]))) +
            0.0346864611*activation1(((activation1((((((((data["mean_change_rate"]) * (((data["mean_change_rate_first_10000"]) + (((data["mean_change_rate_first_10000"]) + (data["mean_change_rate_last_50000"]))))))) + (((data["q01_roll_std_100"]) * (((((data["mean_change_rate_first_10000"]) * (((data["q99_roll_std_10"]) * 2.0)))) + (data["mean_change_rate_first_10000"]))))))) * 2.0)))) / 2.0)) +
            0.0397265106*activation1(activation1((((((data["min_roll_std_100"]) * ((-1.0*((((data["av_change_abs_roll_mean_100"]) * ((((data["q05_roll_mean_100"]) + (data["mean_change_rate_first_10000"]))/2.0))))))))) - (((((((-1.0*((data["classic_sta_lta4_mean"])))) / 2.0)) + ((((data["av_change_rate_roll_mean_100"]) + ((((data["Moving_average_6000_mean"]) + (data["mean_change_rate_first_10000"]))/2.0)))/2.0)))/2.0)))))) +
            0.0399687439*activation1(((activation1((((data["min_first_50000"]) * (((((((((activation1((((((data["q01_roll_std_1000"]) + (activation1(((-1.0*((((((data["skew"]) + (data["classic_sta_lta2_mean"]))) + (data["q95_roll_std_10"])))))))))) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)))))) / 2.0)) +
            0.0365696438*activation1(((activation1((((((((((((((((data["mean_change_rate_last_10000"]) * (((data["ave_roll_std_10"]) - (((data["q95"]) - ((((((data["abs_mean"]) + (((data["iqr"]) / 2.0)))/2.0)) / 2.0)))))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) / 2.0)) +
            0.0393358096*activation1(((((data["max_roll_std_10"]) * (((((data["q95_roll_mean_100"]) + (((data["q95_roll_mean_100"]) + (((data["q05_roll_mean_1000"]) * (data["classic_sta_lta2_mean"]))))))) * (((data["max_roll_std_10"]) * (((((data["skew"]) * (((data["q05_roll_mean_100"]) + (data["classic_sta_lta2_mean"]))))) * 2.0)))))))) * 2.0)) +
            0.0381871462*activation1((((activation1((((((((data["q05_roll_mean_10"]) - (((data["q05"]) + (data["ave_roll_std_100"]))))) * (73.0))) * (73.0))))) + (((((((data["abs_q05"]) * (((data["std_roll_mean_1000"]) * (data["av_change_rate_roll_std_10"]))))) * 2.0)) - (data["q05"]))))/2.0)) +
            0.0383043550*activation1(((data["max_last_50000"]) * (((((data["av_change_rate_roll_std_100"]) * 2.0)) * (activation1((((((((((((((activation1((((data["mean_change_abs"]) - (data["classic_sta_lta3_mean"]))))) - (data["classic_sta_lta3_mean"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))))))) +
            0.0397812054*activation1(activation1((activation1((((data["min_roll_mean_100"]) * (((((data["av_change_rate_roll_mean_1000"]) - (data["min_roll_mean_100"]))) - (((((((data["min_last_50000"]) * (((data["av_change_rate_roll_mean_1000"]) - (activation1((((data["min_roll_std_10"]) - (((data["min_roll_mean_100"]) * 2.0)))))))))) * 2.0)) * 2.0)))))))))) +
            0.0399921872*activation1((-1.0*((((((activation1((((((((((((((((((data["mean_change_abs"]) * 2.0)) - ((((((-1.0*((data["mean_change_rate_last_50000"])))) * 2.0)) - (data["trend"]))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) / 2.0)) / 2.0))))) +
            0.0329673775*activation1(activation1(((-1.0*((((data["std_first_10000"]) * (((((((((data["av_change_abs_roll_mean_10"]) + (data["max_last_10000"]))) * (((((data["max_last_10000"]) * (0.8873239756))) + (data["classic_sta_lta2_mean"]))))) * 2.0)) * (data["max_last_10000"])))))))))) +
            0.0367962494*activation1(((data["avg_last_10000"]) * (((((data["std_roll_mean_1000"]) * ((-1.0*(((((((((data["mean_change_rate_first_50000"]) - (((data["mean_change_abs"]) * (data["max_last_10000"]))))) + (data["max_to_min_diff"]))/2.0)) - (((data["max_last_10000"]) * (((data["max_to_min"]) * (data["q05_roll_mean_100"])))))))))))) * 2.0)))) +
            0.0399921872*activation1(((data["av_change_abs_roll_mean_10"]) * (((((data["av_change_abs_roll_std_1000"]) - ((((data["av_change_abs_roll_mean_10"]) + ((((data["min_roll_std_10"]) + (((((((data["av_change_rate_roll_mean_10"]) * (data["av_change_abs_roll_std_10"]))) + (data["av_change_abs_roll_mean_10"]))) * (data["av_change_abs_roll_mean_10"]))))/2.0)))/2.0)))) * (data["min_roll_std_10"]))))) +
            0.0395311601*activation1(((activation1(((-1.0*((((data["std_first_10000"]) + (data["min_first_10000"])))))))) + (activation1((((data["max_roll_std_10"]) * (((data["q95_roll_mean_100"]) * ((((((((data["mean_change_abs"]) + (data["max_roll_std_10"]))/2.0)) - ((-1.0*((data["min_first_10000"])))))) + (data["mean_change_rate_last_50000"]))))))))))) +
            0.0399296731*activation1(((((data["min_roll_std_10"]) + (((data["min_roll_std_10"]) * (data["med"]))))) * ((((((data["classic_sta_lta1_mean"]) + (data["min_roll_mean_1000"]))/2.0)) - (((data["q05_roll_std_10"]) * ((((((data["med"]) * ((((data["classic_sta_lta1_mean"]) + (data["min_roll_mean_1000"]))/2.0)))) + (data["med"]))/2.0)))))))) +
            0.0399453007*activation1(activation1((((((((activation1((data["min_roll_std_10"]))) * (data["std_roll_mean_1000"]))) * 2.0)) + ((((((((((((data["max_to_min_diff"]) * 2.0)) * (data["std_last_50000"]))) + (data["min_roll_std_10"]))/2.0)) / 2.0)) + (((data["min_roll_std_10"]) * (activation1((data["max_to_min_diff"]))))))))))) +
            0.0394999012*activation1(((data["mean_change_rate_first_10000"]) * (((data["min_first_10000"]) * ((((((((data["min_roll_std_10"]) * (((data["av_change_abs_roll_std_100"]) + (data["av_change_abs_roll_std_100"]))))) + ((((data["mean_change_rate_first_10000"]) + (((data["min_roll_std_10"]) * (data["mean_change_rate_first_10000"]))))/2.0)))/2.0)) + (((data["min_roll_std_10"]) - (data["av_change_abs_roll_std_100"]))))))))) +
            0.0399687439*activation1(((data["max_first_10000"]) * (((((data["std_first_10000"]) * (((data["max_first_10000"]) * (((data["std_first_10000"]) * (((activation1((((data["std_first_10000"]) * (data["av_change_abs_roll_mean_10"]))))) + (data["std_roll_mean_100"]))))))))) - (((((data["av_change_abs_roll_mean_10"]) * (data["min_roll_std_100"]))) / 2.0)))))) +
            0.0397499502*activation1(((data["trend"]) * ((((((data["std_first_50000"]) + (activation1((activation1((((data["av_change_rate_roll_mean_10"]) + (((((((data["av_change_rate_roll_mean_1000"]) + ((((data["min_roll_std_100"]) + (((data["Hilbert_mean"]) + (data["av_change_rate_roll_mean_10"]))))/2.0)))) * 2.0)) * 2.0)))))))))/2.0)) * (data["min_roll_std_100"]))))) +
            0.0274897423*activation1((((activation1((((((((((((((data["q95_roll_std_100"]) - (data["ave_roll_std_10"]))) - (data["abs_max_roll_mean_10"]))) - (data["max_last_50000"]))) - (((data["min_first_50000"]) + (data["ave_roll_std_10"]))))) * 2.0)) * 2.0)))) + (((((data["q95_roll_std_100"]) - (data["ave_roll_std_10"]))) * 2.0)))/2.0)) +
            0.0399531126*activation1(((((data["mean_change_rate_first_10000"]) * (activation1((((((((((data["av_change_abs_roll_mean_10"]) + (data["mean_change_rate_last_10000"]))/2.0)) * (((data["avg_first_10000"]) - (activation1((data["av_change_abs_roll_mean_100"]))))))) + (activation1((activation1((data["avg_first_10000"]))))))/2.0)))))) * ((((-1.0*((data["mean_change_abs"])))) * 2.0)))) +
            0.0340613388*activation1(((data["skew"]) * (((data["max_first_50000"]) * (((((((((activation1((((((((data["iqr"]) * 2.0)) - (data["av_change_abs_roll_mean_10"]))) - (data["std_last_10000"]))))) + (data["av_change_abs_roll_mean_10"]))) - (((data["av_change_rate_roll_mean_10"]) * (data["avg_first_50000"]))))) * 2.0)) * 2.0)))))) +
            0.0399921872*activation1((((((data["min_roll_std_10"]) + (((data["min_roll_std_10"]) * (data["avg_first_10000"]))))/2.0)) * (activation1((((73.0) * (((data["avg_first_10000"]) * (((data["min_roll_std_10"]) + (((73.0) * (((73.0) * ((-1.0*((data["av_change_abs_roll_std_1000"])))))))))))))))))) +
            0.0356475860*activation1(activation1((((data["av_change_abs_roll_mean_1000"]) * ((((((((((((((data["avg_last_50000"]) + (data["kurt"]))) + (0.2783510089))/2.0)) + (data["av_change_rate_roll_std_1000"]))) + (0.2783510089))) + (data["kurt"]))) * ((((activation1((data["avg_first_10000"]))) + (data["mean_change_rate_first_50000"]))/2.0)))))))) +
            0.0329986326*activation1((((((((((data["kurt"]) + (data["q05_roll_mean_10"]))) * (data["min_roll_mean_10"]))) + (activation1((((((data["q05_roll_mean_10"]) - (((((data["max_to_min"]) - (data["kurt"]))) + (((data["q05_roll_mean_10"]) * (data["max_to_min"]))))))) * 2.0)))))/2.0)) / 2.0)) +
            0.0399999991*activation1(((data["std_roll_mean_1000"]) * (((data["med"]) * ((((-3.0) + (((((data["avg_first_10000"]) * 2.0)) * (((data["mean_change_rate_first_10000"]) - (((((((data["std_roll_mean_1000"]) / 2.0)) * (((data["std_roll_mean_1000"]) * (-3.0))))) * (data["abs_max_roll_mean_1000"]))))))))/2.0)))))) +
            0.0303496774*activation1((-1.0*((((activation1((activation1((((((((data["classic_sta_lta2_mean"]) + (((data["av_change_abs_roll_std_10"]) + (((data["av_change_abs_roll_mean_10"]) + (data["classic_sta_lta1_mean"]))))))) * (data["abs_q05"]))) * (3.6923100948))))))) / 2.0))))) +
            0.0399687439*activation1((((-1.0*((data["max_first_50000"])))) * (((((data["std_roll_mean_1000"]) + (((data["av_change_abs_roll_std_10"]) * (((data["ave_roll_mean_10"]) + (((data["med"]) + (1.0))))))))) * (((data["avg_last_50000"]) + (((data["med"]) + (2.0769200325))))))))) +
            0.0399843715*activation1(((activation1((((data["av_change_rate_roll_mean_100"]) * (((activation1((((((((((data["av_change_abs_roll_mean_100"]) * 2.0)) + (data["mean_change_rate_first_50000"]))) * 2.0)) * 2.0)))) - (((data["av_change_abs_roll_mean_100"]) * (((((data["av_change_rate_roll_std_1000"]) - (data["classic_sta_lta2_mean"]))) - (data["classic_sta_lta2_mean"]))))))))))) / 2.0)) +
            0.0399999991*activation1(((data["mean_change_abs"]) * ((((data["av_change_abs_roll_std_100"]) + (activation1((((((((((((((((((data["min_first_10000"]) - (data["av_change_rate_roll_std_10"]))) - (((data["av_change_abs_roll_std_100"]) * (3.6923100948))))) * 2.0)) * 2.0)) * 2.0)) + (data["mean_change_abs"]))) * 2.0)) * 2.0)))))/2.0)))) +
            0.0285446383*activation1(((activation1((((((73.0) * (data["mean_change_rate_first_10000"]))) + (((((((((data["trend"]) + (((data["av_change_abs_roll_mean_10"]) + (data["min_last_50000"]))))) + (data["mean_change_rate_first_10000"]))) * 2.0)) * 2.0)))))) + ((-1.0*((activation1((data["mean_change_rate_first_10000"])))))))) +
            0.0399687439*activation1(((((activation1((((((((((((((((((((activation1(((((((data["kurt"]) + (data["classic_sta_lta4_mean"]))/2.0)) - (data["max_first_10000"]))))) + (data["skew"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) / 2.0)) / 2.0)) +
            0.0398593470*activation1(((activation1(((-1.0*((activation1((((data["avg_first_10000"]) * (((data["min_roll_std_10"]) - (((((data["mean_change_rate_last_10000"]) * (data["min_roll_std_10"]))) + ((((data["mean_change_abs"]) + ((((((data["ave_roll_mean_100"]) + (data["avg_first_10000"]))/2.0)) + (data["skew"]))))/2.0))))))))))))))) / 2.0)) +
            0.0399296731*activation1(((activation1((activation1((activation1((activation1((((((((data["mean_change_abs"]) * ((((data["av_change_abs_roll_mean_100"]) + (0.7446810007))/2.0)))) - (((activation1((data["skew"]))) + ((((data["trend"]) + (0.7446810007))/2.0)))))) * 2.0)))))))))) / 2.0)) +
            0.0387341268*activation1((((((((data["min_roll_std_100"]) * (activation1((((((((data["av_change_rate_roll_std_100"]) * 2.0)) + (((data["abs_q05"]) + (activation1((data["iqr"]))))))) * 2.0)))))) + (activation1((((activation1((((data["iqr"]) * 2.0)))) - (data["min_roll_std_10"]))))))/2.0)) / 2.0)) +
            0.0384997055*activation1(((data["av_change_abs_roll_mean_1000"]) * (((activation1((((((activation1((data["std_first_50000"]))) + ((((-1.0*((data["max_to_min"])))) * (data["std_first_10000"]))))) + (((data["av_change_abs_roll_mean_1000"]) * (data["min_roll_mean_1000"]))))))) + ((((-1.0*((data["min_roll_std_10"])))) * (data["std_first_10000"]))))))) +
            0.0384918936*activation1(((activation1((activation1((((((73.0) * (((73.0) * (((((data["q95_roll_std_100"]) + (data["std_roll_mean_1000"]))) * 2.0)))))) * (((data["std_first_50000"]) + (((activation1((activation1((data["max_roll_std_1000"]))))) * (data["kurt"]))))))))))) / 2.0)) +
            0.0343192033*activation1(((((-1.0) - (activation1((((((((0.8873239756) + (data["mean_change_abs"]))) - (data["Hann_window_mean"]))) * (((((((((data["Hann_window_mean"]) - (data["min_last_10000"]))) * 2.0)) * 2.0)) + (data["av_change_rate_roll_std_100"]))))))))) / 2.0)) +
            0.0372416489*activation1(((activation1((((((data["classic_sta_lta2_mean"]) + (((data["q95_roll_mean_1000"]) * (data["classic_sta_lta2_mean"]))))) + ((((-1.0*((((((data["std_last_10000"]) + (data["MA_400MA_BB_low_mean"]))) + (activation1((((data["min_roll_std_100"]) + (data["classic_sta_lta2_mean"])))))))))) * (data["avg_last_10000"]))))))) / 2.0)) +
            0.0377808139*activation1(activation1((((((data["mean_change_rate_last_10000"]) * (((((((data["mean_change_rate_first_10000"]) + ((((data["min_roll_std_10"]) + ((-1.0*((-3.0)))))/2.0)))/2.0)) + (data["Moving_average_1500_mean"]))/2.0)))) * (activation1((((((data["mean_change_rate_last_10000"]) + (((data["av_change_abs_roll_std_1000"]) + (1.9019600153))))) * 2.0)))))))) +
            0.0369056463*activation1(activation1((activation1((((((data["max_first_50000"]) * (data["max_first_50000"]))) * ((((((((data["max_first_50000"]) + (data["kurt"]))/2.0)) - (data["avg_last_50000"]))) - (((activation1((((data["mean_change_rate_first_10000"]) * 2.0)))) - (((data["kurt"]) - (0.5555559993))))))))))))))

In [15]:
def GPI_activation2(data):
    return (5.591662 +
            0.0399999991*activation2(((data["mean_change_rate_first_10000"]) - (((73.0) * ((((((7.0) * (((((((((data["q95"]) + (((0.3183098733) + (data["q05_roll_std_100"]))))) * 2.0)) * 2.0)) + (((data["q95_roll_std_10"]) + (data["iqr"]))))))) + (data["iqr"]))/2.0)))))) +
            0.0399999991*activation2(((((3.6923100948) + (data["Moving_average_6000_mean"]))) * (((73.0) * (((data["MA_400MA_BB_low_mean"]) - (((((data["Moving_average_6000_mean"]) * (data["q05_roll_std_10"]))) + (((data["iqr"]) + (((data["q05_roll_std_10"]) * (((3.6923100948) * 2.0)))))))))))))) +
            0.0399999991*activation2(((((((((((((((((((((((data["q05"]) * 2.0)) * 2.0)) - (data["iqr"]))) - (data["max_roll_std_1000"]))) * 2.0)) * 2.0)) * 2.0)) - (data["iqr"]))) - (3.6923100948))) - (((data["q05_roll_std_10"]) * (73.0))))) * 2.0)) +
            0.0399999991*activation2(((((((((((((((((((data["q05_roll_std_10"]) * (((((-3.0) * 2.0)) * 2.0)))) - (((data["iqr"]) + (data["max_roll_std_1000"]))))) * 2.0)) * 2.0)) * 2.0)) - (data["max_roll_mean_100"]))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*activation2(((73.0) * ((((data["max_first_50000"]) + ((((data["max_first_50000"]) + (((73.0) * (((data["q05"]) - (((data["q05_roll_std_100"]) + (((0.5555559993) + (((data["mad"]) + (data["q95"]))))))))))))/2.0)))/2.0)))) +
            0.0399999991*activation2(((((((data["q05_roll_std_10"]) * (73.0))) * 2.0)) - ((((((((data["q05_roll_std_10"]) * (73.0))) + (((((((((data["iqr"]) + (((data["MA_1000MA_std_mean"]) + (data["abs_max_roll_mean_10"]))))) * 2.0)) * 2.0)) * 2.0)))/2.0)) * (73.0))))) +
            0.0399999991*activation2(((((data["mean_change_abs"]) - (((((data["q95"]) + (((data["q95"]) + (((((data["q05_roll_std_100"]) / 2.0)) + (((((((0.3183098733) * 2.0)) - (data["q05"]))) - (data["q05"]))))))))) * (73.0))))) * 2.0)) +
            0.0399999991*activation2((-1.0*(((((((7.48210620880126953)) * 2.0)) + (((((data["iqr"]) + ((((-1.0*((data["avg_first_10000"])))) * 2.0)))) + (((73.0) * (((data["iqr"]) + (((((data["q05_roll_std_10"]) * 2.0)) * 2.0))))))))))))) +
            0.0399999991*activation2((((((data["q01_roll_std_10"]) + (((((data["q05_roll_mean_10"]) - (((((data["q05_roll_std_100"]) - (((data["q05"]) - (0.3026320040))))) + (((((((0.5434780121) + (0.3026320040))/2.0)) + (data["iqr"]))/2.0)))))) * (73.0))))/2.0)) * (73.0))) +
            0.0399999991*activation2(((((data["trend"]) + (((data["q95_roll_std_10"]) + (data["trend"]))))) - (((73.0) * (((((((data["ave_roll_std_10"]) + (((((((data["q95_roll_mean_10"]) + (data["q05_roll_std_1000"]))/2.0)) + (data["q05_roll_std_1000"]))/2.0)))/2.0)) + ((((0.5434780121) + (data["q95_roll_mean_10"]))/2.0)))/2.0)))))) +
            0.0399999991*activation2(((73.0) - (((((73.0) * 2.0)) * (((73.0) * ((((((((((data["q05_roll_std_1000"]) / 2.0)) + (data["iqr"]))/2.0)) + (data["q05_roll_std_10"]))) + (((0.2183910012) + (data["q95"]))))))))))) +
            0.0399999991*activation2(((73.0) * (((73.0) * (((((73.0) * (((data["q05_roll_mean_10"]) - (((((data["q05_roll_std_100"]) + (data["q95"]))) + (0.5434780121))))))) + (data["q05_roll_std_100"]))))))) +
            0.0399999991*activation2((-1.0*((((73.0) * (((73.0) * (((73.0) * (((data["q05_roll_std_100"]) + ((((data["q95_roll_mean_10"]) + ((((data["Moving_average_3000_mean"]) + (((((data["iqr"]) + (0.9756100178))) + (data["abs_mean"]))))/2.0)))/2.0))))))))))))) +
            0.0399999991*activation2(((73.0) * (((((data["classic_sta_lta3_mean"]) + (data["mad"]))) + (((((((73.0) * (((data["q05_roll_mean_10"]) - (((data["q05_roll_std_100"]) + (((data["mad"]) + (0.5555559993))))))))) - (data["iqr"]))) - (data["iqr"]))))))) +
            0.0399999991*activation2(((((activation2((data["q95_roll_std_10"]))) + (((((((data["q05_roll_std_10"]) + ((((((0.2439019978) + (data["ave10"]))/2.0)) / 2.0)))) * 2.0)) * 2.0)))) * (((data["q05_roll_std_10"]) - (((73.0) * (((73.0) * (73.0))))))))) +
            0.0399999991*activation2(((3.6923100948) * (((73.0) * (((3.6923100948) * ((((((-1.0*((data["q99_roll_mean_1000"])))) * (data["avg_last_50000"]))) - (((data["q05_roll_std_10"]) * (((3.6923100948) + (((3.6923100948) + (data["abs_max_roll_mean_100"]))))))))))))))) +
            0.0399999991*activation2(((((((data["Moving_average_1500_mean"]) + (((((((((data["MA_400MA_BB_low_mean"]) + ((((-1.0*((data["q05_roll_std_1000"])))) - (data["iqr"]))))) * (73.0))) - (data["min_roll_std_1000"]))) - (data["q05_roll_std_1000"]))))) * (73.0))) * (73.0))) +
            0.0399999991*activation2(((73.0) - (((73.0) * (((73.0) * (((data["q05_roll_std_10"]) + (activation2(((((((data["exp_Moving_average_3000_mean"]) + (((data["iqr"]) + (activation2((((data["q05_roll_std_100"]) + (data["exp_Moving_average_30000_mean"]))))))))/2.0)) / 2.0)))))))))))) +
            0.0399999991*activation2(((73.0) * (((73.0) * (((data["q05_roll_mean_10"]) - (((activation2((activation2((activation2((activation2((activation2((data["q05_roll_std_100"]))))))))))) + ((((data["q95_roll_std_10"]) + (activation2((2.8437500000))))/2.0)))))))))) +
            0.0399999991*activation2(((((((data["iqr"]) / 2.0)) + (((data["mean"]) + (((data["q05_roll_std_100"]) + (((data["q05_roll_std_10"]) * 2.0)))))))) * ((-1.0*((((73.0) - (((((((((data["mean"]) * 2.0)) * 2.0)) * 2.0)) * 2.0))))))))) +
            0.0399999991*activation2(((((((((0.5217390060) - (((((data["q05_roll_std_10"]) * ((-1.0*((7.0)))))) + (data["exp_Moving_average_3000_mean"]))))) * ((-1.0*((73.0)))))) - (((data["classic_sta_lta4_mean"]) * (7.0))))) - ((((7.0) + (data["classic_sta_lta4_mean"]))/2.0)))) +
            0.0399999991*activation2(((((((((((((((((((activation2(((-1.0*((((data["q05_roll_mean_100"]) + (0.2183910012)))))))) - ((((data["q05_roll_std_100"]) + (((data["q05_roll_std_10"]) * 2.0)))/2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) - (data["exp_Moving_average_30000_mean"]))) * 2.0)) +
            0.0399999991*activation2(((data["trend"]) - (((73.0) * ((((3.6923100948) + ((((9.0)) * (((activation2((activation2((activation2((activation2((activation2((activation2((activation2((data["q05_roll_std_100"]))))))))))))))) - (data["q05_roll_mean_10"]))))))/2.0)))))) +
            0.0399999991*activation2(((data["q05_roll_std_1000"]) - (((73.0) * (((data["avg_last_50000"]) + (((((73.0) * 2.0)) * (((data["ave_roll_std_100"]) + (((data["Moving_average_6000_mean"]) + (((data["iqr"]) + (((data["q05_roll_std_1000"]) + (((data["q95"]) * 2.0)))))))))))))))))) +
            0.0399999991*activation2(((((data["q05_roll_mean_10"]) - (2.0769200325))) - ((((((6.08570814132690430)) * (((2.0769200325) + (((((((((data["q05_roll_std_10"]) - (data["q05_roll_mean_10"]))) * 2.0)) * 2.0)) * 2.0)))))) + (((((data["av_change_rate_roll_std_100"]) - (data["avg_last_10000"]))) * 2.0)))))) +
            0.0399999991*activation2((-1.0*((((((((((((((((((((((((((((((((data["q05_roll_std_1000"]) + (data["q05_roll_mean_100"]))/2.0)) * 2.0)) + (data["iqr"]))/2.0)) + (data["q95_roll_mean_10"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0))))) +
            0.0399999991*activation2((((-1.0*((((((((((data["q05_roll_std_100"]) - (activation2(((-1.0*((((data["MA_400MA_std_mean"]) + (((data["exp_Moving_average_300_mean"]) * (((data["q05_roll_std_100"]) * 2.0))))))))))))) * 2.0)) * 2.0)) + (activation2((data["q05_roll_std_100"])))))))) * (73.0))) +
            0.0399999991*activation2(((((data["abs_q05"]) - ((((((2.0769200325) * ((((((73.0) * ((((((0.5555559993) + (((data["q05_roll_std_100"]) + (0.7446810007))))/2.0)) + (data["q05_roll_std_100"]))))) + (0.5555559993))/2.0)))) + (0.5555559993))/2.0)))) * 2.0)) +
            0.0399999991*activation2((((10.75933361053466797)) - (((7.0) * (((7.0) * (((7.0) * ((((((data["q95_roll_mean_10"]) + (data["q05_roll_std_100"]))) + ((((data["q95_roll_mean_10"]) + (((data["iqr"]) + (((data["Moving_average_700_mean"]) * 2.0)))))/2.0)))/2.0)))))))))) +
            0.0399999991*activation2(((73.0) * (((((data["q05_roll_std_1000"]) + ((((data["q05_roll_std_10"]) + ((((((data["q05_roll_std_1000"]) + (activation2((data["q05_roll_std_1000"]))))/2.0)) - (((data["q95_roll_mean_1000"]) - (activation2((73.0))))))))/2.0)))) * (((2.8437500000) - (73.0))))))) +
            0.0399999991*activation2(((((((((-3.0) + (data["Moving_average_3000_mean"]))) - (((73.0) * (((data["ave_roll_mean_10"]) - ((((((data["q05_roll_std_100"]) + (data["q95_roll_mean_10"]))/2.0)) * (((-3.0) - (((data["Moving_average_700_mean"]) + (data["q95_roll_mean_10"]))))))))))))) * 2.0)) * 2.0)) +
            0.0399999991*activation2(((((((data["classic_sta_lta3_mean"]) * 2.0)) + (data["q05_roll_mean_100"]))) - (((73.0) * (((((((0.6582279801) + (data["q05_roll_std_10"]))) - (((data["q05_roll_mean_10"]) * 2.0)))) + ((((-1.0*((data["abs_q05"])))) + (((data["q05_roll_std_100"]) * 2.0)))))))))) +
            0.0399999991*activation2(((((data["skew"]) + (((data["kurt"]) + (data["med"]))))) - ((((7.02540111541748047)) * (((((data["iqr"]) + (((data["q95_roll_std_10"]) + (((((data["q95_roll_mean_10"]) + (((data["iqr"]) * (data["Moving_average_1500_mean"]))))) * 2.0)))))) * 2.0)))))) +
            0.0399999991*activation2(((((data["q05_roll_std_100"]) - (((-2.0) + (((((data["iqr"]) * (-2.0))) - ((-1.0*((data["av_change_rate_roll_std_10"])))))))))) - ((((((data["q05_roll_std_100"]) + (activation2((((data["q05_roll_std_100"]) * (data["ave_roll_mean_10"]))))))/2.0)) * (73.0))))) +
            0.0399999991*activation2(((((((((((data["q05_roll_std_10"]) + (((data["av_change_abs_roll_std_1000"]) - (((data["av_change_abs_roll_mean_10"]) * 2.0)))))) * 2.0)) + (data["av_change_abs_roll_std_1000"]))) * 2.0)) - (((73.0) * (((((data["abs_max_roll_mean_1000"]) + (((0.5555559993) + (data["q05_roll_std_10"]))))) * 2.0)))))) +
            0.0399999991*activation2((((((((((((((((((activation2((((((data["q95_roll_mean_10"]) + (data["q95_roll_std_100"]))) - (data["q05_roll_std_1000"]))))) + (data["q95_roll_std_100"]))/2.0)) + (data["q95_roll_std_100"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) - (((73.0) * (data["q95_roll_mean_10"]))))) * 2.0)) +
            0.0399999991*activation2(((73.0) - (((((((((((data["q05_roll_std_10"]) * 2.0)) - (data["ave10"]))) + (((data["q05_roll_std_100"]) + (0.9756100178))))) * (73.0))) * (((73.0) * 2.0)))))) +
            0.0399999991*activation2(((((data["Moving_average_6000_mean"]) + (((7.0) + (((data["ave10"]) * 2.0)))))) * (((((((((data["MA_700MA_BB_low_mean"]) * (data["avg_last_50000"]))) + ((-1.0*((((data["q95_roll_mean_10"]) * 2.0))))))) - (((data["ave_roll_mean_10"]) + (data["q05_roll_std_100"]))))) * 2.0)))) +
            0.0399999991*activation2(((73.0) * (((data["q05_roll_std_100"]) - (((((((data["iqr"]) - (activation2((((73.0) * (((data["ave10"]) - (((data["iqr"]) + (((((data["q05_roll_std_1000"]) + (0.5434780121))) * 2.0)))))))))))) * 2.0)) * 2.0)))))) +
            0.0399999991*activation2(((((((((((((((data["exp_Moving_average_30000_mean"]) * (((data["exp_Moving_average_30000_mean"]) + ((-1.0*((data["q01_roll_std_10"])))))))) - (((data["q05_roll_std_10"]) + (((data["q05_roll_std_10"]) + (((data["q95_roll_mean_100"]) + (data["q05_roll_std_10"]))))))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*activation2(((73.0) - (((((data["q05_roll_std_100"]) * (73.0))) * (((73.0) * (((data["q05_roll_std_100"]) * (((data["q05_roll_std_100"]) - (((data["q05"]) - ((((0.2525250018) + (2.0))/2.0)))))))))))))) +
            0.0399999991*activation2(((((((((7.0) - ((((data["sum"]) + (((73.0) * (((((data["sum"]) + (data["q05_roll_std_10"]))) + (((((((activation2((data["abs_q05"]))) + (data["q05_roll_std_10"]))) * 2.0)) * 2.0)))))))/2.0)))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*activation2((((((1.0)) - (((((((((data["q05_roll_std_100"]) + (activation2((((data["abs_q05"]) * (((((data["exp_Moving_average_30000_mean"]) - (data["abs_q05"]))) + (((data["q05_roll_std_100"]) + (((data["q95_roll_mean_10"]) * 2.0)))))))))))) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) +
            0.0399999991*activation2(((((activation2((((activation2((data["Moving_average_1500_mean"]))) * 2.0)))) - (((data["Moving_average_1500_mean"]) + ((-1.0*((((((data["q05_roll_std_100"]) + (activation2((0.6582279801))))) * 2.0))))))))) * ((((-1.0*((((data["Moving_average_1500_mean"]) * 2.0))))) - (73.0))))) +
            0.0399999991*activation2(((73.0) - ((((((((((((data["q05_roll_std_100"]) + (((data["q05_roll_std_100"]) * (data["Moving_average_6000_mean"]))))/2.0)) + (((data["Moving_average_6000_mean"]) + (((data["q05_roll_std_100"]) + (data["q95"]))))))) * 2.0)) * 2.0)) * (((73.0) + (data["Moving_average_6000_mean"]))))))) +
            0.0399999991*activation2(((((((((((((data["mean_change_rate_last_10000"]) - (data["mean_change_abs"]))) - (((((((((((((data["q05_roll_std_10"]) + (((0.5434780121) - (data["q05"]))))) * 2.0)) - (data["Hann_window_mean"]))) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*activation2(((((((-1.0*((((data["avg_last_50000"]) * (3.6923100948)))))) + (((73.0) / 2.0)))/2.0)) - (((73.0) * ((((((data["Moving_average_6000_mean"]) + (((data["q05_roll_std_10"]) - (((data["exp_Moving_average_30000_mean"]) * (data["exp_Moving_average_30000_mean"]))))))/2.0)) + (data["q05_roll_std_10"]))))))) +
            0.0399999991*activation2(((73.0) * ((-1.0*((((data["q01_roll_std_1000"]) + (((73.0) * (((data["q05_roll_std_100"]) + ((((1.4411799908) + ((((((data["q01_roll_std_1000"]) - (data["q99_roll_mean_1000"]))) + (((data["q99_roll_mean_1000"]) * (data["q99_roll_mean_1000"]))))/2.0)))/2.0))))))))))))) +
            0.0399999991*activation2((((12.50784111022949219)) + ((((12.50784111022949219)) - ((((((activation2((((data["q05_roll_std_10"]) * (data["q95_roll_mean_1000"]))))) + (((((((data["q05_roll_std_10"]) + (activation2((((data["q05_roll_std_10"]) * (data["exp_Moving_average_30000_mean"]))))))) * 2.0)) * 2.0)))/2.0)) * (73.0))))))) +
            0.0399999991*activation2(((((((data["skew"]) - (((data["ave_roll_mean_100"]) + (((((((data["ave_roll_mean_100"]) * (data["abs_q05"]))) + (((((2.0) + (((((data["q05_roll_std_100"]) - (data["q05_roll_mean_10"]))) * 2.0)))) * 2.0)))) * 2.0)))))) * 2.0)) * 2.0)) +
            0.0399999991*activation2(((((((((data["q95_roll_mean_1000"]) * (data["ave_roll_mean_100"]))) * 2.0)) - (((((data["exp_Moving_average_30000_mean"]) + (((((((((data["exp_Moving_average_30000_mean"]) * (data["q05_roll_std_10"]))) + (((((data["q05_roll_std_10"]) - (0.2439019978))) * 2.0)))) * 2.0)) * 2.0)))) * 2.0)))) * 2.0)) +
            0.0399999991*activation2(((((((((data["q05_roll_std_10"]) - (((((data["q95"]) - (((((data["iqr"]) - (((data["exp_Moving_average_3000_mean"]) * (activation2((((data["q05_roll_std_10"]) * 2.0)))))))) - (((data["q05_roll_std_10"]) * 2.0)))))) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*activation2(((((((data["q95_roll_mean_100"]) + (((data["mean_change_rate_first_10000"]) + (((data["abs_q05"]) + (data["abs_q05"]))))))) * (3.6923100948))) + (((((((((data["q05_roll_std_1000"]) + (0.7446810007))) * (((3.6923100948) - (73.0))))) * 2.0)) * 2.0)))) +
            0.0399999991*activation2((-1.0*((((((73.0) * ((((((data["q01_roll_mean_1000"]) - (((data["q05"]) * 2.0)))) + (((data["q05_roll_std_1000"]) + (((((data["q05_roll_std_100"]) - (data["q95_roll_mean_1000"]))) * (data["Moving_average_700_mean"]))))))/2.0)))) + (((((data["Moving_average_1500_mean"]) * 2.0)) * 2.0))))))) +
            0.0399999991*activation2(((((((((data["q05"]) - (((73.0) * ((((-1.0*((((data["q05"]) * 2.0))))) + (((data["q05_roll_std_1000"]) + (1.9019600153))))))))) + (((((((data["q05"]) + (data["max_to_min"]))) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) +
            0.0399999991*activation2(((((((data["av_change_abs_roll_mean_100"]) - (((data["q01_roll_mean_1000"]) * 2.0)))) - (((73.0) * (((((((data["Hann_window_mean"]) * (((data["q95"]) - (data["q95_roll_mean_1000"]))))) / 2.0)) + (((((data["q95"]) * 2.0)) + (data["q01_roll_mean_1000"]))))))))) * 2.0)) +
            0.0399999991*activation2(((((((((((data["skew"]) - (((((data["q05_roll_mean_10"]) + (((((data["iqr"]) + (((3.1415927410) * (((1.0) - (((data["q05_roll_mean_10"]) * 2.0)))))))) * 2.0)))) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*activation2((((((((6.0)) - (((((((((data["q05_roll_std_100"]) + (((data["q05_roll_std_100"]) * 2.0)))) * 2.0)) + (((((data["ave10"]) * (((((data["q05_roll_std_100"]) * 2.0)) * 2.0)))) + (activation2((data["q05_roll_std_100"]))))))) * 2.0)))) * 2.0)) * 2.0)) +
            0.0399999991*activation2(((((((((data["q05_roll_std_10"]) + (((((((data["q95_roll_mean_1000"]) + (((((((data["skew"]) + (((((((data["abs_q05"]) - (((data["q05_roll_std_10"]) * 2.0)))) * 2.0)) * 2.0)))) * 2.0)) * 2.0)))) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*activation2(((((((((data["max_to_min"]) + (data["q95_roll_std_1000"]))) - (((((73.0) * ((((data["q05_roll_std_1000"]) + (((0.2183910012) + (data["abs_q05"]))))/2.0)))) * (((data["q05_roll_std_1000"]) * (((data["q05_roll_std_10"]) + (data["q01_roll_std_10"]))))))))) * 2.0)) * 2.0)) +
            0.0399999991*activation2(((((((((((data["abs_q05"]) + (data["q05_roll_std_100"]))) - (((7.0) * (((data["iqr"]) - (((data["q99_roll_mean_1000"]) + ((-1.0*((((((((data["q05_roll_std_100"]) + (1.0))) * 2.0)) * 2.0))))))))))))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*activation2((-1.0*((((7.0) * (((73.0) * (((((7.0) * (((((data["Moving_average_6000_mean"]) * (((((-1.0*((data["Moving_average_6000_mean"])))) + (activation2((activation2((data["q01_roll_std_10"]))))))/2.0)))) + (data["q95_roll_mean_10"]))))) + (data["q95_roll_mean_10"])))))))))) +
            0.0399999991*activation2(((((((((((data["skew"]) + (data["ave10"]))) + (data["q95_roll_std_100"]))) - (((7.0) * (((((data["q05_roll_std_100"]) * (data["q95"]))) * ((((((data["q05_roll_std_100"]) * (7.0))) + (7.0))/2.0)))))))) * 2.0)) * 2.0)) +
            0.0399999991*activation2((((((9.64285850524902344)) * (((((data["Moving_average_700_mean"]) - ((((((9.64285850524902344)) * ((((data["q05_roll_std_100"]) + (data["exp_Moving_average_30000_mean"]))/2.0)))) - ((((data["skew"]) + ((3.98402547836303711)))/2.0)))))) * 2.0)))) * (((data["iqr"]) * (data["q05_roll_std_100"]))))) +
            0.0399999991*activation2(((((((((((((((((data["Moving_average_1500_mean"]) - (((activation2((data["exp_Moving_average_30000_mean"]))) + (activation2((((data["abs_q05"]) + ((((data["q95_roll_std_10"]) + (data["q01_roll_std_10"]))/2.0)))))))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) - (data["iqr"]))) * 2.0)) +
            0.0399999991*activation2((((-1.0*((data["q95"])))) * (((73.0) + ((-1.0*((((data["abs_q05"]) * ((((((((-1.0*((data["abs_q05"])))) - (data["min_roll_std_10"]))) - (73.0))) - ((((-1.0*((data["q05_roll_mean_10"])))) * (73.0)))))))))))))) +
            0.0399999991*activation2(((data["q05_roll_std_10"]) * (((((((((data["q05_roll_std_100"]) * ((11.90774154663085938)))) * (((data["ave10"]) - (-2.0))))) - ((11.90774154663085938)))) * ((-1.0*((((data["q05_roll_std_1000"]) + (((data["q05_roll_std_100"]) - (-2.0)))))))))))) +
            0.0399999991*activation2(((((((data["iqr"]) * ((9.63457870483398438)))) * (((data["q95_roll_mean_1000"]) - ((((9.63457870483398438)) * (((data["q05_roll_std_10"]) * ((((data["q01_roll_std_10"]) + (((data["iqr"]) * (data["q95_roll_mean_1000"]))))/2.0)))))))))) - (((-3.0) * (data["q01_roll_std_10"]))))) +
            0.0399999991*activation2(((((((((-1.0) - (((activation2((data["q05_roll_std_10"]))) + (((((((data["q05_roll_std_100"]) - (activation2((((data["classic_sta_lta1_mean"]) + (((((((data["q95_roll_std_100"]) * 2.0)) * 2.0)) * 2.0)))))))) * 2.0)) * 2.0)))))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*activation2(((((((((data["q95_roll_mean_10"]) - (((((((data["q95_roll_mean_10"]) + (((((data["q05_roll_std_10"]) + (((3.6923100948) * ((((data["ave10"]) + (((data["q05_roll_std_10"]) + (-1.0))))/2.0)))))) * 2.0)))) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*activation2(((((data["q05_roll_std_100"]) + (((((data["q95_roll_mean_1000"]) + (((((data["q05_roll_std_100"]) + ((((3.1415927410) + (data["q05_roll_std_1000"]))/2.0)))) * (((((((data["q05_roll_mean_10"]) * 2.0)) * 2.0)) * (((3.1415927410) * (data["q05_roll_std_100"]))))))))) * 2.0)))) * 2.0)) +
            0.0399999991*activation2(((((((((((((data["mean_change_rate_first_10000"]) - (data["Moving_average_6000_mean"]))) * 2.0)) - (data["q01_roll_mean_1000"]))) - (((data["av_change_abs_roll_mean_10"]) + (((data["mean_change_rate_last_50000"]) + (((data["av_change_abs_roll_std_100"]) - ((((((13.93686485290527344)) / 2.0)) * (data["q05"]))))))))))) * 2.0)) * 2.0)) +
            0.0399999991*activation2(((((((((data["q05_roll_std_10"]) * (data["q05_roll_std_10"]))) * ((((-1.0*((((2.0) - ((-1.0*((((data["q05_roll_std_100"]) + (((((data["q05_roll_std_100"]) * (data["abs_q05"]))) * 2.0)))))))))))) * 2.0)))) - ((-1.0*((data["kurt"])))))) * 2.0)) +
            0.0399999991*activation2(((((((((((((((data["q05"]) - (activation2((((data["iqr"]) + (((((data["Moving_average_1500_mean"]) + (((data["q05"]) + (data["Moving_average_3000_mean"]))))) * 2.0)))))))) * 2.0)) + (data["q05"]))) * 2.0)) * 2.0)) - (data["av_change_abs_roll_mean_10"]))) * 2.0)) +
            0.0399999991*activation2(((((((((activation2((((((((((((((data["Moving_average_6000_mean"]) - (data["q05_roll_std_100"]))) * 2.0)) * 2.0)) - (((1.6428600550) + (data["q01_roll_std_10"]))))) * 2.0)) - (data["av_change_abs_roll_std_100"]))))) * 2.0)) - (data["abs_q05"]))) * 2.0)) * 2.0)) +
            0.0399999991*activation2((((((((((((((data["skew"]) + (data["q95_roll_std_10"]))/2.0)) - (((((data["ave_roll_std_100"]) + (((data["max_roll_mean_10"]) * 2.0)))) * (data["abs_q05"]))))) + (((((activation2((data["q95_roll_std_100"]))) - (data["q01_roll_std_10"]))) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*activation2(((data["skew"]) + (((((data["q01_roll_std_100"]) * 2.0)) + (((data["Moving_average_700_mean"]) + (((data["q05_roll_std_10"]) * (((data["q05"]) * ((((((73.0) * (((data["Moving_average_700_mean"]) + (((data["q05_roll_std_100"]) / 2.0)))))) + (data["q05_roll_std_10"]))/2.0)))))))))))) +
            0.0399999991*activation2(((((((data["q05_roll_mean_10"]) * (((data["q05_roll_std_100"]) * (data["q05_roll_std_100"]))))) * (((((data["q05_roll_std_100"]) * 2.0)) * (((data["q05_roll_std_1000"]) * 2.0)))))) - (((((((data["q05_roll_std_1000"]) * 2.0)) * (data["q05_roll_std_100"]))) - (((data["q01_roll_std_1000"]) * 2.0)))))) +
            0.0399999991*activation2(((((((-1.0) + (((((((activation2((data["q01_roll_std_10"]))) * 2.0)) * 2.0)) + (((data["max_to_min_diff"]) + (((data["q05"]) * ((((14.98138904571533203)) + (((data["mean_change_rate_last_10000"]) * 2.0)))))))))))) * (73.0))) * (73.0))) +
            0.0399999991*activation2(((((((data["q05_roll_std_10"]) - (((((data["q05_roll_std_1000"]) + (0.7446810007))) * (((data["q05_roll_std_10"]) * (((((((data["q01_roll_std_10"]) * (((data["ave10"]) + (((data["q95_roll_mean_10"]) * (data["q05_roll_std_10"]))))))) * 2.0)) * 2.0)))))))) * 2.0)) * 2.0)) +
            0.0399999991*activation2(((((((activation2((data["q01_roll_std_100"]))) - (((((((((data["q05_roll_std_1000"]) + (((((data["q95"]) - (activation2((data["q95_roll_std_100"]))))) * 2.0)))) - (activation2((activation2((data["q95"]))))))) - (data["std_roll_mean_1000"]))) * 2.0)))) * 2.0)) * 2.0)) +
            0.0399999991*activation2(((((data["q05_roll_std_10"]) + (((data["ave_roll_mean_1000"]) * 2.0)))) * (((((((data["ave_roll_mean_1000"]) * (data["ave_roll_mean_1000"]))) - (data["q05_roll_std_1000"]))) + (((data["q05_roll_mean_10"]) * (((data["Moving_average_3000_mean"]) + (((((data["q05_roll_std_10"]) * 2.0)) * 2.0)))))))))) +
            0.0399999991*activation2(((((((((((data["q05_roll_std_1000"]) + (((data["q05_roll_std_100"]) - (((data["Moving_average_6000_mean"]) * (data["Moving_average_6000_mean"]))))))) * (((data["q95_roll_std_100"]) * ((((-1.0*((data["iqr"])))) - (((data["Moving_average_6000_mean"]) + (data["Moving_average_1500_mean"]))))))))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*activation2(((((((((data["sum"]) + (0.9756100178))) * 2.0)) * 2.0)) * (((data["q01_roll_std_100"]) - (((activation2((((data["q05_roll_std_10"]) * (data["q05_roll_std_10"]))))) + (((data["q05_roll_std_1000"]) * (((((data["q01_roll_std_100"]) * (data["q01_roll_std_10"]))) * 2.0)))))))))) +
            0.0399999991*activation2((((((((((-1.0*((data["q05_roll_std_100"])))) - (((data["q01_roll_std_100"]) * (data["mean"]))))) * (((data["q05_roll_std_100"]) * (data["q05_roll_std_100"]))))) * (((data["q05_roll_std_100"]) * (data["q05_roll_std_100"]))))) + (((((data["av_change_rate_roll_std_10"]) * (data["max_to_min"]))) * 2.0)))) +
            0.0399999991*activation2(((((((data["q05_roll_std_10"]) * 2.0)) * 2.0)) * (((((0.5434780121) * 2.0)) - (((((data["q05_roll_std_10"]) * (((((((((data["q95_roll_std_10"]) * 2.0)) * (data["q05_roll_std_10"]))) + (data["q05_roll_std_10"]))) * (data["q05_roll_std_10"]))))) + (data["skew"]))))))) +
            0.0399999991*activation2(((((data["abs_q05"]) * (data["q95_roll_std_100"]))) - (((((data["q05_roll_std_100"]) * (((data["q05_roll_std_100"]) * (((data["q05_roll_std_100"]) * (((data["q05_roll_std_100"]) * (((data["abs_q05"]) * (data["q05_roll_std_100"]))))))))))) + ((((data["min_roll_std_100"]) + (data["av_change_abs_roll_mean_100"]))/2.0)))))) +
            0.0399999991*activation2(((((data["Moving_average_6000_mean"]) + (0.8873239756))) * (((((data["q05_roll_std_100"]) + (data["q95_roll_std_1000"]))) * (((1.6428600550) - (((data["q05_roll_std_1000"]) * (((((data["q05_roll_std_100"]) + (((data["exp_Moving_average_3000_mean"]) + (0.9756100178))))) + (data["q05_roll_std_100"]))))))))))) +
            0.0399999991*activation2(((((data["q95_roll_mean_10"]) + (((data["q95_roll_std_1000"]) * (((data["med"]) + (((((data["q05_roll_mean_100"]) + (((data["q95_roll_mean_10"]) * (data["q95_roll_mean_10"]))))) * (data["q05_roll_std_100"]))))))))) * (((((data["exp_Moving_average_30000_mean"]) - (data["q05_roll_std_100"]))) - (data["q05_roll_std_1000"]))))) +
            0.0399999991*activation2(((((((data["min"]) + ((((-1.0*((data["MA_400MA_BB_high_mean"])))) + (activation2((((((0.3183098733) + (((((data["kurt"]) * 2.0)) * 2.0)))) * ((((3.0)) + ((-1.0*((data["kurt"])))))))))))))) * 2.0)) * 2.0)) +
            0.0399999991*activation2(((((((data["min_roll_mean_1000"]) * 2.0)) * 2.0)) + ((((-1.0*((((((data["q05_roll_std_1000"]) + (((data["q01_roll_std_10"]) * (data["q95"]))))) + (((data["avg_last_50000"]) * (((data["q01_roll_std_10"]) - (data["avg_last_50000"])))))))))) * (data["q01_roll_std_100"]))))) +
            0.0399921872*activation2(((((data["q95_roll_std_1000"]) - ((((((((data["av_change_abs_roll_mean_10"]) + (data["std_first_10000"]))/2.0)) + (((data["q95"]) - (data["min_last_10000"]))))) - (((data["min"]) + (((data["q95_roll_std_1000"]) + (data["mean_change_rate_last_10000"]))))))))) - (((data["q95"]) + (data["q95"]))))) +
            0.0399999991*activation2(((((((data["min_roll_mean_1000"]) - (((data["q05_roll_std_1000"]) * (((activation2((((data["med"]) * 2.0)))) + ((((((data["skew"]) * (data["kurt"]))) + (data["q05_roll_std_1000"]))/2.0)))))))) + (((data["kurt"]) * (data["skew"]))))) + (data["min_roll_mean_1000"]))) +
            0.0399921872*activation2(((data["exp_Moving_average_30000_mean"]) * (((((data["max_roll_std_100"]) - (((((data["exp_Moving_average_30000_mean"]) * (data["MA_400MA_std_mean"]))) + (data["q05_roll_mean_10"]))))) * (((((data["mean_change_rate_first_10000"]) * (73.0))) - (((((data["abs_mean"]) * (73.0))) * (73.0))))))))) +
            0.0399999991*activation2(((data["q01_roll_std_10"]) + ((-1.0*((((data["q05_roll_std_1000"]) * (((data["av_change_rate_roll_mean_100"]) + ((((-1.0*((data["classic_sta_lta4_mean"])))) + (((data["med"]) + (((data["q05_roll_std_1000"]) + (((data["iqr"]) * (((data["iqr"]) + (data["max_last_10000"])))))))))))))))))))) +
            0.0399921872*activation2((((((((data["q05"]) + (data["mean_change_rate_first_10000"]))/2.0)) + (((data["kurt"]) - (((((data["kurt"]) * (data["abs_max_roll_std_1000"]))) * (data["kurt"]))))))) + (((((data["av_change_abs_roll_std_100"]) * ((-1.0*((data["classic_sta_lta3_mean"])))))) * 2.0)))) +
            0.0399999991*activation2(((((((((((((((data["skew"]) - ((((data["min_roll_std_1000"]) + (data["q95_roll_mean_100"]))/2.0)))) + (((data["q95_roll_mean_100"]) * (data["q05"]))))) * (data["MA_400MA_std_mean"]))) * (data["MA_400MA_BB_high_mean"]))) * 2.0)) * 2.0)) + ((((data["mean_change_abs"]) + (data["min_roll_std_1000"]))/2.0)))) +
            0.0399921872*activation2((((((4.12892913818359375)) * ((((((data["exp_Moving_average_300_mean"]) + (data["med"]))/2.0)) - (data["q01_roll_std_100"]))))) * (((data["skew"]) + (((0.2439019978) + ((((data["MA_1000MA_std_mean"]) + ((((((data["av_change_abs_roll_std_1000"]) + (data["med"]))) + (data["iqr"]))/2.0)))/2.0)))))))) +
            0.0399999991*activation2(((((((((data["av_change_abs_roll_std_10"]) * (((((data["mean_change_rate_last_50000"]) + (data["av_change_abs_roll_mean_10"]))) + (data["min_last_50000"]))))) * 2.0)) * 2.0)) + ((((((((1.0)) + (((data["av_change_rate_roll_std_10"]) + (data["skew"]))))/2.0)) + (data["q01_roll_mean_1000"]))/2.0)))) +
            0.0399999991*activation2(((((((((data["q05_roll_std_10"]) * (((((((data["med"]) + (data["q05_roll_std_10"]))) * (data["q05_roll_mean_10"]))) * (data["q95_roll_std_1000"]))))) * (data["q05_roll_std_10"]))) + (((((data["q05_roll_mean_10"]) + (data["q95_roll_std_100"]))) + (data["q95_roll_std_100"]))))) * 2.0)) +
            0.0399921872*activation2((((data["skew"]) + (((((((((data["min_roll_std_10"]) * ((-1.0*((data["min_roll_std_10"])))))) + (data["q05_roll_mean_1000"]))) * (data["Hilbert_mean"]))) - ((((data["av_change_rate_roll_std_100"]) + (((data["max_roll_std_100"]) * (((data["skew"]) * (data["skew"]))))))/2.0)))))/2.0)) +
            0.0399999991*activation2(((data["q05_roll_std_10"]) * (((((((((data["q01_roll_mean_1000"]) - (activation2((((data["Moving_average_6000_mean"]) - (((data["ave_roll_std_10"]) * (((data["kurt"]) - ((((data["av_change_abs_roll_mean_10"]) + ((((data["Moving_average_6000_mean"]) + (data["ave_roll_std_10"]))/2.0)))/2.0)))))))))))) * 2.0)) * 2.0)) * 2.0)))) +
            0.0399843715*activation2(((data["min"]) + ((-1.0*((((data["av_change_rate_roll_std_100"]) * ((((((((data["mean_change_rate"]) + (data["min"]))/2.0)) + (data["min_roll_std_1000"]))) + (((activation2((data["min_last_10000"]))) + ((((((data["min_roll_std_100"]) * (data["av_change_rate_roll_mean_10"]))) + (data["av_change_abs_roll_std_10"]))/2.0))))))))))))) +
            0.0399999991*activation2(((data["min_roll_mean_100"]) * (((((data["min_roll_std_1000"]) * (((data["q01_roll_std_1000"]) * (((((((data["q01_roll_std_1000"]) * (data["q99_roll_std_100"]))) + (data["exp_Moving_average_300_mean"]))) + (data["avg_first_10000"]))))))) + (((((data["mean_change_rate"]) * (3.1415927410))) - (data["avg_first_10000"]))))))) +
            0.0399921872*activation2(((((data["q95_roll_mean_1000"]) * (((((((((data["q95_roll_mean_1000"]) + (((data["avg_last_10000"]) + (((3.0) / 2.0)))))/2.0)) + (data["q05_roll_mean_100"]))/2.0)) - (data["q95_roll_std_1000"]))))) * (((((((data["q95_roll_std_1000"]) * 2.0)) + (data["q99_roll_mean_10"]))) + (data["classic_sta_lta4_mean"]))))) +
            0.0399296731*activation2((((((((((((data["q01_roll_std_100"]) + (activation2((((data["q01"]) + ((-1.0*((((data["q01_roll_std_100"]) * 2.0))))))))))/2.0)) * ((-1.0*(((((data["q95"]) + (((data["exp_Moving_average_30000_mean"]) * 2.0)))/2.0))))))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399921872*activation2(((((activation2((data["q99"]))) + ((-1.0*(((((data["max_first_10000"]) + (((data["av_change_rate_roll_mean_1000"]) - (data["q99"]))))/2.0))))))) - ((((((((data["av_change_rate_roll_mean_1000"]) * (data["av_change_rate_roll_mean_1000"]))) + (data["q05_roll_std_10"]))/2.0)) * (((data["q05_roll_std_10"]) * (data["iqr"]))))))) +
            0.0399999991*activation2(((data["min_roll_mean_1000"]) * (((((((((((data["std_roll_std_100"]) + (((data["q05"]) + (((((data["q95_roll_mean_1000"]) * (data["q95_roll_std_1000"]))) + (((data["q95_roll_mean_1000"]) * (((data["q95_roll_mean_1000"]) * (data["q95_roll_std_1000"]))))))))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) +
            0.0399921872*activation2(((((data["mean_change_abs"]) * (data["min_roll_std_10"]))) + (((((((data["std_last_50000"]) * (3.6923100948))) * (((data["q95_roll_mean_1000"]) + (((data["classic_sta_lta4_mean"]) - (data["min_roll_std_10"]))))))) * ((((((data["min_roll_std_10"]) + (data["q05_roll_mean_100"]))/2.0)) - (data["max_roll_std_10"]))))))) +
            0.0399999991*activation2(((((data["q01_roll_std_10"]) + (data["min_roll_std_100"]))) * (((data["abs_q05"]) - (((data["avg_last_50000"]) - (((((data["classic_sta_lta2_mean"]) / 2.0)) + (((data["max_roll_mean_1000"]) * (((((data["avg_last_50000"]) - (activation2((data["min_roll_std_100"]))))) - (data["min_roll_std_100"]))))))))))))) +
            0.0399999991*activation2(((((data["q95_roll_mean_100"]) * (data["min_roll_mean_1000"]))) - (((((data["av_change_abs_roll_std_100"]) * (((((data["classic_sta_lta3_mean"]) + (data["mean_change_rate_first_50000"]))) + (((data["q95_roll_mean_100"]) * (data["classic_sta_lta3_mean"]))))))) + (((data["mean_change_abs"]) * (((data["classic_sta_lta3_mean"]) - (data["mean_change_rate_first_50000"]))))))))) +
            0.0398359038*activation2(((data["q95_roll_mean_10"]) * (((((data["q95_roll_mean_10"]) * (((data["min_roll_mean_1000"]) + (data["av_change_rate_roll_std_10"]))))) - (((data["max_to_min"]) + (((((((data["max_to_min"]) * 2.0)) * 2.0)) * (((data["av_change_rate_roll_std_10"]) + (data["q01_roll_mean_1000"]))))))))))) +
            0.0399999991*activation2(((data["q01_roll_std_10"]) * (((((((data["q01_roll_std_10"]) * (data["skew"]))) + (((data["classic_sta_lta4_mean"]) - (data["classic_sta_lta2_mean"]))))) + ((((data["av_change_abs_roll_mean_1000"]) + (((((-1.0*((data["q01_roll_std_100"])))) + (((data["q01_roll_std_10"]) * ((-1.0*((data["q01_roll_std_10"])))))))/2.0)))/2.0)))))) +
            0.0385153331*activation2((((((data["std_roll_mean_1000"]) + (data["av_change_abs_roll_mean_100"]))) + (((data["max_first_10000"]) * (((data["MA_700MA_std_mean"]) * (((73.0) * (((activation2((data["max_to_min"]))) - (((data["av_change_rate_roll_mean_1000"]) + (((((data["std_roll_mean_1000"]) * 2.0)) * 2.0)))))))))))))/2.0)) +
            0.0399999991*activation2(((((data["std_roll_mean_1000"]) * (((data["av_change_rate_roll_std_1000"]) - (((((((data["skew"]) - (data["min_roll_std_10"]))) - (data["classic_sta_lta4_mean"]))) + (((data["std_roll_mean_1000"]) - (2.0))))))))) + (((data["classic_sta_lta4_mean"]) * (((data["av_change_abs_roll_mean_1000"]) - (data["q99"]))))))) +
            0.0399843715*activation2((-1.0*((((data["kurt"]) * (((data["kurt"]) * ((((((data["av_change_abs_roll_mean_10"]) + (((((data["med"]) - (((data["med"]) * (data["av_change_abs_roll_mean_10"]))))) - (data["min_roll_std_10"]))))/2.0)) + (((data["q99_roll_std_1000"]) * (data["kurt"])))))))))))) +
            0.0399921872*activation2(((data["mean_change_rate_first_10000"]) * (((data["av_change_abs_roll_mean_100"]) * ((-1.0*((((((activation2((((((data["classic_sta_lta3_mean"]) + (data["mean_change_rate_first_10000"]))) * 2.0)))) + (((1.9019600153) - (data["av_change_abs_roll_mean_100"]))))) - (((((data["classic_sta_lta3_mean"]) * 2.0)) * (data["mean_change_rate_first_10000"])))))))))))) +
            0.0399921872*activation2(((((data["av_change_rate_roll_std_100"]) * (((data["classic_sta_lta3_mean"]) * (((((data["min_roll_std_1000"]) - (data["min"]))) * (data["q05"]))))))) + (((data["mean_change_rate_last_10000"]) * (((((data["av_change_abs_roll_std_1000"]) - (data["min"]))) - (((data["classic_sta_lta3_mean"]) * (data["q01_roll_std_1000"]))))))))) +
            0.0399999991*activation2(((data["q95_roll_std_1000"]) + (((data["q05_roll_mean_10"]) - ((-1.0*((((data["q95_roll_std_1000"]) * (((data["q95_roll_std_1000"]) * (((((data["q01_roll_std_10"]) * ((((((((data["q95_roll_std_1000"]) / 2.0)) + (data["q01_roll_mean_1000"]))/2.0)) - (data["med"]))))) * (data["q95_roll_std_1000"])))))))))))))) +
            0.0399843715*activation2(((data["av_change_abs_roll_mean_100"]) * (((data["classic_sta_lta4_mean"]) * (((((((((3.65123486518859863)) + ((((3.65123486518859863)) * (data["abs_max_roll_std_10"]))))/2.0)) + (((((data["min_last_50000"]) * (((data["abs_max_roll_std_1000"]) - (data["av_change_abs_roll_mean_10"]))))) - (data["classic_sta_lta4_mean"]))))) * (data["ave_roll_mean_10"]))))))) +
            0.0399999991*activation2(((((data["max_to_min_diff"]) * (((data["min_roll_std_100"]) * (((data["av_change_rate_roll_std_100"]) + (((data["q01_roll_std_100"]) + (((((((data["av_change_rate_roll_mean_100"]) + (data["Hilbert_mean"]))) * (data["q95_roll_mean_1000"]))) + (((data["q01"]) * (data["min_roll_std_100"]))))))))))))) * 2.0)) +
            0.0399999991*activation2(((((activation2((((((data["q05"]) + (((activation2((data["avg_first_10000"]))) + (data["mean_change_rate_last_10000"]))))) + (data["q05"]))))) + (((((data["kurt"]) * (data["mean_change_rate_last_10000"]))) + (data["q95_roll_std_1000"]))))) * (((data["q01_roll_mean_1000"]) + (data["av_change_rate_roll_mean_100"]))))) +
            0.0399921872*activation2(((data["min_roll_mean_1000"]) + ((((((((((((data["av_change_rate_roll_std_10"]) + (((data["av_change_abs_roll_std_1000"]) * (data["min_roll_mean_1000"]))))/2.0)) * (data["classic_sta_lta3_mean"]))) + (data["av_change_rate_roll_std_10"]))/2.0)) + (((data["abs_max_roll_mean_100"]) - (((data["av_change_abs_roll_std_1000"]) * (data["classic_sta_lta1_mean"]))))))/2.0)))) +
            0.0399999991*activation2(((data["max_to_min"]) * (((((data["MA_400MA_std_mean"]) - (data["q95_roll_std_100"]))) + ((((((data["min_roll_std_10"]) * (((((((data["min_roll_std_10"]) * (data["classic_sta_lta1_mean"]))) - (data["min_roll_std_10"]))) - (data["kurt"]))))) + ((((data["kurt"]) + (data["min_roll_std_10"]))/2.0)))/2.0)))))) +
            0.0399921872*activation2(((data["min_roll_std_10"]) * ((((((((((((activation2((data["min_roll_std_10"]))) + (data["avg_last_10000"]))/2.0)) - (data["q95_roll_std_100"]))) - (data["MA_700MA_BB_high_mean"]))) + (((data["abs_max_roll_mean_100"]) - (data["classic_sta_lta4_mean"]))))) * (((data["min_roll_std_10"]) - (((3.0) / 2.0)))))))) +
            0.0399765596*activation2(((((data["avg_last_10000"]) * (((data["min_roll_mean_1000"]) + (((((data["avg_last_10000"]) + (activation2((data["mean_change_rate_last_50000"]))))) * (((data["min_roll_std_10"]) * (data["min_last_10000"]))))))))) + (activation2((activation2((((((data["mean_change_rate_last_50000"]) * (data["min_roll_std_1000"]))) * 2.0)))))))) +
            0.0399921872*activation2(activation2((activation2((activation2((((((((data["q95_roll_std_10"]) - (data["q05"]))) * (((data["abs_q05"]) * (((data["mean_change_rate_first_10000"]) + (((((data["mean_change_rate_first_10000"]) + (data["q05"]))) * (((data["abs_q05"]) * (data["q95_roll_mean_1000"]))))))))))) * 2.0)))))))) +
            0.0399843715*activation2(((data["q95_roll_std_1000"]) * (((data["max_roll_mean_100"]) + (((((((((data["q01_roll_mean_1000"]) * 2.0)) * 2.0)) * (data["min_roll_mean_1000"]))) * (((((((data["q01_roll_mean_1000"]) + ((-1.0*((data["max_to_min"])))))) * 2.0)) * 2.0)))))))) +
            0.0399765596*activation2(((data["q05_roll_std_100"]) * ((-1.0*((activation2((((((data["med"]) + (((((data["med"]) + (((data["q01_roll_std_10"]) * (((data["avg_first_50000"]) + (data["q01_roll_std_10"]))))))) * ((((1.16074943542480469)) + (data["avg_first_50000"]))))))) * 2.0))))))))) +
            0.0399062298*activation2((-1.0*((((data["classic_sta_lta2_mean"]) * ((((((activation2((data["kurt"]))) + (data["avg_last_50000"]))/2.0)) + (((data["q05_roll_mean_100"]) * ((-1.0*(((((((((data["kurt"]) + (data["q05_roll_mean_100"]))/2.0)) - (activation2((data["kurt"]))))) + (data["std_last_10000"]))))))))))))))) +
            0.0399453007*activation2(((data["std_last_50000"]) * (((((activation2((((((((((((data["av_change_abs_roll_mean_100"]) + (activation2((activation2((((((((((data["skew"]) + (data["Moving_average_6000_mean"]))) * 2.0)) * 2.0)) * 2.0)))))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) * 2.0)))) +
            0.0390935726*activation2((-1.0*((((data["max_roll_mean_100"]) * (((((((activation2((((((data["classic_sta_lta3_mean"]) + ((((data["q99_roll_mean_1000"]) + (data["mean_change_rate_first_10000"]))/2.0)))) + (((data["av_change_abs_roll_mean_10"]) + (((data["classic_sta_lta3_mean"]) * (data["q95_roll_mean_1000"]))))))))) * 2.0)) * 2.0)) * 2.0))))))) +
            0.0399296731*activation2(((data["q95_roll_std_100"]) + ((-1.0*((((data["q95_roll_std_10"]) + ((((((((data["kurt"]) * ((((data["mean_change_rate_last_10000"]) + (data["trend"]))/2.0)))) * (data["kurt"]))) + ((((((data["med"]) + (data["max_first_50000"]))/2.0)) * (data["min_roll_std_100"]))))/2.0))))))))) +
            0.0398984179*activation2(((((activation2((((data["q01"]) - (data["q05"]))))) + (((((((((activation2((data["q05_roll_std_100"]))) + (data["q01"]))) * 2.0)) + (data["q95_roll_mean_1000"]))) * 2.0)))) * ((-1.0*((((data["q01"]) - (data["q05"])))))))) +
            0.0399999991*activation2(activation2((activation2(((-1.0*((((data["mean_change_rate_first_10000"]) * (((data["min_last_10000"]) + (((data["min_last_10000"]) + (((((data["abs_q95"]) + (((data["avg_last_50000"]) + ((((data["min_last_10000"]) + (0.8873239756))/2.0)))))) * ((-1.0*((data["q05_roll_std_100"]))))))))))))))))))) +
            0.0399999991*activation2(((data["min_first_50000"]) * ((((-1.0*((data["av_change_abs_roll_std_100"])))) - (((data["ave_roll_mean_10"]) * ((-1.0*((((data["max_to_min"]) - (((((((data["MA_400MA_BB_low_mean"]) - (((data["av_change_rate_roll_mean_100"]) + (((data["q01_roll_mean_1000"]) * (data["av_change_rate_roll_mean_100"]))))))) * 2.0)) * 2.0))))))))))))) +
            0.0399999991*activation2(((((((((data["max_to_min_diff"]) * (((data["av_change_rate_roll_std_100"]) + (((((data["q95_roll_std_10"]) + (data["max_first_50000"]))) + (((data["mean_change_abs"]) + (((((data["MA_400MA_BB_low_mean"]) * (data["max_first_50000"]))) * (data["max_first_50000"]))))))))))) * 2.0)) * 2.0)) * (data["min_roll_std_100"]))) +
            0.0399999991*activation2(((((data["max"]) * (((data["std_first_50000"]) * ((((((-1.0*((data["max_roll_std_100"])))) + (((((((((data["q001"]) + (((data["q001"]) * (data["classic_sta_lta4_mean"]))))) + (data["classic_sta_lta4_mean"]))) * 2.0)) * 2.0)))) * 2.0)))))) * (data["q05_roll_std_1000"]))) +
            0.0399765596*activation2(((((data["abs_max_roll_std_1000"]) * 2.0)) * (((data["q95_roll_std_10"]) - (((data["q999"]) - (activation2((((data["max_last_10000"]) * (((((((((((((data["abs_max_roll_std_1000"]) + (data["Moving_average_6000_mean"]))) * 2.0)) + (data["mean_change_abs"]))) * 2.0)) * 2.0)) * 2.0)))))))))))) +
            0.0399687439*activation2(((data["min_roll_std_10"]) * ((((((data["avg_last_50000"]) + (((((((data["q01_roll_std_1000"]) * (data["classic_sta_lta4_mean"]))) * (data["mean_change_rate_last_50000"]))) - (((data["min_roll_std_10"]) * (data["av_change_abs_roll_mean_10"]))))))/2.0)) + ((((((data["classic_sta_lta4_mean"]) * (data["Moving_average_6000_mean"]))) + (data["mean_change_rate_last_50000"]))/2.0)))))) +
                   
            0.0398906022*activation2((((activation2((((data["av_change_abs_roll_mean_1000"]) - (((data["Hann_window_mean"]) - (data["skew"]))))))) + (((((((data["min_roll_mean_1000"]) * 2.0)) * 2.0)) * (((data["skew"]) + (((((data["std_roll_mean_1000"]) - (data["q99_roll_mean_10"]))) - (data["med"]))))))))/2.0)) +
                   
            0.0399999991*activation2(((data["std_first_10000"]) * ((-1.0*((((data["min_roll_std_10"]) * (((((((((data["kurt"]) + (((((data["av_change_rate_roll_std_1000"]) - (data["trend"]))) + (data["mean_change_rate_first_10000"]))))) - (data["max_last_10000"]))) + (((data["trend"]) * (data["trend"]))))) * 2.0))))))))) +
                   
            0.0399765596*activation2(((data["q99"]) * ((-1.0*((((data["iqr"]) - (((((((1.9019600153) + (((data["abs_max_roll_mean_100"]) + (data["q99"]))))) * (((((data["std_roll_mean_1000"]) * 2.0)) * (((data["max_roll_mean_1000"]) - (data["std_roll_mean_1000"]))))))) + (data["min_roll_std_1000"])))))))))) +
                   
            0.0399765596*activation2(((((data["mean_change_rate_last_10000"]) + (activation2((((((activation2((data["skew"]))) + (((data["max_roll_std_1000"]) * 2.0)))) * 2.0)))))) * (((data["abs_max_roll_std_10"]) + ((-1.0*((((data["skew"]) * (((((data["abs_max"]) / 2.0)) - (data["ave_roll_std_1000"])))))))))))) +
                   
            0.0355928876*activation2((((((((data["min_roll_std_100"]) * (data["q05_roll_mean_1000"]))) * (data["min_roll_std_1000"]))) + (((((data["min_roll_std_100"]) * (data["min_roll_std_100"]))) - (((((data["max_to_min"]) + ((((((data["q999"]) + (data["min_roll_std_100"]))/2.0)) + (data["q999"]))))) * (data["classic_sta_lta3_mean"]))))))/2.0)) +
                   
            0.0399843715*activation2((((((-1.0*((((((((data["q01_roll_std_10"]) * (data["q05_roll_std_1000"]))) * (data["max_first_50000"]))) * (data["q01_roll_std_10"])))))) * (((data["max_first_50000"]) + (data["max_to_min_diff"]))))) * (((((data["min"]) - (data["q05"]))) - (data["q05"]))))) +
                   
            0.0399765596*activation2(((data["med"]) * (((activation2((((((((((data["av_change_abs_roll_std_100"]) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) - (((((data["min_roll_std_10"]) + ((((data["q01_roll_std_1000"]) + ((((data["max_to_min_diff"]) + (data["min_roll_std_1000"]))/2.0)))/2.0)))) * (((data["av_change_abs_roll_std_100"]) * 2.0)))))))) +
                   
            0.0387106836*activation2((-1.0*((((((data["av_change_rate_roll_mean_10"]) * (((data["q95_roll_std_1000"]) + (((((((((((data["min_roll_mean_1000"]) * 2.0)) * 2.0)) * (((data["q95_roll_std_1000"]) + (data["min_roll_mean_1000"]))))) * 2.0)) * (((data["q95_roll_std_100"]) - (((data["min_roll_mean_1000"]) * 2.0)))))))))) * 2.0))))) +
                   
            0.0399999991*activation2(((data["max_last_10000"]) * ((((((((data["MA_700MA_BB_low_mean"]) * (((((data["avg_first_10000"]) - (data["av_change_abs_roll_mean_1000"]))) - (((data["avg_first_10000"]) * (((data["avg_first_10000"]) - (((data["min_roll_std_1000"]) * 2.0)))))))))) * 2.0)) + (((data["avg_first_10000"]) - (data["mean_change_rate_last_10000"]))))/2.0)))) +
                   
            0.0399999991*activation2(activation2((((((((((data["av_change_rate_roll_std_100"]) * (((data["min"]) * (((data["q05_roll_std_100"]) + (((((data["av_change_abs_roll_std_100"]) + (((data["mean_change_rate_first_10000"]) * (data["av_change_rate_roll_std_100"]))))) + (((data["med"]) - (data["mean_change_rate_first_10000"]))))))))))) * 2.0)) * 2.0)) * 2.0)))) +
                   
            0.0359679610*activation2((-1.0*((((activation2((((((((((((((((((data["min_roll_std_1000"]) - (data["av_change_rate_roll_std_1000"]))) - (((-2.0) + ((-1.0*((data["min_roll_std_1000"])))))))) - (activation2((data["av_change_abs_roll_std_100"]))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) / 2.0))))) +
                   
            0.0373354182*activation2((-1.0*((((data["max_to_min"]) * (((data["avg_first_50000"]) * (((((data["ave_roll_std_100"]) - (((data["classic_sta_lta1_mean"]) + (data["mean_change_rate_first_10000"]))))) * (((data["classic_sta_lta2_mean"]) - ((-1.0*((((data["ave_roll_std_100"]) / 2.0)))))))))))))))) +
                   
            0.0398671627*activation2(((((0.8873239756) + (data["avg_last_50000"]))) * (((data["iqr"]) * (((((0.8873239756) + (data["Moving_average_6000_mean"]))) + (((data["q05_roll_mean_10"]) * (((((((data["ave10"]) + (data["q01_roll_std_10"]))) * (data["q01_roll_std_10"]))) * (data["iqr"]))))))))))) +
                   
            0.0399453007*activation2((((activation2((((((((((((data["mean_change_rate_last_10000"]) + (activation2(((((((-1.0*((data["mean_change_rate_last_50000"])))) * 2.0)) * (data["q95_roll_mean_1000"]))))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) + (activation2((activation2(((((-1.0*((data["mean_change_rate_last_50000"])))) * 2.0)))))))/2.0)) +
                   
            0.0399687439*activation2(((((((data["max_roll_std_1000"]) + (((data["max_roll_std_1000"]) + (data["trend"]))))) + (data["count_big"]))) * (((data["max_to_min"]) * (((((((-1.0*((data["max_to_min"])))) + (data["max_roll_mean_1000"]))/2.0)) - (data["q99_roll_std_10"]))))))) +
                   
            0.0399843715*activation2(((data["classic_sta_lta2_mean"]) * (((data["abs_mean"]) * (((data["classic_sta_lta1_mean"]) * (((((((data["mean_change_rate_first_50000"]) + ((-1.0*((data["mean_change_abs"])))))) + (data["classic_sta_lta4_mean"]))) + (((((((-2.0) + (data["av_change_rate_roll_std_10"]))/2.0)) + (data["av_change_abs_roll_std_1000"]))/2.0)))))))))) +
                   
            0.0399687439*activation2(((data["min_roll_std_100"]) * (((((((data["mean_change_abs"]) + (((((((data["av_change_abs_roll_std_100"]) * (data["min_roll_std_100"]))) + (data["min_roll_std_1000"]))) * (((data["std_roll_mean_1000"]) * (((data["min_last_10000"]) + (data["av_change_abs_roll_std_100"]))))))))) * 2.0)) * (data["av_change_abs_roll_std_100"]))))) +
                   
            0.0399374850*activation2(((((data["min_first_10000"]) - (((data["mean_change_rate_first_10000"]) * (data["kurt"]))))) * (((((((data["abs_trend"]) * (data["kurt"]))) - (data["mean_change_abs"]))) + (((data["std_roll_mean_1000"]) + (((data["std_roll_mean_1000"]) + (((data["min_first_10000"]) + (data["abs_std"]))))))))))) +
                   
            0.0399921872*activation2(((data["q95_roll_std_10"]) * ((-1.0*((((((-1.0*((data["abs_max_roll_mean_100"])))) + ((((-1.0*((((data["classic_sta_lta4_mean"]) / 2.0))))) - (((data["q001"]) * (((data["q95_roll_mean_1000"]) - ((-1.0*((((((((data["classic_sta_lta4_mean"]) / 2.0)) / 2.0)) / 2.0))))))))))))/2.0))))))) +
                   
            0.0399843715*activation2(((((data["mean_change_rate_last_10000"]) * (((data["abs_max"]) * (((((((((((data["min_roll_std_1000"]) + (data["max_to_min"]))) - (data["count_big"]))) * 2.0)) - (data["max_to_min"]))) + (((data["av_change_abs_roll_std_1000"]) + (data["mean_change_abs"]))))))))) + (data["count_big"]))) +
                   
            0.0383434258*activation2(((((data["med"]) * ((-1.0*((((((((data["trend"]) * (((activation2((data["av_change_abs_roll_std_100"]))) - (data["mean_change_abs"]))))) + (data["mean_change_abs"]))) * 2.0))))))) * (((((data["av_change_abs_roll_std_100"]) + (activation2((data["min_last_10000"]))))) + (data["av_change_abs_roll_std_100"]))))) +
                   
            0.0399999991*activation2(activation2((((((((data["min_roll_mean_100"]) * 2.0)) * 2.0)) * (((((((data["max_last_10000"]) - (0.5434780121))) * (data["av_change_rate_roll_mean_100"]))) + (((data["av_change_rate_roll_std_10"]) - (((data["min_roll_std_10"]) * (data["av_change_rate_roll_std_10"]))))))))))) +
                   
            0.0399921872*activation2((-1.0*((((((((((data["min_roll_std_100"]) + (data["av_change_abs_roll_std_10"]))/2.0)) * (((data["av_change_abs_roll_mean_1000"]) + (data["av_change_abs_roll_std_10"]))))) + (((data["av_change_abs_roll_std_100"]) * (((data["classic_sta_lta4_mean"]) * (((data["av_change_abs_roll_std_10"]) + (((data["min_roll_std_1000"]) * (data["av_change_abs_roll_std_100"]))))))))))/2.0))))) +
                   
            0.0293416679*activation2(activation2((((data["abs_q95"]) * (((data["q01_roll_std_10"]) * (((data["q01_roll_std_10"]) * (((((data["max_last_50000"]) - ((((((data["av_change_abs_roll_mean_10"]) - (data["classic_sta_lta4_mean"]))) + (data["av_change_abs_roll_mean_100"]))/2.0)))) - ((-1.0*((((data["av_change_abs_roll_mean_10"]) * (data["classic_sta_lta4_mean"])))))))))))))))) +
                   
            0.0399765596*activation2(((((data["trend"]) * (data["max_first_10000"]))) * (((data["med"]) + (((data["av_change_rate_roll_mean_1000"]) + (((((2.0769200325) - (((activation2((((((((((data["classic_sta_lta1_mean"]) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) * 2.0)))) / 2.0)))))))) +
                   
            0.0399921872*activation2(((((data["mean_change_rate_last_50000"]) * ((-1.0*((((data["min_first_10000"]) - (data["std_first_10000"])))))))) * ((((((((((-1.0*((data["std_first_10000"])))) - (data["min_last_50000"]))) - (data["abs_max_roll_std_10"]))) - (((data["mean_change_rate_last_50000"]) * (data["std_first_10000"]))))) - (data["classic_sta_lta2_mean"]))))) +
                   
            0.0399921872*activation2(((data["av_change_abs_roll_mean_10"]) * (((data["abs_trend"]) + ((((((((data["mean_change_abs"]) * (data["min_last_10000"]))) + (((data["mean_change_rate_first_10000"]) * (data["max_first_10000"]))))) + ((((data["av_change_abs_roll_mean_10"]) + (((((data["med"]) - (data["q95_roll_mean_1000"]))) - (data["mean_change_rate_first_10000"]))))/2.0)))/2.0)))))) +
            0.0399531126*activation2(((data["ave_roll_std_100"]) * (((data["min_roll_std_100"]) * (((((data["max_roll_mean_100"]) + ((((data["max_to_min_diff"]) + (data["q99"]))/2.0)))) * (((((((data["max_to_min_diff"]) * ((((-1.0*((data["std_first_50000"])))) * (data["q95_roll_std_100"]))))) - (data["skew"]))) * 2.0)))))))) +
            0.0399843715*activation2(((data["std_roll_std_100"]) * (((data["min_roll_std_1000"]) * (((data["max_roll_mean_10"]) - (((data["kurt"]) * ((((((data["avg_last_10000"]) * (data["avg_last_10000"]))) + ((((data["q99_roll_mean_100"]) + ((((((data["avg_last_10000"]) * (data["avg_last_10000"]))) + (data["avg_first_10000"]))/2.0)))/2.0)))/2.0)))))))))) +
            0.0353584699*activation2(((data["med"]) * (((activation2((((activation2(((((((-1.0*((data["max_to_min"])))) - (data["trend"]))) * 2.0)))) - (((data["min_roll_std_100"]) * (data["med"]))))))) - (activation2((activation2((data["min_roll_std_100"]))))))))) +
            0.0399765596*activation2((((((((data["ave_roll_mean_10"]) + (data["min_roll_std_100"]))/2.0)) * (((((data["classic_sta_lta4_mean"]) + (data["kurt"]))) + (((((data["q95_roll_mean_10"]) + (((data["q95"]) * (data["q95"]))))) * (data["sum"]))))))) * ((((data["q05_roll_std_100"]) + (data["ave_roll_mean_10"]))/2.0)))) +
            0.0399843715*activation2(((data["min_roll_std_10"]) * ((((((-1.0*((((data["av_change_abs_roll_std_10"]) * (data["av_change_abs_roll_mean_100"])))))) * (data["av_change_abs_roll_std_10"]))) + (activation2(((((-1.0*((data["q05"])))) + (((data["min_roll_std_1000"]) * ((-1.0*((((data["av_change_abs_roll_mean_100"]) * (data["av_change_abs_roll_mean_100"])))))))))))))))) +
            0.0399999991*activation2((-1.0*((((data["classic_sta_lta1_mean"]) * (((data["max_last_50000"]) + (((((data["abs_q95"]) + (((data["av_change_abs_roll_std_10"]) * 2.0)))) * (((((data["abs_std"]) + (data["max_last_50000"]))) * (((((data["iqr"]) * 2.0)) * (data["avg_last_10000"])))))))))))))) +
            0.0388200805*activation2(((((data["min_first_10000"]) * (((data["mean_change_rate"]) * (((((data["min_roll_std_100"]) + (data["max_last_50000"]))) + (((((data["mean_change_rate"]) * (((data["mean_change_abs"]) * (((data["min_roll_std_100"]) + (data["min_first_10000"]))))))) - (data["mean_change_abs"]))))))))) * 2.0)) +
            0.0389529206*activation2(((activation2((activation2((((data["max_to_min"]) + (data["max_to_min"]))))))) - (activation2((((((((((((((((((((data["min_first_10000"]) * 2.0)) + (data["max_to_min"]))) * 2.0)) * 2.0)) + (data["max_last_50000"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)))))) +
            0.0399843715*activation2((-1.0*((((((data["max_first_50000"]) * 2.0)) * (((activation2(((((((-1.0*((((data["q95_roll_std_1000"]) + (((data["q01_roll_std_10"]) + (data["av_change_abs_roll_std_1000"])))))))) * 2.0)) * 2.0)))) + (((data["q01_roll_std_10"]) + (((data["max_first_50000"]) * (data["mean_change_abs"])))))))))))) +
            0.0399453007*activation2((((((data["av_change_abs_roll_std_1000"]) + (data["mean_change_abs"]))/2.0)) * (((data["av_change_abs_roll_std_100"]) + (((data["trend"]) + (((((data["av_change_abs_roll_mean_1000"]) + ((((data["av_change_abs_roll_std_100"]) + (((data["av_change_rate_roll_std_100"]) + (((data["av_change_abs_roll_std_1000"]) + (data["min_roll_std_10"]))))))/2.0)))) * (data["av_change_abs_roll_mean_1000"]))))))))) +
            0.0399140455*activation2(((data["std_first_50000"]) * (((data["mean_change_rate_last_50000"]) * ((-1.0*((((data["avg_first_50000"]) * ((((data["max_first_10000"]) + (((data["min_roll_mean_10"]) - (((data["avg_first_50000"]) - (((((data["mean_change_rate_last_50000"]) - (data["min_last_10000"]))) * (data["min_last_10000"]))))))))/2.0))))))))))) +
            0.0399999991*activation2(((((data["min_last_10000"]) + (activation2((((data["ave_roll_mean_1000"]) + (((((((data["av_change_abs_roll_mean_100"]) + (data["av_change_abs_roll_std_10"]))) * (data["ave_roll_mean_1000"]))) * (((data["min_roll_std_1000"]) * 2.0)))))))))) * (((data["min_roll_std_1000"]) * (((((data["std_roll_mean_1000"]) * 2.0)) * 2.0)))))) +
            0.0399765596*activation2((((((data["av_change_abs_roll_mean_100"]) + (data["q99_roll_std_10"]))/2.0)) * (activation2((((activation2((((((((((((((data["avg_first_10000"]) + (data["max_roll_std_1000"]))) + (data["count_big"]))) * (73.0))) - (data["av_change_rate_roll_mean_100"]))) * 2.0)) + (data["avg_first_10000"]))))) * 2.0)))))) +
            0.0371556953*activation2(((((data["max_last_10000"]) * (((((data["std_last_50000"]) * (data["avg_last_50000"]))) + (((((((data["min_roll_std_10"]) * 2.0)) + (((data["min_last_10000"]) * (((data["max_first_50000"]) * (((((data["min_roll_std_10"]) * 2.0)) * 2.0)))))))) * (data["max_first_10000"]))))))) * 2.0)) +
            0.0399609283*activation2(((data["av_change_abs_roll_std_10"]) * ((((((((((data["min_last_50000"]) + (((data["trend"]) * (data["av_change_abs_roll_mean_10"]))))/2.0)) + (data["max_to_min"]))/2.0)) + (((data["min_last_50000"]) - (((data["min_last_50000"]) * (((data["min_last_50000"]) * (data["q05_roll_mean_1000"]))))))))/2.0)))) +
            0.0391638987*activation2(((((data["trend"]) - ((((data["min_roll_std_100"]) + (data["q001"]))/2.0)))) * (((activation2((activation2((((((data["classic_sta_lta3_mean"]) + ((((data["classic_sta_lta3_mean"]) + ((((data["kurt"]) + (data["q95_roll_mean_100"]))/2.0)))/2.0)))) * 2.0)))))) * ((-1.0*((data["min_roll_std_100"])))))))) +
            0.0398749746*activation2(((data["ave10"]) * ((-1.0*((((data["av_change_abs_roll_mean_10"]) * (((data["av_change_abs_roll_mean_100"]) + ((-1.0*(((((data["av_change_abs_roll_mean_10"]) + (((((data["classic_sta_lta2_mean"]) * 2.0)) + (((data["mean_change_rate_last_10000"]) * (data["av_change_rate_roll_mean_100"]))))))/2.0)))))))))))))) +
            0.0385778472*activation2((-1.0*(((((activation2((((((((data["trend"]) + (((data["min_roll_std_10"]) + (data["Moving_average_6000_mean"]))))) * 2.0)) * 2.0)))) + (((((-1.0*((((data["q95_roll_std_100"]) * (data["ave10"])))))) + (((data["mean_change_rate_first_50000"]) * (data["av_change_abs_roll_mean_10"]))))/2.0)))/2.0))))) +
            0.0399921872*activation2(((((((((data["min_roll_std_10"]) * ((((data["ave_roll_std_10"]) + (data["avg_last_50000"]))/2.0)))) + (((((((((data["avg_last_10000"]) + (data["min_roll_std_10"]))/2.0)) + (data["min_roll_std_10"]))/2.0)) - (data["ave_roll_std_10"]))))/2.0)) + (((((data["q95_roll_std_100"]) - (data["ave_roll_std_10"]))) * 2.0)))/2.0)) +
            0.0399921872*activation2((((((-1.0*((((data["mean_change_rate_first_10000"]) * 2.0))))) * ((((((((((data["av_change_rate_roll_std_10"]) * (data["avg_last_10000"]))) + (((data["min_last_10000"]) * (data["av_change_rate_roll_std_10"]))))) - (((data["avg_first_10000"]) / 2.0)))) + (data["av_change_rate_roll_std_1000"]))/2.0)))) * (data["av_change_abs_roll_std_1000"]))) +
            0.0399453007*activation2(((data["av_change_abs_roll_std_1000"]) * (((data["avg_last_10000"]) * (((((((data["avg_last_10000"]) * (((data["avg_last_10000"]) * (((((data["med"]) * (((data["q001"]) - (data["classic_sta_lta1_mean"]))))) - (data["min_roll_std_10"]))))))) - (data["max_roll_mean_10"]))) - (data["min_roll_std_10"]))))))) +
            0.0398749746*activation2(activation2((activation2((((((((((((-1.0*((data["av_change_abs_roll_std_100"])))) + ((-1.0*((data["mean_change_rate_first_50000"])))))/2.0)) / 2.0)) - (((data["mean_change_abs"]) * (((((((data["avg_last_50000"]) * (data["q99_roll_mean_10"]))) - (data["mean_change_rate_first_50000"]))) - (data["std_last_10000"]))))))) * 2.0)))))) +
            0.0363899209*activation2(((data["max_last_10000"]) * (((((activation2((((((((data["av_change_rate_roll_mean_100"]) + (((data["min_roll_std_1000"]) * 2.0)))) * 2.0)) * 2.0)))) + (((data["av_change_abs_roll_std_100"]) + ((-1.0*((data["mean_change_abs"])))))))) + (((((data["av_change_rate_roll_mean_10"]) * (data["max_last_50000"]))) * 2.0)))))) +
            0.0399999991*activation2(((data["min_roll_std_100"]) * (((activation2((data["min_last_10000"]))) + (((((activation2((data["std_first_10000"]))) - ((((activation2((data["min_last_10000"]))) + (((data["exp_Moving_average_30000_mean"]) * (data["min_last_10000"]))))/2.0)))) * ((((((data["avg_last_50000"]) + (data["mean_change_abs"]))/2.0)) * 2.0)))))))) +
            0.0351396762*activation2(activation2((((((data["av_change_abs_roll_std_1000"]) * (data["mean_change_rate_last_10000"]))) + (((data["med"]) * (activation2((((data["av_change_rate_roll_std_100"]) - (((((data["avg_last_10000"]) - (((data["mean_change_rate_first_10000"]) - (((data["avg_last_10000"]) * (data["mean_change_rate_last_10000"]))))))) - (data["av_change_abs_roll_std_1000"]))))))))))))) +
            0.0399921872*activation2((((activation2((((((((((data["min_roll_std_10"]) + (((data["mean_change_rate_first_50000"]) + (((((data["mean_change_rate_first_50000"]) + (data["mean_change_rate_first_50000"]))) * (data["classic_sta_lta4_mean"]))))))) * (data["av_change_rate_roll_mean_1000"]))) * 2.0)) * 2.0)))) + (((data["av_change_abs_roll_mean_10"]) * (activation2((data["med"]))))))/2.0)) +
            0.0379214697*activation2((-1.0*((((((data["max_to_min_diff"]) * (((((((((((((data["min_last_10000"]) - (data["med"]))) - (data["mean_change_rate_first_10000"]))) + (data["q05_roll_std_100"]))) + (data["q01_roll_std_1000"]))) * 2.0)) * 2.0)))) * ((-1.0*((((data["mean_change_rate_first_10000"]) * (data["min_last_10000"]))))))))))) +
            0.0399218611*activation2(((data["min_first_10000"]) * (activation2((((((((((activation2((((data["max_to_min"]) * 2.0)))) + (((((((data["min_first_50000"]) * 2.0)) + (((activation2((activation2((data["mean_change_rate_last_10000"]))))) + (data["min_roll_mean_10"]))))) * 2.0)))) * 2.0)) * 2.0)) * 2.0)))))) +
            0.0328423530*activation2((-1.0*((((activation2((((data["kurt"]) * (((((((data["mean_change_rate_last_50000"]) * 2.0)) * 2.0)) * (data["skew"]))))))) + (activation2((((data["skew"]) * (((((data["mean_change_rate_last_50000"]) * 2.0)) * (data["skew"])))))))))))) +
            0.0346083194*activation2((((((data["av_change_abs_roll_std_10"]) / 2.0)) + ((-1.0*((((activation2((data["q05_roll_mean_1000"]))) + (activation2((((((data["min_roll_std_100"]) - (data["mean_change_rate_last_50000"]))) - (((((data["q95_roll_mean_1000"]) - (data["av_change_abs_roll_std_10"]))) - (((data["av_change_abs_roll_mean_1000"]) - (data["mean_change_rate_last_50000"])))))))))))))))/2.0)) +
            0.0395155288*activation2(((data["avg_last_50000"]) * ((-1.0*(((((activation2((activation2((((((((((((data["av_change_rate_roll_mean_100"]) * ((-1.0*((data["max_to_min"])))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)))))) + ((((((data["av_change_rate_roll_std_10"]) * (data["avg_last_10000"]))) + (data["av_change_rate_roll_std_10"]))/2.0)))/2.0))))))) +
            0.0316311792*activation2(((data["max_first_50000"]) - (((data["classic_sta_lta3_mean"]) * (((((data["max_to_min_diff"]) + (data["mean_change_abs"]))) - (((data["trend"]) * (((((data["mean_change_abs"]) - (((activation2((data["max_to_min_diff"]))) * 2.0)))) - (data["av_change_abs_roll_mean_100"]))))))))))) +
            0.0399609283*activation2(((data["trend"]) * (((((((data["min_roll_std_1000"]) * (data["std_last_10000"]))) * ((((data["av_change_abs_roll_mean_10"]) + (((data["std_last_50000"]) - ((((((data["av_change_rate_roll_mean_10"]) * (data["q05_roll_std_1000"]))) + (data["min_roll_std_1000"]))/2.0)))))/2.0)))) + (((data["min_roll_std_1000"]) - (data["q05_roll_std_1000"]))))))) +
            0.0362180099*activation2((((activation2((((((((((((data["q01_roll_std_100"]) - (data["mean_change_rate_last_10000"]))) - ((((data["av_change_abs_roll_mean_10"]) + (data["abs_max_roll_mean_100"]))/2.0)))) - (0.2525250018))) * (data["min_roll_std_10"]))) * (73.0))))) + (((data["min_roll_mean_1000"]) + (data["abs_max_roll_mean_100"]))))/2.0)) +
            0.0399999991*activation2((((activation2((((73.0) * (((((73.0) * (data["av_change_abs_roll_std_100"]))) - (data["av_change_abs_roll_mean_100"]))))))) + (activation2((((((((((data["av_change_rate_roll_mean_100"]) - (data["av_change_abs_roll_std_100"]))) - (data["av_change_abs_roll_std_100"]))) - (data["std_first_50000"]))) - (data["av_change_abs_roll_std_100"]))))))/2.0)) +
            0.0374213718*activation2((((-1.0*((((data["q05"]) * (((data["max_last_10000"]) * (((((data["min_roll_std_10"]) - (data["std_last_10000"]))) - (data["std_last_10000"])))))))))) + (((data["max_last_50000"]) * (((data["mean_change_rate_last_10000"]) - (activation2((((((data["av_change_rate_roll_std_10"]) * 2.0)) * 2.0)))))))))) +
            0.0331470966*activation2(((((((((data["abs_max_roll_std_1000"]) * 2.0)) * 2.0)) * 2.0)) * (((((data["mean_change_rate_last_50000"]) * ((((((data["abs_max_roll_std_1000"]) * (((data["mean_change_abs"]) * 2.0)))) + ((((data["std_first_50000"]) + (data["mean_change_rate_last_50000"]))/2.0)))/2.0)))) + (((data["mean_change_abs"]) * (data["av_change_rate_roll_std_1000"]))))))) +
            0.0396796241*activation2(((activation2((activation2((((((data["avg_first_10000"]) * (((((data["Moving_average_3000_mean"]) + (data["skew"]))) * 2.0)))) + (activation2(((-1.0*((((((data["ave_roll_mean_100"]) + (((data["av_change_abs_roll_std_1000"]) + (((data["avg_first_10000"]) * 2.0)))))) * 2.0))))))))))))) / 2.0)) +
            0.0399765596*activation2(((activation2((((((data["av_change_abs_roll_mean_10"]) + (data["exp_Moving_average_300_mean"]))) * (((((data["classic_sta_lta4_mean"]) + (data["av_change_abs_roll_mean_10"]))) + (((((data["av_change_abs_roll_mean_10"]) + (((activation2((((activation2((data["avg_last_50000"]))) * 2.0)))) * 2.0)))) * (data["av_change_abs_roll_mean_10"]))))))))) / 2.0)) +
            0.0396014862*activation2((((((data["max_last_50000"]) + (((((activation2((activation2((((((activation2((activation2(((((data["q95"]) + (data["mean_change_abs"]))/2.0)))))) + (((data["av_change_abs_roll_std_100"]) * 2.0)))) * 2.0)))))) * 2.0)) - (data["av_change_abs_roll_std_100"]))))/2.0)) * (data["classic_sta_lta4_mean"]))) +
            0.0317015015*activation2(((activation2((((((data["avg_last_10000"]) - (((data["iqr"]) - (((((data["avg_last_10000"]) - (((data["iqr"]) * (data["av_change_rate_roll_std_100"]))))) - (((((((((data["min_roll_std_100"]) * (data["av_change_rate_roll_std_100"]))) * 2.0)) * 2.0)) * 2.0)))))))) * 2.0)))) / 2.0)) +
            0.0399296731*activation2((-1.0*((((data["q05_roll_std_1000"]) * (((((((((data["avg_first_10000"]) + (data["mean_change_abs"]))/2.0)) * (data["std_last_50000"]))) + (((((((data["avg_last_10000"]) + (((data["mean_change_rate_first_10000"]) * (data["mean_change_abs"]))))/2.0)) + (data["max_to_min_diff"]))/2.0)))/2.0))))))) +
            0.0377104878*activation2((((activation2((activation2((((((((data["count_big"]) + (((data["min_last_10000"]) * (data["min_first_50000"]))))) * 2.0)) * 2.0)))))) + ((((data["count_big"]) + ((-1.0*((((data["max_roll_mean_100"]) - (activation2((((data["min_last_10000"]) * (data["avg_last_10000"])))))))))))/2.0)))/2.0)) +
            0.0309357308*activation2(((activation2((((((((data["max_to_min"]) * (data["classic_sta_lta2_mean"]))) + (((data["classic_sta_lta2_mean"]) * (((data["max_to_min"]) * (data["classic_sta_lta2_mean"]))))))) * 2.0)))) / 2.0)) +
            0.0355303772*activation2(((((activation2((activation2((((((((((data["av_change_abs_roll_std_10"]) - (activation2((((data["skew"]) * 2.0)))))) * (data["mean_change_rate_first_10000"]))) * (data["mean_change_rate_first_10000"]))) * (((data["mean_change_rate_first_10000"]) + (data["av_change_abs_roll_std_10"]))))))))) * (data["mean_change_rate_first_10000"]))) * (data["mean_change_rate_first_10000"]))) +
            0.0399843715*activation2(((data["av_change_abs_roll_std_1000"]) * (((((((data["trend"]) + (activation2((((data["min_roll_std_1000"]) * ((((8.0)) + (((data["min_roll_std_1000"]) * ((8.0)))))))))))/2.0)) + ((-1.0*((((data["q01_roll_std_1000"]) + (((data["q01_roll_std_1000"]) * (data["mean_change_rate_first_50000"])))))))))/2.0)))) +
            0.0398437195*activation2((-1.0*((((data["std_first_10000"]) * ((((((data["q99"]) / 2.0)) + (((data["skew"]) + (((((data["std_first_10000"]) * 2.0)) * (((data["std_first_10000"]) * (((data["std_first_10000"]) * (((data["std_first_10000"]) - (((data["MA_700MA_std_mean"]) * 2.0)))))))))))))/2.0))))))) +
            0.0399843715*activation2((-1.0*((((data["mean_change_abs"]) * ((((((data["av_change_abs_roll_mean_1000"]) + ((((((((data["abs_max_roll_mean_10"]) + (((((data["min_roll_mean_100"]) * (data["av_change_abs_roll_mean_1000"]))) * (data["min_roll_std_10"]))))) - ((((data["min_roll_std_10"]) + (data["av_change_abs_roll_std_100"]))/2.0)))) + (data["skew"]))/2.0)))/2.0)) / 2.0))))))) +
            0.0295057632*activation2(((((((((((((((data["q99_roll_mean_100"]) + (((data["q99_roll_mean_100"]) + (data["q95_roll_mean_10"]))))) * (((data["q05_roll_std_100"]) + ((((((data["av_change_abs_roll_std_1000"]) - (data["min_roll_std_100"]))) + (data["av_change_abs_roll_mean_100"]))/2.0)))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * (data["av_change_abs_roll_std_1000"]))) +
            0.0346864611*activation2(((activation2((((((((data["mean_change_rate"]) * (((data["mean_change_rate_first_10000"]) + (((data["mean_change_rate_first_10000"]) + (data["mean_change_rate_last_50000"]))))))) + (((data["q01_roll_std_100"]) * (((((data["mean_change_rate_first_10000"]) * (((data["q99_roll_std_10"]) * 2.0)))) + (data["mean_change_rate_first_10000"]))))))) * 2.0)))) / 2.0)) +
            0.0397265106*activation2(activation2((((((data["min_roll_std_100"]) * ((-1.0*((((data["av_change_abs_roll_mean_100"]) * ((((data["q05_roll_mean_100"]) + (data["mean_change_rate_first_10000"]))/2.0))))))))) - (((((((-1.0*((data["classic_sta_lta4_mean"])))) / 2.0)) + ((((data["av_change_rate_roll_mean_100"]) + ((((data["Moving_average_6000_mean"]) + (data["mean_change_rate_first_10000"]))/2.0)))/2.0)))/2.0)))))) +
            0.0399687439*activation2(((activation2((((data["min_first_50000"]) * (((((((((activation2((((((data["q01_roll_std_1000"]) + (activation2(((-1.0*((((((data["skew"]) + (data["classic_sta_lta2_mean"]))) + (data["q95_roll_std_10"])))))))))) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)))))) / 2.0)) +
            0.0365696438*activation2(((activation2((((((((((((((((data["mean_change_rate_last_10000"]) * (((data["ave_roll_std_10"]) - (((data["q95"]) - ((((((data["abs_mean"]) + (((data["iqr"]) / 2.0)))/2.0)) / 2.0)))))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) / 2.0)) +
            0.0393358096*activation2(((((data["max_roll_std_10"]) * (((((data["q95_roll_mean_100"]) + (((data["q95_roll_mean_100"]) + (((data["q05_roll_mean_1000"]) * (data["classic_sta_lta2_mean"]))))))) * (((data["max_roll_std_10"]) * (((((data["skew"]) * (((data["q05_roll_mean_100"]) + (data["classic_sta_lta2_mean"]))))) * 2.0)))))))) * 2.0)) +
            0.0381871462*activation2((((activation2((((((((data["q05_roll_mean_10"]) - (((data["q05"]) + (data["ave_roll_std_100"]))))) * (73.0))) * (73.0))))) + (((((((data["abs_q05"]) * (((data["std_roll_mean_1000"]) * (data["av_change_rate_roll_std_10"]))))) * 2.0)) - (data["q05"]))))/2.0)) +
            0.0383043550*activation2(((data["max_last_50000"]) * (((((data["av_change_rate_roll_std_100"]) * 2.0)) * (activation2((((((((((((((activation2((((data["mean_change_abs"]) - (data["classic_sta_lta3_mean"]))))) - (data["classic_sta_lta3_mean"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))))))) +
            0.0397812054*activation2(activation2((activation2((((data["min_roll_mean_100"]) * (((((data["av_change_rate_roll_mean_1000"]) - (data["min_roll_mean_100"]))) - (((((((data["min_last_50000"]) * (((data["av_change_rate_roll_mean_1000"]) - (activation2((((data["min_roll_std_10"]) - (((data["min_roll_mean_100"]) * 2.0)))))))))) * 2.0)) * 2.0)))))))))) +
            0.0399921872*activation2((-1.0*((((((activation2((((((((((((((((((data["mean_change_abs"]) * 2.0)) - ((((((-1.0*((data["mean_change_rate_last_50000"])))) * 2.0)) - (data["trend"]))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) / 2.0)) / 2.0))))) +
            0.0329673775*activation2(activation2(((-1.0*((((data["std_first_10000"]) * (((((((((data["av_change_abs_roll_mean_10"]) + (data["max_last_10000"]))) * (((((data["max_last_10000"]) * (0.8873239756))) + (data["classic_sta_lta2_mean"]))))) * 2.0)) * (data["max_last_10000"])))))))))) +
            0.0367962494*activation2(((data["avg_last_10000"]) * (((((data["std_roll_mean_1000"]) * ((-1.0*(((((((((data["mean_change_rate_first_50000"]) - (((data["mean_change_abs"]) * (data["max_last_10000"]))))) + (data["max_to_min_diff"]))/2.0)) - (((data["max_last_10000"]) * (((data["max_to_min"]) * (data["q05_roll_mean_100"])))))))))))) * 2.0)))) +
            0.0399921872*activation2(((data["av_change_abs_roll_mean_10"]) * (((((data["av_change_abs_roll_std_1000"]) - ((((data["av_change_abs_roll_mean_10"]) + ((((data["min_roll_std_10"]) + (((((((data["av_change_rate_roll_mean_10"]) * (data["av_change_abs_roll_std_10"]))) + (data["av_change_abs_roll_mean_10"]))) * (data["av_change_abs_roll_mean_10"]))))/2.0)))/2.0)))) * (data["min_roll_std_10"]))))) +
            0.0395311601*activation2(((activation2(((-1.0*((((data["std_first_10000"]) + (data["min_first_10000"])))))))) + (activation2((((data["max_roll_std_10"]) * (((data["q95_roll_mean_100"]) * ((((((((data["mean_change_abs"]) + (data["max_roll_std_10"]))/2.0)) - ((-1.0*((data["min_first_10000"])))))) + (data["mean_change_rate_last_50000"]))))))))))) +
            0.0399296731*activation2(((((data["min_roll_std_10"]) + (((data["min_roll_std_10"]) * (data["med"]))))) * ((((((data["classic_sta_lta1_mean"]) + (data["min_roll_mean_1000"]))/2.0)) - (((data["q05_roll_std_10"]) * ((((((data["med"]) * ((((data["classic_sta_lta1_mean"]) + (data["min_roll_mean_1000"]))/2.0)))) + (data["med"]))/2.0)))))))) +
            0.0399453007*activation2(activation2((((((((activation2((data["min_roll_std_10"]))) * (data["std_roll_mean_1000"]))) * 2.0)) + ((((((((((((data["max_to_min_diff"]) * 2.0)) * (data["std_last_50000"]))) + (data["min_roll_std_10"]))/2.0)) / 2.0)) + (((data["min_roll_std_10"]) * (activation2((data["max_to_min_diff"]))))))))))) +
            0.0394999012*activation2(((data["mean_change_rate_first_10000"]) * (((data["min_first_10000"]) * ((((((((data["min_roll_std_10"]) * (((data["av_change_abs_roll_std_100"]) + (data["av_change_abs_roll_std_100"]))))) + ((((data["mean_change_rate_first_10000"]) + (((data["min_roll_std_10"]) * (data["mean_change_rate_first_10000"]))))/2.0)))/2.0)) + (((data["min_roll_std_10"]) - (data["av_change_abs_roll_std_100"]))))))))) +
            0.0399687439*activation2(((data["max_first_10000"]) * (((((data["std_first_10000"]) * (((data["max_first_10000"]) * (((data["std_first_10000"]) * (((activation2((((data["std_first_10000"]) * (data["av_change_abs_roll_mean_10"]))))) + (data["std_roll_mean_100"]))))))))) - (((((data["av_change_abs_roll_mean_10"]) * (data["min_roll_std_100"]))) / 2.0)))))) +
            0.0397499502*activation2(((data["trend"]) * ((((((data["std_first_50000"]) + (activation2((activation2((((data["av_change_rate_roll_mean_10"]) + (((((((data["av_change_rate_roll_mean_1000"]) + ((((data["min_roll_std_100"]) + (((data["Hilbert_mean"]) + (data["av_change_rate_roll_mean_10"]))))/2.0)))) * 2.0)) * 2.0)))))))))/2.0)) * (data["min_roll_std_100"]))))) +
            0.0274897423*activation2((((activation2((((((((((((((data["q95_roll_std_100"]) - (data["ave_roll_std_10"]))) - (data["abs_max_roll_mean_10"]))) - (data["max_last_50000"]))) - (((data["min_first_50000"]) + (data["ave_roll_std_10"]))))) * 2.0)) * 2.0)))) + (((((data["q95_roll_std_100"]) - (data["ave_roll_std_10"]))) * 2.0)))/2.0)) +
            0.0399531126*activation2(((((data["mean_change_rate_first_10000"]) * (activation2((((((((((data["av_change_abs_roll_mean_10"]) + (data["mean_change_rate_last_10000"]))/2.0)) * (((data["avg_first_10000"]) - (activation2((data["av_change_abs_roll_mean_100"]))))))) + (activation2((activation2((data["avg_first_10000"]))))))/2.0)))))) * ((((-1.0*((data["mean_change_abs"])))) * 2.0)))) +
            0.0340613388*activation2(((data["skew"]) * (((data["max_first_50000"]) * (((((((((activation2((((((((data["iqr"]) * 2.0)) - (data["av_change_abs_roll_mean_10"]))) - (data["std_last_10000"]))))) + (data["av_change_abs_roll_mean_10"]))) - (((data["av_change_rate_roll_mean_10"]) * (data["avg_first_50000"]))))) * 2.0)) * 2.0)))))) +
            0.0399921872*activation2((((((data["min_roll_std_10"]) + (((data["min_roll_std_10"]) * (data["avg_first_10000"]))))/2.0)) * (activation2((((73.0) * (((data["avg_first_10000"]) * (((data["min_roll_std_10"]) + (((73.0) * (((73.0) * ((-1.0*((data["av_change_abs_roll_std_1000"])))))))))))))))))) +
            0.0356475860*activation2(activation2((((data["av_change_abs_roll_mean_1000"]) * ((((((((((((((data["avg_last_50000"]) + (data["kurt"]))) + (0.2783510089))/2.0)) + (data["av_change_rate_roll_std_1000"]))) + (0.2783510089))) + (data["kurt"]))) * ((((activation2((data["avg_first_10000"]))) + (data["mean_change_rate_first_50000"]))/2.0)))))))) +
            0.0329986326*activation2((((((((((data["kurt"]) + (data["q05_roll_mean_10"]))) * (data["min_roll_mean_10"]))) + (activation2((((((data["q05_roll_mean_10"]) - (((((data["max_to_min"]) - (data["kurt"]))) + (((data["q05_roll_mean_10"]) * (data["max_to_min"]))))))) * 2.0)))))/2.0)) / 2.0)) +
            0.0399999991*activation2(((data["std_roll_mean_1000"]) * (((data["med"]) * ((((-3.0) + (((((data["avg_first_10000"]) * 2.0)) * (((data["mean_change_rate_first_10000"]) - (((((((data["std_roll_mean_1000"]) / 2.0)) * (((data["std_roll_mean_1000"]) * (-3.0))))) * (data["abs_max_roll_mean_1000"]))))))))/2.0)))))) +
            0.0303496774*activation2((-1.0*((((activation2((activation2((((((((data["classic_sta_lta2_mean"]) + (((data["av_change_abs_roll_std_10"]) + (((data["av_change_abs_roll_mean_10"]) + (data["classic_sta_lta1_mean"]))))))) * (data["abs_q05"]))) * (3.6923100948))))))) / 2.0))))) +
            0.0399687439*activation2((((-1.0*((data["max_first_50000"])))) * (((((data["std_roll_mean_1000"]) + (((data["av_change_abs_roll_std_10"]) * (((data["ave_roll_mean_10"]) + (((data["med"]) + (1.0))))))))) * (((data["avg_last_50000"]) + (((data["med"]) + (2.0769200325))))))))) +
            0.0399843715*activation2(((activation2((((data["av_change_rate_roll_mean_100"]) * (((activation2((((((((((data["av_change_abs_roll_mean_100"]) * 2.0)) + (data["mean_change_rate_first_50000"]))) * 2.0)) * 2.0)))) - (((data["av_change_abs_roll_mean_100"]) * (((((data["av_change_rate_roll_std_1000"]) - (data["classic_sta_lta2_mean"]))) - (data["classic_sta_lta2_mean"]))))))))))) / 2.0)) +
            0.0399999991*activation2(((data["mean_change_abs"]) * ((((data["av_change_abs_roll_std_100"]) + (activation2((((((((((((((((((data["min_first_10000"]) - (data["av_change_rate_roll_std_10"]))) - (((data["av_change_abs_roll_std_100"]) * (3.6923100948))))) * 2.0)) * 2.0)) * 2.0)) + (data["mean_change_abs"]))) * 2.0)) * 2.0)))))/2.0)))) +
            0.0285446383*activation2(((activation2((((((73.0) * (data["mean_change_rate_first_10000"]))) + (((((((((data["trend"]) + (((data["av_change_abs_roll_mean_10"]) + (data["min_last_50000"]))))) + (data["mean_change_rate_first_10000"]))) * 2.0)) * 2.0)))))) + ((-1.0*((activation2((data["mean_change_rate_first_10000"])))))))) +
            0.0399687439*activation2(((((activation2((((((((((((((((((((activation2(((((((data["kurt"]) + (data["classic_sta_lta4_mean"]))/2.0)) - (data["max_first_10000"]))))) + (data["skew"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) / 2.0)) / 2.0)) +
            0.0398593470*activation2(((activation2(((-1.0*((activation2((((data["avg_first_10000"]) * (((data["min_roll_std_10"]) - (((((data["mean_change_rate_last_10000"]) * (data["min_roll_std_10"]))) + ((((data["mean_change_abs"]) + ((((((data["ave_roll_mean_100"]) + (data["avg_first_10000"]))/2.0)) + (data["skew"]))))/2.0))))))))))))))) / 2.0)) +
            0.0399296731*activation2(((activation2((activation2((activation2((activation2((((((((data["mean_change_abs"]) * ((((data["av_change_abs_roll_mean_100"]) + (0.7446810007))/2.0)))) - (((activation2((data["skew"]))) + ((((data["trend"]) + (0.7446810007))/2.0)))))) * 2.0)))))))))) / 2.0)) +
            0.0387341268*activation2((((((((data["min_roll_std_100"]) * (activation2((((((((data["av_change_rate_roll_std_100"]) * 2.0)) + (((data["abs_q05"]) + (activation2((data["iqr"]))))))) * 2.0)))))) + (activation2((((activation2((((data["iqr"]) * 2.0)))) - (data["min_roll_std_10"]))))))/2.0)) / 2.0)) +
            0.0384997055*activation2(((data["av_change_abs_roll_mean_1000"]) * (((activation2((((((activation2((data["std_first_50000"]))) + ((((-1.0*((data["max_to_min"])))) * (data["std_first_10000"]))))) + (((data["av_change_abs_roll_mean_1000"]) * (data["min_roll_mean_1000"]))))))) + ((((-1.0*((data["min_roll_std_10"])))) * (data["std_first_10000"]))))))) +
            0.0384918936*activation2(((activation2((activation2((((((73.0) * (((73.0) * (((((data["q95_roll_std_100"]) + (data["std_roll_mean_1000"]))) * 2.0)))))) * (((data["std_first_50000"]) + (((activation2((activation2((data["max_roll_std_1000"]))))) * (data["kurt"]))))))))))) / 2.0)) +
            0.0343192033*activation2(((((-1.0) - (activation2((((((((0.8873239756) + (data["mean_change_abs"]))) - (data["Hann_window_mean"]))) * (((((((((data["Hann_window_mean"]) - (data["min_last_10000"]))) * 2.0)) * 2.0)) + (data["av_change_rate_roll_std_100"]))))))))) / 2.0)) +
            0.0372416489*activation2(((activation2((((((data["classic_sta_lta2_mean"]) + (((data["q95_roll_mean_1000"]) * (data["classic_sta_lta2_mean"]))))) + ((((-1.0*((((((data["std_last_10000"]) + (data["MA_400MA_BB_low_mean"]))) + (activation2((((data["min_roll_std_100"]) + (data["classic_sta_lta2_mean"])))))))))) * (data["avg_last_10000"]))))))) / 2.0)) +
            0.0377808139*activation2(activation2((((((data["mean_change_rate_last_10000"]) * (((((((data["mean_change_rate_first_10000"]) + ((((data["min_roll_std_10"]) + ((-1.0*((-3.0)))))/2.0)))/2.0)) + (data["Moving_average_1500_mean"]))/2.0)))) * (activation2((((((data["mean_change_rate_last_10000"]) + (((data["av_change_abs_roll_std_1000"]) + (1.9019600153))))) * 2.0)))))))) +
            0.0369056463*activation2(activation2((activation2((((((data["max_first_50000"]) * (data["max_first_50000"]))) * ((((((((data["max_first_50000"]) + (data["kurt"]))/2.0)) - (data["avg_last_50000"]))) - (((activation2((((data["mean_change_rate_first_10000"]) * 2.0)))) - (((data["kurt"]) - (0.5555559993))))))))))))))

In [16]:
def GPI_erf(data):
    return (5.591662 +
            0.0399999991*erf(((data["mean_change_rate_first_10000"]) - (((73.0) * ((((((7.0) * (((((((((data["q95"]) + (((0.3183098733) + (data["q05_roll_std_100"]))))) * 2.0)) * 2.0)) + (((data["q95_roll_std_10"]) + (data["iqr"]))))))) + (data["iqr"]))/2.0)))))) +
            0.0399999991*erf(((((3.6923100948) + (data["Moving_average_6000_mean"]))) * (((73.0) * (((data["MA_400MA_BB_low_mean"]) - (((((data["Moving_average_6000_mean"]) * (data["q05_roll_std_10"]))) + (((data["iqr"]) + (((data["q05_roll_std_10"]) * (((3.6923100948) * 2.0)))))))))))))) +
            0.0399999991*erf(((((((((((((((((((((((data["q05"]) * 2.0)) * 2.0)) - (data["iqr"]))) - (data["max_roll_std_1000"]))) * 2.0)) * 2.0)) * 2.0)) - (data["iqr"]))) - (3.6923100948))) - (((data["q05_roll_std_10"]) * (73.0))))) * 2.0)) +
            0.0399999991*erf(((((((((((((((((((data["q05_roll_std_10"]) * (((((-3.0) * 2.0)) * 2.0)))) - (((data["iqr"]) + (data["max_roll_std_1000"]))))) * 2.0)) * 2.0)) * 2.0)) - (data["max_roll_mean_100"]))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*erf(((73.0) * ((((data["max_first_50000"]) + ((((data["max_first_50000"]) + (((73.0) * (((data["q05"]) - (((data["q05_roll_std_100"]) + (((0.5555559993) + (((data["mad"]) + (data["q95"]))))))))))))/2.0)))/2.0)))) +
            0.0399999991*erf(((((((data["q05_roll_std_10"]) * (73.0))) * 2.0)) - ((((((((data["q05_roll_std_10"]) * (73.0))) + (((((((((data["iqr"]) + (((data["MA_1000MA_std_mean"]) + (data["abs_max_roll_mean_10"]))))) * 2.0)) * 2.0)) * 2.0)))/2.0)) * (73.0))))) +
            0.0399999991*erf(((((data["mean_change_abs"]) - (((((data["q95"]) + (((data["q95"]) + (((((data["q05_roll_std_100"]) / 2.0)) + (((((((0.3183098733) * 2.0)) - (data["q05"]))) - (data["q05"]))))))))) * (73.0))))) * 2.0)) +
            0.0399999991*erf((-1.0*(((((((7.48210620880126953)) * 2.0)) + (((((data["iqr"]) + ((((-1.0*((data["avg_first_10000"])))) * 2.0)))) + (((73.0) * (((data["iqr"]) + (((((data["q05_roll_std_10"]) * 2.0)) * 2.0))))))))))))) +
            0.0399999991*erf((((((data["q01_roll_std_10"]) + (((((data["q05_roll_mean_10"]) - (((((data["q05_roll_std_100"]) - (((data["q05"]) - (0.3026320040))))) + (((((((0.5434780121) + (0.3026320040))/2.0)) + (data["iqr"]))/2.0)))))) * (73.0))))/2.0)) * (73.0))) +
            0.0399999991*erf(((((data["trend"]) + (((data["q95_roll_std_10"]) + (data["trend"]))))) - (((73.0) * (((((((data["ave_roll_std_10"]) + (((((((data["q95_roll_mean_10"]) + (data["q05_roll_std_1000"]))/2.0)) + (data["q05_roll_std_1000"]))/2.0)))/2.0)) + ((((0.5434780121) + (data["q95_roll_mean_10"]))/2.0)))/2.0)))))) +
            0.0399999991*erf(((73.0) - (((((73.0) * 2.0)) * (((73.0) * ((((((((((data["q05_roll_std_1000"]) / 2.0)) + (data["iqr"]))/2.0)) + (data["q05_roll_std_10"]))) + (((0.2183910012) + (data["q95"]))))))))))) +
            0.0399999991*erf(((73.0) * (((73.0) * (((((73.0) * (((data["q05_roll_mean_10"]) - (((((data["q05_roll_std_100"]) + (data["q95"]))) + (0.5434780121))))))) + (data["q05_roll_std_100"]))))))) +
            0.0399999991*erf((-1.0*((((73.0) * (((73.0) * (((73.0) * (((data["q05_roll_std_100"]) + ((((data["q95_roll_mean_10"]) + ((((data["Moving_average_3000_mean"]) + (((((data["iqr"]) + (0.9756100178))) + (data["abs_mean"]))))/2.0)))/2.0))))))))))))) +
            0.0399999991*erf(((73.0) * (((((data["classic_sta_lta3_mean"]) + (data["mad"]))) + (((((((73.0) * (((data["q05_roll_mean_10"]) - (((data["q05_roll_std_100"]) + (((data["mad"]) + (0.5555559993))))))))) - (data["iqr"]))) - (data["iqr"]))))))) +
            0.0399999991*erf(((((erf((data["q95_roll_std_10"]))) + (((((((data["q05_roll_std_10"]) + ((((((0.2439019978) + (data["ave10"]))/2.0)) / 2.0)))) * 2.0)) * 2.0)))) * (((data["q05_roll_std_10"]) - (((73.0) * (((73.0) * (73.0))))))))) +
            0.0399999991*erf(((3.6923100948) * (((73.0) * (((3.6923100948) * ((((((-1.0*((data["q99_roll_mean_1000"])))) * (data["avg_last_50000"]))) - (((data["q05_roll_std_10"]) * (((3.6923100948) + (((3.6923100948) + (data["abs_max_roll_mean_100"]))))))))))))))) +
            0.0399999991*erf(((((((data["Moving_average_1500_mean"]) + (((((((((data["MA_400MA_BB_low_mean"]) + ((((-1.0*((data["q05_roll_std_1000"])))) - (data["iqr"]))))) * (73.0))) - (data["min_roll_std_1000"]))) - (data["q05_roll_std_1000"]))))) * (73.0))) * (73.0))) +
            0.0399999991*erf(((73.0) - (((73.0) * (((73.0) * (((data["q05_roll_std_10"]) + (erf(((((((data["exp_Moving_average_3000_mean"]) + (((data["iqr"]) + (erf((((data["q05_roll_std_100"]) + (data["exp_Moving_average_30000_mean"]))))))))/2.0)) / 2.0)))))))))))) +
            0.0399999991*erf(((73.0) * (((73.0) * (((data["q05_roll_mean_10"]) - (((erf((erf((erf((erf((erf((data["q05_roll_std_100"]))))))))))) + ((((data["q95_roll_std_10"]) + (erf((2.8437500000))))/2.0)))))))))) +
            0.0399999991*erf(((((((data["iqr"]) / 2.0)) + (((data["mean"]) + (((data["q05_roll_std_100"]) + (((data["q05_roll_std_10"]) * 2.0)))))))) * ((-1.0*((((73.0) - (((((((((data["mean"]) * 2.0)) * 2.0)) * 2.0)) * 2.0))))))))) +
            0.0399999991*erf(((((((((0.5217390060) - (((((data["q05_roll_std_10"]) * ((-1.0*((7.0)))))) + (data["exp_Moving_average_3000_mean"]))))) * ((-1.0*((73.0)))))) - (((data["classic_sta_lta4_mean"]) * (7.0))))) - ((((7.0) + (data["classic_sta_lta4_mean"]))/2.0)))) +
            0.0399999991*erf(((((((((((((((((((erf(((-1.0*((((data["q05_roll_mean_100"]) + (0.2183910012)))))))) - ((((data["q05_roll_std_100"]) + (((data["q05_roll_std_10"]) * 2.0)))/2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) - (data["exp_Moving_average_30000_mean"]))) * 2.0)) +
            0.0399999991*erf(((data["trend"]) - (((73.0) * ((((3.6923100948) + ((((9.0)) * (((erf((erf((erf((erf((erf((erf((erf((data["q05_roll_std_100"]))))))))))))))) - (data["q05_roll_mean_10"]))))))/2.0)))))) +
            0.0399999991*erf(((data["q05_roll_std_1000"]) - (((73.0) * (((data["avg_last_50000"]) + (((((73.0) * 2.0)) * (((data["ave_roll_std_100"]) + (((data["Moving_average_6000_mean"]) + (((data["iqr"]) + (((data["q05_roll_std_1000"]) + (((data["q95"]) * 2.0)))))))))))))))))) +
            0.0399999991*erf(((((data["q05_roll_mean_10"]) - (2.0769200325))) - ((((((6.08570814132690430)) * (((2.0769200325) + (((((((((data["q05_roll_std_10"]) - (data["q05_roll_mean_10"]))) * 2.0)) * 2.0)) * 2.0)))))) + (((((data["av_change_rate_roll_std_100"]) - (data["avg_last_10000"]))) * 2.0)))))) +
            0.0399999991*erf((-1.0*((((((((((((((((((((((((((((((((data["q05_roll_std_1000"]) + (data["q05_roll_mean_100"]))/2.0)) * 2.0)) + (data["iqr"]))/2.0)) + (data["q95_roll_mean_10"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0))))) +
            0.0399999991*erf((((-1.0*((((((((((data["q05_roll_std_100"]) - (erf(((-1.0*((((data["MA_400MA_std_mean"]) + (((data["exp_Moving_average_300_mean"]) * (((data["q05_roll_std_100"]) * 2.0))))))))))))) * 2.0)) * 2.0)) + (erf((data["q05_roll_std_100"])))))))) * (73.0))) +
            0.0399999991*erf(((((data["abs_q05"]) - ((((((2.0769200325) * ((((((73.0) * ((((((0.5555559993) + (((data["q05_roll_std_100"]) + (0.7446810007))))/2.0)) + (data["q05_roll_std_100"]))))) + (0.5555559993))/2.0)))) + (0.5555559993))/2.0)))) * 2.0)) +
            0.0399999991*erf((((10.75933361053466797)) - (((7.0) * (((7.0) * (((7.0) * ((((((data["q95_roll_mean_10"]) + (data["q05_roll_std_100"]))) + ((((data["q95_roll_mean_10"]) + (((data["iqr"]) + (((data["Moving_average_700_mean"]) * 2.0)))))/2.0)))/2.0)))))))))) +
            0.0399999991*erf(((73.0) * (((((data["q05_roll_std_1000"]) + ((((data["q05_roll_std_10"]) + ((((((data["q05_roll_std_1000"]) + (erf((data["q05_roll_std_1000"]))))/2.0)) - (((data["q95_roll_mean_1000"]) - (erf((73.0))))))))/2.0)))) * (((2.8437500000) - (73.0))))))) +
            0.0399999991*erf(((((((((-3.0) + (data["Moving_average_3000_mean"]))) - (((73.0) * (((data["ave_roll_mean_10"]) - ((((((data["q05_roll_std_100"]) + (data["q95_roll_mean_10"]))/2.0)) * (((-3.0) - (((data["Moving_average_700_mean"]) + (data["q95_roll_mean_10"]))))))))))))) * 2.0)) * 2.0)) +
            0.0399999991*erf(((((((data["classic_sta_lta3_mean"]) * 2.0)) + (data["q05_roll_mean_100"]))) - (((73.0) * (((((((0.6582279801) + (data["q05_roll_std_10"]))) - (((data["q05_roll_mean_10"]) * 2.0)))) + ((((-1.0*((data["abs_q05"])))) + (((data["q05_roll_std_100"]) * 2.0)))))))))) +
            0.0399999991*erf(((((data["skew"]) + (((data["kurt"]) + (data["med"]))))) - ((((7.02540111541748047)) * (((((data["iqr"]) + (((data["q95_roll_std_10"]) + (((((data["q95_roll_mean_10"]) + (((data["iqr"]) * (data["Moving_average_1500_mean"]))))) * 2.0)))))) * 2.0)))))) +
            0.0399999991*erf(((((data["q05_roll_std_100"]) - (((-2.0) + (((((data["iqr"]) * (-2.0))) - ((-1.0*((data["av_change_rate_roll_std_10"])))))))))) - ((((((data["q05_roll_std_100"]) + (erf((((data["q05_roll_std_100"]) * (data["ave_roll_mean_10"]))))))/2.0)) * (73.0))))) +
            0.0399999991*erf(((((((((((data["q05_roll_std_10"]) + (((data["av_change_abs_roll_std_1000"]) - (((data["av_change_abs_roll_mean_10"]) * 2.0)))))) * 2.0)) + (data["av_change_abs_roll_std_1000"]))) * 2.0)) - (((73.0) * (((((data["abs_max_roll_mean_1000"]) + (((0.5555559993) + (data["q05_roll_std_10"]))))) * 2.0)))))) +
            0.0399999991*erf((((((((((((((((((erf((((((data["q95_roll_mean_10"]) + (data["q95_roll_std_100"]))) - (data["q05_roll_std_1000"]))))) + (data["q95_roll_std_100"]))/2.0)) + (data["q95_roll_std_100"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) - (((73.0) * (data["q95_roll_mean_10"]))))) * 2.0)) +
            0.0399999991*erf(((73.0) - (((((((((((data["q05_roll_std_10"]) * 2.0)) - (data["ave10"]))) + (((data["q05_roll_std_100"]) + (0.9756100178))))) * (73.0))) * (((73.0) * 2.0)))))) +
            0.0399999991*erf(((((data["Moving_average_6000_mean"]) + (((7.0) + (((data["ave10"]) * 2.0)))))) * (((((((((data["MA_700MA_BB_low_mean"]) * (data["avg_last_50000"]))) + ((-1.0*((((data["q95_roll_mean_10"]) * 2.0))))))) - (((data["ave_roll_mean_10"]) + (data["q05_roll_std_100"]))))) * 2.0)))) +
            0.0399999991*erf(((73.0) * (((data["q05_roll_std_100"]) - (((((((data["iqr"]) - (erf((((73.0) * (((data["ave10"]) - (((data["iqr"]) + (((((data["q05_roll_std_1000"]) + (0.5434780121))) * 2.0)))))))))))) * 2.0)) * 2.0)))))) +
            0.0399999991*erf(((((((((((((((data["exp_Moving_average_30000_mean"]) * (((data["exp_Moving_average_30000_mean"]) + ((-1.0*((data["q01_roll_std_10"])))))))) - (((data["q05_roll_std_10"]) + (((data["q05_roll_std_10"]) + (((data["q95_roll_mean_100"]) + (data["q05_roll_std_10"]))))))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*erf(((73.0) - (((((data["q05_roll_std_100"]) * (73.0))) * (((73.0) * (((data["q05_roll_std_100"]) * (((data["q05_roll_std_100"]) - (((data["q05"]) - ((((0.2525250018) + (2.0))/2.0)))))))))))))) +
            0.0399999991*erf(((((((((7.0) - ((((data["sum"]) + (((73.0) * (((((data["sum"]) + (data["q05_roll_std_10"]))) + (((((((erf((data["abs_q05"]))) + (data["q05_roll_std_10"]))) * 2.0)) * 2.0)))))))/2.0)))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*erf((((((1.0)) - (((((((((data["q05_roll_std_100"]) + (erf((((data["abs_q05"]) * (((((data["exp_Moving_average_30000_mean"]) - (data["abs_q05"]))) + (((data["q05_roll_std_100"]) + (((data["q95_roll_mean_10"]) * 2.0)))))))))))) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) +
            0.0399999991*erf(((((erf((((erf((data["Moving_average_1500_mean"]))) * 2.0)))) - (((data["Moving_average_1500_mean"]) + ((-1.0*((((((data["q05_roll_std_100"]) + (erf((0.6582279801))))) * 2.0))))))))) * ((((-1.0*((((data["Moving_average_1500_mean"]) * 2.0))))) - (73.0))))) +
            0.0399999991*erf(((73.0) - ((((((((((((data["q05_roll_std_100"]) + (((data["q05_roll_std_100"]) * (data["Moving_average_6000_mean"]))))/2.0)) + (((data["Moving_average_6000_mean"]) + (((data["q05_roll_std_100"]) + (data["q95"]))))))) * 2.0)) * 2.0)) * (((73.0) + (data["Moving_average_6000_mean"]))))))) +
            0.0399999991*erf(((((((((((((data["mean_change_rate_last_10000"]) - (data["mean_change_abs"]))) - (((((((((((((data["q05_roll_std_10"]) + (((0.5434780121) - (data["q05"]))))) * 2.0)) - (data["Hann_window_mean"]))) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*erf(((((((-1.0*((((data["avg_last_50000"]) * (3.6923100948)))))) + (((73.0) / 2.0)))/2.0)) - (((73.0) * ((((((data["Moving_average_6000_mean"]) + (((data["q05_roll_std_10"]) - (((data["exp_Moving_average_30000_mean"]) * (data["exp_Moving_average_30000_mean"]))))))/2.0)) + (data["q05_roll_std_10"]))))))) +
            0.0399999991*erf(((73.0) * ((-1.0*((((data["q01_roll_std_1000"]) + (((73.0) * (((data["q05_roll_std_100"]) + ((((1.4411799908) + ((((((data["q01_roll_std_1000"]) - (data["q99_roll_mean_1000"]))) + (((data["q99_roll_mean_1000"]) * (data["q99_roll_mean_1000"]))))/2.0)))/2.0))))))))))))) +
            0.0399999991*erf((((12.50784111022949219)) + ((((12.50784111022949219)) - ((((((erf((((data["q05_roll_std_10"]) * (data["q95_roll_mean_1000"]))))) + (((((((data["q05_roll_std_10"]) + (erf((((data["q05_roll_std_10"]) * (data["exp_Moving_average_30000_mean"]))))))) * 2.0)) * 2.0)))/2.0)) * (73.0))))))) +
            0.0399999991*erf(((((((data["skew"]) - (((data["ave_roll_mean_100"]) + (((((((data["ave_roll_mean_100"]) * (data["abs_q05"]))) + (((((2.0) + (((((data["q05_roll_std_100"]) - (data["q05_roll_mean_10"]))) * 2.0)))) * 2.0)))) * 2.0)))))) * 2.0)) * 2.0)) +
            0.0399999991*erf(((((((((data["q95_roll_mean_1000"]) * (data["ave_roll_mean_100"]))) * 2.0)) - (((((data["exp_Moving_average_30000_mean"]) + (((((((((data["exp_Moving_average_30000_mean"]) * (data["q05_roll_std_10"]))) + (((((data["q05_roll_std_10"]) - (0.2439019978))) * 2.0)))) * 2.0)) * 2.0)))) * 2.0)))) * 2.0)) +
            0.0399999991*erf(((((((((data["q05_roll_std_10"]) - (((((data["q95"]) - (((((data["iqr"]) - (((data["exp_Moving_average_3000_mean"]) * (erf((((data["q05_roll_std_10"]) * 2.0)))))))) - (((data["q05_roll_std_10"]) * 2.0)))))) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*erf(((((((data["q95_roll_mean_100"]) + (((data["mean_change_rate_first_10000"]) + (((data["abs_q05"]) + (data["abs_q05"]))))))) * (3.6923100948))) + (((((((((data["q05_roll_std_1000"]) + (0.7446810007))) * (((3.6923100948) - (73.0))))) * 2.0)) * 2.0)))) +
            0.0399999991*erf((-1.0*((((((73.0) * ((((((data["q01_roll_mean_1000"]) - (((data["q05"]) * 2.0)))) + (((data["q05_roll_std_1000"]) + (((((data["q05_roll_std_100"]) - (data["q95_roll_mean_1000"]))) * (data["Moving_average_700_mean"]))))))/2.0)))) + (((((data["Moving_average_1500_mean"]) * 2.0)) * 2.0))))))) +
            0.0399999991*erf(((((((((data["q05"]) - (((73.0) * ((((-1.0*((((data["q05"]) * 2.0))))) + (((data["q05_roll_std_1000"]) + (1.9019600153))))))))) + (((((((data["q05"]) + (data["max_to_min"]))) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) +
            0.0399999991*erf(((((((data["av_change_abs_roll_mean_100"]) - (((data["q01_roll_mean_1000"]) * 2.0)))) - (((73.0) * (((((((data["Hann_window_mean"]) * (((data["q95"]) - (data["q95_roll_mean_1000"]))))) / 2.0)) + (((((data["q95"]) * 2.0)) + (data["q01_roll_mean_1000"]))))))))) * 2.0)) +
            0.0399999991*erf(((((((((((data["skew"]) - (((((data["q05_roll_mean_10"]) + (((((data["iqr"]) + (((3.1415927410) * (((1.0) - (((data["q05_roll_mean_10"]) * 2.0)))))))) * 2.0)))) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*erf((((((((6.0)) - (((((((((data["q05_roll_std_100"]) + (((data["q05_roll_std_100"]) * 2.0)))) * 2.0)) + (((((data["ave10"]) * (((((data["q05_roll_std_100"]) * 2.0)) * 2.0)))) + (erf((data["q05_roll_std_100"]))))))) * 2.0)))) * 2.0)) * 2.0)) +
            0.0399999991*erf(((((((((data["q05_roll_std_10"]) + (((((((data["q95_roll_mean_1000"]) + (((((((data["skew"]) + (((((((data["abs_q05"]) - (((data["q05_roll_std_10"]) * 2.0)))) * 2.0)) * 2.0)))) * 2.0)) * 2.0)))) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*erf(((((((((data["max_to_min"]) + (data["q95_roll_std_1000"]))) - (((((73.0) * ((((data["q05_roll_std_1000"]) + (((0.2183910012) + (data["abs_q05"]))))/2.0)))) * (((data["q05_roll_std_1000"]) * (((data["q05_roll_std_10"]) + (data["q01_roll_std_10"]))))))))) * 2.0)) * 2.0)) +
            0.0399999991*erf(((((((((((data["abs_q05"]) + (data["q05_roll_std_100"]))) - (((7.0) * (((data["iqr"]) - (((data["q99_roll_mean_1000"]) + ((-1.0*((((((((data["q05_roll_std_100"]) + (1.0))) * 2.0)) * 2.0))))))))))))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*erf((-1.0*((((7.0) * (((73.0) * (((((7.0) * (((((data["Moving_average_6000_mean"]) * (((((-1.0*((data["Moving_average_6000_mean"])))) + (erf((erf((data["q01_roll_std_10"]))))))/2.0)))) + (data["q95_roll_mean_10"]))))) + (data["q95_roll_mean_10"])))))))))) +
            0.0399999991*erf(((((((((((data["skew"]) + (data["ave10"]))) + (data["q95_roll_std_100"]))) - (((7.0) * (((((data["q05_roll_std_100"]) * (data["q95"]))) * ((((((data["q05_roll_std_100"]) * (7.0))) + (7.0))/2.0)))))))) * 2.0)) * 2.0)) +
            0.0399999991*erf((((((9.64285850524902344)) * (((((data["Moving_average_700_mean"]) - ((((((9.64285850524902344)) * ((((data["q05_roll_std_100"]) + (data["exp_Moving_average_30000_mean"]))/2.0)))) - ((((data["skew"]) + ((3.98402547836303711)))/2.0)))))) * 2.0)))) * (((data["iqr"]) * (data["q05_roll_std_100"]))))) +
            0.0399999991*erf(((((((((((((((((data["Moving_average_1500_mean"]) - (((erf((data["exp_Moving_average_30000_mean"]))) + (erf((((data["abs_q05"]) + ((((data["q95_roll_std_10"]) + (data["q01_roll_std_10"]))/2.0)))))))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) - (data["iqr"]))) * 2.0)) +
            0.0399999991*erf((((-1.0*((data["q95"])))) * (((73.0) + ((-1.0*((((data["abs_q05"]) * ((((((((-1.0*((data["abs_q05"])))) - (data["min_roll_std_10"]))) - (73.0))) - ((((-1.0*((data["q05_roll_mean_10"])))) * (73.0)))))))))))))) +
            0.0399999991*erf(((data["q05_roll_std_10"]) * (((((((((data["q05_roll_std_100"]) * ((11.90774154663085938)))) * (((data["ave10"]) - (-2.0))))) - ((11.90774154663085938)))) * ((-1.0*((((data["q05_roll_std_1000"]) + (((data["q05_roll_std_100"]) - (-2.0)))))))))))) +
            0.0399999991*erf(((((((data["iqr"]) * ((9.63457870483398438)))) * (((data["q95_roll_mean_1000"]) - ((((9.63457870483398438)) * (((data["q05_roll_std_10"]) * ((((data["q01_roll_std_10"]) + (((data["iqr"]) * (data["q95_roll_mean_1000"]))))/2.0)))))))))) - (((-3.0) * (data["q01_roll_std_10"]))))) +
            0.0399999991*erf(((((((((-1.0) - (((erf((data["q05_roll_std_10"]))) + (((((((data["q05_roll_std_100"]) - (erf((((data["classic_sta_lta1_mean"]) + (((((((data["q95_roll_std_100"]) * 2.0)) * 2.0)) * 2.0)))))))) * 2.0)) * 2.0)))))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*erf(((((((((data["q95_roll_mean_10"]) - (((((((data["q95_roll_mean_10"]) + (((((data["q05_roll_std_10"]) + (((3.6923100948) * ((((data["ave10"]) + (((data["q05_roll_std_10"]) + (-1.0))))/2.0)))))) * 2.0)))) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*erf(((((data["q05_roll_std_100"]) + (((((data["q95_roll_mean_1000"]) + (((((data["q05_roll_std_100"]) + ((((3.1415927410) + (data["q05_roll_std_1000"]))/2.0)))) * (((((((data["q05_roll_mean_10"]) * 2.0)) * 2.0)) * (((3.1415927410) * (data["q05_roll_std_100"]))))))))) * 2.0)))) * 2.0)) +
            0.0399999991*erf(((((((((((((data["mean_change_rate_first_10000"]) - (data["Moving_average_6000_mean"]))) * 2.0)) - (data["q01_roll_mean_1000"]))) - (((data["av_change_abs_roll_mean_10"]) + (((data["mean_change_rate_last_50000"]) + (((data["av_change_abs_roll_std_100"]) - ((((((13.93686485290527344)) / 2.0)) * (data["q05"]))))))))))) * 2.0)) * 2.0)) +
            0.0399999991*erf(((((((((data["q05_roll_std_10"]) * (data["q05_roll_std_10"]))) * ((((-1.0*((((2.0) - ((-1.0*((((data["q05_roll_std_100"]) + (((((data["q05_roll_std_100"]) * (data["abs_q05"]))) * 2.0)))))))))))) * 2.0)))) - ((-1.0*((data["kurt"])))))) * 2.0)) +
            0.0399999991*erf(((((((((((((((data["q05"]) - (erf((((data["iqr"]) + (((((data["Moving_average_1500_mean"]) + (((data["q05"]) + (data["Moving_average_3000_mean"]))))) * 2.0)))))))) * 2.0)) + (data["q05"]))) * 2.0)) * 2.0)) - (data["av_change_abs_roll_mean_10"]))) * 2.0)) +
            0.0399999991*erf(((((((((erf((((((((((((((data["Moving_average_6000_mean"]) - (data["q05_roll_std_100"]))) * 2.0)) * 2.0)) - (((1.6428600550) + (data["q01_roll_std_10"]))))) * 2.0)) - (data["av_change_abs_roll_std_100"]))))) * 2.0)) - (data["abs_q05"]))) * 2.0)) * 2.0)) +
            0.0399999991*erf((((((((((((((data["skew"]) + (data["q95_roll_std_10"]))/2.0)) - (((((data["ave_roll_std_100"]) + (((data["max_roll_mean_10"]) * 2.0)))) * (data["abs_q05"]))))) + (((((erf((data["q95_roll_std_100"]))) - (data["q01_roll_std_10"]))) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*erf(((data["skew"]) + (((((data["q01_roll_std_100"]) * 2.0)) + (((data["Moving_average_700_mean"]) + (((data["q05_roll_std_10"]) * (((data["q05"]) * ((((((73.0) * (((data["Moving_average_700_mean"]) + (((data["q05_roll_std_100"]) / 2.0)))))) + (data["q05_roll_std_10"]))/2.0)))))))))))) +
            0.0399999991*erf(((((((data["q05_roll_mean_10"]) * (((data["q05_roll_std_100"]) * (data["q05_roll_std_100"]))))) * (((((data["q05_roll_std_100"]) * 2.0)) * (((data["q05_roll_std_1000"]) * 2.0)))))) - (((((((data["q05_roll_std_1000"]) * 2.0)) * (data["q05_roll_std_100"]))) - (((data["q01_roll_std_1000"]) * 2.0)))))) +
            0.0399999991*erf(((((((-1.0) + (((((((erf((data["q01_roll_std_10"]))) * 2.0)) * 2.0)) + (((data["max_to_min_diff"]) + (((data["q05"]) * ((((14.98138904571533203)) + (((data["mean_change_rate_last_10000"]) * 2.0)))))))))))) * (73.0))) * (73.0))) +
            0.0399999991*erf(((((((data["q05_roll_std_10"]) - (((((data["q05_roll_std_1000"]) + (0.7446810007))) * (((data["q05_roll_std_10"]) * (((((((data["q01_roll_std_10"]) * (((data["ave10"]) + (((data["q95_roll_mean_10"]) * (data["q05_roll_std_10"]))))))) * 2.0)) * 2.0)))))))) * 2.0)) * 2.0)) +
            0.0399999991*erf(((((((erf((data["q01_roll_std_100"]))) - (((((((((data["q05_roll_std_1000"]) + (((((data["q95"]) - (erf((data["q95_roll_std_100"]))))) * 2.0)))) - (erf((erf((data["q95"]))))))) - (data["std_roll_mean_1000"]))) * 2.0)))) * 2.0)) * 2.0)) +
            0.0399999991*erf(((((data["q05_roll_std_10"]) + (((data["ave_roll_mean_1000"]) * 2.0)))) * (((((((data["ave_roll_mean_1000"]) * (data["ave_roll_mean_1000"]))) - (data["q05_roll_std_1000"]))) + (((data["q05_roll_mean_10"]) * (((data["Moving_average_3000_mean"]) + (((((data["q05_roll_std_10"]) * 2.0)) * 2.0)))))))))) +
            0.0399999991*erf(((((((((((data["q05_roll_std_1000"]) + (((data["q05_roll_std_100"]) - (((data["Moving_average_6000_mean"]) * (data["Moving_average_6000_mean"]))))))) * (((data["q95_roll_std_100"]) * ((((-1.0*((data["iqr"])))) - (((data["Moving_average_6000_mean"]) + (data["Moving_average_1500_mean"]))))))))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*erf(((((((((data["sum"]) + (0.9756100178))) * 2.0)) * 2.0)) * (((data["q01_roll_std_100"]) - (((erf((((data["q05_roll_std_10"]) * (data["q05_roll_std_10"]))))) + (((data["q05_roll_std_1000"]) * (((((data["q01_roll_std_100"]) * (data["q01_roll_std_10"]))) * 2.0)))))))))) +
            0.0399999991*erf((((((((((-1.0*((data["q05_roll_std_100"])))) - (((data["q01_roll_std_100"]) * (data["mean"]))))) * (((data["q05_roll_std_100"]) * (data["q05_roll_std_100"]))))) * (((data["q05_roll_std_100"]) * (data["q05_roll_std_100"]))))) + (((((data["av_change_rate_roll_std_10"]) * (data["max_to_min"]))) * 2.0)))) +
            0.0399999991*erf(((((((data["q05_roll_std_10"]) * 2.0)) * 2.0)) * (((((0.5434780121) * 2.0)) - (((((data["q05_roll_std_10"]) * (((((((((data["q95_roll_std_10"]) * 2.0)) * (data["q05_roll_std_10"]))) + (data["q05_roll_std_10"]))) * (data["q05_roll_std_10"]))))) + (data["skew"]))))))) +
            0.0399999991*erf(((((data["abs_q05"]) * (data["q95_roll_std_100"]))) - (((((data["q05_roll_std_100"]) * (((data["q05_roll_std_100"]) * (((data["q05_roll_std_100"]) * (((data["q05_roll_std_100"]) * (((data["abs_q05"]) * (data["q05_roll_std_100"]))))))))))) + ((((data["min_roll_std_100"]) + (data["av_change_abs_roll_mean_100"]))/2.0)))))) +
            0.0399999991*erf(((((data["Moving_average_6000_mean"]) + (0.8873239756))) * (((((data["q05_roll_std_100"]) + (data["q95_roll_std_1000"]))) * (((1.6428600550) - (((data["q05_roll_std_1000"]) * (((((data["q05_roll_std_100"]) + (((data["exp_Moving_average_3000_mean"]) + (0.9756100178))))) + (data["q05_roll_std_100"]))))))))))) +
            0.0399999991*erf(((((data["q95_roll_mean_10"]) + (((data["q95_roll_std_1000"]) * (((data["med"]) + (((((data["q05_roll_mean_100"]) + (((data["q95_roll_mean_10"]) * (data["q95_roll_mean_10"]))))) * (data["q05_roll_std_100"]))))))))) * (((((data["exp_Moving_average_30000_mean"]) - (data["q05_roll_std_100"]))) - (data["q05_roll_std_1000"]))))) +
            0.0399999991*erf(((((((data["min"]) + ((((-1.0*((data["MA_400MA_BB_high_mean"])))) + (erf((((((0.3183098733) + (((((data["kurt"]) * 2.0)) * 2.0)))) * ((((3.0)) + ((-1.0*((data["kurt"])))))))))))))) * 2.0)) * 2.0)) +
            0.0399999991*erf(((((((data["min_roll_mean_1000"]) * 2.0)) * 2.0)) + ((((-1.0*((((((data["q05_roll_std_1000"]) + (((data["q01_roll_std_10"]) * (data["q95"]))))) + (((data["avg_last_50000"]) * (((data["q01_roll_std_10"]) - (data["avg_last_50000"])))))))))) * (data["q01_roll_std_100"]))))) +
            0.0399921872*erf(((((data["q95_roll_std_1000"]) - ((((((((data["av_change_abs_roll_mean_10"]) + (data["std_first_10000"]))/2.0)) + (((data["q95"]) - (data["min_last_10000"]))))) - (((data["min"]) + (((data["q95_roll_std_1000"]) + (data["mean_change_rate_last_10000"]))))))))) - (((data["q95"]) + (data["q95"]))))) +
            0.0399999991*erf(((((((data["min_roll_mean_1000"]) - (((data["q05_roll_std_1000"]) * (((erf((((data["med"]) * 2.0)))) + ((((((data["skew"]) * (data["kurt"]))) + (data["q05_roll_std_1000"]))/2.0)))))))) + (((data["kurt"]) * (data["skew"]))))) + (data["min_roll_mean_1000"]))) +
            0.0399921872*erf(((data["exp_Moving_average_30000_mean"]) * (((((data["max_roll_std_100"]) - (((((data["exp_Moving_average_30000_mean"]) * (data["MA_400MA_std_mean"]))) + (data["q05_roll_mean_10"]))))) * (((((data["mean_change_rate_first_10000"]) * (73.0))) - (((((data["abs_mean"]) * (73.0))) * (73.0))))))))) +
            0.0399999991*erf(((data["q01_roll_std_10"]) + ((-1.0*((((data["q05_roll_std_1000"]) * (((data["av_change_rate_roll_mean_100"]) + ((((-1.0*((data["classic_sta_lta4_mean"])))) + (((data["med"]) + (((data["q05_roll_std_1000"]) + (((data["iqr"]) * (((data["iqr"]) + (data["max_last_10000"])))))))))))))))))))) +
            0.0399921872*erf((((((((data["q05"]) + (data["mean_change_rate_first_10000"]))/2.0)) + (((data["kurt"]) - (((((data["kurt"]) * (data["abs_max_roll_std_1000"]))) * (data["kurt"]))))))) + (((((data["av_change_abs_roll_std_100"]) * ((-1.0*((data["classic_sta_lta3_mean"])))))) * 2.0)))) +
            0.0399999991*erf(((((((((((((((data["skew"]) - ((((data["min_roll_std_1000"]) + (data["q95_roll_mean_100"]))/2.0)))) + (((data["q95_roll_mean_100"]) * (data["q05"]))))) * (data["MA_400MA_std_mean"]))) * (data["MA_400MA_BB_high_mean"]))) * 2.0)) * 2.0)) + ((((data["mean_change_abs"]) + (data["min_roll_std_1000"]))/2.0)))) +
            0.0399921872*erf((((((4.12892913818359375)) * ((((((data["exp_Moving_average_300_mean"]) + (data["med"]))/2.0)) - (data["q01_roll_std_100"]))))) * (((data["skew"]) + (((0.2439019978) + ((((data["MA_1000MA_std_mean"]) + ((((((data["av_change_abs_roll_std_1000"]) + (data["med"]))) + (data["iqr"]))/2.0)))/2.0)))))))) +
            0.0399999991*erf(((((((((data["av_change_abs_roll_std_10"]) * (((((data["mean_change_rate_last_50000"]) + (data["av_change_abs_roll_mean_10"]))) + (data["min_last_50000"]))))) * 2.0)) * 2.0)) + ((((((((1.0)) + (((data["av_change_rate_roll_std_10"]) + (data["skew"]))))/2.0)) + (data["q01_roll_mean_1000"]))/2.0)))) +
            0.0399999991*erf(((((((((data["q05_roll_std_10"]) * (((((((data["med"]) + (data["q05_roll_std_10"]))) * (data["q05_roll_mean_10"]))) * (data["q95_roll_std_1000"]))))) * (data["q05_roll_std_10"]))) + (((((data["q05_roll_mean_10"]) + (data["q95_roll_std_100"]))) + (data["q95_roll_std_100"]))))) * 2.0)) +
            0.0399921872*erf((((data["skew"]) + (((((((((data["min_roll_std_10"]) * ((-1.0*((data["min_roll_std_10"])))))) + (data["q05_roll_mean_1000"]))) * (data["Hilbert_mean"]))) - ((((data["av_change_rate_roll_std_100"]) + (((data["max_roll_std_100"]) * (((data["skew"]) * (data["skew"]))))))/2.0)))))/2.0)) +
            0.0399999991*erf(((data["q05_roll_std_10"]) * (((((((((data["q01_roll_mean_1000"]) - (erf((((data["Moving_average_6000_mean"]) - (((data["ave_roll_std_10"]) * (((data["kurt"]) - ((((data["av_change_abs_roll_mean_10"]) + ((((data["Moving_average_6000_mean"]) + (data["ave_roll_std_10"]))/2.0)))/2.0)))))))))))) * 2.0)) * 2.0)) * 2.0)))) +
            0.0399843715*erf(((data["min"]) + ((-1.0*((((data["av_change_rate_roll_std_100"]) * ((((((((data["mean_change_rate"]) + (data["min"]))/2.0)) + (data["min_roll_std_1000"]))) + (((erf((data["min_last_10000"]))) + ((((((data["min_roll_std_100"]) * (data["av_change_rate_roll_mean_10"]))) + (data["av_change_abs_roll_std_10"]))/2.0))))))))))))) +
            0.0399999991*erf(((data["min_roll_mean_100"]) * (((((data["min_roll_std_1000"]) * (((data["q01_roll_std_1000"]) * (((((((data["q01_roll_std_1000"]) * (data["q99_roll_std_100"]))) + (data["exp_Moving_average_300_mean"]))) + (data["avg_first_10000"]))))))) + (((((data["mean_change_rate"]) * (3.1415927410))) - (data["avg_first_10000"]))))))) +
            0.0399921872*erf(((((data["q95_roll_mean_1000"]) * (((((((((data["q95_roll_mean_1000"]) + (((data["avg_last_10000"]) + (((3.0) / 2.0)))))/2.0)) + (data["q05_roll_mean_100"]))/2.0)) - (data["q95_roll_std_1000"]))))) * (((((((data["q95_roll_std_1000"]) * 2.0)) + (data["q99_roll_mean_10"]))) + (data["classic_sta_lta4_mean"]))))) +
            0.0399296731*erf((((((((((((data["q01_roll_std_100"]) + (erf((((data["q01"]) + ((-1.0*((((data["q01_roll_std_100"]) * 2.0))))))))))/2.0)) * ((-1.0*(((((data["q95"]) + (((data["exp_Moving_average_30000_mean"]) * 2.0)))/2.0))))))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399921872*erf(((((erf((data["q99"]))) + ((-1.0*(((((data["max_first_10000"]) + (((data["av_change_rate_roll_mean_1000"]) - (data["q99"]))))/2.0))))))) - ((((((((data["av_change_rate_roll_mean_1000"]) * (data["av_change_rate_roll_mean_1000"]))) + (data["q05_roll_std_10"]))/2.0)) * (((data["q05_roll_std_10"]) * (data["iqr"]))))))) +
            0.0399999991*erf(((data["min_roll_mean_1000"]) * (((((((((((data["std_roll_std_100"]) + (((data["q05"]) + (((((data["q95_roll_mean_1000"]) * (data["q95_roll_std_1000"]))) + (((data["q95_roll_mean_1000"]) * (((data["q95_roll_mean_1000"]) * (data["q95_roll_std_1000"]))))))))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) +
            0.0399921872*erf(((((data["mean_change_abs"]) * (data["min_roll_std_10"]))) + (((((((data["std_last_50000"]) * (3.6923100948))) * (((data["q95_roll_mean_1000"]) + (((data["classic_sta_lta4_mean"]) - (data["min_roll_std_10"]))))))) * ((((((data["min_roll_std_10"]) + (data["q05_roll_mean_100"]))/2.0)) - (data["max_roll_std_10"]))))))) +
            0.0399999991*erf(((((data["q01_roll_std_10"]) + (data["min_roll_std_100"]))) * (((data["abs_q05"]) - (((data["avg_last_50000"]) - (((((data["classic_sta_lta2_mean"]) / 2.0)) + (((data["max_roll_mean_1000"]) * (((((data["avg_last_50000"]) - (erf((data["min_roll_std_100"]))))) - (data["min_roll_std_100"]))))))))))))) +
            0.0399999991*erf(((((data["q95_roll_mean_100"]) * (data["min_roll_mean_1000"]))) - (((((data["av_change_abs_roll_std_100"]) * (((((data["classic_sta_lta3_mean"]) + (data["mean_change_rate_first_50000"]))) + (((data["q95_roll_mean_100"]) * (data["classic_sta_lta3_mean"]))))))) + (((data["mean_change_abs"]) * (((data["classic_sta_lta3_mean"]) - (data["mean_change_rate_first_50000"]))))))))) +
            0.0398359038*erf(((data["q95_roll_mean_10"]) * (((((data["q95_roll_mean_10"]) * (((data["min_roll_mean_1000"]) + (data["av_change_rate_roll_std_10"]))))) - (((data["max_to_min"]) + (((((((data["max_to_min"]) * 2.0)) * 2.0)) * (((data["av_change_rate_roll_std_10"]) + (data["q01_roll_mean_1000"]))))))))))) +
            0.0399999991*erf(((data["q01_roll_std_10"]) * (((((((data["q01_roll_std_10"]) * (data["skew"]))) + (((data["classic_sta_lta4_mean"]) - (data["classic_sta_lta2_mean"]))))) + ((((data["av_change_abs_roll_mean_1000"]) + (((((-1.0*((data["q01_roll_std_100"])))) + (((data["q01_roll_std_10"]) * ((-1.0*((data["q01_roll_std_10"])))))))/2.0)))/2.0)))))) +
            0.0385153331*erf((((((data["std_roll_mean_1000"]) + (data["av_change_abs_roll_mean_100"]))) + (((data["max_first_10000"]) * (((data["MA_700MA_std_mean"]) * (((73.0) * (((erf((data["max_to_min"]))) - (((data["av_change_rate_roll_mean_1000"]) + (((((data["std_roll_mean_1000"]) * 2.0)) * 2.0)))))))))))))/2.0)) +
            0.0399999991*erf(((((data["std_roll_mean_1000"]) * (((data["av_change_rate_roll_std_1000"]) - (((((((data["skew"]) - (data["min_roll_std_10"]))) - (data["classic_sta_lta4_mean"]))) + (((data["std_roll_mean_1000"]) - (2.0))))))))) + (((data["classic_sta_lta4_mean"]) * (((data["av_change_abs_roll_mean_1000"]) - (data["q99"]))))))) +
            0.0399843715*erf((-1.0*((((data["kurt"]) * (((data["kurt"]) * ((((((data["av_change_abs_roll_mean_10"]) + (((((data["med"]) - (((data["med"]) * (data["av_change_abs_roll_mean_10"]))))) - (data["min_roll_std_10"]))))/2.0)) + (((data["q99_roll_std_1000"]) * (data["kurt"])))))))))))) +
            0.0399921872*erf(((data["mean_change_rate_first_10000"]) * (((data["av_change_abs_roll_mean_100"]) * ((-1.0*((((((erf((((((data["classic_sta_lta3_mean"]) + (data["mean_change_rate_first_10000"]))) * 2.0)))) + (((1.9019600153) - (data["av_change_abs_roll_mean_100"]))))) - (((((data["classic_sta_lta3_mean"]) * 2.0)) * (data["mean_change_rate_first_10000"])))))))))))) +
            0.0399921872*erf(((((data["av_change_rate_roll_std_100"]) * (((data["classic_sta_lta3_mean"]) * (((((data["min_roll_std_1000"]) - (data["min"]))) * (data["q05"]))))))) + (((data["mean_change_rate_last_10000"]) * (((((data["av_change_abs_roll_std_1000"]) - (data["min"]))) - (((data["classic_sta_lta3_mean"]) * (data["q01_roll_std_1000"]))))))))) +
            0.0399999991*erf(((data["q95_roll_std_1000"]) + (((data["q05_roll_mean_10"]) - ((-1.0*((((data["q95_roll_std_1000"]) * (((data["q95_roll_std_1000"]) * (((((data["q01_roll_std_10"]) * ((((((((data["q95_roll_std_1000"]) / 2.0)) + (data["q01_roll_mean_1000"]))/2.0)) - (data["med"]))))) * (data["q95_roll_std_1000"])))))))))))))) +
            0.0399843715*erf(((data["av_change_abs_roll_mean_100"]) * (((data["classic_sta_lta4_mean"]) * (((((((((3.65123486518859863)) + ((((3.65123486518859863)) * (data["abs_max_roll_std_10"]))))/2.0)) + (((((data["min_last_50000"]) * (((data["abs_max_roll_std_1000"]) - (data["av_change_abs_roll_mean_10"]))))) - (data["classic_sta_lta4_mean"]))))) * (data["ave_roll_mean_10"]))))))) +
            0.0399999991*erf(((((data["max_to_min_diff"]) * (((data["min_roll_std_100"]) * (((data["av_change_rate_roll_std_100"]) + (((data["q01_roll_std_100"]) + (((((((data["av_change_rate_roll_mean_100"]) + (data["Hilbert_mean"]))) * (data["q95_roll_mean_1000"]))) + (((data["q01"]) * (data["min_roll_std_100"]))))))))))))) * 2.0)) +
            0.0399999991*erf(((((erf((((((data["q05"]) + (((erf((data["avg_first_10000"]))) + (data["mean_change_rate_last_10000"]))))) + (data["q05"]))))) + (((((data["kurt"]) * (data["mean_change_rate_last_10000"]))) + (data["q95_roll_std_1000"]))))) * (((data["q01_roll_mean_1000"]) + (data["av_change_rate_roll_mean_100"]))))) +
            0.0399921872*erf(((data["min_roll_mean_1000"]) + ((((((((((((data["av_change_rate_roll_std_10"]) + (((data["av_change_abs_roll_std_1000"]) * (data["min_roll_mean_1000"]))))/2.0)) * (data["classic_sta_lta3_mean"]))) + (data["av_change_rate_roll_std_10"]))/2.0)) + (((data["abs_max_roll_mean_100"]) - (((data["av_change_abs_roll_std_1000"]) * (data["classic_sta_lta1_mean"]))))))/2.0)))) +
            0.0399999991*erf(((data["max_to_min"]) * (((((data["MA_400MA_std_mean"]) - (data["q95_roll_std_100"]))) + ((((((data["min_roll_std_10"]) * (((((((data["min_roll_std_10"]) * (data["classic_sta_lta1_mean"]))) - (data["min_roll_std_10"]))) - (data["kurt"]))))) + ((((data["kurt"]) + (data["min_roll_std_10"]))/2.0)))/2.0)))))) +
            0.0399921872*erf(((data["min_roll_std_10"]) * ((((((((((((erf((data["min_roll_std_10"]))) + (data["avg_last_10000"]))/2.0)) - (data["q95_roll_std_100"]))) - (data["MA_700MA_BB_high_mean"]))) + (((data["abs_max_roll_mean_100"]) - (data["classic_sta_lta4_mean"]))))) * (((data["min_roll_std_10"]) - (((3.0) / 2.0)))))))) +
            0.0399765596*erf(((((data["avg_last_10000"]) * (((data["min_roll_mean_1000"]) + (((((data["avg_last_10000"]) + (erf((data["mean_change_rate_last_50000"]))))) * (((data["min_roll_std_10"]) * (data["min_last_10000"]))))))))) + (erf((erf((((((data["mean_change_rate_last_50000"]) * (data["min_roll_std_1000"]))) * 2.0)))))))) +
            0.0399921872*erf(erf((erf((erf((((((((data["q95_roll_std_10"]) - (data["q05"]))) * (((data["abs_q05"]) * (((data["mean_change_rate_first_10000"]) + (((((data["mean_change_rate_first_10000"]) + (data["q05"]))) * (((data["abs_q05"]) * (data["q95_roll_mean_1000"]))))))))))) * 2.0)))))))) +
            0.0399843715*erf(((data["q95_roll_std_1000"]) * (((data["max_roll_mean_100"]) + (((((((((data["q01_roll_mean_1000"]) * 2.0)) * 2.0)) * (data["min_roll_mean_1000"]))) * (((((((data["q01_roll_mean_1000"]) + ((-1.0*((data["max_to_min"])))))) * 2.0)) * 2.0)))))))) +
            0.0399765596*erf(((data["q05_roll_std_100"]) * ((-1.0*((erf((((((data["med"]) + (((((data["med"]) + (((data["q01_roll_std_10"]) * (((data["avg_first_50000"]) + (data["q01_roll_std_10"]))))))) * ((((1.16074943542480469)) + (data["avg_first_50000"]))))))) * 2.0))))))))) +
            0.0399062298*erf((-1.0*((((data["classic_sta_lta2_mean"]) * ((((((erf((data["kurt"]))) + (data["avg_last_50000"]))/2.0)) + (((data["q05_roll_mean_100"]) * ((-1.0*(((((((((data["kurt"]) + (data["q05_roll_mean_100"]))/2.0)) - (erf((data["kurt"]))))) + (data["std_last_10000"]))))))))))))))) +
            0.0399453007*erf(((data["std_last_50000"]) * (((((erf((((((((((((data["av_change_abs_roll_mean_100"]) + (erf((erf((((((((((data["skew"]) + (data["Moving_average_6000_mean"]))) * 2.0)) * 2.0)) * 2.0)))))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) * 2.0)))) +
            0.0390935726*erf((-1.0*((((data["max_roll_mean_100"]) * (((((((erf((((((data["classic_sta_lta3_mean"]) + ((((data["q99_roll_mean_1000"]) + (data["mean_change_rate_first_10000"]))/2.0)))) + (((data["av_change_abs_roll_mean_10"]) + (((data["classic_sta_lta3_mean"]) * (data["q95_roll_mean_1000"]))))))))) * 2.0)) * 2.0)) * 2.0))))))) +
            0.0399296731*erf(((data["q95_roll_std_100"]) + ((-1.0*((((data["q95_roll_std_10"]) + ((((((((data["kurt"]) * ((((data["mean_change_rate_last_10000"]) + (data["trend"]))/2.0)))) * (data["kurt"]))) + ((((((data["med"]) + (data["max_first_50000"]))/2.0)) * (data["min_roll_std_100"]))))/2.0))))))))) +
            0.0398984179*erf(((((erf((((data["q01"]) - (data["q05"]))))) + (((((((((erf((data["q05_roll_std_100"]))) + (data["q01"]))) * 2.0)) + (data["q95_roll_mean_1000"]))) * 2.0)))) * ((-1.0*((((data["q01"]) - (data["q05"])))))))) +
            0.0399999991*erf(erf((erf(((-1.0*((((data["mean_change_rate_first_10000"]) * (((data["min_last_10000"]) + (((data["min_last_10000"]) + (((((data["abs_q95"]) + (((data["avg_last_50000"]) + ((((data["min_last_10000"]) + (0.8873239756))/2.0)))))) * ((-1.0*((data["q05_roll_std_100"]))))))))))))))))))) +
            0.0399999991*erf(((data["min_first_50000"]) * ((((-1.0*((data["av_change_abs_roll_std_100"])))) - (((data["ave_roll_mean_10"]) * ((-1.0*((((data["max_to_min"]) - (((((((data["MA_400MA_BB_low_mean"]) - (((data["av_change_rate_roll_mean_100"]) + (((data["q01_roll_mean_1000"]) * (data["av_change_rate_roll_mean_100"]))))))) * 2.0)) * 2.0))))))))))))) +
            0.0399999991*erf(((((((((data["max_to_min_diff"]) * (((data["av_change_rate_roll_std_100"]) + (((((data["q95_roll_std_10"]) + (data["max_first_50000"]))) + (((data["mean_change_abs"]) + (((((data["MA_400MA_BB_low_mean"]) * (data["max_first_50000"]))) * (data["max_first_50000"]))))))))))) * 2.0)) * 2.0)) * (data["min_roll_std_100"]))) +
            0.0399999991*erf(((((data["max"]) * (((data["std_first_50000"]) * ((((((-1.0*((data["max_roll_std_100"])))) + (((((((((data["q001"]) + (((data["q001"]) * (data["classic_sta_lta4_mean"]))))) + (data["classic_sta_lta4_mean"]))) * 2.0)) * 2.0)))) * 2.0)))))) * (data["q05_roll_std_1000"]))) +
            0.0399765596*erf(((((data["abs_max_roll_std_1000"]) * 2.0)) * (((data["q95_roll_std_10"]) - (((data["q999"]) - (erf((((data["max_last_10000"]) * (((((((((((((data["abs_max_roll_std_1000"]) + (data["Moving_average_6000_mean"]))) * 2.0)) + (data["mean_change_abs"]))) * 2.0)) * 2.0)) * 2.0)))))))))))) +
            0.0399687439*erf(((data["min_roll_std_10"]) * ((((((data["avg_last_50000"]) + (((((((data["q01_roll_std_1000"]) * (data["classic_sta_lta4_mean"]))) * (data["mean_change_rate_last_50000"]))) - (((data["min_roll_std_10"]) * (data["av_change_abs_roll_mean_10"]))))))/2.0)) + ((((((data["classic_sta_lta4_mean"]) * (data["Moving_average_6000_mean"]))) + (data["mean_change_rate_last_50000"]))/2.0)))))) +
                   
            0.0398906022*erf((((erf((((data["av_change_abs_roll_mean_1000"]) - (((data["Hann_window_mean"]) - (data["skew"]))))))) + (((((((data["min_roll_mean_1000"]) * 2.0)) * 2.0)) * (((data["skew"]) + (((((data["std_roll_mean_1000"]) - (data["q99_roll_mean_10"]))) - (data["med"]))))))))/2.0)) +
                   
            0.0399999991*erf(((data["std_first_10000"]) * ((-1.0*((((data["min_roll_std_10"]) * (((((((((data["kurt"]) + (((((data["av_change_rate_roll_std_1000"]) - (data["trend"]))) + (data["mean_change_rate_first_10000"]))))) - (data["max_last_10000"]))) + (((data["trend"]) * (data["trend"]))))) * 2.0))))))))) +
                   
            0.0399765596*erf(((data["q99"]) * ((-1.0*((((data["iqr"]) - (((((((1.9019600153) + (((data["abs_max_roll_mean_100"]) + (data["q99"]))))) * (((((data["std_roll_mean_1000"]) * 2.0)) * (((data["max_roll_mean_1000"]) - (data["std_roll_mean_1000"]))))))) + (data["min_roll_std_1000"])))))))))) +
                   
            0.0399765596*erf(((((data["mean_change_rate_last_10000"]) + (erf((((((erf((data["skew"]))) + (((data["max_roll_std_1000"]) * 2.0)))) * 2.0)))))) * (((data["abs_max_roll_std_10"]) + ((-1.0*((((data["skew"]) * (((((data["abs_max"]) / 2.0)) - (data["ave_roll_std_1000"])))))))))))) +
                   
            0.0355928876*erf((((((((data["min_roll_std_100"]) * (data["q05_roll_mean_1000"]))) * (data["min_roll_std_1000"]))) + (((((data["min_roll_std_100"]) * (data["min_roll_std_100"]))) - (((((data["max_to_min"]) + ((((((data["q999"]) + (data["min_roll_std_100"]))/2.0)) + (data["q999"]))))) * (data["classic_sta_lta3_mean"]))))))/2.0)) +
                   
            0.0399843715*erf((((((-1.0*((((((((data["q01_roll_std_10"]) * (data["q05_roll_std_1000"]))) * (data["max_first_50000"]))) * (data["q01_roll_std_10"])))))) * (((data["max_first_50000"]) + (data["max_to_min_diff"]))))) * (((((data["min"]) - (data["q05"]))) - (data["q05"]))))) +
                   
            0.0399765596*erf(((data["med"]) * (((erf((((((((((data["av_change_abs_roll_std_100"]) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) - (((((data["min_roll_std_10"]) + ((((data["q01_roll_std_1000"]) + ((((data["max_to_min_diff"]) + (data["min_roll_std_1000"]))/2.0)))/2.0)))) * (((data["av_change_abs_roll_std_100"]) * 2.0)))))))) +
                   
            0.0387106836*erf((-1.0*((((((data["av_change_rate_roll_mean_10"]) * (((data["q95_roll_std_1000"]) + (((((((((((data["min_roll_mean_1000"]) * 2.0)) * 2.0)) * (((data["q95_roll_std_1000"]) + (data["min_roll_mean_1000"]))))) * 2.0)) * (((data["q95_roll_std_100"]) - (((data["min_roll_mean_1000"]) * 2.0)))))))))) * 2.0))))) +
                   
            0.0399999991*erf(((data["max_last_10000"]) * ((((((((data["MA_700MA_BB_low_mean"]) * (((((data["avg_first_10000"]) - (data["av_change_abs_roll_mean_1000"]))) - (((data["avg_first_10000"]) * (((data["avg_first_10000"]) - (((data["min_roll_std_1000"]) * 2.0)))))))))) * 2.0)) + (((data["avg_first_10000"]) - (data["mean_change_rate_last_10000"]))))/2.0)))) +
                   
            0.0399999991*erf(erf((((((((((data["av_change_rate_roll_std_100"]) * (((data["min"]) * (((data["q05_roll_std_100"]) + (((((data["av_change_abs_roll_std_100"]) + (((data["mean_change_rate_first_10000"]) * (data["av_change_rate_roll_std_100"]))))) + (((data["med"]) - (data["mean_change_rate_first_10000"]))))))))))) * 2.0)) * 2.0)) * 2.0)))) +
                   
            0.0359679610*erf((-1.0*((((erf((((((((((((((((((data["min_roll_std_1000"]) - (data["av_change_rate_roll_std_1000"]))) - (((-2.0) + ((-1.0*((data["min_roll_std_1000"])))))))) - (erf((data["av_change_abs_roll_std_100"]))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) / 2.0))))) +
                   
            0.0373354182*erf((-1.0*((((data["max_to_min"]) * (((data["avg_first_50000"]) * (((((data["ave_roll_std_100"]) - (((data["classic_sta_lta1_mean"]) + (data["mean_change_rate_first_10000"]))))) * (((data["classic_sta_lta2_mean"]) - ((-1.0*((((data["ave_roll_std_100"]) / 2.0)))))))))))))))) +
                   
            0.0398671627*erf(((((0.8873239756) + (data["avg_last_50000"]))) * (((data["iqr"]) * (((((0.8873239756) + (data["Moving_average_6000_mean"]))) + (((data["q05_roll_mean_10"]) * (((((((data["ave10"]) + (data["q01_roll_std_10"]))) * (data["q01_roll_std_10"]))) * (data["iqr"]))))))))))) +
                   
            0.0399453007*erf((((erf((((((((((((data["mean_change_rate_last_10000"]) + (erf(((((((-1.0*((data["mean_change_rate_last_50000"])))) * 2.0)) * (data["q95_roll_mean_1000"]))))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) + (erf((erf(((((-1.0*((data["mean_change_rate_last_50000"])))) * 2.0)))))))/2.0)) +
                   
            0.0399687439*erf(((((((data["max_roll_std_1000"]) + (((data["max_roll_std_1000"]) + (data["trend"]))))) + (data["count_big"]))) * (((data["max_to_min"]) * (((((((-1.0*((data["max_to_min"])))) + (data["max_roll_mean_1000"]))/2.0)) - (data["q99_roll_std_10"]))))))) +
                   
            0.0399843715*erf(((data["classic_sta_lta2_mean"]) * (((data["abs_mean"]) * (((data["classic_sta_lta1_mean"]) * (((((((data["mean_change_rate_first_50000"]) + ((-1.0*((data["mean_change_abs"])))))) + (data["classic_sta_lta4_mean"]))) + (((((((-2.0) + (data["av_change_rate_roll_std_10"]))/2.0)) + (data["av_change_abs_roll_std_1000"]))/2.0)))))))))) +
                   
            0.0399687439*erf(((data["min_roll_std_100"]) * (((((((data["mean_change_abs"]) + (((((((data["av_change_abs_roll_std_100"]) * (data["min_roll_std_100"]))) + (data["min_roll_std_1000"]))) * (((data["std_roll_mean_1000"]) * (((data["min_last_10000"]) + (data["av_change_abs_roll_std_100"]))))))))) * 2.0)) * (data["av_change_abs_roll_std_100"]))))) +
                   
            0.0399374850*erf(((((data["min_first_10000"]) - (((data["mean_change_rate_first_10000"]) * (data["kurt"]))))) * (((((((data["abs_trend"]) * (data["kurt"]))) - (data["mean_change_abs"]))) + (((data["std_roll_mean_1000"]) + (((data["std_roll_mean_1000"]) + (((data["min_first_10000"]) + (data["abs_std"]))))))))))) +
                   
            0.0399921872*erf(((data["q95_roll_std_10"]) * ((-1.0*((((((-1.0*((data["abs_max_roll_mean_100"])))) + ((((-1.0*((((data["classic_sta_lta4_mean"]) / 2.0))))) - (((data["q001"]) * (((data["q95_roll_mean_1000"]) - ((-1.0*((((((((data["classic_sta_lta4_mean"]) / 2.0)) / 2.0)) / 2.0))))))))))))/2.0))))))) +
                   
            0.0399843715*erf(((((data["mean_change_rate_last_10000"]) * (((data["abs_max"]) * (((((((((((data["min_roll_std_1000"]) + (data["max_to_min"]))) - (data["count_big"]))) * 2.0)) - (data["max_to_min"]))) + (((data["av_change_abs_roll_std_1000"]) + (data["mean_change_abs"]))))))))) + (data["count_big"]))) +
                   
            0.0383434258*erf(((((data["med"]) * ((-1.0*((((((((data["trend"]) * (((erf((data["av_change_abs_roll_std_100"]))) - (data["mean_change_abs"]))))) + (data["mean_change_abs"]))) * 2.0))))))) * (((((data["av_change_abs_roll_std_100"]) + (erf((data["min_last_10000"]))))) + (data["av_change_abs_roll_std_100"]))))) +
                   
            0.0399999991*erf(erf((((((((data["min_roll_mean_100"]) * 2.0)) * 2.0)) * (((((((data["max_last_10000"]) - (0.5434780121))) * (data["av_change_rate_roll_mean_100"]))) + (((data["av_change_rate_roll_std_10"]) - (((data["min_roll_std_10"]) * (data["av_change_rate_roll_std_10"]))))))))))) +
                   
            0.0399921872*erf((-1.0*((((((((((data["min_roll_std_100"]) + (data["av_change_abs_roll_std_10"]))/2.0)) * (((data["av_change_abs_roll_mean_1000"]) + (data["av_change_abs_roll_std_10"]))))) + (((data["av_change_abs_roll_std_100"]) * (((data["classic_sta_lta4_mean"]) * (((data["av_change_abs_roll_std_10"]) + (((data["min_roll_std_1000"]) * (data["av_change_abs_roll_std_100"]))))))))))/2.0))))) +
                   
            0.0293416679*erf(erf((((data["abs_q95"]) * (((data["q01_roll_std_10"]) * (((data["q01_roll_std_10"]) * (((((data["max_last_50000"]) - ((((((data["av_change_abs_roll_mean_10"]) - (data["classic_sta_lta4_mean"]))) + (data["av_change_abs_roll_mean_100"]))/2.0)))) - ((-1.0*((((data["av_change_abs_roll_mean_10"]) * (data["classic_sta_lta4_mean"])))))))))))))))) +
                   
            0.0399765596*erf(((((data["trend"]) * (data["max_first_10000"]))) * (((data["med"]) + (((data["av_change_rate_roll_mean_1000"]) + (((((2.0769200325) - (((erf((((((((((data["classic_sta_lta1_mean"]) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) * 2.0)))) / 2.0)))))))) +
                   
            0.0399921872*erf(((((data["mean_change_rate_last_50000"]) * ((-1.0*((((data["min_first_10000"]) - (data["std_first_10000"])))))))) * ((((((((((-1.0*((data["std_first_10000"])))) - (data["min_last_50000"]))) - (data["abs_max_roll_std_10"]))) - (((data["mean_change_rate_last_50000"]) * (data["std_first_10000"]))))) - (data["classic_sta_lta2_mean"]))))) +
                   
            0.0399921872*erf(((data["av_change_abs_roll_mean_10"]) * (((data["abs_trend"]) + ((((((((data["mean_change_abs"]) * (data["min_last_10000"]))) + (((data["mean_change_rate_first_10000"]) * (data["max_first_10000"]))))) + ((((data["av_change_abs_roll_mean_10"]) + (((((data["med"]) - (data["q95_roll_mean_1000"]))) - (data["mean_change_rate_first_10000"]))))/2.0)))/2.0)))))) +
            0.0399531126*erf(((data["ave_roll_std_100"]) * (((data["min_roll_std_100"]) * (((((data["max_roll_mean_100"]) + ((((data["max_to_min_diff"]) + (data["q99"]))/2.0)))) * (((((((data["max_to_min_diff"]) * ((((-1.0*((data["std_first_50000"])))) * (data["q95_roll_std_100"]))))) - (data["skew"]))) * 2.0)))))))) +
            0.0399843715*erf(((data["std_roll_std_100"]) * (((data["min_roll_std_1000"]) * (((data["max_roll_mean_10"]) - (((data["kurt"]) * ((((((data["avg_last_10000"]) * (data["avg_last_10000"]))) + ((((data["q99_roll_mean_100"]) + ((((((data["avg_last_10000"]) * (data["avg_last_10000"]))) + (data["avg_first_10000"]))/2.0)))/2.0)))/2.0)))))))))) +
            0.0353584699*erf(((data["med"]) * (((erf((((erf(((((((-1.0*((data["max_to_min"])))) - (data["trend"]))) * 2.0)))) - (((data["min_roll_std_100"]) * (data["med"]))))))) - (erf((erf((data["min_roll_std_100"]))))))))) +
            0.0399765596*erf((((((((data["ave_roll_mean_10"]) + (data["min_roll_std_100"]))/2.0)) * (((((data["classic_sta_lta4_mean"]) + (data["kurt"]))) + (((((data["q95_roll_mean_10"]) + (((data["q95"]) * (data["q95"]))))) * (data["sum"]))))))) * ((((data["q05_roll_std_100"]) + (data["ave_roll_mean_10"]))/2.0)))) +
            0.0399843715*erf(((data["min_roll_std_10"]) * ((((((-1.0*((((data["av_change_abs_roll_std_10"]) * (data["av_change_abs_roll_mean_100"])))))) * (data["av_change_abs_roll_std_10"]))) + (erf(((((-1.0*((data["q05"])))) + (((data["min_roll_std_1000"]) * ((-1.0*((((data["av_change_abs_roll_mean_100"]) * (data["av_change_abs_roll_mean_100"])))))))))))))))) +
            0.0399999991*erf((-1.0*((((data["classic_sta_lta1_mean"]) * (((data["max_last_50000"]) + (((((data["abs_q95"]) + (((data["av_change_abs_roll_std_10"]) * 2.0)))) * (((((data["abs_std"]) + (data["max_last_50000"]))) * (((((data["iqr"]) * 2.0)) * (data["avg_last_10000"])))))))))))))) +
            0.0388200805*erf(((((data["min_first_10000"]) * (((data["mean_change_rate"]) * (((((data["min_roll_std_100"]) + (data["max_last_50000"]))) + (((((data["mean_change_rate"]) * (((data["mean_change_abs"]) * (((data["min_roll_std_100"]) + (data["min_first_10000"]))))))) - (data["mean_change_abs"]))))))))) * 2.0)) +
            0.0389529206*erf(((erf((erf((((data["max_to_min"]) + (data["max_to_min"]))))))) - (erf((((((((((((((((((((data["min_first_10000"]) * 2.0)) + (data["max_to_min"]))) * 2.0)) * 2.0)) + (data["max_last_50000"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)))))) +
            0.0399843715*erf((-1.0*((((((data["max_first_50000"]) * 2.0)) * (((erf(((((((-1.0*((((data["q95_roll_std_1000"]) + (((data["q01_roll_std_10"]) + (data["av_change_abs_roll_std_1000"])))))))) * 2.0)) * 2.0)))) + (((data["q01_roll_std_10"]) + (((data["max_first_50000"]) * (data["mean_change_abs"])))))))))))) +
            0.0399453007*erf((((((data["av_change_abs_roll_std_1000"]) + (data["mean_change_abs"]))/2.0)) * (((data["av_change_abs_roll_std_100"]) + (((data["trend"]) + (((((data["av_change_abs_roll_mean_1000"]) + ((((data["av_change_abs_roll_std_100"]) + (((data["av_change_rate_roll_std_100"]) + (((data["av_change_abs_roll_std_1000"]) + (data["min_roll_std_10"]))))))/2.0)))) * (data["av_change_abs_roll_mean_1000"]))))))))) +
            0.0399140455*erf(((data["std_first_50000"]) * (((data["mean_change_rate_last_50000"]) * ((-1.0*((((data["avg_first_50000"]) * ((((data["max_first_10000"]) + (((data["min_roll_mean_10"]) - (((data["avg_first_50000"]) - (((((data["mean_change_rate_last_50000"]) - (data["min_last_10000"]))) * (data["min_last_10000"]))))))))/2.0))))))))))) +
            0.0399999991*erf(((((data["min_last_10000"]) + (erf((((data["ave_roll_mean_1000"]) + (((((((data["av_change_abs_roll_mean_100"]) + (data["av_change_abs_roll_std_10"]))) * (data["ave_roll_mean_1000"]))) * (((data["min_roll_std_1000"]) * 2.0)))))))))) * (((data["min_roll_std_1000"]) * (((((data["std_roll_mean_1000"]) * 2.0)) * 2.0)))))) +
            0.0399765596*erf((((((data["av_change_abs_roll_mean_100"]) + (data["q99_roll_std_10"]))/2.0)) * (erf((((erf((((((((((((((data["avg_first_10000"]) + (data["max_roll_std_1000"]))) + (data["count_big"]))) * (73.0))) - (data["av_change_rate_roll_mean_100"]))) * 2.0)) + (data["avg_first_10000"]))))) * 2.0)))))) +
            0.0371556953*erf(((((data["max_last_10000"]) * (((((data["std_last_50000"]) * (data["avg_last_50000"]))) + (((((((data["min_roll_std_10"]) * 2.0)) + (((data["min_last_10000"]) * (((data["max_first_50000"]) * (((((data["min_roll_std_10"]) * 2.0)) * 2.0)))))))) * (data["max_first_10000"]))))))) * 2.0)) +
            0.0399609283*erf(((data["av_change_abs_roll_std_10"]) * ((((((((((data["min_last_50000"]) + (((data["trend"]) * (data["av_change_abs_roll_mean_10"]))))/2.0)) + (data["max_to_min"]))/2.0)) + (((data["min_last_50000"]) - (((data["min_last_50000"]) * (((data["min_last_50000"]) * (data["q05_roll_mean_1000"]))))))))/2.0)))) +
            0.0391638987*erf(((((data["trend"]) - ((((data["min_roll_std_100"]) + (data["q001"]))/2.0)))) * (((erf((erf((((((data["classic_sta_lta3_mean"]) + ((((data["classic_sta_lta3_mean"]) + ((((data["kurt"]) + (data["q95_roll_mean_100"]))/2.0)))/2.0)))) * 2.0)))))) * ((-1.0*((data["min_roll_std_100"])))))))) +
            0.0398749746*erf(((data["ave10"]) * ((-1.0*((((data["av_change_abs_roll_mean_10"]) * (((data["av_change_abs_roll_mean_100"]) + ((-1.0*(((((data["av_change_abs_roll_mean_10"]) + (((((data["classic_sta_lta2_mean"]) * 2.0)) + (((data["mean_change_rate_last_10000"]) * (data["av_change_rate_roll_mean_100"]))))))/2.0)))))))))))))) +
            0.0385778472*erf((-1.0*(((((erf((((((((data["trend"]) + (((data["min_roll_std_10"]) + (data["Moving_average_6000_mean"]))))) * 2.0)) * 2.0)))) + (((((-1.0*((((data["q95_roll_std_100"]) * (data["ave10"])))))) + (((data["mean_change_rate_first_50000"]) * (data["av_change_abs_roll_mean_10"]))))/2.0)))/2.0))))) +
            0.0399921872*erf(((((((((data["min_roll_std_10"]) * ((((data["ave_roll_std_10"]) + (data["avg_last_50000"]))/2.0)))) + (((((((((data["avg_last_10000"]) + (data["min_roll_std_10"]))/2.0)) + (data["min_roll_std_10"]))/2.0)) - (data["ave_roll_std_10"]))))/2.0)) + (((((data["q95_roll_std_100"]) - (data["ave_roll_std_10"]))) * 2.0)))/2.0)) +
            0.0399921872*erf((((((-1.0*((((data["mean_change_rate_first_10000"]) * 2.0))))) * ((((((((((data["av_change_rate_roll_std_10"]) * (data["avg_last_10000"]))) + (((data["min_last_10000"]) * (data["av_change_rate_roll_std_10"]))))) - (((data["avg_first_10000"]) / 2.0)))) + (data["av_change_rate_roll_std_1000"]))/2.0)))) * (data["av_change_abs_roll_std_1000"]))) +
            0.0399453007*erf(((data["av_change_abs_roll_std_1000"]) * (((data["avg_last_10000"]) * (((((((data["avg_last_10000"]) * (((data["avg_last_10000"]) * (((((data["med"]) * (((data["q001"]) - (data["classic_sta_lta1_mean"]))))) - (data["min_roll_std_10"]))))))) - (data["max_roll_mean_10"]))) - (data["min_roll_std_10"]))))))) +
            0.0398749746*erf(erf((erf((((((((((((-1.0*((data["av_change_abs_roll_std_100"])))) + ((-1.0*((data["mean_change_rate_first_50000"])))))/2.0)) / 2.0)) - (((data["mean_change_abs"]) * (((((((data["avg_last_50000"]) * (data["q99_roll_mean_10"]))) - (data["mean_change_rate_first_50000"]))) - (data["std_last_10000"]))))))) * 2.0)))))) +
            0.0363899209*erf(((data["max_last_10000"]) * (((((erf((((((((data["av_change_rate_roll_mean_100"]) + (((data["min_roll_std_1000"]) * 2.0)))) * 2.0)) * 2.0)))) + (((data["av_change_abs_roll_std_100"]) + ((-1.0*((data["mean_change_abs"])))))))) + (((((data["av_change_rate_roll_mean_10"]) * (data["max_last_50000"]))) * 2.0)))))) +
            0.0399999991*erf(((data["min_roll_std_100"]) * (((erf((data["min_last_10000"]))) + (((((erf((data["std_first_10000"]))) - ((((erf((data["min_last_10000"]))) + (((data["exp_Moving_average_30000_mean"]) * (data["min_last_10000"]))))/2.0)))) * ((((((data["avg_last_50000"]) + (data["mean_change_abs"]))/2.0)) * 2.0)))))))) +
            0.0351396762*erf(erf((((((data["av_change_abs_roll_std_1000"]) * (data["mean_change_rate_last_10000"]))) + (((data["med"]) * (erf((((data["av_change_rate_roll_std_100"]) - (((((data["avg_last_10000"]) - (((data["mean_change_rate_first_10000"]) - (((data["avg_last_10000"]) * (data["mean_change_rate_last_10000"]))))))) - (data["av_change_abs_roll_std_1000"]))))))))))))) +
            0.0399921872*erf((((erf((((((((((data["min_roll_std_10"]) + (((data["mean_change_rate_first_50000"]) + (((((data["mean_change_rate_first_50000"]) + (data["mean_change_rate_first_50000"]))) * (data["classic_sta_lta4_mean"]))))))) * (data["av_change_rate_roll_mean_1000"]))) * 2.0)) * 2.0)))) + (((data["av_change_abs_roll_mean_10"]) * (erf((data["med"]))))))/2.0)) +
            0.0379214697*erf((-1.0*((((((data["max_to_min_diff"]) * (((((((((((((data["min_last_10000"]) - (data["med"]))) - (data["mean_change_rate_first_10000"]))) + (data["q05_roll_std_100"]))) + (data["q01_roll_std_1000"]))) * 2.0)) * 2.0)))) * ((-1.0*((((data["mean_change_rate_first_10000"]) * (data["min_last_10000"]))))))))))) +
            0.0399218611*erf(((data["min_first_10000"]) * (erf((((((((((erf((((data["max_to_min"]) * 2.0)))) + (((((((data["min_first_50000"]) * 2.0)) + (((erf((erf((data["mean_change_rate_last_10000"]))))) + (data["min_roll_mean_10"]))))) * 2.0)))) * 2.0)) * 2.0)) * 2.0)))))) +
            0.0328423530*erf((-1.0*((((erf((((data["kurt"]) * (((((((data["mean_change_rate_last_50000"]) * 2.0)) * 2.0)) * (data["skew"]))))))) + (erf((((data["skew"]) * (((((data["mean_change_rate_last_50000"]) * 2.0)) * (data["skew"])))))))))))) +
            0.0346083194*erf((((((data["av_change_abs_roll_std_10"]) / 2.0)) + ((-1.0*((((erf((data["q05_roll_mean_1000"]))) + (erf((((((data["min_roll_std_100"]) - (data["mean_change_rate_last_50000"]))) - (((((data["q95_roll_mean_1000"]) - (data["av_change_abs_roll_std_10"]))) - (((data["av_change_abs_roll_mean_1000"]) - (data["mean_change_rate_last_50000"])))))))))))))))/2.0)) +
            0.0395155288*erf(((data["avg_last_50000"]) * ((-1.0*(((((erf((erf((((((((((((data["av_change_rate_roll_mean_100"]) * ((-1.0*((data["max_to_min"])))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)))))) + ((((((data["av_change_rate_roll_std_10"]) * (data["avg_last_10000"]))) + (data["av_change_rate_roll_std_10"]))/2.0)))/2.0))))))) +
            0.0316311792*erf(((data["max_first_50000"]) - (((data["classic_sta_lta3_mean"]) * (((((data["max_to_min_diff"]) + (data["mean_change_abs"]))) - (((data["trend"]) * (((((data["mean_change_abs"]) - (((erf((data["max_to_min_diff"]))) * 2.0)))) - (data["av_change_abs_roll_mean_100"]))))))))))) +
            0.0399609283*erf(((data["trend"]) * (((((((data["min_roll_std_1000"]) * (data["std_last_10000"]))) * ((((data["av_change_abs_roll_mean_10"]) + (((data["std_last_50000"]) - ((((((data["av_change_rate_roll_mean_10"]) * (data["q05_roll_std_1000"]))) + (data["min_roll_std_1000"]))/2.0)))))/2.0)))) + (((data["min_roll_std_1000"]) - (data["q05_roll_std_1000"]))))))) +
            0.0362180099*erf((((erf((((((((((((data["q01_roll_std_100"]) - (data["mean_change_rate_last_10000"]))) - ((((data["av_change_abs_roll_mean_10"]) + (data["abs_max_roll_mean_100"]))/2.0)))) - (0.2525250018))) * (data["min_roll_std_10"]))) * (73.0))))) + (((data["min_roll_mean_1000"]) + (data["abs_max_roll_mean_100"]))))/2.0)) +
            0.0399999991*erf((((erf((((73.0) * (((((73.0) * (data["av_change_abs_roll_std_100"]))) - (data["av_change_abs_roll_mean_100"]))))))) + (erf((((((((((data["av_change_rate_roll_mean_100"]) - (data["av_change_abs_roll_std_100"]))) - (data["av_change_abs_roll_std_100"]))) - (data["std_first_50000"]))) - (data["av_change_abs_roll_std_100"]))))))/2.0)) +
            0.0374213718*erf((((-1.0*((((data["q05"]) * (((data["max_last_10000"]) * (((((data["min_roll_std_10"]) - (data["std_last_10000"]))) - (data["std_last_10000"])))))))))) + (((data["max_last_50000"]) * (((data["mean_change_rate_last_10000"]) - (erf((((((data["av_change_rate_roll_std_10"]) * 2.0)) * 2.0)))))))))) +
            0.0331470966*erf(((((((((data["abs_max_roll_std_1000"]) * 2.0)) * 2.0)) * 2.0)) * (((((data["mean_change_rate_last_50000"]) * ((((((data["abs_max_roll_std_1000"]) * (((data["mean_change_abs"]) * 2.0)))) + ((((data["std_first_50000"]) + (data["mean_change_rate_last_50000"]))/2.0)))/2.0)))) + (((data["mean_change_abs"]) * (data["av_change_rate_roll_std_1000"]))))))) +
            0.0396796241*erf(((erf((erf((((((data["avg_first_10000"]) * (((((data["Moving_average_3000_mean"]) + (data["skew"]))) * 2.0)))) + (erf(((-1.0*((((((data["ave_roll_mean_100"]) + (((data["av_change_abs_roll_std_1000"]) + (((data["avg_first_10000"]) * 2.0)))))) * 2.0))))))))))))) / 2.0)) +
            0.0399765596*erf(((erf((((((data["av_change_abs_roll_mean_10"]) + (data["exp_Moving_average_300_mean"]))) * (((((data["classic_sta_lta4_mean"]) + (data["av_change_abs_roll_mean_10"]))) + (((((data["av_change_abs_roll_mean_10"]) + (((erf((((erf((data["avg_last_50000"]))) * 2.0)))) * 2.0)))) * (data["av_change_abs_roll_mean_10"]))))))))) / 2.0)) +
            0.0396014862*erf((((((data["max_last_50000"]) + (((((erf((erf((((((erf((erf(((((data["q95"]) + (data["mean_change_abs"]))/2.0)))))) + (((data["av_change_abs_roll_std_100"]) * 2.0)))) * 2.0)))))) * 2.0)) - (data["av_change_abs_roll_std_100"]))))/2.0)) * (data["classic_sta_lta4_mean"]))) +
            0.0317015015*erf(((erf((((((data["avg_last_10000"]) - (((data["iqr"]) - (((((data["avg_last_10000"]) - (((data["iqr"]) * (data["av_change_rate_roll_std_100"]))))) - (((((((((data["min_roll_std_100"]) * (data["av_change_rate_roll_std_100"]))) * 2.0)) * 2.0)) * 2.0)))))))) * 2.0)))) / 2.0)) +
            0.0399296731*erf((-1.0*((((data["q05_roll_std_1000"]) * (((((((((data["avg_first_10000"]) + (data["mean_change_abs"]))/2.0)) * (data["std_last_50000"]))) + (((((((data["avg_last_10000"]) + (((data["mean_change_rate_first_10000"]) * (data["mean_change_abs"]))))/2.0)) + (data["max_to_min_diff"]))/2.0)))/2.0))))))) +
            0.0377104878*erf((((erf((erf((((((((data["count_big"]) + (((data["min_last_10000"]) * (data["min_first_50000"]))))) * 2.0)) * 2.0)))))) + ((((data["count_big"]) + ((-1.0*((((data["max_roll_mean_100"]) - (erf((((data["min_last_10000"]) * (data["avg_last_10000"])))))))))))/2.0)))/2.0)) +
            0.0309357308*erf(((erf((((((((data["max_to_min"]) * (data["classic_sta_lta2_mean"]))) + (((data["classic_sta_lta2_mean"]) * (((data["max_to_min"]) * (data["classic_sta_lta2_mean"]))))))) * 2.0)))) / 2.0)) +
            0.0355303772*erf(((((erf((erf((((((((((data["av_change_abs_roll_std_10"]) - (erf((((data["skew"]) * 2.0)))))) * (data["mean_change_rate_first_10000"]))) * (data["mean_change_rate_first_10000"]))) * (((data["mean_change_rate_first_10000"]) + (data["av_change_abs_roll_std_10"]))))))))) * (data["mean_change_rate_first_10000"]))) * (data["mean_change_rate_first_10000"]))) +
            0.0399843715*erf(((data["av_change_abs_roll_std_1000"]) * (((((((data["trend"]) + (erf((((data["min_roll_std_1000"]) * ((((8.0)) + (((data["min_roll_std_1000"]) * ((8.0)))))))))))/2.0)) + ((-1.0*((((data["q01_roll_std_1000"]) + (((data["q01_roll_std_1000"]) * (data["mean_change_rate_first_50000"])))))))))/2.0)))) +
            0.0398437195*erf((-1.0*((((data["std_first_10000"]) * ((((((data["q99"]) / 2.0)) + (((data["skew"]) + (((((data["std_first_10000"]) * 2.0)) * (((data["std_first_10000"]) * (((data["std_first_10000"]) * (((data["std_first_10000"]) - (((data["MA_700MA_std_mean"]) * 2.0)))))))))))))/2.0))))))) +
            0.0399843715*erf((-1.0*((((data["mean_change_abs"]) * ((((((data["av_change_abs_roll_mean_1000"]) + ((((((((data["abs_max_roll_mean_10"]) + (((((data["min_roll_mean_100"]) * (data["av_change_abs_roll_mean_1000"]))) * (data["min_roll_std_10"]))))) - ((((data["min_roll_std_10"]) + (data["av_change_abs_roll_std_100"]))/2.0)))) + (data["skew"]))/2.0)))/2.0)) / 2.0))))))) +
            0.0295057632*erf(((((((((((((((data["q99_roll_mean_100"]) + (((data["q99_roll_mean_100"]) + (data["q95_roll_mean_10"]))))) * (((data["q05_roll_std_100"]) + ((((((data["av_change_abs_roll_std_1000"]) - (data["min_roll_std_100"]))) + (data["av_change_abs_roll_mean_100"]))/2.0)))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * (data["av_change_abs_roll_std_1000"]))) +
            0.0346864611*erf(((erf((((((((data["mean_change_rate"]) * (((data["mean_change_rate_first_10000"]) + (((data["mean_change_rate_first_10000"]) + (data["mean_change_rate_last_50000"]))))))) + (((data["q01_roll_std_100"]) * (((((data["mean_change_rate_first_10000"]) * (((data["q99_roll_std_10"]) * 2.0)))) + (data["mean_change_rate_first_10000"]))))))) * 2.0)))) / 2.0)) +
            0.0397265106*erf(erf((((((data["min_roll_std_100"]) * ((-1.0*((((data["av_change_abs_roll_mean_100"]) * ((((data["q05_roll_mean_100"]) + (data["mean_change_rate_first_10000"]))/2.0))))))))) - (((((((-1.0*((data["classic_sta_lta4_mean"])))) / 2.0)) + ((((data["av_change_rate_roll_mean_100"]) + ((((data["Moving_average_6000_mean"]) + (data["mean_change_rate_first_10000"]))/2.0)))/2.0)))/2.0)))))) +
            0.0399687439*erf(((erf((((data["min_first_50000"]) * (((((((((erf((((((data["q01_roll_std_1000"]) + (erf(((-1.0*((((((data["skew"]) + (data["classic_sta_lta2_mean"]))) + (data["q95_roll_std_10"])))))))))) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)))))) / 2.0)) +
            0.0365696438*erf(((erf((((((((((((((((data["mean_change_rate_last_10000"]) * (((data["ave_roll_std_10"]) - (((data["q95"]) - ((((((data["abs_mean"]) + (((data["iqr"]) / 2.0)))/2.0)) / 2.0)))))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) / 2.0)) +
            0.0393358096*erf(((((data["max_roll_std_10"]) * (((((data["q95_roll_mean_100"]) + (((data["q95_roll_mean_100"]) + (((data["q05_roll_mean_1000"]) * (data["classic_sta_lta2_mean"]))))))) * (((data["max_roll_std_10"]) * (((((data["skew"]) * (((data["q05_roll_mean_100"]) + (data["classic_sta_lta2_mean"]))))) * 2.0)))))))) * 2.0)) +
            0.0381871462*erf((((erf((((((((data["q05_roll_mean_10"]) - (((data["q05"]) + (data["ave_roll_std_100"]))))) * (73.0))) * (73.0))))) + (((((((data["abs_q05"]) * (((data["std_roll_mean_1000"]) * (data["av_change_rate_roll_std_10"]))))) * 2.0)) - (data["q05"]))))/2.0)) +
            0.0383043550*erf(((data["max_last_50000"]) * (((((data["av_change_rate_roll_std_100"]) * 2.0)) * (erf((((((((((((((erf((((data["mean_change_abs"]) - (data["classic_sta_lta3_mean"]))))) - (data["classic_sta_lta3_mean"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))))))) +
            0.0397812054*erf(erf((erf((((data["min_roll_mean_100"]) * (((((data["av_change_rate_roll_mean_1000"]) - (data["min_roll_mean_100"]))) - (((((((data["min_last_50000"]) * (((data["av_change_rate_roll_mean_1000"]) - (erf((((data["min_roll_std_10"]) - (((data["min_roll_mean_100"]) * 2.0)))))))))) * 2.0)) * 2.0)))))))))) +
            0.0399921872*erf((-1.0*((((((erf((((((((((((((((((data["mean_change_abs"]) * 2.0)) - ((((((-1.0*((data["mean_change_rate_last_50000"])))) * 2.0)) - (data["trend"]))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) / 2.0)) / 2.0))))) +
            0.0329673775*erf(erf(((-1.0*((((data["std_first_10000"]) * (((((((((data["av_change_abs_roll_mean_10"]) + (data["max_last_10000"]))) * (((((data["max_last_10000"]) * (0.8873239756))) + (data["classic_sta_lta2_mean"]))))) * 2.0)) * (data["max_last_10000"])))))))))) +
            0.0367962494*erf(((data["avg_last_10000"]) * (((((data["std_roll_mean_1000"]) * ((-1.0*(((((((((data["mean_change_rate_first_50000"]) - (((data["mean_change_abs"]) * (data["max_last_10000"]))))) + (data["max_to_min_diff"]))/2.0)) - (((data["max_last_10000"]) * (((data["max_to_min"]) * (data["q05_roll_mean_100"])))))))))))) * 2.0)))) +
            0.0399921872*erf(((data["av_change_abs_roll_mean_10"]) * (((((data["av_change_abs_roll_std_1000"]) - ((((data["av_change_abs_roll_mean_10"]) + ((((data["min_roll_std_10"]) + (((((((data["av_change_rate_roll_mean_10"]) * (data["av_change_abs_roll_std_10"]))) + (data["av_change_abs_roll_mean_10"]))) * (data["av_change_abs_roll_mean_10"]))))/2.0)))/2.0)))) * (data["min_roll_std_10"]))))) +
            0.0395311601*erf(((erf(((-1.0*((((data["std_first_10000"]) + (data["min_first_10000"])))))))) + (erf((((data["max_roll_std_10"]) * (((data["q95_roll_mean_100"]) * ((((((((data["mean_change_abs"]) + (data["max_roll_std_10"]))/2.0)) - ((-1.0*((data["min_first_10000"])))))) + (data["mean_change_rate_last_50000"]))))))))))) +
            0.0399296731*erf(((((data["min_roll_std_10"]) + (((data["min_roll_std_10"]) * (data["med"]))))) * ((((((data["classic_sta_lta1_mean"]) + (data["min_roll_mean_1000"]))/2.0)) - (((data["q05_roll_std_10"]) * ((((((data["med"]) * ((((data["classic_sta_lta1_mean"]) + (data["min_roll_mean_1000"]))/2.0)))) + (data["med"]))/2.0)))))))) +
            0.0399453007*erf(erf((((((((erf((data["min_roll_std_10"]))) * (data["std_roll_mean_1000"]))) * 2.0)) + ((((((((((((data["max_to_min_diff"]) * 2.0)) * (data["std_last_50000"]))) + (data["min_roll_std_10"]))/2.0)) / 2.0)) + (((data["min_roll_std_10"]) * (erf((data["max_to_min_diff"]))))))))))) +
            0.0394999012*erf(((data["mean_change_rate_first_10000"]) * (((data["min_first_10000"]) * ((((((((data["min_roll_std_10"]) * (((data["av_change_abs_roll_std_100"]) + (data["av_change_abs_roll_std_100"]))))) + ((((data["mean_change_rate_first_10000"]) + (((data["min_roll_std_10"]) * (data["mean_change_rate_first_10000"]))))/2.0)))/2.0)) + (((data["min_roll_std_10"]) - (data["av_change_abs_roll_std_100"]))))))))) +
            0.0399687439*erf(((data["max_first_10000"]) * (((((data["std_first_10000"]) * (((data["max_first_10000"]) * (((data["std_first_10000"]) * (((erf((((data["std_first_10000"]) * (data["av_change_abs_roll_mean_10"]))))) + (data["std_roll_mean_100"]))))))))) - (((((data["av_change_abs_roll_mean_10"]) * (data["min_roll_std_100"]))) / 2.0)))))) +
            0.0397499502*erf(((data["trend"]) * ((((((data["std_first_50000"]) + (erf((erf((((data["av_change_rate_roll_mean_10"]) + (((((((data["av_change_rate_roll_mean_1000"]) + ((((data["min_roll_std_100"]) + (((data["Hilbert_mean"]) + (data["av_change_rate_roll_mean_10"]))))/2.0)))) * 2.0)) * 2.0)))))))))/2.0)) * (data["min_roll_std_100"]))))) +
            0.0274897423*erf((((erf((((((((((((((data["q95_roll_std_100"]) - (data["ave_roll_std_10"]))) - (data["abs_max_roll_mean_10"]))) - (data["max_last_50000"]))) - (((data["min_first_50000"]) + (data["ave_roll_std_10"]))))) * 2.0)) * 2.0)))) + (((((data["q95_roll_std_100"]) - (data["ave_roll_std_10"]))) * 2.0)))/2.0)) +
            0.0399531126*erf(((((data["mean_change_rate_first_10000"]) * (erf((((((((((data["av_change_abs_roll_mean_10"]) + (data["mean_change_rate_last_10000"]))/2.0)) * (((data["avg_first_10000"]) - (erf((data["av_change_abs_roll_mean_100"]))))))) + (erf((erf((data["avg_first_10000"]))))))/2.0)))))) * ((((-1.0*((data["mean_change_abs"])))) * 2.0)))) +
            0.0340613388*erf(((data["skew"]) * (((data["max_first_50000"]) * (((((((((erf((((((((data["iqr"]) * 2.0)) - (data["av_change_abs_roll_mean_10"]))) - (data["std_last_10000"]))))) + (data["av_change_abs_roll_mean_10"]))) - (((data["av_change_rate_roll_mean_10"]) * (data["avg_first_50000"]))))) * 2.0)) * 2.0)))))) +
            0.0399921872*erf((((((data["min_roll_std_10"]) + (((data["min_roll_std_10"]) * (data["avg_first_10000"]))))/2.0)) * (erf((((73.0) * (((data["avg_first_10000"]) * (((data["min_roll_std_10"]) + (((73.0) * (((73.0) * ((-1.0*((data["av_change_abs_roll_std_1000"])))))))))))))))))) +
            0.0356475860*erf(erf((((data["av_change_abs_roll_mean_1000"]) * ((((((((((((((data["avg_last_50000"]) + (data["kurt"]))) + (0.2783510089))/2.0)) + (data["av_change_rate_roll_std_1000"]))) + (0.2783510089))) + (data["kurt"]))) * ((((erf((data["avg_first_10000"]))) + (data["mean_change_rate_first_50000"]))/2.0)))))))) +
            0.0329986326*erf((((((((((data["kurt"]) + (data["q05_roll_mean_10"]))) * (data["min_roll_mean_10"]))) + (erf((((((data["q05_roll_mean_10"]) - (((((data["max_to_min"]) - (data["kurt"]))) + (((data["q05_roll_mean_10"]) * (data["max_to_min"]))))))) * 2.0)))))/2.0)) / 2.0)) +
            0.0399999991*erf(((data["std_roll_mean_1000"]) * (((data["med"]) * ((((-3.0) + (((((data["avg_first_10000"]) * 2.0)) * (((data["mean_change_rate_first_10000"]) - (((((((data["std_roll_mean_1000"]) / 2.0)) * (((data["std_roll_mean_1000"]) * (-3.0))))) * (data["abs_max_roll_mean_1000"]))))))))/2.0)))))) +
            0.0303496774*erf((-1.0*((((erf((erf((((((((data["classic_sta_lta2_mean"]) + (((data["av_change_abs_roll_std_10"]) + (((data["av_change_abs_roll_mean_10"]) + (data["classic_sta_lta1_mean"]))))))) * (data["abs_q05"]))) * (3.6923100948))))))) / 2.0))))) +
            0.0399687439*erf((((-1.0*((data["max_first_50000"])))) * (((((data["std_roll_mean_1000"]) + (((data["av_change_abs_roll_std_10"]) * (((data["ave_roll_mean_10"]) + (((data["med"]) + (1.0))))))))) * (((data["avg_last_50000"]) + (((data["med"]) + (2.0769200325))))))))) +
            0.0399843715*erf(((erf((((data["av_change_rate_roll_mean_100"]) * (((erf((((((((((data["av_change_abs_roll_mean_100"]) * 2.0)) + (data["mean_change_rate_first_50000"]))) * 2.0)) * 2.0)))) - (((data["av_change_abs_roll_mean_100"]) * (((((data["av_change_rate_roll_std_1000"]) - (data["classic_sta_lta2_mean"]))) - (data["classic_sta_lta2_mean"]))))))))))) / 2.0)) +
            0.0399999991*erf(((data["mean_change_abs"]) * ((((data["av_change_abs_roll_std_100"]) + (erf((((((((((((((((((data["min_first_10000"]) - (data["av_change_rate_roll_std_10"]))) - (((data["av_change_abs_roll_std_100"]) * (3.6923100948))))) * 2.0)) * 2.0)) * 2.0)) + (data["mean_change_abs"]))) * 2.0)) * 2.0)))))/2.0)))) +
            0.0285446383*erf(((erf((((((73.0) * (data["mean_change_rate_first_10000"]))) + (((((((((data["trend"]) + (((data["av_change_abs_roll_mean_10"]) + (data["min_last_50000"]))))) + (data["mean_change_rate_first_10000"]))) * 2.0)) * 2.0)))))) + ((-1.0*((erf((data["mean_change_rate_first_10000"])))))))) +
            0.0399687439*erf(((((erf((((((((((((((((((((erf(((((((data["kurt"]) + (data["classic_sta_lta4_mean"]))/2.0)) - (data["max_first_10000"]))))) + (data["skew"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) / 2.0)) / 2.0)) +
            0.0398593470*erf(((erf(((-1.0*((erf((((data["avg_first_10000"]) * (((data["min_roll_std_10"]) - (((((data["mean_change_rate_last_10000"]) * (data["min_roll_std_10"]))) + ((((data["mean_change_abs"]) + ((((((data["ave_roll_mean_100"]) + (data["avg_first_10000"]))/2.0)) + (data["skew"]))))/2.0))))))))))))))) / 2.0)) +
            0.0399296731*erf(((erf((erf((erf((erf((((((((data["mean_change_abs"]) * ((((data["av_change_abs_roll_mean_100"]) + (0.7446810007))/2.0)))) - (((erf((data["skew"]))) + ((((data["trend"]) + (0.7446810007))/2.0)))))) * 2.0)))))))))) / 2.0)) +
            0.0387341268*erf((((((((data["min_roll_std_100"]) * (erf((((((((data["av_change_rate_roll_std_100"]) * 2.0)) + (((data["abs_q05"]) + (erf((data["iqr"]))))))) * 2.0)))))) + (erf((((erf((((data["iqr"]) * 2.0)))) - (data["min_roll_std_10"]))))))/2.0)) / 2.0)) +
            0.0384997055*erf(((data["av_change_abs_roll_mean_1000"]) * (((erf((((((erf((data["std_first_50000"]))) + ((((-1.0*((data["max_to_min"])))) * (data["std_first_10000"]))))) + (((data["av_change_abs_roll_mean_1000"]) * (data["min_roll_mean_1000"]))))))) + ((((-1.0*((data["min_roll_std_10"])))) * (data["std_first_10000"]))))))) +
            0.0384918936*erf(((erf((erf((((((73.0) * (((73.0) * (((((data["q95_roll_std_100"]) + (data["std_roll_mean_1000"]))) * 2.0)))))) * (((data["std_first_50000"]) + (((erf((erf((data["max_roll_std_1000"]))))) * (data["kurt"]))))))))))) / 2.0)) +
            0.0343192033*erf(((((-1.0) - (erf((((((((0.8873239756) + (data["mean_change_abs"]))) - (data["Hann_window_mean"]))) * (((((((((data["Hann_window_mean"]) - (data["min_last_10000"]))) * 2.0)) * 2.0)) + (data["av_change_rate_roll_std_100"]))))))))) / 2.0)) +
            0.0372416489*erf(((erf((((((data["classic_sta_lta2_mean"]) + (((data["q95_roll_mean_1000"]) * (data["classic_sta_lta2_mean"]))))) + ((((-1.0*((((((data["std_last_10000"]) + (data["MA_400MA_BB_low_mean"]))) + (erf((((data["min_roll_std_100"]) + (data["classic_sta_lta2_mean"])))))))))) * (data["avg_last_10000"]))))))) / 2.0)) +
            0.0377808139*erf(erf((((((data["mean_change_rate_last_10000"]) * (((((((data["mean_change_rate_first_10000"]) + ((((data["min_roll_std_10"]) + ((-1.0*((-3.0)))))/2.0)))/2.0)) + (data["Moving_average_1500_mean"]))/2.0)))) * (erf((((((data["mean_change_rate_last_10000"]) + (((data["av_change_abs_roll_std_1000"]) + (1.9019600153))))) * 2.0)))))))) +
            0.0369056463*erf(erf((erf((((((data["max_first_50000"]) * (data["max_first_50000"]))) * ((((((((data["max_first_50000"]) + (data["kurt"]))/2.0)) - (data["avg_last_50000"]))) - (((erf((((data["mean_change_rate_first_10000"]) * 2.0)))) - (((data["kurt"]) - (0.5555559993))))))))))))))

In [6]:
def GPI_tanh(data):
    return (5.591662 +
            0.0399999991*np.tanh(((data["mean_change_rate_first_10000"]) - (((73.0) * ((((((7.0) * (((((((((data["q95"]) + (((0.3183098733) + (data["q05_roll_std_100"]))))) * 2.0)) * 2.0)) + (((data["q95_roll_std_10"]) + (data["iqr"]))))))) + (data["iqr"]))/2.0)))))) +
            0.0399999991*np.tanh(((((3.6923100948) + (data["Moving_average_6000_mean"]))) * (((73.0) * (((data["MA_400MA_BB_low_mean"]) - (((((data["Moving_average_6000_mean"]) * (data["q05_roll_std_10"]))) + (((data["iqr"]) + (((data["q05_roll_std_10"]) * (((3.6923100948) * 2.0)))))))))))))) +
            0.0399999991*np.tanh(((((((((((((((((((((((data["q05"]) * 2.0)) * 2.0)) - (data["iqr"]))) - (data["max_roll_std_1000"]))) * 2.0)) * 2.0)) * 2.0)) - (data["iqr"]))) - (3.6923100948))) - (((data["q05_roll_std_10"]) * (73.0))))) * 2.0)) +
            0.0399999991*np.tanh(((((((((((((((((((data["q05_roll_std_10"]) * (((((-3.0) * 2.0)) * 2.0)))) - (((data["iqr"]) + (data["max_roll_std_1000"]))))) * 2.0)) * 2.0)) * 2.0)) - (data["max_roll_mean_100"]))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*np.tanh(((73.0) * ((((data["max_first_50000"]) + ((((data["max_first_50000"]) + (((73.0) * (((data["q05"]) - (((data["q05_roll_std_100"]) + (((0.5555559993) + (((data["mad"]) + (data["q95"]))))))))))))/2.0)))/2.0)))) +
            0.0399999991*np.tanh(((((((data["q05_roll_std_10"]) * (73.0))) * 2.0)) - ((((((((data["q05_roll_std_10"]) * (73.0))) + (((((((((data["iqr"]) + (((data["MA_1000MA_std_mean"]) + (data["abs_max_roll_mean_10"]))))) * 2.0)) * 2.0)) * 2.0)))/2.0)) * (73.0))))) +
            0.0399999991*np.tanh(((((data["mean_change_abs"]) - (((((data["q95"]) + (((data["q95"]) + (((((data["q05_roll_std_100"]) / 2.0)) + (((((((0.3183098733) * 2.0)) - (data["q05"]))) - (data["q05"]))))))))) * (73.0))))) * 2.0)) +
            0.0399999991*np.tanh((-1.0*(((((((7.48210620880126953)) * 2.0)) + (((((data["iqr"]) + ((((-1.0*((data["avg_first_10000"])))) * 2.0)))) + (((73.0) * (((data["iqr"]) + (((((data["q05_roll_std_10"]) * 2.0)) * 2.0))))))))))))) +
            0.0399999991*np.tanh((((((data["q01_roll_std_10"]) + (((((data["q05_roll_mean_10"]) - (((((data["q05_roll_std_100"]) - (((data["q05"]) - (0.3026320040))))) + (((((((0.5434780121) + (0.3026320040))/2.0)) + (data["iqr"]))/2.0)))))) * (73.0))))/2.0)) * (73.0))) +
            0.0399999991*np.tanh(((((data["trend"]) + (((data["q95_roll_std_10"]) + (data["trend"]))))) - (((73.0) * (((((((data["ave_roll_std_10"]) + (((((((data["q95_roll_mean_10"]) + (data["q05_roll_std_1000"]))/2.0)) + (data["q05_roll_std_1000"]))/2.0)))/2.0)) + ((((0.5434780121) + (data["q95_roll_mean_10"]))/2.0)))/2.0)))))) +
            0.0399999991*np.tanh(((73.0) - (((((73.0) * 2.0)) * (((73.0) * ((((((((((data["q05_roll_std_1000"]) / 2.0)) + (data["iqr"]))/2.0)) + (data["q05_roll_std_10"]))) + (((0.2183910012) + (data["q95"]))))))))))) +
            0.0399999991*np.tanh(((73.0) * (((73.0) * (((((73.0) * (((data["q05_roll_mean_10"]) - (((((data["q05_roll_std_100"]) + (data["q95"]))) + (0.5434780121))))))) + (data["q05_roll_std_100"]))))))) +
            0.0399999991*np.tanh((-1.0*((((73.0) * (((73.0) * (((73.0) * (((data["q05_roll_std_100"]) + ((((data["q95_roll_mean_10"]) + ((((data["Moving_average_3000_mean"]) + (((((data["iqr"]) + (0.9756100178))) + (data["abs_mean"]))))/2.0)))/2.0))))))))))))) +
            0.0399999991*np.tanh(((73.0) * (((((data["classic_sta_lta3_mean"]) + (data["mad"]))) + (((((((73.0) * (((data["q05_roll_mean_10"]) - (((data["q05_roll_std_100"]) + (((data["mad"]) + (0.5555559993))))))))) - (data["iqr"]))) - (data["iqr"]))))))) +
            0.0399999991*np.tanh(((((np.tanh((data["q95_roll_std_10"]))) + (((((((data["q05_roll_std_10"]) + ((((((0.2439019978) + (data["ave10"]))/2.0)) / 2.0)))) * 2.0)) * 2.0)))) * (((data["q05_roll_std_10"]) - (((73.0) * (((73.0) * (73.0))))))))) +
            0.0399999991*np.tanh(((3.6923100948) * (((73.0) * (((3.6923100948) * ((((((-1.0*((data["q99_roll_mean_1000"])))) * (data["avg_last_50000"]))) - (((data["q05_roll_std_10"]) * (((3.6923100948) + (((3.6923100948) + (data["abs_max_roll_mean_100"]))))))))))))))) +
            0.0399999991*np.tanh(((((((data["Moving_average_1500_mean"]) + (((((((((data["MA_400MA_BB_low_mean"]) + ((((-1.0*((data["q05_roll_std_1000"])))) - (data["iqr"]))))) * (73.0))) - (data["min_roll_std_1000"]))) - (data["q05_roll_std_1000"]))))) * (73.0))) * (73.0))) +
            0.0399999991*np.tanh(((73.0) - (((73.0) * (((73.0) * (((data["q05_roll_std_10"]) + (np.tanh(((((((data["exp_Moving_average_3000_mean"]) + (((data["iqr"]) + (np.tanh((((data["q05_roll_std_100"]) + (data["exp_Moving_average_30000_mean"]))))))))/2.0)) / 2.0)))))))))))) +
            0.0399999991*np.tanh(((73.0) * (((73.0) * (((data["q05_roll_mean_10"]) - (((np.tanh((np.tanh((np.tanh((np.tanh((np.tanh((data["q05_roll_std_100"]))))))))))) + ((((data["q95_roll_std_10"]) + (np.tanh((2.8437500000))))/2.0)))))))))) +
            0.0399999991*np.tanh(((((((data["iqr"]) / 2.0)) + (((data["mean"]) + (((data["q05_roll_std_100"]) + (((data["q05_roll_std_10"]) * 2.0)))))))) * ((-1.0*((((73.0) - (((((((((data["mean"]) * 2.0)) * 2.0)) * 2.0)) * 2.0))))))))) +
            0.0399999991*np.tanh(((((((((0.5217390060) - (((((data["q05_roll_std_10"]) * ((-1.0*((7.0)))))) + (data["exp_Moving_average_3000_mean"]))))) * ((-1.0*((73.0)))))) - (((data["classic_sta_lta4_mean"]) * (7.0))))) - ((((7.0) + (data["classic_sta_lta4_mean"]))/2.0)))) +
            0.0399999991*np.tanh(((((((((((((((((((np.tanh(((-1.0*((((data["q05_roll_mean_100"]) + (0.2183910012)))))))) - ((((data["q05_roll_std_100"]) + (((data["q05_roll_std_10"]) * 2.0)))/2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) - (data["exp_Moving_average_30000_mean"]))) * 2.0)) +
            0.0399999991*np.tanh(((data["trend"]) - (((73.0) * ((((3.6923100948) + ((((9.0)) * (((np.tanh((np.tanh((np.tanh((np.tanh((np.tanh((np.tanh((np.tanh((data["q05_roll_std_100"]))))))))))))))) - (data["q05_roll_mean_10"]))))))/2.0)))))) +
            0.0399999991*np.tanh(((data["q05_roll_std_1000"]) - (((73.0) * (((data["avg_last_50000"]) + (((((73.0) * 2.0)) * (((data["ave_roll_std_100"]) + (((data["Moving_average_6000_mean"]) + (((data["iqr"]) + (((data["q05_roll_std_1000"]) + (((data["q95"]) * 2.0)))))))))))))))))) +
            0.0399999991*np.tanh(((((data["q05_roll_mean_10"]) - (2.0769200325))) - ((((((6.08570814132690430)) * (((2.0769200325) + (((((((((data["q05_roll_std_10"]) - (data["q05_roll_mean_10"]))) * 2.0)) * 2.0)) * 2.0)))))) + (((((data["av_change_rate_roll_std_100"]) - (data["avg_last_10000"]))) * 2.0)))))) +
            0.0399999991*np.tanh((-1.0*((((((((((((((((((((((((((((((((data["q05_roll_std_1000"]) + (data["q05_roll_mean_100"]))/2.0)) * 2.0)) + (data["iqr"]))/2.0)) + (data["q95_roll_mean_10"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0))))) +
            0.0399999991*np.tanh((((-1.0*((((((((((data["q05_roll_std_100"]) - (np.tanh(((-1.0*((((data["MA_400MA_std_mean"]) + (((data["exp_Moving_average_300_mean"]) * (((data["q05_roll_std_100"]) * 2.0))))))))))))) * 2.0)) * 2.0)) + (np.tanh((data["q05_roll_std_100"])))))))) * (73.0))) +
            0.0399999991*np.tanh(((((data["abs_q05"]) - ((((((2.0769200325) * ((((((73.0) * ((((((0.5555559993) + (((data["q05_roll_std_100"]) + (0.7446810007))))/2.0)) + (data["q05_roll_std_100"]))))) + (0.5555559993))/2.0)))) + (0.5555559993))/2.0)))) * 2.0)) +
            0.0399999991*np.tanh((((10.75933361053466797)) - (((7.0) * (((7.0) * (((7.0) * ((((((data["q95_roll_mean_10"]) + (data["q05_roll_std_100"]))) + ((((data["q95_roll_mean_10"]) + (((data["iqr"]) + (((data["Moving_average_700_mean"]) * 2.0)))))/2.0)))/2.0)))))))))) +
            0.0399999991*np.tanh(((73.0) * (((((data["q05_roll_std_1000"]) + ((((data["q05_roll_std_10"]) + ((((((data["q05_roll_std_1000"]) + (np.tanh((data["q05_roll_std_1000"]))))/2.0)) - (((data["q95_roll_mean_1000"]) - (np.tanh((73.0))))))))/2.0)))) * (((2.8437500000) - (73.0))))))) +
            0.0399999991*np.tanh(((((((((-3.0) + (data["Moving_average_3000_mean"]))) - (((73.0) * (((data["ave_roll_mean_10"]) - ((((((data["q05_roll_std_100"]) + (data["q95_roll_mean_10"]))/2.0)) * (((-3.0) - (((data["Moving_average_700_mean"]) + (data["q95_roll_mean_10"]))))))))))))) * 2.0)) * 2.0)) +
            0.0399999991*np.tanh(((((((data["classic_sta_lta3_mean"]) * 2.0)) + (data["q05_roll_mean_100"]))) - (((73.0) * (((((((0.6582279801) + (data["q05_roll_std_10"]))) - (((data["q05_roll_mean_10"]) * 2.0)))) + ((((-1.0*((data["abs_q05"])))) + (((data["q05_roll_std_100"]) * 2.0)))))))))) +
            0.0399999991*np.tanh(((((data["skew"]) + (((data["kurt"]) + (data["med"]))))) - ((((7.02540111541748047)) * (((((data["iqr"]) + (((data["q95_roll_std_10"]) + (((((data["q95_roll_mean_10"]) + (((data["iqr"]) * (data["Moving_average_1500_mean"]))))) * 2.0)))))) * 2.0)))))) +
            0.0399999991*np.tanh(((((data["q05_roll_std_100"]) - (((-2.0) + (((((data["iqr"]) * (-2.0))) - ((-1.0*((data["av_change_rate_roll_std_10"])))))))))) - ((((((data["q05_roll_std_100"]) + (np.tanh((((data["q05_roll_std_100"]) * (data["ave_roll_mean_10"]))))))/2.0)) * (73.0))))) +
            0.0399999991*np.tanh(((((((((((data["q05_roll_std_10"]) + (((data["av_change_abs_roll_std_1000"]) - (((data["av_change_abs_roll_mean_10"]) * 2.0)))))) * 2.0)) + (data["av_change_abs_roll_std_1000"]))) * 2.0)) - (((73.0) * (((((data["abs_max_roll_mean_1000"]) + (((0.5555559993) + (data["q05_roll_std_10"]))))) * 2.0)))))) +
            0.0399999991*np.tanh((((((((((((((((((np.tanh((((((data["q95_roll_mean_10"]) + (data["q95_roll_std_100"]))) - (data["q05_roll_std_1000"]))))) + (data["q95_roll_std_100"]))/2.0)) + (data["q95_roll_std_100"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) - (((73.0) * (data["q95_roll_mean_10"]))))) * 2.0)) +
            0.0399999991*np.tanh(((73.0) - (((((((((((data["q05_roll_std_10"]) * 2.0)) - (data["ave10"]))) + (((data["q05_roll_std_100"]) + (0.9756100178))))) * (73.0))) * (((73.0) * 2.0)))))) +
            0.0399999991*np.tanh(((((data["Moving_average_6000_mean"]) + (((7.0) + (((data["ave10"]) * 2.0)))))) * (((((((((data["MA_700MA_BB_low_mean"]) * (data["avg_last_50000"]))) + ((-1.0*((((data["q95_roll_mean_10"]) * 2.0))))))) - (((data["ave_roll_mean_10"]) + (data["q05_roll_std_100"]))))) * 2.0)))) +
            0.0399999991*np.tanh(((73.0) * (((data["q05_roll_std_100"]) - (((((((data["iqr"]) - (np.tanh((((73.0) * (((data["ave10"]) - (((data["iqr"]) + (((((data["q05_roll_std_1000"]) + (0.5434780121))) * 2.0)))))))))))) * 2.0)) * 2.0)))))) +
            0.0399999991*np.tanh(((((((((((((((data["exp_Moving_average_30000_mean"]) * (((data["exp_Moving_average_30000_mean"]) + ((-1.0*((data["q01_roll_std_10"])))))))) - (((data["q05_roll_std_10"]) + (((data["q05_roll_std_10"]) + (((data["q95_roll_mean_100"]) + (data["q05_roll_std_10"]))))))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*np.tanh(((73.0) - (((((data["q05_roll_std_100"]) * (73.0))) * (((73.0) * (((data["q05_roll_std_100"]) * (((data["q05_roll_std_100"]) - (((data["q05"]) - ((((0.2525250018) + (2.0))/2.0)))))))))))))) +
            0.0399999991*np.tanh(((((((((7.0) - ((((data["sum"]) + (((73.0) * (((((data["sum"]) + (data["q05_roll_std_10"]))) + (((((((np.tanh((data["abs_q05"]))) + (data["q05_roll_std_10"]))) * 2.0)) * 2.0)))))))/2.0)))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*np.tanh((((((1.0)) - (((((((((data["q05_roll_std_100"]) + (np.tanh((((data["abs_q05"]) * (((((data["exp_Moving_average_30000_mean"]) - (data["abs_q05"]))) + (((data["q05_roll_std_100"]) + (((data["q95_roll_mean_10"]) * 2.0)))))))))))) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) +
            0.0399999991*np.tanh(((((np.tanh((((np.tanh((data["Moving_average_1500_mean"]))) * 2.0)))) - (((data["Moving_average_1500_mean"]) + ((-1.0*((((((data["q05_roll_std_100"]) + (np.tanh((0.6582279801))))) * 2.0))))))))) * ((((-1.0*((((data["Moving_average_1500_mean"]) * 2.0))))) - (73.0))))) +
            0.0399999991*np.tanh(((73.0) - ((((((((((((data["q05_roll_std_100"]) + (((data["q05_roll_std_100"]) * (data["Moving_average_6000_mean"]))))/2.0)) + (((data["Moving_average_6000_mean"]) + (((data["q05_roll_std_100"]) + (data["q95"]))))))) * 2.0)) * 2.0)) * (((73.0) + (data["Moving_average_6000_mean"]))))))) +
            0.0399999991*np.tanh(((((((((((((data["mean_change_rate_last_10000"]) - (data["mean_change_abs"]))) - (((((((((((((data["q05_roll_std_10"]) + (((0.5434780121) - (data["q05"]))))) * 2.0)) - (data["Hann_window_mean"]))) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*np.tanh(((((((-1.0*((((data["avg_last_50000"]) * (3.6923100948)))))) + (((73.0) / 2.0)))/2.0)) - (((73.0) * ((((((data["Moving_average_6000_mean"]) + (((data["q05_roll_std_10"]) - (((data["exp_Moving_average_30000_mean"]) * (data["exp_Moving_average_30000_mean"]))))))/2.0)) + (data["q05_roll_std_10"]))))))) +
            0.0399999991*np.tanh(((73.0) * ((-1.0*((((data["q01_roll_std_1000"]) + (((73.0) * (((data["q05_roll_std_100"]) + ((((1.4411799908) + ((((((data["q01_roll_std_1000"]) - (data["q99_roll_mean_1000"]))) + (((data["q99_roll_mean_1000"]) * (data["q99_roll_mean_1000"]))))/2.0)))/2.0))))))))))))) +
            0.0399999991*np.tanh((((12.50784111022949219)) + ((((12.50784111022949219)) - ((((((np.tanh((((data["q05_roll_std_10"]) * (data["q95_roll_mean_1000"]))))) + (((((((data["q05_roll_std_10"]) + (np.tanh((((data["q05_roll_std_10"]) * (data["exp_Moving_average_30000_mean"]))))))) * 2.0)) * 2.0)))/2.0)) * (73.0))))))) +
            0.0399999991*np.tanh(((((((data["skew"]) - (((data["ave_roll_mean_100"]) + (((((((data["ave_roll_mean_100"]) * (data["abs_q05"]))) + (((((2.0) + (((((data["q05_roll_std_100"]) - (data["q05_roll_mean_10"]))) * 2.0)))) * 2.0)))) * 2.0)))))) * 2.0)) * 2.0)) +
            0.0399999991*np.tanh(((((((((data["q95_roll_mean_1000"]) * (data["ave_roll_mean_100"]))) * 2.0)) - (((((data["exp_Moving_average_30000_mean"]) + (((((((((data["exp_Moving_average_30000_mean"]) * (data["q05_roll_std_10"]))) + (((((data["q05_roll_std_10"]) - (0.2439019978))) * 2.0)))) * 2.0)) * 2.0)))) * 2.0)))) * 2.0)) +
            0.0399999991*np.tanh(((((((((data["q05_roll_std_10"]) - (((((data["q95"]) - (((((data["iqr"]) - (((data["exp_Moving_average_3000_mean"]) * (np.tanh((((data["q05_roll_std_10"]) * 2.0)))))))) - (((data["q05_roll_std_10"]) * 2.0)))))) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*np.tanh(((((((data["q95_roll_mean_100"]) + (((data["mean_change_rate_first_10000"]) + (((data["abs_q05"]) + (data["abs_q05"]))))))) * (3.6923100948))) + (((((((((data["q05_roll_std_1000"]) + (0.7446810007))) * (((3.6923100948) - (73.0))))) * 2.0)) * 2.0)))) +
            0.0399999991*np.tanh((-1.0*((((((73.0) * ((((((data["q01_roll_mean_1000"]) - (((data["q05"]) * 2.0)))) + (((data["q05_roll_std_1000"]) + (((((data["q05_roll_std_100"]) - (data["q95_roll_mean_1000"]))) * (data["Moving_average_700_mean"]))))))/2.0)))) + (((((data["Moving_average_1500_mean"]) * 2.0)) * 2.0))))))) +
            0.0399999991*np.tanh(((((((((data["q05"]) - (((73.0) * ((((-1.0*((((data["q05"]) * 2.0))))) + (((data["q05_roll_std_1000"]) + (1.9019600153))))))))) + (((((((data["q05"]) + (data["max_to_min"]))) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) +
            0.0399999991*np.tanh(((((((data["av_change_abs_roll_mean_100"]) - (((data["q01_roll_mean_1000"]) * 2.0)))) - (((73.0) * (((((((data["Hann_window_mean"]) * (((data["q95"]) - (data["q95_roll_mean_1000"]))))) / 2.0)) + (((((data["q95"]) * 2.0)) + (data["q01_roll_mean_1000"]))))))))) * 2.0)) +
            0.0399999991*np.tanh(((((((((((data["skew"]) - (((((data["q05_roll_mean_10"]) + (((((data["iqr"]) + (((3.1415927410) * (((1.0) - (((data["q05_roll_mean_10"]) * 2.0)))))))) * 2.0)))) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*np.tanh((((((((6.0)) - (((((((((data["q05_roll_std_100"]) + (((data["q05_roll_std_100"]) * 2.0)))) * 2.0)) + (((((data["ave10"]) * (((((data["q05_roll_std_100"]) * 2.0)) * 2.0)))) + (np.tanh((data["q05_roll_std_100"]))))))) * 2.0)))) * 2.0)) * 2.0)) +
            0.0399999991*np.tanh(((((((((data["q05_roll_std_10"]) + (((((((data["q95_roll_mean_1000"]) + (((((((data["skew"]) + (((((((data["abs_q05"]) - (((data["q05_roll_std_10"]) * 2.0)))) * 2.0)) * 2.0)))) * 2.0)) * 2.0)))) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*np.tanh(((((((((data["max_to_min"]) + (data["q95_roll_std_1000"]))) - (((((73.0) * ((((data["q05_roll_std_1000"]) + (((0.2183910012) + (data["abs_q05"]))))/2.0)))) * (((data["q05_roll_std_1000"]) * (((data["q05_roll_std_10"]) + (data["q01_roll_std_10"]))))))))) * 2.0)) * 2.0)) +
            0.0399999991*np.tanh(((((((((((data["abs_q05"]) + (data["q05_roll_std_100"]))) - (((7.0) * (((data["iqr"]) - (((data["q99_roll_mean_1000"]) + ((-1.0*((((((((data["q05_roll_std_100"]) + (1.0))) * 2.0)) * 2.0))))))))))))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*np.tanh((-1.0*((((7.0) * (((73.0) * (((((7.0) * (((((data["Moving_average_6000_mean"]) * (((((-1.0*((data["Moving_average_6000_mean"])))) + (np.tanh((np.tanh((data["q01_roll_std_10"]))))))/2.0)))) + (data["q95_roll_mean_10"]))))) + (data["q95_roll_mean_10"])))))))))) +
            0.0399999991*np.tanh(((((((((((data["skew"]) + (data["ave10"]))) + (data["q95_roll_std_100"]))) - (((7.0) * (((((data["q05_roll_std_100"]) * (data["q95"]))) * ((((((data["q05_roll_std_100"]) * (7.0))) + (7.0))/2.0)))))))) * 2.0)) * 2.0)) +
            0.0399999991*np.tanh((((((9.64285850524902344)) * (((((data["Moving_average_700_mean"]) - ((((((9.64285850524902344)) * ((((data["q05_roll_std_100"]) + (data["exp_Moving_average_30000_mean"]))/2.0)))) - ((((data["skew"]) + ((3.98402547836303711)))/2.0)))))) * 2.0)))) * (((data["iqr"]) * (data["q05_roll_std_100"]))))) +
            0.0399999991*np.tanh(((((((((((((((((data["Moving_average_1500_mean"]) - (((np.tanh((data["exp_Moving_average_30000_mean"]))) + (np.tanh((((data["abs_q05"]) + ((((data["q95_roll_std_10"]) + (data["q01_roll_std_10"]))/2.0)))))))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) - (data["iqr"]))) * 2.0)) +
            0.0399999991*np.tanh((((-1.0*((data["q95"])))) * (((73.0) + ((-1.0*((((data["abs_q05"]) * ((((((((-1.0*((data["abs_q05"])))) - (data["min_roll_std_10"]))) - (73.0))) - ((((-1.0*((data["q05_roll_mean_10"])))) * (73.0)))))))))))))) +
            0.0399999991*np.tanh(((data["q05_roll_std_10"]) * (((((((((data["q05_roll_std_100"]) * ((11.90774154663085938)))) * (((data["ave10"]) - (-2.0))))) - ((11.90774154663085938)))) * ((-1.0*((((data["q05_roll_std_1000"]) + (((data["q05_roll_std_100"]) - (-2.0)))))))))))) +
            0.0399999991*np.tanh(((((((data["iqr"]) * ((9.63457870483398438)))) * (((data["q95_roll_mean_1000"]) - ((((9.63457870483398438)) * (((data["q05_roll_std_10"]) * ((((data["q01_roll_std_10"]) + (((data["iqr"]) * (data["q95_roll_mean_1000"]))))/2.0)))))))))) - (((-3.0) * (data["q01_roll_std_10"]))))) +
            0.0399999991*np.tanh(((((((((-1.0) - (((np.tanh((data["q05_roll_std_10"]))) + (((((((data["q05_roll_std_100"]) - (np.tanh((((data["classic_sta_lta1_mean"]) + (((((((data["q95_roll_std_100"]) * 2.0)) * 2.0)) * 2.0)))))))) * 2.0)) * 2.0)))))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*np.tanh(((((((((data["q95_roll_mean_10"]) - (((((((data["q95_roll_mean_10"]) + (((((data["q05_roll_std_10"]) + (((3.6923100948) * ((((data["ave10"]) + (((data["q05_roll_std_10"]) + (-1.0))))/2.0)))))) * 2.0)))) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*np.tanh(((((data["q05_roll_std_100"]) + (((((data["q95_roll_mean_1000"]) + (((((data["q05_roll_std_100"]) + ((((3.1415927410) + (data["q05_roll_std_1000"]))/2.0)))) * (((((((data["q05_roll_mean_10"]) * 2.0)) * 2.0)) * (((3.1415927410) * (data["q05_roll_std_100"]))))))))) * 2.0)))) * 2.0)) +
            0.0399999991*np.tanh(((((((((((((data["mean_change_rate_first_10000"]) - (data["Moving_average_6000_mean"]))) * 2.0)) - (data["q01_roll_mean_1000"]))) - (((data["av_change_abs_roll_mean_10"]) + (((data["mean_change_rate_last_50000"]) + (((data["av_change_abs_roll_std_100"]) - ((((((13.93686485290527344)) / 2.0)) * (data["q05"]))))))))))) * 2.0)) * 2.0)) +
            0.0399999991*np.tanh(((((((((data["q05_roll_std_10"]) * (data["q05_roll_std_10"]))) * ((((-1.0*((((2.0) - ((-1.0*((((data["q05_roll_std_100"]) + (((((data["q05_roll_std_100"]) * (data["abs_q05"]))) * 2.0)))))))))))) * 2.0)))) - ((-1.0*((data["kurt"])))))) * 2.0)) +
            0.0399999991*np.tanh(((((((((((((((data["q05"]) - (np.tanh((((data["iqr"]) + (((((data["Moving_average_1500_mean"]) + (((data["q05"]) + (data["Moving_average_3000_mean"]))))) * 2.0)))))))) * 2.0)) + (data["q05"]))) * 2.0)) * 2.0)) - (data["av_change_abs_roll_mean_10"]))) * 2.0)) +
            0.0399999991*np.tanh(((((((((np.tanh((((((((((((((data["Moving_average_6000_mean"]) - (data["q05_roll_std_100"]))) * 2.0)) * 2.0)) - (((1.6428600550) + (data["q01_roll_std_10"]))))) * 2.0)) - (data["av_change_abs_roll_std_100"]))))) * 2.0)) - (data["abs_q05"]))) * 2.0)) * 2.0)) +
            0.0399999991*np.tanh((((((((((((((data["skew"]) + (data["q95_roll_std_10"]))/2.0)) - (((((data["ave_roll_std_100"]) + (((data["max_roll_mean_10"]) * 2.0)))) * (data["abs_q05"]))))) + (((((np.tanh((data["q95_roll_std_100"]))) - (data["q01_roll_std_10"]))) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*np.tanh(((data["skew"]) + (((((data["q01_roll_std_100"]) * 2.0)) + (((data["Moving_average_700_mean"]) + (((data["q05_roll_std_10"]) * (((data["q05"]) * ((((((73.0) * (((data["Moving_average_700_mean"]) + (((data["q05_roll_std_100"]) / 2.0)))))) + (data["q05_roll_std_10"]))/2.0)))))))))))) +
            0.0399999991*np.tanh(((((((data["q05_roll_mean_10"]) * (((data["q05_roll_std_100"]) * (data["q05_roll_std_100"]))))) * (((((data["q05_roll_std_100"]) * 2.0)) * (((data["q05_roll_std_1000"]) * 2.0)))))) - (((((((data["q05_roll_std_1000"]) * 2.0)) * (data["q05_roll_std_100"]))) - (((data["q01_roll_std_1000"]) * 2.0)))))) +
            0.0399999991*np.tanh(((((((-1.0) + (((((((np.tanh((data["q01_roll_std_10"]))) * 2.0)) * 2.0)) + (((data["max_to_min_diff"]) + (((data["q05"]) * ((((14.98138904571533203)) + (((data["mean_change_rate_last_10000"]) * 2.0)))))))))))) * (73.0))) * (73.0))) +
            0.0399999991*np.tanh(((((((data["q05_roll_std_10"]) - (((((data["q05_roll_std_1000"]) + (0.7446810007))) * (((data["q05_roll_std_10"]) * (((((((data["q01_roll_std_10"]) * (((data["ave10"]) + (((data["q95_roll_mean_10"]) * (data["q05_roll_std_10"]))))))) * 2.0)) * 2.0)))))))) * 2.0)) * 2.0)) +
            0.0399999991*np.tanh(((((((np.tanh((data["q01_roll_std_100"]))) - (((((((((data["q05_roll_std_1000"]) + (((((data["q95"]) - (np.tanh((data["q95_roll_std_100"]))))) * 2.0)))) - (np.tanh((np.tanh((data["q95"]))))))) - (data["std_roll_mean_1000"]))) * 2.0)))) * 2.0)) * 2.0)) +
            0.0399999991*np.tanh(((((data["q05_roll_std_10"]) + (((data["ave_roll_mean_1000"]) * 2.0)))) * (((((((data["ave_roll_mean_1000"]) * (data["ave_roll_mean_1000"]))) - (data["q05_roll_std_1000"]))) + (((data["q05_roll_mean_10"]) * (((data["Moving_average_3000_mean"]) + (((((data["q05_roll_std_10"]) * 2.0)) * 2.0)))))))))) +
            0.0399999991*np.tanh(((((((((((data["q05_roll_std_1000"]) + (((data["q05_roll_std_100"]) - (((data["Moving_average_6000_mean"]) * (data["Moving_average_6000_mean"]))))))) * (((data["q95_roll_std_100"]) * ((((-1.0*((data["iqr"])))) - (((data["Moving_average_6000_mean"]) + (data["Moving_average_1500_mean"]))))))))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*np.tanh(((((((((data["sum"]) + (0.9756100178))) * 2.0)) * 2.0)) * (((data["q01_roll_std_100"]) - (((np.tanh((((data["q05_roll_std_10"]) * (data["q05_roll_std_10"]))))) + (((data["q05_roll_std_1000"]) * (((((data["q01_roll_std_100"]) * (data["q01_roll_std_10"]))) * 2.0)))))))))) +
            0.0399999991*np.tanh((((((((((-1.0*((data["q05_roll_std_100"])))) - (((data["q01_roll_std_100"]) * (data["mean"]))))) * (((data["q05_roll_std_100"]) * (data["q05_roll_std_100"]))))) * (((data["q05_roll_std_100"]) * (data["q05_roll_std_100"]))))) + (((((data["av_change_rate_roll_std_10"]) * (data["max_to_min"]))) * 2.0)))) +
            0.0399999991*np.tanh(((((((data["q05_roll_std_10"]) * 2.0)) * 2.0)) * (((((0.5434780121) * 2.0)) - (((((data["q05_roll_std_10"]) * (((((((((data["q95_roll_std_10"]) * 2.0)) * (data["q05_roll_std_10"]))) + (data["q05_roll_std_10"]))) * (data["q05_roll_std_10"]))))) + (data["skew"]))))))) +
            0.0399999991*np.tanh(((((data["abs_q05"]) * (data["q95_roll_std_100"]))) - (((((data["q05_roll_std_100"]) * (((data["q05_roll_std_100"]) * (((data["q05_roll_std_100"]) * (((data["q05_roll_std_100"]) * (((data["abs_q05"]) * (data["q05_roll_std_100"]))))))))))) + ((((data["min_roll_std_100"]) + (data["av_change_abs_roll_mean_100"]))/2.0)))))) +
            0.0399999991*np.tanh(((((data["Moving_average_6000_mean"]) + (0.8873239756))) * (((((data["q05_roll_std_100"]) + (data["q95_roll_std_1000"]))) * (((1.6428600550) - (((data["q05_roll_std_1000"]) * (((((data["q05_roll_std_100"]) + (((data["exp_Moving_average_3000_mean"]) + (0.9756100178))))) + (data["q05_roll_std_100"]))))))))))) +
            0.0399999991*np.tanh(((((data["q95_roll_mean_10"]) + (((data["q95_roll_std_1000"]) * (((data["med"]) + (((((data["q05_roll_mean_100"]) + (((data["q95_roll_mean_10"]) * (data["q95_roll_mean_10"]))))) * (data["q05_roll_std_100"]))))))))) * (((((data["exp_Moving_average_30000_mean"]) - (data["q05_roll_std_100"]))) - (data["q05_roll_std_1000"]))))) +
            0.0399999991*np.tanh(((((((data["min"]) + ((((-1.0*((data["MA_400MA_BB_high_mean"])))) + (np.tanh((((((0.3183098733) + (((((data["kurt"]) * 2.0)) * 2.0)))) * ((((3.0)) + ((-1.0*((data["kurt"])))))))))))))) * 2.0)) * 2.0)) +
            0.0399999991*np.tanh(((((((data["min_roll_mean_1000"]) * 2.0)) * 2.0)) + ((((-1.0*((((((data["q05_roll_std_1000"]) + (((data["q01_roll_std_10"]) * (data["q95"]))))) + (((data["avg_last_50000"]) * (((data["q01_roll_std_10"]) - (data["avg_last_50000"])))))))))) * (data["q01_roll_std_100"]))))) +
            0.0399921872*np.tanh(((((data["q95_roll_std_1000"]) - ((((((((data["av_change_abs_roll_mean_10"]) + (data["std_first_10000"]))/2.0)) + (((data["q95"]) - (data["min_last_10000"]))))) - (((data["min"]) + (((data["q95_roll_std_1000"]) + (data["mean_change_rate_last_10000"]))))))))) - (((data["q95"]) + (data["q95"]))))) +
            0.0399999991*np.tanh(((((((data["min_roll_mean_1000"]) - (((data["q05_roll_std_1000"]) * (((np.tanh((((data["med"]) * 2.0)))) + ((((((data["skew"]) * (data["kurt"]))) + (data["q05_roll_std_1000"]))/2.0)))))))) + (((data["kurt"]) * (data["skew"]))))) + (data["min_roll_mean_1000"]))) +
            0.0399921872*np.tanh(((data["exp_Moving_average_30000_mean"]) * (((((data["max_roll_std_100"]) - (((((data["exp_Moving_average_30000_mean"]) * (data["MA_400MA_std_mean"]))) + (data["q05_roll_mean_10"]))))) * (((((data["mean_change_rate_first_10000"]) * (73.0))) - (((((data["abs_mean"]) * (73.0))) * (73.0))))))))) +
            0.0399999991*np.tanh(((data["q01_roll_std_10"]) + ((-1.0*((((data["q05_roll_std_1000"]) * (((data["av_change_rate_roll_mean_100"]) + ((((-1.0*((data["classic_sta_lta4_mean"])))) + (((data["med"]) + (((data["q05_roll_std_1000"]) + (((data["iqr"]) * (((data["iqr"]) + (data["max_last_10000"])))))))))))))))))))) +
            0.0399921872*np.tanh((((((((data["q05"]) + (data["mean_change_rate_first_10000"]))/2.0)) + (((data["kurt"]) - (((((data["kurt"]) * (data["abs_max_roll_std_1000"]))) * (data["kurt"]))))))) + (((((data["av_change_abs_roll_std_100"]) * ((-1.0*((data["classic_sta_lta3_mean"])))))) * 2.0)))) +
            0.0399999991*np.tanh(((((((((((((((data["skew"]) - ((((data["min_roll_std_1000"]) + (data["q95_roll_mean_100"]))/2.0)))) + (((data["q95_roll_mean_100"]) * (data["q05"]))))) * (data["MA_400MA_std_mean"]))) * (data["MA_400MA_BB_high_mean"]))) * 2.0)) * 2.0)) + ((((data["mean_change_abs"]) + (data["min_roll_std_1000"]))/2.0)))) +
            0.0399921872*np.tanh((((((4.12892913818359375)) * ((((((data["exp_Moving_average_300_mean"]) + (data["med"]))/2.0)) - (data["q01_roll_std_100"]))))) * (((data["skew"]) + (((0.2439019978) + ((((data["MA_1000MA_std_mean"]) + ((((((data["av_change_abs_roll_std_1000"]) + (data["med"]))) + (data["iqr"]))/2.0)))/2.0)))))))) +
            0.0399999991*np.tanh(((((((((data["av_change_abs_roll_std_10"]) * (((((data["mean_change_rate_last_50000"]) + (data["av_change_abs_roll_mean_10"]))) + (data["min_last_50000"]))))) * 2.0)) * 2.0)) + ((((((((1.0)) + (((data["av_change_rate_roll_std_10"]) + (data["skew"]))))/2.0)) + (data["q01_roll_mean_1000"]))/2.0)))) +
            0.0399999991*np.tanh(((((((((data["q05_roll_std_10"]) * (((((((data["med"]) + (data["q05_roll_std_10"]))) * (data["q05_roll_mean_10"]))) * (data["q95_roll_std_1000"]))))) * (data["q05_roll_std_10"]))) + (((((data["q05_roll_mean_10"]) + (data["q95_roll_std_100"]))) + (data["q95_roll_std_100"]))))) * 2.0)) +
            0.0399921872*np.tanh((((data["skew"]) + (((((((((data["min_roll_std_10"]) * ((-1.0*((data["min_roll_std_10"])))))) + (data["q05_roll_mean_1000"]))) * (data["Hilbert_mean"]))) - ((((data["av_change_rate_roll_std_100"]) + (((data["max_roll_std_100"]) * (((data["skew"]) * (data["skew"]))))))/2.0)))))/2.0)) +
            0.0399999991*np.tanh(((data["q05_roll_std_10"]) * (((((((((data["q01_roll_mean_1000"]) - (np.tanh((((data["Moving_average_6000_mean"]) - (((data["ave_roll_std_10"]) * (((data["kurt"]) - ((((data["av_change_abs_roll_mean_10"]) + ((((data["Moving_average_6000_mean"]) + (data["ave_roll_std_10"]))/2.0)))/2.0)))))))))))) * 2.0)) * 2.0)) * 2.0)))) +
            0.0399843715*np.tanh(((data["min"]) + ((-1.0*((((data["av_change_rate_roll_std_100"]) * ((((((((data["mean_change_rate"]) + (data["min"]))/2.0)) + (data["min_roll_std_1000"]))) + (((np.tanh((data["min_last_10000"]))) + ((((((data["min_roll_std_100"]) * (data["av_change_rate_roll_mean_10"]))) + (data["av_change_abs_roll_std_10"]))/2.0))))))))))))) +
            0.0399999991*np.tanh(((data["min_roll_mean_100"]) * (((((data["min_roll_std_1000"]) * (((data["q01_roll_std_1000"]) * (((((((data["q01_roll_std_1000"]) * (data["q99_roll_std_100"]))) + (data["exp_Moving_average_300_mean"]))) + (data["avg_first_10000"]))))))) + (((((data["mean_change_rate"]) * (3.1415927410))) - (data["avg_first_10000"]))))))) +
            0.0399921872*np.tanh(((((data["q95_roll_mean_1000"]) * (((((((((data["q95_roll_mean_1000"]) + (((data["avg_last_10000"]) + (((3.0) / 2.0)))))/2.0)) + (data["q05_roll_mean_100"]))/2.0)) - (data["q95_roll_std_1000"]))))) * (((((((data["q95_roll_std_1000"]) * 2.0)) + (data["q99_roll_mean_10"]))) + (data["classic_sta_lta4_mean"]))))) +
            0.0399296731*np.tanh((((((((((((data["q01_roll_std_100"]) + (np.tanh((((data["q01"]) + ((-1.0*((((data["q01_roll_std_100"]) * 2.0))))))))))/2.0)) * ((-1.0*(((((data["q95"]) + (((data["exp_Moving_average_30000_mean"]) * 2.0)))/2.0))))))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399921872*np.tanh(((((np.tanh((data["q99"]))) + ((-1.0*(((((data["max_first_10000"]) + (((data["av_change_rate_roll_mean_1000"]) - (data["q99"]))))/2.0))))))) - ((((((((data["av_change_rate_roll_mean_1000"]) * (data["av_change_rate_roll_mean_1000"]))) + (data["q05_roll_std_10"]))/2.0)) * (((data["q05_roll_std_10"]) * (data["iqr"]))))))) +
            0.0399999991*np.tanh(((data["min_roll_mean_1000"]) * (((((((((((data["std_roll_std_100"]) + (((data["q05"]) + (((((data["q95_roll_mean_1000"]) * (data["q95_roll_std_1000"]))) + (((data["q95_roll_mean_1000"]) * (((data["q95_roll_mean_1000"]) * (data["q95_roll_std_1000"]))))))))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) +
            0.0399921872*np.tanh(((((data["mean_change_abs"]) * (data["min_roll_std_10"]))) + (((((((data["std_last_50000"]) * (3.6923100948))) * (((data["q95_roll_mean_1000"]) + (((data["classic_sta_lta4_mean"]) - (data["min_roll_std_10"]))))))) * ((((((data["min_roll_std_10"]) + (data["q05_roll_mean_100"]))/2.0)) - (data["max_roll_std_10"]))))))) +
            0.0399999991*np.tanh(((((data["q01_roll_std_10"]) + (data["min_roll_std_100"]))) * (((data["abs_q05"]) - (((data["avg_last_50000"]) - (((((data["classic_sta_lta2_mean"]) / 2.0)) + (((data["max_roll_mean_1000"]) * (((((data["avg_last_50000"]) - (np.tanh((data["min_roll_std_100"]))))) - (data["min_roll_std_100"]))))))))))))) +
            0.0399999991*np.tanh(((((data["q95_roll_mean_100"]) * (data["min_roll_mean_1000"]))) - (((((data["av_change_abs_roll_std_100"]) * (((((data["classic_sta_lta3_mean"]) + (data["mean_change_rate_first_50000"]))) + (((data["q95_roll_mean_100"]) * (data["classic_sta_lta3_mean"]))))))) + (((data["mean_change_abs"]) * (((data["classic_sta_lta3_mean"]) - (data["mean_change_rate_first_50000"]))))))))) +
            0.0398359038*np.tanh(((data["q95_roll_mean_10"]) * (((((data["q95_roll_mean_10"]) * (((data["min_roll_mean_1000"]) + (data["av_change_rate_roll_std_10"]))))) - (((data["max_to_min"]) + (((((((data["max_to_min"]) * 2.0)) * 2.0)) * (((data["av_change_rate_roll_std_10"]) + (data["q01_roll_mean_1000"]))))))))))) +
            0.0399999991*np.tanh(((data["q01_roll_std_10"]) * (((((((data["q01_roll_std_10"]) * (data["skew"]))) + (((data["classic_sta_lta4_mean"]) - (data["classic_sta_lta2_mean"]))))) + ((((data["av_change_abs_roll_mean_1000"]) + (((((-1.0*((data["q01_roll_std_100"])))) + (((data["q01_roll_std_10"]) * ((-1.0*((data["q01_roll_std_10"])))))))/2.0)))/2.0)))))) +
            0.0385153331*np.tanh((((((data["std_roll_mean_1000"]) + (data["av_change_abs_roll_mean_100"]))) + (((data["max_first_10000"]) * (((data["MA_700MA_std_mean"]) * (((73.0) * (((np.tanh((data["max_to_min"]))) - (((data["av_change_rate_roll_mean_1000"]) + (((((data["std_roll_mean_1000"]) * 2.0)) * 2.0)))))))))))))/2.0)) +
            0.0399999991*np.tanh(((((data["std_roll_mean_1000"]) * (((data["av_change_rate_roll_std_1000"]) - (((((((data["skew"]) - (data["min_roll_std_10"]))) - (data["classic_sta_lta4_mean"]))) + (((data["std_roll_mean_1000"]) - (2.0))))))))) + (((data["classic_sta_lta4_mean"]) * (((data["av_change_abs_roll_mean_1000"]) - (data["q99"]))))))) +
            0.0399843715*np.tanh((-1.0*((((data["kurt"]) * (((data["kurt"]) * ((((((data["av_change_abs_roll_mean_10"]) + (((((data["med"]) - (((data["med"]) * (data["av_change_abs_roll_mean_10"]))))) - (data["min_roll_std_10"]))))/2.0)) + (((data["q99_roll_std_1000"]) * (data["kurt"])))))))))))) +
            0.0399921872*np.tanh(((data["mean_change_rate_first_10000"]) * (((data["av_change_abs_roll_mean_100"]) * ((-1.0*((((((np.tanh((((((data["classic_sta_lta3_mean"]) + (data["mean_change_rate_first_10000"]))) * 2.0)))) + (((1.9019600153) - (data["av_change_abs_roll_mean_100"]))))) - (((((data["classic_sta_lta3_mean"]) * 2.0)) * (data["mean_change_rate_first_10000"])))))))))))) +
            0.0399921872*np.tanh(((((data["av_change_rate_roll_std_100"]) * (((data["classic_sta_lta3_mean"]) * (((((data["min_roll_std_1000"]) - (data["min"]))) * (data["q05"]))))))) + (((data["mean_change_rate_last_10000"]) * (((((data["av_change_abs_roll_std_1000"]) - (data["min"]))) - (((data["classic_sta_lta3_mean"]) * (data["q01_roll_std_1000"]))))))))) +
            0.0399999991*np.tanh(((data["q95_roll_std_1000"]) + (((data["q05_roll_mean_10"]) - ((-1.0*((((data["q95_roll_std_1000"]) * (((data["q95_roll_std_1000"]) * (((((data["q01_roll_std_10"]) * ((((((((data["q95_roll_std_1000"]) / 2.0)) + (data["q01_roll_mean_1000"]))/2.0)) - (data["med"]))))) * (data["q95_roll_std_1000"])))))))))))))) +
            0.0399843715*np.tanh(((data["av_change_abs_roll_mean_100"]) * (((data["classic_sta_lta4_mean"]) * (((((((((3.65123486518859863)) + ((((3.65123486518859863)) * (data["abs_max_roll_std_10"]))))/2.0)) + (((((data["min_last_50000"]) * (((data["abs_max_roll_std_1000"]) - (data["av_change_abs_roll_mean_10"]))))) - (data["classic_sta_lta4_mean"]))))) * (data["ave_roll_mean_10"]))))))) +
            0.0399999991*np.tanh(((((data["max_to_min_diff"]) * (((data["min_roll_std_100"]) * (((data["av_change_rate_roll_std_100"]) + (((data["q01_roll_std_100"]) + (((((((data["av_change_rate_roll_mean_100"]) + (data["Hilbert_mean"]))) * (data["q95_roll_mean_1000"]))) + (((data["q01"]) * (data["min_roll_std_100"]))))))))))))) * 2.0)) +
            0.0399999991*np.tanh(((((np.tanh((((((data["q05"]) + (((np.tanh((data["avg_first_10000"]))) + (data["mean_change_rate_last_10000"]))))) + (data["q05"]))))) + (((((data["kurt"]) * (data["mean_change_rate_last_10000"]))) + (data["q95_roll_std_1000"]))))) * (((data["q01_roll_mean_1000"]) + (data["av_change_rate_roll_mean_100"]))))) +
            0.0399921872*np.tanh(((data["min_roll_mean_1000"]) + ((((((((((((data["av_change_rate_roll_std_10"]) + (((data["av_change_abs_roll_std_1000"]) * (data["min_roll_mean_1000"]))))/2.0)) * (data["classic_sta_lta3_mean"]))) + (data["av_change_rate_roll_std_10"]))/2.0)) + (((data["abs_max_roll_mean_100"]) - (((data["av_change_abs_roll_std_1000"]) * (data["classic_sta_lta1_mean"]))))))/2.0)))) +
            0.0399999991*np.tanh(((data["max_to_min"]) * (((((data["MA_400MA_std_mean"]) - (data["q95_roll_std_100"]))) + ((((((data["min_roll_std_10"]) * (((((((data["min_roll_std_10"]) * (data["classic_sta_lta1_mean"]))) - (data["min_roll_std_10"]))) - (data["kurt"]))))) + ((((data["kurt"]) + (data["min_roll_std_10"]))/2.0)))/2.0)))))) +
            0.0399921872*np.tanh(((data["min_roll_std_10"]) * ((((((((((((np.tanh((data["min_roll_std_10"]))) + (data["avg_last_10000"]))/2.0)) - (data["q95_roll_std_100"]))) - (data["MA_700MA_BB_high_mean"]))) + (((data["abs_max_roll_mean_100"]) - (data["classic_sta_lta4_mean"]))))) * (((data["min_roll_std_10"]) - (((3.0) / 2.0)))))))) +
            0.0399765596*np.tanh(((((data["avg_last_10000"]) * (((data["min_roll_mean_1000"]) + (((((data["avg_last_10000"]) + (np.tanh((data["mean_change_rate_last_50000"]))))) * (((data["min_roll_std_10"]) * (data["min_last_10000"]))))))))) + (np.tanh((np.tanh((((((data["mean_change_rate_last_50000"]) * (data["min_roll_std_1000"]))) * 2.0)))))))) +
            0.0399921872*np.tanh(np.tanh((np.tanh((np.tanh((((((((data["q95_roll_std_10"]) - (data["q05"]))) * (((data["abs_q05"]) * (((data["mean_change_rate_first_10000"]) + (((((data["mean_change_rate_first_10000"]) + (data["q05"]))) * (((data["abs_q05"]) * (data["q95_roll_mean_1000"]))))))))))) * 2.0)))))))) +
            0.0399843715*np.tanh(((data["q95_roll_std_1000"]) * (((data["max_roll_mean_100"]) + (((((((((data["q01_roll_mean_1000"]) * 2.0)) * 2.0)) * (data["min_roll_mean_1000"]))) * (((((((data["q01_roll_mean_1000"]) + ((-1.0*((data["max_to_min"])))))) * 2.0)) * 2.0)))))))) +
            0.0399765596*np.tanh(((data["q05_roll_std_100"]) * ((-1.0*((np.tanh((((((data["med"]) + (((((data["med"]) + (((data["q01_roll_std_10"]) * (((data["avg_first_50000"]) + (data["q01_roll_std_10"]))))))) * ((((1.16074943542480469)) + (data["avg_first_50000"]))))))) * 2.0))))))))) +
            0.0399062298*np.tanh((-1.0*((((data["classic_sta_lta2_mean"]) * ((((((np.tanh((data["kurt"]))) + (data["avg_last_50000"]))/2.0)) + (((data["q05_roll_mean_100"]) * ((-1.0*(((((((((data["kurt"]) + (data["q05_roll_mean_100"]))/2.0)) - (np.tanh((data["kurt"]))))) + (data["std_last_10000"]))))))))))))))) +
            0.0399453007*np.tanh(((data["std_last_50000"]) * (((((np.tanh((((((((((((data["av_change_abs_roll_mean_100"]) + (np.tanh((np.tanh((((((((((data["skew"]) + (data["Moving_average_6000_mean"]))) * 2.0)) * 2.0)) * 2.0)))))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) * 2.0)))) +
            0.0390935726*np.tanh((-1.0*((((data["max_roll_mean_100"]) * (((((((np.tanh((((((data["classic_sta_lta3_mean"]) + ((((data["q99_roll_mean_1000"]) + (data["mean_change_rate_first_10000"]))/2.0)))) + (((data["av_change_abs_roll_mean_10"]) + (((data["classic_sta_lta3_mean"]) * (data["q95_roll_mean_1000"]))))))))) * 2.0)) * 2.0)) * 2.0))))))) +
            0.0399296731*np.tanh(((data["q95_roll_std_100"]) + ((-1.0*((((data["q95_roll_std_10"]) + ((((((((data["kurt"]) * ((((data["mean_change_rate_last_10000"]) + (data["trend"]))/2.0)))) * (data["kurt"]))) + ((((((data["med"]) + (data["max_first_50000"]))/2.0)) * (data["min_roll_std_100"]))))/2.0))))))))) +
            0.0398984179*np.tanh(((((np.tanh((((data["q01"]) - (data["q05"]))))) + (((((((((np.tanh((data["q05_roll_std_100"]))) + (data["q01"]))) * 2.0)) + (data["q95_roll_mean_1000"]))) * 2.0)))) * ((-1.0*((((data["q01"]) - (data["q05"])))))))) +
            0.0399999991*np.tanh(np.tanh((np.tanh(((-1.0*((((data["mean_change_rate_first_10000"]) * (((data["min_last_10000"]) + (((data["min_last_10000"]) + (((((data["abs_q95"]) + (((data["avg_last_50000"]) + ((((data["min_last_10000"]) + (0.8873239756))/2.0)))))) * ((-1.0*((data["q05_roll_std_100"]))))))))))))))))))) +
            0.0399999991*np.tanh(((data["min_first_50000"]) * ((((-1.0*((data["av_change_abs_roll_std_100"])))) - (((data["ave_roll_mean_10"]) * ((-1.0*((((data["max_to_min"]) - (((((((data["MA_400MA_BB_low_mean"]) - (((data["av_change_rate_roll_mean_100"]) + (((data["q01_roll_mean_1000"]) * (data["av_change_rate_roll_mean_100"]))))))) * 2.0)) * 2.0))))))))))))) +
            0.0399999991*np.tanh(((((((((data["max_to_min_diff"]) * (((data["av_change_rate_roll_std_100"]) + (((((data["q95_roll_std_10"]) + (data["max_first_50000"]))) + (((data["mean_change_abs"]) + (((((data["MA_400MA_BB_low_mean"]) * (data["max_first_50000"]))) * (data["max_first_50000"]))))))))))) * 2.0)) * 2.0)) * (data["min_roll_std_100"]))) +
            0.0399999991*np.tanh(((((data["max"]) * (((data["std_first_50000"]) * ((((((-1.0*((data["max_roll_std_100"])))) + (((((((((data["q001"]) + (((data["q001"]) * (data["classic_sta_lta4_mean"]))))) + (data["classic_sta_lta4_mean"]))) * 2.0)) * 2.0)))) * 2.0)))))) * (data["q05_roll_std_1000"]))) +
            0.0399765596*np.tanh(((((data["abs_max_roll_std_1000"]) * 2.0)) * (((data["q95_roll_std_10"]) - (((data["q999"]) - (np.tanh((((data["max_last_10000"]) * (((((((((((((data["abs_max_roll_std_1000"]) + (data["Moving_average_6000_mean"]))) * 2.0)) + (data["mean_change_abs"]))) * 2.0)) * 2.0)) * 2.0)))))))))))) +
            0.0399687439*np.tanh(((data["min_roll_std_10"]) * ((((((data["avg_last_50000"]) + (((((((data["q01_roll_std_1000"]) * (data["classic_sta_lta4_mean"]))) * (data["mean_change_rate_last_50000"]))) - (((data["min_roll_std_10"]) * (data["av_change_abs_roll_mean_10"]))))))/2.0)) + ((((((data["classic_sta_lta4_mean"]) * (data["Moving_average_6000_mean"]))) + (data["mean_change_rate_last_50000"]))/2.0)))))) +
            0.0398906022*np.tanh((((np.tanh((((data["av_change_abs_roll_mean_1000"]) - (((data["Hann_window_mean"]) - (data["skew"]))))))) + (((((((data["min_roll_mean_1000"]) * 2.0)) * 2.0)) * (((data["skew"]) + (((((data["std_roll_mean_1000"]) - (data["q99_roll_mean_10"]))) - (data["med"]))))))))/2.0)) +
            0.0399999991*np.tanh(((data["std_first_10000"]) * ((-1.0*((((data["min_roll_std_10"]) * (((((((((data["kurt"]) + (((((data["av_change_rate_roll_std_1000"]) - (data["trend"]))) + (data["mean_change_rate_first_10000"]))))) - (data["max_last_10000"]))) + (((data["trend"]) * (data["trend"]))))) * 2.0))))))))) +
            0.0399765596*np.tanh(((data["q99"]) * ((-1.0*((((data["iqr"]) - (((((((1.9019600153) + (((data["abs_max_roll_mean_100"]) + (data["q99"]))))) * (((((data["std_roll_mean_1000"]) * 2.0)) * (((data["max_roll_mean_1000"]) - (data["std_roll_mean_1000"]))))))) + (data["min_roll_std_1000"])))))))))) +
            0.0399765596*np.tanh(((((data["mean_change_rate_last_10000"]) + (np.tanh((((((np.tanh((data["skew"]))) + (((data["max_roll_std_1000"]) * 2.0)))) * 2.0)))))) * (((data["abs_max_roll_std_10"]) + ((-1.0*((((data["skew"]) * (((((data["abs_max"]) / 2.0)) - (data["ave_roll_std_1000"])))))))))))) +
            0.0355928876*np.tanh((((((((data["min_roll_std_100"]) * (data["q05_roll_mean_1000"]))) * (data["min_roll_std_1000"]))) + (((((data["min_roll_std_100"]) * (data["min_roll_std_100"]))) - (((((data["max_to_min"]) + ((((((data["q999"]) + (data["min_roll_std_100"]))/2.0)) + (data["q999"]))))) * (data["classic_sta_lta3_mean"]))))))/2.0)) +
            0.0399843715*np.tanh((((((-1.0*((((((((data["q01_roll_std_10"]) * (data["q05_roll_std_1000"]))) * (data["max_first_50000"]))) * (data["q01_roll_std_10"])))))) * (((data["max_first_50000"]) + (data["max_to_min_diff"]))))) * (((((data["min"]) - (data["q05"]))) - (data["q05"]))))) +
            0.0399765596*np.tanh(((data["med"]) * (((np.tanh((((((((((data["av_change_abs_roll_std_100"]) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) - (((((data["min_roll_std_10"]) + ((((data["q01_roll_std_1000"]) + ((((data["max_to_min_diff"]) + (data["min_roll_std_1000"]))/2.0)))/2.0)))) * (((data["av_change_abs_roll_std_100"]) * 2.0)))))))) +
            0.0387106836*np.tanh((-1.0*((((((data["av_change_rate_roll_mean_10"]) * (((data["q95_roll_std_1000"]) + (((((((((((data["min_roll_mean_1000"]) * 2.0)) * 2.0)) * (((data["q95_roll_std_1000"]) + (data["min_roll_mean_1000"]))))) * 2.0)) * (((data["q95_roll_std_100"]) - (((data["min_roll_mean_1000"]) * 2.0)))))))))) * 2.0))))) +
            0.0399999991*np.tanh(((data["max_last_10000"]) * ((((((((data["MA_700MA_BB_low_mean"]) * (((((data["avg_first_10000"]) - (data["av_change_abs_roll_mean_1000"]))) - (((data["avg_first_10000"]) * (((data["avg_first_10000"]) - (((data["min_roll_std_1000"]) * 2.0)))))))))) * 2.0)) + (((data["avg_first_10000"]) - (data["mean_change_rate_last_10000"]))))/2.0)))) +
            0.0399999991*np.tanh(np.tanh((((((((((data["av_change_rate_roll_std_100"]) * (((data["min"]) * (((data["q05_roll_std_100"]) + (((((data["av_change_abs_roll_std_100"]) + (((data["mean_change_rate_first_10000"]) * (data["av_change_rate_roll_std_100"]))))) + (((data["med"]) - (data["mean_change_rate_first_10000"]))))))))))) * 2.0)) * 2.0)) * 2.0)))) +
            0.0359679610*np.tanh((-1.0*((((np.tanh((((((((((((((((((data["min_roll_std_1000"]) - (data["av_change_rate_roll_std_1000"]))) - (((-2.0) + ((-1.0*((data["min_roll_std_1000"])))))))) - (np.tanh((data["av_change_abs_roll_std_100"]))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) / 2.0))))) +
            0.0373354182*np.tanh((-1.0*((((data["max_to_min"]) * (((data["avg_first_50000"]) * (((((data["ave_roll_std_100"]) - (((data["classic_sta_lta1_mean"]) + (data["mean_change_rate_first_10000"]))))) * (((data["classic_sta_lta2_mean"]) - ((-1.0*((((data["ave_roll_std_100"]) / 2.0)))))))))))))))) +
            0.0398671627*np.tanh(((((0.8873239756) + (data["avg_last_50000"]))) * (((data["iqr"]) * (((((0.8873239756) + (data["Moving_average_6000_mean"]))) + (((data["q05_roll_mean_10"]) * (((((((data["ave10"]) + (data["q01_roll_std_10"]))) * (data["q01_roll_std_10"]))) * (data["iqr"]))))))))))) +
            0.0399453007*np.tanh((((np.tanh((((((((((((data["mean_change_rate_last_10000"]) + (np.tanh(((((((-1.0*((data["mean_change_rate_last_50000"])))) * 2.0)) * (data["q95_roll_mean_1000"]))))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) + (np.tanh((np.tanh(((((-1.0*((data["mean_change_rate_last_50000"])))) * 2.0)))))))/2.0)) +
            0.0399687439*np.tanh(((((((data["max_roll_std_1000"]) + (((data["max_roll_std_1000"]) + (data["trend"]))))) + (data["count_big"]))) * (((data["max_to_min"]) * (((((((-1.0*((data["max_to_min"])))) + (data["max_roll_mean_1000"]))/2.0)) - (data["q99_roll_std_10"]))))))) +
            0.0399843715*np.tanh(((data["classic_sta_lta2_mean"]) * (((data["abs_mean"]) * (((data["classic_sta_lta1_mean"]) * (((((((data["mean_change_rate_first_50000"]) + ((-1.0*((data["mean_change_abs"])))))) + (data["classic_sta_lta4_mean"]))) + (((((((-2.0) + (data["av_change_rate_roll_std_10"]))/2.0)) + (data["av_change_abs_roll_std_1000"]))/2.0)))))))))) +
            0.0399687439*np.tanh(((data["min_roll_std_100"]) * (((((((data["mean_change_abs"]) + (((((((data["av_change_abs_roll_std_100"]) * (data["min_roll_std_100"]))) + (data["min_roll_std_1000"]))) * (((data["std_roll_mean_1000"]) * (((data["min_last_10000"]) + (data["av_change_abs_roll_std_100"]))))))))) * 2.0)) * (data["av_change_abs_roll_std_100"]))))) +
            0.0399374850*np.tanh(((((data["min_first_10000"]) - (((data["mean_change_rate_first_10000"]) * (data["kurt"]))))) * (((((((data["abs_trend"]) * (data["kurt"]))) - (data["mean_change_abs"]))) + (((data["std_roll_mean_1000"]) + (((data["std_roll_mean_1000"]) + (((data["min_first_10000"]) + (data["abs_std"]))))))))))) +
            0.0399921872*np.tanh(((data["q95_roll_std_10"]) * ((-1.0*((((((-1.0*((data["abs_max_roll_mean_100"])))) + ((((-1.0*((((data["classic_sta_lta4_mean"]) / 2.0))))) - (((data["q001"]) * (((data["q95_roll_mean_1000"]) - ((-1.0*((((((((data["classic_sta_lta4_mean"]) / 2.0)) / 2.0)) / 2.0))))))))))))/2.0))))))) +
            0.0399843715*np.tanh(((((data["mean_change_rate_last_10000"]) * (((data["abs_max"]) * (((((((((((data["min_roll_std_1000"]) + (data["max_to_min"]))) - (data["count_big"]))) * 2.0)) - (data["max_to_min"]))) + (((data["av_change_abs_roll_std_1000"]) + (data["mean_change_abs"]))))))))) + (data["count_big"]))) +
            0.0383434258*np.tanh(((((data["med"]) * ((-1.0*((((((((data["trend"]) * (((np.tanh((data["av_change_abs_roll_std_100"]))) - (data["mean_change_abs"]))))) + (data["mean_change_abs"]))) * 2.0))))))) * (((((data["av_change_abs_roll_std_100"]) + (np.tanh((data["min_last_10000"]))))) + (data["av_change_abs_roll_std_100"]))))) +
            0.0399999991*np.tanh(np.tanh((((((((data["min_roll_mean_100"]) * 2.0)) * 2.0)) * (((((((data["max_last_10000"]) - (0.5434780121))) * (data["av_change_rate_roll_mean_100"]))) + (((data["av_change_rate_roll_std_10"]) - (((data["min_roll_std_10"]) * (data["av_change_rate_roll_std_10"]))))))))))) +
            0.0399921872*np.tanh((-1.0*((((((((((data["min_roll_std_100"]) + (data["av_change_abs_roll_std_10"]))/2.0)) * (((data["av_change_abs_roll_mean_1000"]) + (data["av_change_abs_roll_std_10"]))))) + (((data["av_change_abs_roll_std_100"]) * (((data["classic_sta_lta4_mean"]) * (((data["av_change_abs_roll_std_10"]) + (((data["min_roll_std_1000"]) * (data["av_change_abs_roll_std_100"]))))))))))/2.0))))) +
            0.0293416679*np.tanh(np.tanh((((data["abs_q95"]) * (((data["q01_roll_std_10"]) * (((data["q01_roll_std_10"]) * (((((data["max_last_50000"]) - ((((((data["av_change_abs_roll_mean_10"]) - (data["classic_sta_lta4_mean"]))) + (data["av_change_abs_roll_mean_100"]))/2.0)))) - ((-1.0*((((data["av_change_abs_roll_mean_10"]) * (data["classic_sta_lta4_mean"])))))))))))))))) +
            0.0399765596*np.tanh(((((data["trend"]) * (data["max_first_10000"]))) * (((data["med"]) + (((data["av_change_rate_roll_mean_1000"]) + (((((2.0769200325) - (((np.tanh((((((((((data["classic_sta_lta1_mean"]) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) * 2.0)))) / 2.0)))))))) +
            0.0399921872*np.tanh(((((data["mean_change_rate_last_50000"]) * ((-1.0*((((data["min_first_10000"]) - (data["std_first_10000"])))))))) * ((((((((((-1.0*((data["std_first_10000"])))) - (data["min_last_50000"]))) - (data["abs_max_roll_std_10"]))) - (((data["mean_change_rate_last_50000"]) * (data["std_first_10000"]))))) - (data["classic_sta_lta2_mean"]))))) +
            0.0399921872*np.tanh(((data["av_change_abs_roll_mean_10"]) * (((data["abs_trend"]) + ((((((((data["mean_change_abs"]) * (data["min_last_10000"]))) + (((data["mean_change_rate_first_10000"]) * (data["max_first_10000"]))))) + ((((data["av_change_abs_roll_mean_10"]) + (((((data["med"]) - (data["q95_roll_mean_1000"]))) - (data["mean_change_rate_first_10000"]))))/2.0)))/2.0)))))) +
            0.0399531126*np.tanh(((data["ave_roll_std_100"]) * (((data["min_roll_std_100"]) * (((((data["max_roll_mean_100"]) + ((((data["max_to_min_diff"]) + (data["q99"]))/2.0)))) * (((((((data["max_to_min_diff"]) * ((((-1.0*((data["std_first_50000"])))) * (data["q95_roll_std_100"]))))) - (data["skew"]))) * 2.0)))))))) +
            0.0399843715*np.tanh(((data["std_roll_std_100"]) * (((data["min_roll_std_1000"]) * (((data["max_roll_mean_10"]) - (((data["kurt"]) * ((((((data["avg_last_10000"]) * (data["avg_last_10000"]))) + ((((data["q99_roll_mean_100"]) + ((((((data["avg_last_10000"]) * (data["avg_last_10000"]))) + (data["avg_first_10000"]))/2.0)))/2.0)))/2.0)))))))))) +
            0.0353584699*np.tanh(((data["med"]) * (((np.tanh((((np.tanh(((((((-1.0*((data["max_to_min"])))) - (data["trend"]))) * 2.0)))) - (((data["min_roll_std_100"]) * (data["med"]))))))) - (np.tanh((np.tanh((data["min_roll_std_100"]))))))))) +
            0.0399765596*np.tanh((((((((data["ave_roll_mean_10"]) + (data["min_roll_std_100"]))/2.0)) * (((((data["classic_sta_lta4_mean"]) + (data["kurt"]))) + (((((data["q95_roll_mean_10"]) + (((data["q95"]) * (data["q95"]))))) * (data["sum"]))))))) * ((((data["q05_roll_std_100"]) + (data["ave_roll_mean_10"]))/2.0)))) +
            0.0399843715*np.tanh(((data["min_roll_std_10"]) * ((((((-1.0*((((data["av_change_abs_roll_std_10"]) * (data["av_change_abs_roll_mean_100"])))))) * (data["av_change_abs_roll_std_10"]))) + (np.tanh(((((-1.0*((data["q05"])))) + (((data["min_roll_std_1000"]) * ((-1.0*((((data["av_change_abs_roll_mean_100"]) * (data["av_change_abs_roll_mean_100"])))))))))))))))) +
            0.0399999991*np.tanh((-1.0*((((data["classic_sta_lta1_mean"]) * (((data["max_last_50000"]) + (((((data["abs_q95"]) + (((data["av_change_abs_roll_std_10"]) * 2.0)))) * (((((data["abs_std"]) + (data["max_last_50000"]))) * (((((data["iqr"]) * 2.0)) * (data["avg_last_10000"])))))))))))))) +
            0.0388200805*np.tanh(((((data["min_first_10000"]) * (((data["mean_change_rate"]) * (((((data["min_roll_std_100"]) + (data["max_last_50000"]))) + (((((data["mean_change_rate"]) * (((data["mean_change_abs"]) * (((data["min_roll_std_100"]) + (data["min_first_10000"]))))))) - (data["mean_change_abs"]))))))))) * 2.0)) +
            0.0389529206*np.tanh(((np.tanh((np.tanh((((data["max_to_min"]) + (data["max_to_min"]))))))) - (np.tanh((((((((((((((((((((data["min_first_10000"]) * 2.0)) + (data["max_to_min"]))) * 2.0)) * 2.0)) + (data["max_last_50000"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)))))) +
            0.0399843715*np.tanh((-1.0*((((((data["max_first_50000"]) * 2.0)) * (((np.tanh(((((((-1.0*((((data["q95_roll_std_1000"]) + (((data["q01_roll_std_10"]) + (data["av_change_abs_roll_std_1000"])))))))) * 2.0)) * 2.0)))) + (((data["q01_roll_std_10"]) + (((data["max_first_50000"]) * (data["mean_change_abs"])))))))))))) +
            0.0399453007*np.tanh((((((data["av_change_abs_roll_std_1000"]) + (data["mean_change_abs"]))/2.0)) * (((data["av_change_abs_roll_std_100"]) + (((data["trend"]) + (((((data["av_change_abs_roll_mean_1000"]) + ((((data["av_change_abs_roll_std_100"]) + (((data["av_change_rate_roll_std_100"]) + (((data["av_change_abs_roll_std_1000"]) + (data["min_roll_std_10"]))))))/2.0)))) * (data["av_change_abs_roll_mean_1000"]))))))))) +
            0.0399140455*np.tanh(((data["std_first_50000"]) * (((data["mean_change_rate_last_50000"]) * ((-1.0*((((data["avg_first_50000"]) * ((((data["max_first_10000"]) + (((data["min_roll_mean_10"]) - (((data["avg_first_50000"]) - (((((data["mean_change_rate_last_50000"]) - (data["min_last_10000"]))) * (data["min_last_10000"]))))))))/2.0))))))))))) +
            0.0399999991*np.tanh(((((data["min_last_10000"]) + (np.tanh((((data["ave_roll_mean_1000"]) + (((((((data["av_change_abs_roll_mean_100"]) + (data["av_change_abs_roll_std_10"]))) * (data["ave_roll_mean_1000"]))) * (((data["min_roll_std_1000"]) * 2.0)))))))))) * (((data["min_roll_std_1000"]) * (((((data["std_roll_mean_1000"]) * 2.0)) * 2.0)))))) +
            0.0399765596*np.tanh((((((data["av_change_abs_roll_mean_100"]) + (data["q99_roll_std_10"]))/2.0)) * (np.tanh((((np.tanh((((((((((((((data["avg_first_10000"]) + (data["max_roll_std_1000"]))) + (data["count_big"]))) * (73.0))) - (data["av_change_rate_roll_mean_100"]))) * 2.0)) + (data["avg_first_10000"]))))) * 2.0)))))) +
            0.0371556953*np.tanh(((((data["max_last_10000"]) * (((((data["std_last_50000"]) * (data["avg_last_50000"]))) + (((((((data["min_roll_std_10"]) * 2.0)) + (((data["min_last_10000"]) * (((data["max_first_50000"]) * (((((data["min_roll_std_10"]) * 2.0)) * 2.0)))))))) * (data["max_first_10000"]))))))) * 2.0)) +
            0.0399609283*np.tanh(((data["av_change_abs_roll_std_10"]) * ((((((((((data["min_last_50000"]) + (((data["trend"]) * (data["av_change_abs_roll_mean_10"]))))/2.0)) + (data["max_to_min"]))/2.0)) + (((data["min_last_50000"]) - (((data["min_last_50000"]) * (((data["min_last_50000"]) * (data["q05_roll_mean_1000"]))))))))/2.0)))) +
            0.0391638987*np.tanh(((((data["trend"]) - ((((data["min_roll_std_100"]) + (data["q001"]))/2.0)))) * (((np.tanh((np.tanh((((((data["classic_sta_lta3_mean"]) + ((((data["classic_sta_lta3_mean"]) + ((((data["kurt"]) + (data["q95_roll_mean_100"]))/2.0)))/2.0)))) * 2.0)))))) * ((-1.0*((data["min_roll_std_100"])))))))) +
            0.0398749746*np.tanh(((data["ave10"]) * ((-1.0*((((data["av_change_abs_roll_mean_10"]) * (((data["av_change_abs_roll_mean_100"]) + ((-1.0*(((((data["av_change_abs_roll_mean_10"]) + (((((data["classic_sta_lta2_mean"]) * 2.0)) + (((data["mean_change_rate_last_10000"]) * (data["av_change_rate_roll_mean_100"]))))))/2.0)))))))))))))) +
            0.0385778472*np.tanh((-1.0*(((((np.tanh((((((((data["trend"]) + (((data["min_roll_std_10"]) + (data["Moving_average_6000_mean"]))))) * 2.0)) * 2.0)))) + (((((-1.0*((((data["q95_roll_std_100"]) * (data["ave10"])))))) + (((data["mean_change_rate_first_50000"]) * (data["av_change_abs_roll_mean_10"]))))/2.0)))/2.0))))) +
            0.0399921872*np.tanh(((((((((data["min_roll_std_10"]) * ((((data["ave_roll_std_10"]) + (data["avg_last_50000"]))/2.0)))) + (((((((((data["avg_last_10000"]) + (data["min_roll_std_10"]))/2.0)) + (data["min_roll_std_10"]))/2.0)) - (data["ave_roll_std_10"]))))/2.0)) + (((((data["q95_roll_std_100"]) - (data["ave_roll_std_10"]))) * 2.0)))/2.0)) +
            0.0399921872*np.tanh((((((-1.0*((((data["mean_change_rate_first_10000"]) * 2.0))))) * ((((((((((data["av_change_rate_roll_std_10"]) * (data["avg_last_10000"]))) + (((data["min_last_10000"]) * (data["av_change_rate_roll_std_10"]))))) - (((data["avg_first_10000"]) / 2.0)))) + (data["av_change_rate_roll_std_1000"]))/2.0)))) * (data["av_change_abs_roll_std_1000"]))) +
            0.0399453007*np.tanh(((data["av_change_abs_roll_std_1000"]) * (((data["avg_last_10000"]) * (((((((data["avg_last_10000"]) * (((data["avg_last_10000"]) * (((((data["med"]) * (((data["q001"]) - (data["classic_sta_lta1_mean"]))))) - (data["min_roll_std_10"]))))))) - (data["max_roll_mean_10"]))) - (data["min_roll_std_10"]))))))) +
            0.0398749746*np.tanh(np.tanh((np.tanh((((((((((((-1.0*((data["av_change_abs_roll_std_100"])))) + ((-1.0*((data["mean_change_rate_first_50000"])))))/2.0)) / 2.0)) - (((data["mean_change_abs"]) * (((((((data["avg_last_50000"]) * (data["q99_roll_mean_10"]))) - (data["mean_change_rate_first_50000"]))) - (data["std_last_10000"]))))))) * 2.0)))))) +
            0.0363899209*np.tanh(((data["max_last_10000"]) * (((((np.tanh((((((((data["av_change_rate_roll_mean_100"]) + (((data["min_roll_std_1000"]) * 2.0)))) * 2.0)) * 2.0)))) + (((data["av_change_abs_roll_std_100"]) + ((-1.0*((data["mean_change_abs"])))))))) + (((((data["av_change_rate_roll_mean_10"]) * (data["max_last_50000"]))) * 2.0)))))) +
            0.0399999991*np.tanh(((data["min_roll_std_100"]) * (((np.tanh((data["min_last_10000"]))) + (((((np.tanh((data["std_first_10000"]))) - ((((np.tanh((data["min_last_10000"]))) + (((data["exp_Moving_average_30000_mean"]) * (data["min_last_10000"]))))/2.0)))) * ((((((data["avg_last_50000"]) + (data["mean_change_abs"]))/2.0)) * 2.0)))))))) +
            0.0351396762*np.tanh(np.tanh((((((data["av_change_abs_roll_std_1000"]) * (data["mean_change_rate_last_10000"]))) + (((data["med"]) * (np.tanh((((data["av_change_rate_roll_std_100"]) - (((((data["avg_last_10000"]) - (((data["mean_change_rate_first_10000"]) - (((data["avg_last_10000"]) * (data["mean_change_rate_last_10000"]))))))) - (data["av_change_abs_roll_std_1000"]))))))))))))) +
            0.0399921872*np.tanh((((np.tanh((((((((((data["min_roll_std_10"]) + (((data["mean_change_rate_first_50000"]) + (((((data["mean_change_rate_first_50000"]) + (data["mean_change_rate_first_50000"]))) * (data["classic_sta_lta4_mean"]))))))) * (data["av_change_rate_roll_mean_1000"]))) * 2.0)) * 2.0)))) + (((data["av_change_abs_roll_mean_10"]) * (np.tanh((data["med"]))))))/2.0)) +
            0.0379214697*np.tanh((-1.0*((((((data["max_to_min_diff"]) * (((((((((((((data["min_last_10000"]) - (data["med"]))) - (data["mean_change_rate_first_10000"]))) + (data["q05_roll_std_100"]))) + (data["q01_roll_std_1000"]))) * 2.0)) * 2.0)))) * ((-1.0*((((data["mean_change_rate_first_10000"]) * (data["min_last_10000"]))))))))))) +
            0.0399218611*np.tanh(((data["min_first_10000"]) * (np.tanh((((((((((np.tanh((((data["max_to_min"]) * 2.0)))) + (((((((data["min_first_50000"]) * 2.0)) + (((np.tanh((np.tanh((data["mean_change_rate_last_10000"]))))) + (data["min_roll_mean_10"]))))) * 2.0)))) * 2.0)) * 2.0)) * 2.0)))))) +
            0.0328423530*np.tanh((-1.0*((((np.tanh((((data["kurt"]) * (((((((data["mean_change_rate_last_50000"]) * 2.0)) * 2.0)) * (data["skew"]))))))) + (np.tanh((((data["skew"]) * (((((data["mean_change_rate_last_50000"]) * 2.0)) * (data["skew"])))))))))))) +
            0.0346083194*np.tanh((((((data["av_change_abs_roll_std_10"]) / 2.0)) + ((-1.0*((((np.tanh((data["q05_roll_mean_1000"]))) + (np.tanh((((((data["min_roll_std_100"]) - (data["mean_change_rate_last_50000"]))) - (((((data["q95_roll_mean_1000"]) - (data["av_change_abs_roll_std_10"]))) - (((data["av_change_abs_roll_mean_1000"]) - (data["mean_change_rate_last_50000"])))))))))))))))/2.0)) +
            0.0395155288*np.tanh(((data["avg_last_50000"]) * ((-1.0*(((((np.tanh((np.tanh((((((((((((data["av_change_rate_roll_mean_100"]) * ((-1.0*((data["max_to_min"])))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)))))) + ((((((data["av_change_rate_roll_std_10"]) * (data["avg_last_10000"]))) + (data["av_change_rate_roll_std_10"]))/2.0)))/2.0))))))) +
            0.0316311792*np.tanh(((data["max_first_50000"]) - (((data["classic_sta_lta3_mean"]) * (((((data["max_to_min_diff"]) + (data["mean_change_abs"]))) - (((data["trend"]) * (((((data["mean_change_abs"]) - (((np.tanh((data["max_to_min_diff"]))) * 2.0)))) - (data["av_change_abs_roll_mean_100"]))))))))))) +
            0.0399609283*np.tanh(((data["trend"]) * (((((((data["min_roll_std_1000"]) * (data["std_last_10000"]))) * ((((data["av_change_abs_roll_mean_10"]) + (((data["std_last_50000"]) - ((((((data["av_change_rate_roll_mean_10"]) * (data["q05_roll_std_1000"]))) + (data["min_roll_std_1000"]))/2.0)))))/2.0)))) + (((data["min_roll_std_1000"]) - (data["q05_roll_std_1000"]))))))) +
            0.0362180099*np.tanh((((np.tanh((((((((((((data["q01_roll_std_100"]) - (data["mean_change_rate_last_10000"]))) - ((((data["av_change_abs_roll_mean_10"]) + (data["abs_max_roll_mean_100"]))/2.0)))) - (0.2525250018))) * (data["min_roll_std_10"]))) * (73.0))))) + (((data["min_roll_mean_1000"]) + (data["abs_max_roll_mean_100"]))))/2.0)) +
            0.0399999991*np.tanh((((np.tanh((((73.0) * (((((73.0) * (data["av_change_abs_roll_std_100"]))) - (data["av_change_abs_roll_mean_100"]))))))) + (np.tanh((((((((((data["av_change_rate_roll_mean_100"]) - (data["av_change_abs_roll_std_100"]))) - (data["av_change_abs_roll_std_100"]))) - (data["std_first_50000"]))) - (data["av_change_abs_roll_std_100"]))))))/2.0)) +
            0.0374213718*np.tanh((((-1.0*((((data["q05"]) * (((data["max_last_10000"]) * (((((data["min_roll_std_10"]) - (data["std_last_10000"]))) - (data["std_last_10000"])))))))))) + (((data["max_last_50000"]) * (((data["mean_change_rate_last_10000"]) - (np.tanh((((((data["av_change_rate_roll_std_10"]) * 2.0)) * 2.0)))))))))) +
            0.0331470966*np.tanh(((((((((data["abs_max_roll_std_1000"]) * 2.0)) * 2.0)) * 2.0)) * (((((data["mean_change_rate_last_50000"]) * ((((((data["abs_max_roll_std_1000"]) * (((data["mean_change_abs"]) * 2.0)))) + ((((data["std_first_50000"]) + (data["mean_change_rate_last_50000"]))/2.0)))/2.0)))) + (((data["mean_change_abs"]) * (data["av_change_rate_roll_std_1000"]))))))) +
            0.0396796241*np.tanh(((np.tanh((np.tanh((((((data["avg_first_10000"]) * (((((data["Moving_average_3000_mean"]) + (data["skew"]))) * 2.0)))) + (np.tanh(((-1.0*((((((data["ave_roll_mean_100"]) + (((data["av_change_abs_roll_std_1000"]) + (((data["avg_first_10000"]) * 2.0)))))) * 2.0))))))))))))) / 2.0)) +
            0.0399765596*np.tanh(((np.tanh((((((data["av_change_abs_roll_mean_10"]) + (data["exp_Moving_average_300_mean"]))) * (((((data["classic_sta_lta4_mean"]) + (data["av_change_abs_roll_mean_10"]))) + (((((data["av_change_abs_roll_mean_10"]) + (((np.tanh((((np.tanh((data["avg_last_50000"]))) * 2.0)))) * 2.0)))) * (data["av_change_abs_roll_mean_10"]))))))))) / 2.0)) +
            0.0396014862*np.tanh((((((data["max_last_50000"]) + (((((np.tanh((np.tanh((((((np.tanh((np.tanh(((((data["q95"]) + (data["mean_change_abs"]))/2.0)))))) + (((data["av_change_abs_roll_std_100"]) * 2.0)))) * 2.0)))))) * 2.0)) - (data["av_change_abs_roll_std_100"]))))/2.0)) * (data["classic_sta_lta4_mean"]))) +
            0.0317015015*np.tanh(((np.tanh((((((data["avg_last_10000"]) - (((data["iqr"]) - (((((data["avg_last_10000"]) - (((data["iqr"]) * (data["av_change_rate_roll_std_100"]))))) - (((((((((data["min_roll_std_100"]) * (data["av_change_rate_roll_std_100"]))) * 2.0)) * 2.0)) * 2.0)))))))) * 2.0)))) / 2.0)) +
            0.0399296731*np.tanh((-1.0*((((data["q05_roll_std_1000"]) * (((((((((data["avg_first_10000"]) + (data["mean_change_abs"]))/2.0)) * (data["std_last_50000"]))) + (((((((data["avg_last_10000"]) + (((data["mean_change_rate_first_10000"]) * (data["mean_change_abs"]))))/2.0)) + (data["max_to_min_diff"]))/2.0)))/2.0))))))) +
            0.0377104878*np.tanh((((np.tanh((np.tanh((((((((data["count_big"]) + (((data["min_last_10000"]) * (data["min_first_50000"]))))) * 2.0)) * 2.0)))))) + ((((data["count_big"]) + ((-1.0*((((data["max_roll_mean_100"]) - (np.tanh((((data["min_last_10000"]) * (data["avg_last_10000"])))))))))))/2.0)))/2.0)) +
            0.0309357308*np.tanh(((np.tanh((((((((data["max_to_min"]) * (data["classic_sta_lta2_mean"]))) + (((data["classic_sta_lta2_mean"]) * (((data["max_to_min"]) * (data["classic_sta_lta2_mean"]))))))) * 2.0)))) / 2.0)) +
            0.0355303772*np.tanh(((((np.tanh((np.tanh((((((((((data["av_change_abs_roll_std_10"]) - (np.tanh((((data["skew"]) * 2.0)))))) * (data["mean_change_rate_first_10000"]))) * (data["mean_change_rate_first_10000"]))) * (((data["mean_change_rate_first_10000"]) + (data["av_change_abs_roll_std_10"]))))))))) * (data["mean_change_rate_first_10000"]))) * (data["mean_change_rate_first_10000"]))) +
            0.0399843715*np.tanh(((data["av_change_abs_roll_std_1000"]) * (((((((data["trend"]) + (np.tanh((((data["min_roll_std_1000"]) * ((((8.0)) + (((data["min_roll_std_1000"]) * ((8.0)))))))))))/2.0)) + ((-1.0*((((data["q01_roll_std_1000"]) + (((data["q01_roll_std_1000"]) * (data["mean_change_rate_first_50000"])))))))))/2.0)))) +
            0.0398437195*np.tanh((-1.0*((((data["std_first_10000"]) * ((((((data["q99"]) / 2.0)) + (((data["skew"]) + (((((data["std_first_10000"]) * 2.0)) * (((data["std_first_10000"]) * (((data["std_first_10000"]) * (((data["std_first_10000"]) - (((data["MA_700MA_std_mean"]) * 2.0)))))))))))))/2.0))))))) +
            0.0399843715*np.tanh((-1.0*((((data["mean_change_abs"]) * ((((((data["av_change_abs_roll_mean_1000"]) + ((((((((data["abs_max_roll_mean_10"]) + (((((data["min_roll_mean_100"]) * (data["av_change_abs_roll_mean_1000"]))) * (data["min_roll_std_10"]))))) - ((((data["min_roll_std_10"]) + (data["av_change_abs_roll_std_100"]))/2.0)))) + (data["skew"]))/2.0)))/2.0)) / 2.0))))))) +
            0.0295057632*np.tanh(((((((((((((((data["q99_roll_mean_100"]) + (((data["q99_roll_mean_100"]) + (data["q95_roll_mean_10"]))))) * (((data["q05_roll_std_100"]) + ((((((data["av_change_abs_roll_std_1000"]) - (data["min_roll_std_100"]))) + (data["av_change_abs_roll_mean_100"]))/2.0)))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * (data["av_change_abs_roll_std_1000"]))) +
            0.0346864611*np.tanh(((np.tanh((((((((data["mean_change_rate"]) * (((data["mean_change_rate_first_10000"]) + (((data["mean_change_rate_first_10000"]) + (data["mean_change_rate_last_50000"]))))))) + (((data["q01_roll_std_100"]) * (((((data["mean_change_rate_first_10000"]) * (((data["q99_roll_std_10"]) * 2.0)))) + (data["mean_change_rate_first_10000"]))))))) * 2.0)))) / 2.0)) +
            0.0397265106*np.tanh(np.tanh((((((data["min_roll_std_100"]) * ((-1.0*((((data["av_change_abs_roll_mean_100"]) * ((((data["q05_roll_mean_100"]) + (data["mean_change_rate_first_10000"]))/2.0))))))))) - (((((((-1.0*((data["classic_sta_lta4_mean"])))) / 2.0)) + ((((data["av_change_rate_roll_mean_100"]) + ((((data["Moving_average_6000_mean"]) + (data["mean_change_rate_first_10000"]))/2.0)))/2.0)))/2.0)))))) +
            0.0399687439*np.tanh(((np.tanh((((data["min_first_50000"]) * (((((((((np.tanh((((((data["q01_roll_std_1000"]) + (np.tanh(((-1.0*((((((data["skew"]) + (data["classic_sta_lta2_mean"]))) + (data["q95_roll_std_10"])))))))))) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)))))) / 2.0)) +
            0.0365696438*np.tanh(((np.tanh((((((((((((((((data["mean_change_rate_last_10000"]) * (((data["ave_roll_std_10"]) - (((data["q95"]) - ((((((data["abs_mean"]) + (((data["iqr"]) / 2.0)))/2.0)) / 2.0)))))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) / 2.0)) +
            0.0393358096*np.tanh(((((data["max_roll_std_10"]) * (((((data["q95_roll_mean_100"]) + (((data["q95_roll_mean_100"]) + (((data["q05_roll_mean_1000"]) * (data["classic_sta_lta2_mean"]))))))) * (((data["max_roll_std_10"]) * (((((data["skew"]) * (((data["q05_roll_mean_100"]) + (data["classic_sta_lta2_mean"]))))) * 2.0)))))))) * 2.0)) +
            0.0381871462*np.tanh((((np.tanh((((((((data["q05_roll_mean_10"]) - (((data["q05"]) + (data["ave_roll_std_100"]))))) * (73.0))) * (73.0))))) + (((((((data["abs_q05"]) * (((data["std_roll_mean_1000"]) * (data["av_change_rate_roll_std_10"]))))) * 2.0)) - (data["q05"]))))/2.0)) +
            0.0383043550*np.tanh(((data["max_last_50000"]) * (((((data["av_change_rate_roll_std_100"]) * 2.0)) * (np.tanh((((((((((((((np.tanh((((data["mean_change_abs"]) - (data["classic_sta_lta3_mean"]))))) - (data["classic_sta_lta3_mean"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))))))) +
            0.0397812054*np.tanh(np.tanh((np.tanh((((data["min_roll_mean_100"]) * (((((data["av_change_rate_roll_mean_1000"]) - (data["min_roll_mean_100"]))) - (((((((data["min_last_50000"]) * (((data["av_change_rate_roll_mean_1000"]) - (np.tanh((((data["min_roll_std_10"]) - (((data["min_roll_mean_100"]) * 2.0)))))))))) * 2.0)) * 2.0)))))))))) +
            0.0399921872*np.tanh((-1.0*((((((np.tanh((((((((((((((((((data["mean_change_abs"]) * 2.0)) - ((((((-1.0*((data["mean_change_rate_last_50000"])))) * 2.0)) - (data["trend"]))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) / 2.0)) / 2.0))))) +
            0.0329673775*np.tanh(np.tanh(((-1.0*((((data["std_first_10000"]) * (((((((((data["av_change_abs_roll_mean_10"]) + (data["max_last_10000"]))) * (((((data["max_last_10000"]) * (0.8873239756))) + (data["classic_sta_lta2_mean"]))))) * 2.0)) * (data["max_last_10000"])))))))))) +
            0.0367962494*np.tanh(((data["avg_last_10000"]) * (((((data["std_roll_mean_1000"]) * ((-1.0*(((((((((data["mean_change_rate_first_50000"]) - (((data["mean_change_abs"]) * (data["max_last_10000"]))))) + (data["max_to_min_diff"]))/2.0)) - (((data["max_last_10000"]) * (((data["max_to_min"]) * (data["q05_roll_mean_100"])))))))))))) * 2.0)))) +
            0.0399921872*np.tanh(((data["av_change_abs_roll_mean_10"]) * (((((data["av_change_abs_roll_std_1000"]) - ((((data["av_change_abs_roll_mean_10"]) + ((((data["min_roll_std_10"]) + (((((((data["av_change_rate_roll_mean_10"]) * (data["av_change_abs_roll_std_10"]))) + (data["av_change_abs_roll_mean_10"]))) * (data["av_change_abs_roll_mean_10"]))))/2.0)))/2.0)))) * (data["min_roll_std_10"]))))) +
            0.0395311601*np.tanh(((np.tanh(((-1.0*((((data["std_first_10000"]) + (data["min_first_10000"])))))))) + (np.tanh((((data["max_roll_std_10"]) * (((data["q95_roll_mean_100"]) * ((((((((data["mean_change_abs"]) + (data["max_roll_std_10"]))/2.0)) - ((-1.0*((data["min_first_10000"])))))) + (data["mean_change_rate_last_50000"]))))))))))) +
            0.0399296731*np.tanh(((((data["min_roll_std_10"]) + (((data["min_roll_std_10"]) * (data["med"]))))) * ((((((data["classic_sta_lta1_mean"]) + (data["min_roll_mean_1000"]))/2.0)) - (((data["q05_roll_std_10"]) * ((((((data["med"]) * ((((data["classic_sta_lta1_mean"]) + (data["min_roll_mean_1000"]))/2.0)))) + (data["med"]))/2.0)))))))) +
            0.0399453007*np.tanh(np.tanh((((((((np.tanh((data["min_roll_std_10"]))) * (data["std_roll_mean_1000"]))) * 2.0)) + ((((((((((((data["max_to_min_diff"]) * 2.0)) * (data["std_last_50000"]))) + (data["min_roll_std_10"]))/2.0)) / 2.0)) + (((data["min_roll_std_10"]) * (np.tanh((data["max_to_min_diff"]))))))))))) +
            0.0394999012*np.tanh(((data["mean_change_rate_first_10000"]) * (((data["min_first_10000"]) * ((((((((data["min_roll_std_10"]) * (((data["av_change_abs_roll_std_100"]) + (data["av_change_abs_roll_std_100"]))))) + ((((data["mean_change_rate_first_10000"]) + (((data["min_roll_std_10"]) * (data["mean_change_rate_first_10000"]))))/2.0)))/2.0)) + (((data["min_roll_std_10"]) - (data["av_change_abs_roll_std_100"]))))))))) +
            0.0399687439*np.tanh(((data["max_first_10000"]) * (((((data["std_first_10000"]) * (((data["max_first_10000"]) * (((data["std_first_10000"]) * (((np.tanh((((data["std_first_10000"]) * (data["av_change_abs_roll_mean_10"]))))) + (data["std_roll_mean_100"]))))))))) - (((((data["av_change_abs_roll_mean_10"]) * (data["min_roll_std_100"]))) / 2.0)))))) +
            0.0397499502*np.tanh(((data["trend"]) * ((((((data["std_first_50000"]) + (np.tanh((np.tanh((((data["av_change_rate_roll_mean_10"]) + (((((((data["av_change_rate_roll_mean_1000"]) + ((((data["min_roll_std_100"]) + (((data["Hilbert_mean"]) + (data["av_change_rate_roll_mean_10"]))))/2.0)))) * 2.0)) * 2.0)))))))))/2.0)) * (data["min_roll_std_100"]))))) +
            0.0274897423*np.tanh((((np.tanh((((((((((((((data["q95_roll_std_100"]) - (data["ave_roll_std_10"]))) - (data["abs_max_roll_mean_10"]))) - (data["max_last_50000"]))) - (((data["min_first_50000"]) + (data["ave_roll_std_10"]))))) * 2.0)) * 2.0)))) + (((((data["q95_roll_std_100"]) - (data["ave_roll_std_10"]))) * 2.0)))/2.0)) +
            0.0399531126*np.tanh(((((data["mean_change_rate_first_10000"]) * (np.tanh((((((((((data["av_change_abs_roll_mean_10"]) + (data["mean_change_rate_last_10000"]))/2.0)) * (((data["avg_first_10000"]) - (np.tanh((data["av_change_abs_roll_mean_100"]))))))) + (np.tanh((np.tanh((data["avg_first_10000"]))))))/2.0)))))) * ((((-1.0*((data["mean_change_abs"])))) * 2.0)))) +
            0.0340613388*np.tanh(((data["skew"]) * (((data["max_first_50000"]) * (((((((((np.tanh((((((((data["iqr"]) * 2.0)) - (data["av_change_abs_roll_mean_10"]))) - (data["std_last_10000"]))))) + (data["av_change_abs_roll_mean_10"]))) - (((data["av_change_rate_roll_mean_10"]) * (data["avg_first_50000"]))))) * 2.0)) * 2.0)))))) +
            0.0399921872*np.tanh((((((data["min_roll_std_10"]) + (((data["min_roll_std_10"]) * (data["avg_first_10000"]))))/2.0)) * (np.tanh((((73.0) * (((data["avg_first_10000"]) * (((data["min_roll_std_10"]) + (((73.0) * (((73.0) * ((-1.0*((data["av_change_abs_roll_std_1000"])))))))))))))))))) +
            0.0356475860*np.tanh(np.tanh((((data["av_change_abs_roll_mean_1000"]) * ((((((((((((((data["avg_last_50000"]) + (data["kurt"]))) + (0.2783510089))/2.0)) + (data["av_change_rate_roll_std_1000"]))) + (0.2783510089))) + (data["kurt"]))) * ((((np.tanh((data["avg_first_10000"]))) + (data["mean_change_rate_first_50000"]))/2.0)))))))) +
            0.0329986326*np.tanh((((((((((data["kurt"]) + (data["q05_roll_mean_10"]))) * (data["min_roll_mean_10"]))) + (np.tanh((((((data["q05_roll_mean_10"]) - (((((data["max_to_min"]) - (data["kurt"]))) + (((data["q05_roll_mean_10"]) * (data["max_to_min"]))))))) * 2.0)))))/2.0)) / 2.0)) +
            0.0399999991*np.tanh(((data["std_roll_mean_1000"]) * (((data["med"]) * ((((-3.0) + (((((data["avg_first_10000"]) * 2.0)) * (((data["mean_change_rate_first_10000"]) - (((((((data["std_roll_mean_1000"]) / 2.0)) * (((data["std_roll_mean_1000"]) * (-3.0))))) * (data["abs_max_roll_mean_1000"]))))))))/2.0)))))) +
            0.0303496774*np.tanh((-1.0*((((np.tanh((np.tanh((((((((data["classic_sta_lta2_mean"]) + (((data["av_change_abs_roll_std_10"]) + (((data["av_change_abs_roll_mean_10"]) + (data["classic_sta_lta1_mean"]))))))) * (data["abs_q05"]))) * (3.6923100948))))))) / 2.0))))) +
            0.0399687439*np.tanh((((-1.0*((data["max_first_50000"])))) * (((((data["std_roll_mean_1000"]) + (((data["av_change_abs_roll_std_10"]) * (((data["ave_roll_mean_10"]) + (((data["med"]) + (1.0))))))))) * (((data["avg_last_50000"]) + (((data["med"]) + (2.0769200325))))))))) +
            0.0399843715*np.tanh(((np.tanh((((data["av_change_rate_roll_mean_100"]) * (((np.tanh((((((((((data["av_change_abs_roll_mean_100"]) * 2.0)) + (data["mean_change_rate_first_50000"]))) * 2.0)) * 2.0)))) - (((data["av_change_abs_roll_mean_100"]) * (((((data["av_change_rate_roll_std_1000"]) - (data["classic_sta_lta2_mean"]))) - (data["classic_sta_lta2_mean"]))))))))))) / 2.0)) +
            0.0399999991*np.tanh(((data["mean_change_abs"]) * ((((data["av_change_abs_roll_std_100"]) + (np.tanh((((((((((((((((((data["min_first_10000"]) - (data["av_change_rate_roll_std_10"]))) - (((data["av_change_abs_roll_std_100"]) * (3.6923100948))))) * 2.0)) * 2.0)) * 2.0)) + (data["mean_change_abs"]))) * 2.0)) * 2.0)))))/2.0)))) +
            0.0285446383*np.tanh(((np.tanh((((((73.0) * (data["mean_change_rate_first_10000"]))) + (((((((((data["trend"]) + (((data["av_change_abs_roll_mean_10"]) + (data["min_last_50000"]))))) + (data["mean_change_rate_first_10000"]))) * 2.0)) * 2.0)))))) + ((-1.0*((np.tanh((data["mean_change_rate_first_10000"])))))))) +
            0.0399687439*np.tanh(((((np.tanh((((((((((((((((((((np.tanh(((((((data["kurt"]) + (data["classic_sta_lta4_mean"]))/2.0)) - (data["max_first_10000"]))))) + (data["skew"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) / 2.0)) / 2.0)) +
            0.0398593470*np.tanh(((np.tanh(((-1.0*((np.tanh((((data["avg_first_10000"]) * (((data["min_roll_std_10"]) - (((((data["mean_change_rate_last_10000"]) * (data["min_roll_std_10"]))) + ((((data["mean_change_abs"]) + ((((((data["ave_roll_mean_100"]) + (data["avg_first_10000"]))/2.0)) + (data["skew"]))))/2.0))))))))))))))) / 2.0)) +
            0.0399296731*np.tanh(((np.tanh((np.tanh((np.tanh((np.tanh((((((((data["mean_change_abs"]) * ((((data["av_change_abs_roll_mean_100"]) + (0.7446810007))/2.0)))) - (((np.tanh((data["skew"]))) + ((((data["trend"]) + (0.7446810007))/2.0)))))) * 2.0)))))))))) / 2.0)) +
            0.0387341268*np.tanh((((((((data["min_roll_std_100"]) * (np.tanh((((((((data["av_change_rate_roll_std_100"]) * 2.0)) + (((data["abs_q05"]) + (np.tanh((data["iqr"]))))))) * 2.0)))))) + (np.tanh((((np.tanh((((data["iqr"]) * 2.0)))) - (data["min_roll_std_10"]))))))/2.0)) / 2.0)) +
            0.0384997055*np.tanh(((data["av_change_abs_roll_mean_1000"]) * (((np.tanh((((((np.tanh((data["std_first_50000"]))) + ((((-1.0*((data["max_to_min"])))) * (data["std_first_10000"]))))) + (((data["av_change_abs_roll_mean_1000"]) * (data["min_roll_mean_1000"]))))))) + ((((-1.0*((data["min_roll_std_10"])))) * (data["std_first_10000"]))))))) +
            0.0384918936*np.tanh(((np.tanh((np.tanh((((((73.0) * (((73.0) * (((((data["q95_roll_std_100"]) + (data["std_roll_mean_1000"]))) * 2.0)))))) * (((data["std_first_50000"]) + (((np.tanh((np.tanh((data["max_roll_std_1000"]))))) * (data["kurt"]))))))))))) / 2.0)) +
            0.0343192033*np.tanh(((((-1.0) - (np.tanh((((((((0.8873239756) + (data["mean_change_abs"]))) - (data["Hann_window_mean"]))) * (((((((((data["Hann_window_mean"]) - (data["min_last_10000"]))) * 2.0)) * 2.0)) + (data["av_change_rate_roll_std_100"]))))))))) / 2.0)) +
            0.0372416489*np.tanh(((np.tanh((((((data["classic_sta_lta2_mean"]) + (((data["q95_roll_mean_1000"]) * (data["classic_sta_lta2_mean"]))))) + ((((-1.0*((((((data["std_last_10000"]) + (data["MA_400MA_BB_low_mean"]))) + (np.tanh((((data["min_roll_std_100"]) + (data["classic_sta_lta2_mean"])))))))))) * (data["avg_last_10000"]))))))) / 2.0)) +
            0.0377808139*np.tanh(np.tanh((((((data["mean_change_rate_last_10000"]) * (((((((data["mean_change_rate_first_10000"]) + ((((data["min_roll_std_10"]) + ((-1.0*((-3.0)))))/2.0)))/2.0)) + (data["Moving_average_1500_mean"]))/2.0)))) * (np.tanh((((((data["mean_change_rate_last_10000"]) + (((data["av_change_abs_roll_std_1000"]) + (1.9019600153))))) * 2.0)))))))) +
            0.0369056463*np.tanh(np.tanh((np.tanh((((((data["max_first_50000"]) * (data["max_first_50000"]))) * ((((((((data["max_first_50000"]) + (data["kurt"]))/2.0)) - (data["avg_last_50000"]))) - (((np.tanh((((data["mean_change_rate_first_10000"]) * 2.0)))) - (((data["kurt"]) - (0.5555559993))))))))))))))

In [17]:
alldata = pd.concat([X_tr,
                     X_test])

scaler = StandardScaler()
alldata = pd.DataFrame(scaler.fit_transform(alldata), columns=alldata.columns)

In [18]:
mean_absolute_error(y_tr,GPI_activation(alldata[:X_tr.shape[0]])) 

1.9135853993245366

In [27]:
mean_absolute_error(y_tr,GPI_activation1(alldata[:X_tr.shape[0]])) 

1.8719039823775079

In [19]:
mean_absolute_error(y_tr,GPI_activation2(alldata[:X_tr.shape[0]]))

1.8848160683887825

In [20]:
mean_absolute_error(y_tr,GPI_erf(alldata[:X_tr.shape[0]]))

1.8576862445099833

In [21]:
mean_absolute_error(y_tr,GPI_tanh(alldata[:X_tr.shape[0]])) 

1.8640317460810856

In [23]:
submission.time_to_failure = GPI_activation(alldata[X_tr.shape[0]:]).values
submission.to_csv('gp1_submission.csv',index=True)
submission.head()

,time_to_failure
seg_id,
seg_00030f,3.234512284696534
seg_0012b5,5.114868525120073
seg_00184e,4.803023345748058
seg_003339,8.222042936510514
seg_0042cc,6.700822916987057


In [22]:
submission.time_to_failure = GPI_activation1(alldata[X_tr.shape[0]:]).values
submission.to_csv('gp2_submission.csv',index=True)
submission.head()

,time_to_failure
seg_id,
seg_00030f,3.083893929092591
seg_0012b5,5.084677125780372
seg_00184e,4.838270839979383
seg_003339,7.948219133460869
seg_0042cc,6.633578310344631


In [24]:
submission.time_to_failure = GPI_activation2(alldata[X_tr.shape[0]:]).values 
submission.to_csv('gp3_demisubmission.csv',index=True)
submission.head()

,time_to_failure
seg_id,
seg_00030f,3.104928009385487
seg_0012b5,5.086738103308617
seg_00184e,4.797447604269748
seg_003339,8.078895629689793
seg_0042cc,6.647867286216952


In [25]:
submission.time_to_failure = GPI_erf(alldata[X_tr.shape[0]:]).values 
submission.to_csv('gp4_demisubmission.csv',index=True)
submission.head()

,time_to_failure
seg_id,
seg_00030f,3.074744837235399
seg_0012b5,5.062844513373911
seg_00184e,4.888794785203718
seg_003339,7.844473929510527
seg_0042cc,6.690332174816142


In [26]:
submission.time_to_failure = GPI_tanh(alldata[X_tr.shape[0]:]).values 
submission.to_csv('gp5_demisubmission.csv',index=True)
submission.head()

,time_to_failure
seg_id,
seg_00030f,3.083095164530614
seg_0012b5,5.079934847428867
seg_00184e,4.867285433765740
seg_003339,7.879076333935096
seg_0042cc,6.621312866118349
